<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/BM25_topK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q rank_bm25

In [18]:
import string
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi
from sklearn.utils import shuffle
from tqdm.autonotebook import tqdm

In [10]:
corpus = ['quem está obrigado a apresentar a declaração de ajuste anual relativa ao exercício de 2020,  ano-calendário de 2019?   está obrigada a apresentar a declaração de ajuste anual   referente ao exercício de 2020, a pessoa  física residente no brasil que, no ano-calendário de 2019:   1 - recebeu rendimentos tributáveis, sujeitos ao ajuste na declaração, cuja soma foi superior a $ 28.559,70  (vinte e oito mil, quinhentos e cinquenta e nove reais e setenta centavos);   2 - recebeu rendimentos isentos, não tributáveis ou tributados exclusivamente na fonte, cuja soma foi superior  a $ 40.000,00 (quarenta mil reais);   3 - obteve, em qualquer mês, ganho de capital na alienação de bens ou direitos, sujeito à incidência do  imposto, ou realizou operações em bolsas de valores, de mercadorias, de futuros e assemelhadas;   4 - relativamente à atividade rural:   a) obteve receita bruta em valor superior a $ 142.798,50 (cento e quarenta e dois mil, setecentos e noventa  e oito reais e cinquenta centavos);   b) pretenda compensar, no ano-calendário de 2019 ou posteriores, prejuízos de anos-calendário anteriores  ou do próprio ano-calendário de 2019;   5 - teve, em 31 de dezembro, a posse ou a propriedade de bens ou direitos, inclusive terra nua, de valor total  superior a $ 300.000,00 (trezentos mil reais);   6 - passou à condição de residente no brasil em qualquer mês e nesta condição se encontrava em 31 de  dezembro; ou   7 - optou pela isenção do imposto sobre a renda incidente sobre o ganho de capital auferido na venda de  imóveis residenciais, cujo produto da venda seja destinado à aplicação na aquisição de imóveis residenciais  localizados no país, no prazo de 180 (cento e oitenta) dias contados da celebração do contrato de venda, nos  termos do art. 39 da lei nº 11.196, de 21 de novembro de 2005.   atenção:    apresentação da declaração    declaração de ajuste anual   deve ser elaborada, exclusivamente, com o uso de:   i - computador, mediante a utilização do programa gerador da declaração   relativo ao  exercício de 2020, disponível no sítio da secretaria especial da receita federal do brasil  , na  internet, no endereço http://receita.economia.gov.br;   ii - computador, mediante acesso ao serviço “meu imposto de renda (extrato da )”,  disponível no centro virtual de atendimento (e-) no sítio da  na internet, no endereço de  que trata o item “i” deste “atenção”; ou   iii - dispositivos móveis, tais como tablets e smartphones, mediante acesso ao “meu imposto de  renda”, por meio do respectivo  disponível nas lojas de aplicativos google play, para o sistema  operacional android, ou app store, para o sistema operacional ios.   inicialmente, estava previsto a obrigatoriedade de se informar o número constante no recibo de  entrega da última declaração apresentada, relativa ao exercício de 2019, ano-calendário 2018, com  algumas exceções. entretanto, em função da pandemia de covid-19, não é mais obrigatório  informar esse número de recibo na apresentação da  original em nenhuma hipótese.    acesso ao serviço “meu imposto de renda (extrato da )” com a utilização de computador  será feito com certificação digital: a) pelo contribuinte; ou b) por seu representante, com procuração  eletrônica ou com a procuração de que trata a instrução normativa  nº 1.751, de 16 de outubro  de 2017.   é vedado o preenchimento e a apresentação da declaração por meio do “meu imposto de renda”,  disponível para dispositivos móveis, nas hipóteses de os declarantes ou seus dependentes  informados nessa declaração, no ano-calendário de 2019:    – terem auferido:   a) rendimentos tributáveis:   1. sujeitos ao ajuste anual, cuja soma foi superior a $ 5.000.000,00 (cinco milhões de reais),  apenas na hipótese de utilização do serviço por meio de dispositivos móveis;   2. recebidos do exterior;      23   b) os seguintes rendimentos sujeitos à tributação exclusiva ou definitiva:   1. ganhos de capital na alienação de bens ou direitos;   2. ganhos de capital na alienação de bens, direitos e aplicações financeiras adquiridos em  moeda estrangeira;   3. ganhos de capital na alienação de moeda estrangeira mantida em espécie; ou   4. ganhos líquidos em operações de renda variável realizadas em bolsa de valores, de  mercadorias, de futuros e assemelhadas, e fundos de investimento imobiliário;   c) os seguintes rendimentos isentos e não tributáveis:   1. rendimentos cuja soma foi superior a $ 5.000.000,00 (cinco milhões de reais), apenas na  hipótese de utilização do serviço por meio de dispositivos móveis;   2. parcela isenta correspondente à atividade rural;   3. recuperação de prejuízos em renda variável (bolsa de valores, de mercadorias, de futuros e  assemelhados e fundos de investimento imobiliário);   4. lucro na venda de imóvel residencial para aquisição de outro imóvel residencial;   5. lucro na alienação de imóvel residencial adquirido após o ano de 1969; ou   d) rendimentos tributados exclusivamente na fonte, cuja soma foi superior a $ 5.000.000,00 (cinco  milhões de reais), apenas na hipótese de utilização do serviço por meio de dispositivos móveis;     - terem-se sujeitado:   a) ao imposto pago no exterior ou ao recolhimento do imposto sobre a renda na fonte de que trata  os §§ 1º e 2º do art. 2º da lei nº 11.033, de 21 de dezembro de 2004; ou   b) ao preenchimento dos demonstrativos referentes à atividade rural, ao ganho de capital na  alienação de bens e direitos, ao ganho de capital em moeda estrangeira ou à renda variável; ou    - terem realizado pagamentos de rendimentos a pessoas jurídicas, quando constituam dedução  na declaração, ou a pessoas físicas, quando constituam, ou não, dedução na declaração, cuja soma  foi superior a $ 5.000.000,00 (cinco milhões de reais), apenas na hipótese de utilização do serviço  por meio de dispositivos móveis, em cada caso ou no total.   dispensa da entrega da declaração   a -  pessoa física que se enquadrar em uma ou mais das hipóteses de obrigatoriedade previstas  nos itens 1 a 7 fica dispensada de apresentar a declaração de ajuste anual  , caso conste  como dependente em declaração apresentada por outra pessoa física, na qual tenham sido  informados seus rendimentos, bens e direitos, caso os possua;   b – a pessoa física que se enquadrar apenas na hipótese prevista no item 5 e que, na constância  da sociedade conjugal ou da união estável, os bens comuns tenham sido declarados pelo outro  cônjuge companheiro, fica dispensada de apresentar declaração de ajuste anual  , desde que  o valor total dos seus bens privativos não exceda $ 300.000,00 (trezentos mil reais)',
 'pessoa física desobrigada pode apresentar a declaração de ajuste anual  ?   sim.  pessoa física, ainda que desobrigada, pode apresentar a declaração de ajuste anual  , sendo  vedado a um mesmo contribuinte constar simultaneamente em mais de uma declaração de ajuste anual, seja  como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de  2019.',
 'contribuinte que é titular ou sócio de empresa está obrigado a apresentar a declaração de  ajuste anual do exercício 2020?   não, a menos que se enquadre nas hipóteses previstas na resposta à pergunta 001. não é a condição de  titular ou sócio de empresa, por si só, que obriga à apresentação de declaração de ajuste anual.   atenção:    ficam dispensadas da obrigatoriedade de entrega da declaração de ajuste anual relativa aos  exercícios de 2006 a 2009, anos-calendário de 2005 a 2008, as pessoas físicas sócias  exclusivamente de pessoas jurídicas que tiveram sua inscrição no cadastro nacional da pessoa  jurídica   baixada, nos termos previstos na instrução normativa nº 1.035, de 28 de maio de  2010, desde que a única condição de obrigatoriedade para entrega da referida declaração seja a  participação, em qualquer mês do referido período, no quadro societário de sociedade empresária  ou simples, como sócio ou acionista, ou como titular de empresa individual.',
 'contribuinte, que participou de quadro societário de sociedade anônima ou que foi associado  de cooperativa em 2019, deve apresentar, por esses motivos, a declaração de ajuste anual do  exercício de 2020?   não, a menos que esteja obrigado a declarar caso se enquadre nas hipóteses previstas na pergunta 001.  não é o fato de ter participado de quadro societário de sociedade anônima ou ter sido associado de  cooperativa, por si só, que obriga à apresentação de declaração de ajuste anual.',
 'contribuinte que constou como responsável perante a secretaria especial da receita federal  do brasil   por cadastro nacional da pessoa jurídica   de associações (bairros, creches,  clubes etc.) no ano-calendário de 2019, deve apresentar a declaração de ajuste anual do exercício  de 2020?   esse contribuinte está obrigado a declarar caso se enquadre nas hipóteses previstas na pergunta 001. não  é o fato de ter constado como responsável perante a secretaria especial da receita federal do brasil    por cadastro nacional da pessoa jurídica   de associações (bairros, creches, clubes etc.), por si só,  que obriga a apresentação de declaração de ajuste anual.',
 'existe limite de idade para a obrigatoriedade ou dispensa de apresentação da declaração de  ajuste anual?   não há limitação quanto à idade. ',
 'para verificação da obrigatoriedade de entrega da declaração de ajuste anual, qual é o critério  a ser utilizado para avaliar os bens e direitos, no caso de contribuinte dispensado de apresentar a  declaração nos últimos cinco anos?   é o custo de aquisição. tratando-se de bens e direitos cuja aquisição tenha ocorrido até 1995, o custo de  aquisição pode ser atualizado até 31/12/1995, tomando-se por base o valor da ufir vigente em 01/01/1996,  não se lhe aplicando qualquer atualização a partir dessa data.   tratando-se de bens e direitos cuja aquisição tenha ocorrido após 31/12/1995, ao custo de aquisição não é  aplicada qualquer atualização.   atenção:     pessoa física sujeita à apresentação da declaração de ajuste anual deve nela relacionar os bens  e direitos que, no brasil ou no exterior, constituam, em 31 de dezembro de 2018 e de 2019, seu  patrimônio e o de seus dependentes relacionados na declaração, bem como os bens e direitos  adquiridos e alienados no decorrer do ano-calendário de 2019.   devem ser declarados quaisquer recebíveis que constituam créditos do declarante, tais como  cheques ou assemelhados.   devem também ser informados as dívidas e os ônus reais existentes em 31 de dezembro de 2018  e de 2019, do declarante e de seus dependentes relacionados na declaração de ajuste anual, bem  como os constituídos e os extintos no decorrer do ano-calendário de 2019.   fica dispensada, em relação a valores existentes em 31 de dezembro de 2019, a inclusão de:    - saldos de contas correntes bancárias e demais aplicações financeiras, cujo valor unitário não  exceda a $ 140,00 (cento e quarenta reais);    - bens móveis, exceto veículos automotores, embarcações e aeronaves, bem como os direitos,  cujo valor unitário de aquisição seja inferior a $ 5.000,00 (cinco mil reais);    - conjunto de ações e quotas de uma mesma empresa, negociadas ou não em bolsa de valores,  bem como ouro, ativo financeiro, cujo valor de constituição ou de aquisição seja inferior a  $ 1.000,00 (um mil reais);    - dívidas e ônus reais, cujo valor seja igual ou inferior a $ 5.000,00 (cinco mil reais).',
 'posse ou a propriedade de bens e direitos relativos à atividade rural de valor superior a  $ 300.000,00, exceto terra nua, obriga o contribuinte à apresentação da declaração de ajuste anual  do exercício de 2020?   os bens vinculados à atividade rural, tais como maquinários, semoventes, safra em estoque, não integram o  limite para efeito de obrigatoriedade de apresentação da declaração de ajuste anual, exceto para aqueles  contribuintes que mantiveram tais bens na declaração de bens e direitos da referida declaração de ajuste.',
 'contribuinte deve apresentar uma declaração de ajuste anual para cada fonte pagadora  dos rendimentos que auferir?   não.  contribuinte deve apresentar somente uma declaração de ajuste anual, independentemente do  número de fontes pagadoras, informando todos os rendimentos recebidos durante o ano-calendário de 2019.',
 'dependente que possui caderneta de poupança em valor superior a $ 300.000,00 está  obrigado a declarar?   está obrigado a apresentar a declaração de ajuste anual, o contribuinte que, em 31 de dezembro de  2019, teve a posse ou a propriedade de bens e direitos, inclusive terra nua, de valor total superior a  $ 300.000,00. portanto, o titular de caderneta de poupança com saldo superior a $ 300.000,00 está  obrigado a apresentar a declaração.   fica dispensada de apresentar a , a pessoa física que, embora se enquadre em qualquer das hipóteses  de obrigatoriedade, conste como dependente em declaração apresentada por outra pessoa física, na qual  tenham sido informados seus rendimentos, bens e direitos, caso os possua.',
 'contribuinte com doença grave está desobrigado de apresentar a declaração?   não.  isenção relativa à doença grave especificada em lei não desobriga, por si só, o contribuinte de  apresentar declaração.   (instrução normativa  nº 1.924, de 19 de fevereiro de 2020)   consulte as perguntas 001 e 267              -',
 'que se considera como opção pelo desconto simplificado?    opção pelo desconto simplificado implica a substituição de todas as deduções admitidas na legislação  tributária, correspondente à dedução de 20% do valor dos rendimentos tributáveis na declaração de ajuste  anual, limitado a $ 16.754,34 (dezesseis mil, setecentos e cinquenta e quatro reais e trinta e quatro  centavos). não necessita de comprovação e pode ser utilizado independentemente do montante dos  rendimentos recebidos e do número de fontes pagadoras.    valor utilizado a título de desconto simplificado não justifica variação patrimonial, sendo considerado  rendimento consumido.',
 'um cônjuge ou companheiro que apresenta declaração utilizando-se das deduções legais  pode incluir um dependente comum se o outro cônjuge ou companheiro apresentou a declaração  utilizando-se do desconto simplificado e não incluiu tal dependente?   sim.  apresentação de declaração com opção pelo desconto simplificado por um dos cônjuges ou  companheiros, em que não há a inclusão de dependente comum, não impede que o outro cônjuge ou  companheiro apresente declaração com a utilização das deduções legais, incluindo o dependente comum na  declaração e utilizando as deduções a ele relacionadas e vice-versa.    desconto simplificado substitui todas as deduções previstas na legislação tributária às quais o contribuinte  faria jus caso optasse pela declaração com base nas deduções legais, entretanto, não substitui as deduções  relacionadas a pessoas que, embora possam ser consideradas dependentes perante a legislação tributária,  não constam da declaração.',
 'quem pode optar pelo desconto simplificado na apresentação da declaração de ajuste  anual?   qualquer contribuinte pode optar pelo desconto simplificado. entretanto, após o prazo para a apresentação  da declaração, não será admitida a mudança na forma de tributação de declaração já apresentada.   (lei nº 9.250, de 26 de dezembro de 1995, art.10, inciso ; instrução normativa  nº 1.131, de  20 de fevereiro de 2011.',
 'contribuinte que tem mais de uma fonte pagadora pode optar pelo desconto simplificado?   sim.  contribuinte que tem mais de uma fonte pagadora pode optar pelo desconto simplificado. ele deve  preencher a declaração de ajuste anual, informando nos campos pertinentes os nomes e os números de  inscrição no cadastro nacional da pessoa jurídica   ou no cadastro de pessoas físicas   de todas  as fontes, bem como indicar os rendimentos de todas as fontes e os respectivos impostos retidos.',
 'contribuinte que em 2019 recebeu rendimentos tributáveis na declaração de ajuste anual  e obteve receita da atividade rural, mas com resultado negativo (prejuízo), pode optar pelo desconto  simplificado?   sim, ele pode optar pelo desconto simplificado na declaração de ajuste anual e informar o prejuízo na ficha  atividade rural.',
 'contribuinte que optar pelo desconto simplificado deve preencher as fichas “pagamentos  efetuados” e “doações efetuadas”?   independentemente da forma de tributação escolhida pelo contribuinte, deve-se preencher as fichas   “pagamentos efetuados” e “doações efetuadas” incluindo todos os pagamentos e doações efetuados a:   - pessoas físicas, tais como pensão alimentícia, aluguéis, arrendamento rural, instrução, pagamentos a   profissionais autônomos (médicos, dentistas, psicólogos, advogados, engenheiros, arquitetos, corretores,   professores, mecânicos, e outros);   - pessoas jurídicas, quando constituam exclusão ou dedução na declaração do contribuinte.    falta das informações relativas ao preenchimento da ficha “pagamentos efetuados” sujeita o contribuinte à  multa de 20% do valor não declarado.',
 'desconto simplificado substitui a parcela de isenção referente a rendimentos de  aposentadoria recebidos por contribuinte maior de 65 anos?   não.  parcela isenta referente a rendimentos de aposentadoria recebidos por contribuinte maior de 65 anos  deve ser informada na declaração de ajuste anual na ficha correspondente aos rendimentos isentos e não  tributáveis.  desconto simplificado aplica-se apenas aos rendimentos tributáveis e substitui as deduções  legais cabíveis, limitado a $ 16.754,34.',
 'contribuinte que optar pelo desconto simplificado pode excluir os honorários advocatícios  pagos referentes a rendimentos recebidos acumuladamente, por decisão judicial?   sim.  contribuinte, independentemente da opção pelo desconto simplificado ou não, pode informar como rendimento tributável o valor recebido, excluindo os honorários pagos na proporção dos rendimentos  tributáveis.',
 'contribuinte que optar pelo desconto simplificado pode excluir as despesas com  condomínio, taxas, impostos, em relação a aluguéis recebidos?   sim.  contribuinte, independentemente da opção pelo desconto simplificado ou não, deve informar como  rendimento tributável o valor dos aluguéis recebidos, podendo excluir os impostos, as taxas e os emolumentos  incidentes sobre o bem que produzir o rendimento, desde que o ônus desses encargos tenha sido  exclusivamente do declarante.',
 'qual é o prazo de apresentação da declaração de ajuste anual do exercício de 2020?    declaração de ajuste anual deve ser apresentada no período de 2 de março a 30 de junho de 2020.    serviço de recepção da declaração, pela internet, será interrompido às 23h59min59s (vinte e três horas,  cinquenta e nove minutos e cinquenta e nove segundos), horário de brasília, do último dia do prazo  estabelecido.',
 'qual é o prazo de apresentação da declaração de ajuste anual para a pessoa física ausente  do brasil?    pessoa física que se encontra no exterior deve apresentar sua declaração até 30 de junho de 2020.',
 'contribuinte que na data final da apresentação da declaração se encontra em viagem, fora de  seu domicílio fiscal, tem direito à prorrogação desse prazo?   não. recomenda-se que o contribuinte apresente sua declaração no prazo legal, no local onde se encontrar,  indicando, no campo próprio, seu domicílio fiscal permanente.',
 'qual é a penalidade aplicável na apresentação da declaração de ajuste anual depois do prazo  ou na sua não apresentação?    contribuinte obrigado a apresentar a declaração, no caso de apresentação após o prazo previsto ou da não  apresentação, fica sujeito ao pagamento de multa por atraso, calculada da seguinte forma: existindo imposto devido, multa de 1% ao mês-calendário ou fração de atraso, incidente sobre o  imposto devido, ainda que integralmente pago, observados os valores mínimo de $ 165,74 e máximo  de 20% do imposto devido;     inexistindo imposto devido, multa de $ 165,74.    multa terá por termo inicial o primeiro dia subsequente ao fixado para a entrega da declaração e por termo  final o mês da entrega ou, no caso de não apresentação, do lançamento de ofício.     no caso do não pagamento da multa por atraso na entrega dentro do vencimento estabelecido na  notificação de lançamento emitida pelo  ou pelo serviço “meu imposto de renda (extrato da  )” ou, ainda, pelo aplicativo “meu imposto de renda” (consultar os itens “i”, “ii” e “iii” do “atenção”  da pergunta 001), a multa, com os respectivos acréscimos legais decorrentes do não pagamento,  será deduzida do valor do imposto a ser restituído para as declarações com direito a restituição.   atenção:   a)  entrega de declaração de ajuste anual retificadora não está sujeita à multa por atraso na  entrega;       30   b)  contribuinte que deixou de apresentar, no prazo previsto, a declaração de ajuste anual,  quando estava obrigado a fazê-lo, deverá fazer o download, do sítio da  na internet, do  programa relativo ao ano-calendário correspondente e após preencher a declaração de acordo com  as instruções vigentes para aquele ano, apresentá-la:  b.1) pela internet, mediante utilização do programa de transmissão receitanet;  b.2) a partir do exercício de 2017, pela internet, no respectivo programa , na opção “entregar  declaração”;  b.3) utilizando o serviço “meu imposto de renda (extrato da ” ou  “meu imposto de  renda” (consultar os itens “ii” e “iii” do “atenção” da pergunta 001), na hipótese de apresentação  de declaração original; ou  b.4) em mídia removível, nas unidades da , durante o seu horário de expediente.',
 'contribuinte não obrigado à entrega da declaração de ajuste anual está sujeito à multa por  atraso na apresentação da declaração?   não é devida a multa por atraso na apresentação da declaração para quem está desobrigado de apresentar  a declaração de ajuste anual.',
 'qual é o programa utilizado para preenchimento da declaração de ajuste anual do ano- calendário de 2019, exercício de 2020?   para o ano-calendário de 2019, exercício de 2020, deve ser utilizado o programa  2020 para  preenchimento da declaração de ajuste anual. 2020, entre outras condições, observa os limites legais, apura automaticamente o imposto a pagar ou  a restituir e informa ao contribuinte a opção de declaração que lhe é mais favorável.   para demais formas de elaboração da declaração de ajuste anual consulte o “atenção” contido na pergunta  001.   atenção:    da elaboração   a)  declaração de ajuste anual, para o ano-calendário de 2019, exercício de 2020, deve ser  elaborada com o uso do computador mediante a utilização do programa  2020, localizado no  sítio http://receita.economia.gov.br;   da declaração de ajuste anual pré-preenchida   b)  contribuinte pode utilizar a declaração de ajuste anual pré-preenchida, desde que:   b.1) inicie uma nova declaração, com a utilização de computador:   b.1.1) por meio do irpf2020, a partir da tela de entrada do programa, aba “nova”, selecionando a  opção “iniciar declaração a partir da pré-preenchida”; ou   b.1.2) mediante acesso ao serviço “meu imposto de renda (extrato da )” e a partir da tela  inicial do e-, selecionando, dentro do menu “declarações e demonstrativos”, o item “meu  imposto de renda (extrato da )” e, em seguida, dentro do menu “declaração”, o item  “preencher declaração online” e, posteriormente, “importar declaração pré-preenchida;   b.2) no momento da importação do arquivo referido no item “c” (abaixo), as fontes pagadoras ou as  pessoas jurídicas ou equiparadas, conforme o caso, tenham enviado para a  informações  relativas ao contribuinte referentes ao exercício de 2020, ano-calendário de 2019, por meio da:   b.2.1) declaração do imposto sobre a renda retido na fonte (dirf);      31   b.2.2) declaração de serviços médicos e de saúde (dmed); ou   b.2.3) declaração de informações sobre atividades imobiliárias (dimob).   c)   disponibiliza ao contribuinte um arquivo a ser importado para a declaração de ajuste  anual, já contendo algumas informações relativas a rendimentos, deduções, bens e direitos e  dívidas e ônus reais.   c.1) o acesso às informações do arquivo de que trata o item “b.2” a ser importado para a declaração  de ajuste anual, dar-se-á somente com certificado digital e pode ser feito pelo:   - contribuinte; ou   - representante do contribuinte com procuração eletrônica ou procuração de que trata a instrução  normativa  nº 1.751, de 16 de outubro de 2017.   c.2) o arquivo deve ser obtido no centro virtual de atendimento (e-) no sítio da , na  internet, no endereço referido no item “a” deste atenção;   d) é de inteira responsabilidade do contribuinte a verificação da correção de todos os dados pré- preenchidos na declaração de ajuste anual, devendo ele realizar as alterações, inclusões e  exclusões das informações necessárias, se for o caso.   e)  possibilidade de utilização da declaração de ajuste anual pré-preenchida não se aplica à  declaração de ajuste anual elaborada com a utilização do aplicativo “meu imposto de renda” por  meio de dispositivos móveis de que trata o item “iii” contido no “atenção” da pergunta 001.',
 'onde obter o programa  2020?    programa  2020 para a declaração de ajuste anual pode ser obtido no sítio da secretaria especial da  receita federal do brasil   na internet, no endereço http://receita.economia.gov.br.   localize o programa  2020 a partir da barra de menu na opção “onde encontro”; "download de  aplicativos”; “ você";  – declaração do imposto de renda de pessoa física” e siga as  orientações para download constantes no sítio da  na internet.  ',
 'qual é o equipamento necessário para a utilização do programa  2020?       foi desenvolvido em java multiplataforma e pode ser executado em diversos sistemas operacionais  desde que tenha instalada no computador a máquina virtual java  , versão 1.8 ou superior.',
 'como instalar o programa  2020?   1 - acesse o sítio da secretaria especial da receita federal do brasil   na internet, no endereço  http://receita.economia.gov.br;   2 - localize o programa  2020;   3 - siga as orientações para download constantes no sítio da  na internet. ',
 'uais são os meios a serem utilizados para transmissão da declaração de ajuste anual do  ano-calendário de 2019, exercício de 2020?    declaração de ajuste anual deve ser apresentada no período de 2 de março a 30 de junho de 2020, pela  internet, mediante a utilização do próprio programa gerador da declaração   da declaração ou mediante  a utilização do programa de transmissão receitanet, disponível no sítio da , no endereço  http://receita.economia.gov.br ou do serviço “meu imposto de renda (extrato da )” ou, ainda, pelo  aplicativo “meu imposto de renda”, disponível para dispositivos móveis (consulte os itens “ii” e “iii” do  “atenção” da pergunta 001).    comprovação da apresentação da declaração de ajuste anual é feita por meio de recibo gravado após a  transmissão, em disco rígido de computador, em mídia removível ou no dispositivo móvel que contenha a  declaração transmitida, cuja impressão fica a cargo do contribuinte.   procedimentos para a transmissão, utilizando o próprio :   a) a partir da barra de menu na opção “declaração”; “entregar declaração”;   b) selecionar a declaração a ser transmitida;   c) seguir as orientações do programa.   procedimentos para a transmissão, utilizando o receitanet:   a) gravar a declaração no disco rígido ou em mídia removível, por meio do 2020, a partir da barra  de menu na opção “ferramentas”; “cópia de segurança”; “gravar”;   b) selecionar a opção “cópia da declaração para entrega à ”;   c) selecionar a declaração a ser gravada;   d) por meio do receitanet, selecionar a declaração gravada anteriormente para transmissão, seguindo as  orientações do programa.   atenção:    recibo de entrega, contendo o carimbo de recepção, é gravado automaticamente no disco rígido  ou mídia removível, no ato da transmissão.',
 'onde obter outras informações sobre a transmissão da declaração pela internet, utilizando o  receitanet?   estão disponíveis no sítio da secretaria especial da receita federal do brasil   na internet, no endereço  http://receita.economia.gov.br (acesso a partir da barra de menu na opção “onde encontro”; “receitanet”), as  respostas para as principais dúvidas e problemas que possam ocorrer em relação à transmissão da  declaração pela internet, utilizando o receitanet.',
 'qual o local de apresentação no prazo da declaração de ajuste anual do exercício de 2020?    declaração deve ser apresentada pela internet, por meio da funcionalidade “entregar declaração” do    2020, ou mediante a utilização do programa de transmissão receitanet, disponível no sítio da  na  internet ou do serviço “meu imposto de renda (extrato da )” ou, ainda, do aplicativo “meu imposto de  renda”, disponível para dispositivos móveis (consulte os itens “ii” e “iii” do “atenção” da pergunta 001)    declaração pode ser transmitida até as 23h59min59s (vinte e três horas, cinquenta e nove minutos e  cinquenta e nove segundos), horário de brasília, do dia 30 de junho de 2020.   http://receita.economia.gov.br/    33   atenção:     comprovação da apresentação da declaração de ajuste anual é feita por meio de recibo gravado  depois da transmissão, em disco rígido de computador, em mídia removível ou no dispositivo móvel  que contenha a declaração transmitida, cuja impressão fica a cargo do contribuinte e deve ser feita  mediante a utilização do.',
 'quem deve transmitir a declaração de ajuste anual do exercício de 2020 com a utilização de  certificado digital?   deve transmitir a declaração de ajuste anual, com a utilização de certificado digital, o contribuinte que se  enquadrou, no ano-calendário de 2019, em pelo menos uma das seguintes situações:    - recebeu rendimentos:   a) tributáveis sujeitos ao ajuste anual, cuja soma foi superior a $ 5.000.000,00 (cinco milhões de reais);   b) isentos e não tributáveis, cuja soma foi superior a $ 5.000.000,00 (cinco milhões de reais);   c) tributados exclusivamente na fonte, cuja soma foi superior a $ 5.000.000,00 (cinco milhões de reais); ou    - realizou pagamentos de rendimentos a pessoas jurídicas, quando constituam dedução na declaração, ou  a pessoas físicas, quando constituam, ou não, dedução na declaração, cuja soma foi superior a  $ 5.000.000,00 (cinco milhões de reais), em cada caso ou no total.    declaração de ajuste anual relativa a espólio, independentemente de ser inicial ou intermediária, ou a  declaração final de espólio, que se enquadre nas hipóteses contidas nos itens  e  deve ser apresentada,  em mídia removível, em uma unidade da , durante o seu horário de expediente, sem a necessidade de  utilização de certificado digital.   atenção:     disposto nas orientações constantes desta resposta não se aplica à declaração de ajuste anual  elaborada com a utilização de computador, mediante acesso ao serviço “meu imposto de renda  (extrato da )” de que trata o item “ii” do “atenção” da pergunta 001.',
 'qual é o local de apresentação da declaração de ajuste anual para a pessoa física residente  no brasil que esteja no exterior?    declaração de contribuinte residente no brasil que esteja no exterior deve ser enviada pela internet.    serviço de recepção da declaração, transmitida pela internet, será interrompido às 23h59min59s (vinte e  três horas, cinquenta e nove minutos e cinquenta e nove segundos), horário de brasília, do último dia do prazo  estabelecido.',
 'qual é o local de apresentação da declaração de ajuste anual do exercício de 2020  apresentada fora do prazo?    declaração, após 30 de junho de 2020, deve ser apresentada:   a) pela internet, mediante utilização do programa de transmissão receitanet;   b) a partir do exercício de 2017, pela internet, no respectivo programa , na opção “entregar declaração”;   c) utilizando o serviço “meu imposto de renda (extrato da )” ou o aplicativo “meu imposto de renda”  (consulte os itens “ii” e “iii” do “atenção” da pergunta 001), na hipótese de apresentação de declaração  original; ou   d) em mídia removível, nas unidades da , durante o seu horário de expediente.',
 'em quais instituições financeiras podem ser apresentadas a declaração de ajuste anual de  pessoas físicas do exercício de 2020?   as declarações de ajuste anual, desde o exercício de 2015, mesmo que no prazo, não poderão ser  apresentadas em instituições financeiras.',
 'como apresentar as declarações de anos anteriores?   utilize o programa relativo ao exercício correspondente à declaração, disponível na internet, no sítio da  secretaria especial da receita federal do brasil  , no endereço http://receita.economia.gov.br, a partir  da barra de menu na opção “onde encontro”; "download de aplicativos”; “ você –  - declaração  do imposto de renda de pessoa física" e siga as orientações para download constantes no sítio da  na  internet.   as declarações de exercícios anteriores devem ser apresentadas pela internet ou entregues em mídia  removível nas unidades da secretaria especial da receita federal do brasil. ',
 'contribuinte pode retificar sua declaração de rendimentos?   sim, desde que não esteja sob procedimento de ofício. se apresentada após o prazo final (30/06/2020), a  declaração de ajuste anual   retificadora deve ser apresentada observando-se a mesma natureza da  declaração original, não se admitindo troca de opção por outra forma de tributação.    atenção:    contribuinte poderá retificar a  apresentada, independente da opção da forma de tributação  (utilizando as deduções legais ou utilizando o desconto simplificado), para declaração final de  espólio   ou para declaração de saída definitiva do país  , conforme o caso.   da mesma forma, também é possível a retificação de  ou de  para .   ressalte-se, porém, que na hipótese de o contribuinte ter apresentado:   a) dentro do prazo, uma  e após o prazo apresentar  ou  e, posteriormente, retificar  esta novamente para uma , deverá, necessariamente, utilizar a mesma forma de tributação da  última  apresentada dentro do prazo;      35   b) após o prazo, uma  original e depois apresentar  ou  e, posteriormente, retificar  esta novamente para uma , deverá, necessariamente, utilizar a mesma forma de tributação da   original.    contribuinte deve informar o número do recibo de entrega da última declaração apresentada, relativa ao  mesmo ano-calendário. esse número é obrigatório e pode ser obtido no recibo de entrega impresso ou  visualizado por meio do menu declaração, opção abrir, do programa irpf2020.   atenção:     declaração retificadora tem a mesma natureza da declaração originariamente apresentada,  substituindo-a integralmente e, portanto, deve conter todas as informações anteriormente  declaradas com as alterações e exclusões necessárias, bem como as informações adicionais, se  for o caso.    nas hipóteses de redução de débitos já inscritos em dívida ativa da união bem como de redução  de débitos objeto de pedido de parcelamento deferido, admite-se a retificação da declaração tão  somente após autorização administrativa, desde que haja prova inequívoca da ocorrência de erro  no preenchimento da declaração, e enquanto não extinto o crédito tributário.',
 'há limite de prazo para a retificação da declaração?   sim. extingue-se em cinco anos o direito de o contribuinte retificar a declaração de rendimentos, inclusive  quanto ao valor dos bens e direitos declarados.   atenção:    sobre o termo inicial da contagem do prazo de 5 anos:    a) se não tiver havido algum imposto pago antecipadamente (carnê-leão, imposto complementar,  ): 5 (cinco) anos a partir de 1º de janeiro do ano seguinte ao ano inicial de apresentação da  declaração;   b) se tiver havido algum imposto pago antecipadamente: 5 (cinco) anos a partir do ano inicial de  apresentação da declaração.',
 'onde deve ser apresentada a declaração retificadora?   até 30 de junho de 2020, a declaração retificadora deve ser enviada pela internet, mediante a utilização do   ou do serviço “meu imposto de renda (extrato da )” ou, ainda, por meio de dispositivos móveis  mediante acesso ao aplicativo “meu imposto de renda”.   após 30 de junho de 2020, além dos meios citados acima, também pode ser apresentada em mídia removível  nas unidades da secretaria especial da receita federal do brasil  , sem a interrupção do pagamento do  imposto.   atenção:    transmissão da declaração de ajuste anual retificadora elaborada mediante utilização do   pode ser feita também com a utilização do programa de transmissão receitanet, disponível no sítio  da  (http://receita.economia.gov.br).',
 'contribuinte pode retificar sua declaração para troca da opção da forma de tributação?    escolha da forma de tributação é uma opção do contribuinte, a qual se torna definitiva com a apresentação  da declaração de ajuste anual  . desse modo, é permitida a retificação da declaração de rendimentos  visando à troca de opção por outra forma de tributação, somente até 30 de junho de 2020.   atenção:    contribuinte poderá retificar a  apresentada, independente da opção da forma de tributação  (utilizando as deduções legais ou utilizando o desconto simplificado), para declaração final de  espólio   ou para declaração de saída definitiva do país  , conforme o caso.   da mesma forma, também é possível a retificação de  ou de  para .   ressalte-se, porém, que na hipótese de o contribuinte ter apresentado:   a) dentro do prazo, uma  e após o prazo apresentar  ou  e, posteriormente, retificar  esta novamente para uma , deverá, necessariamente, utilizar a mesma forma de tributação da  última  apresentada dentro do prazo;   b) após o prazo, uma  original e depois apresentar  ou  e, posteriormente, retificar  esta novamente para uma , deverá, necessariamente, utilizar a mesma forma de tributação da   original.   ',
 'contribuinte pode retificar a sua declaração, relativa ao ano-calendário de 2019, a fim de  incluir ou alterar os prejuízos acumulados na atividade rural ainda que tenha optado pelo desconto  simplificado?   sim. entretanto, após o prazo para a apresentação da declaração, não será admitida a mudança na forma de  tributação de declaração já apresentada.',
 'como proceder quando a declaração retificadora for relativa a exercícios anteriores?    contribuinte deve apresentar declaração preenchida no programa  correspondente ao exercício que  deseja retificar, não sendo admitida a retificação que tenha por objetivo a troca da opção por outra forma de  tributação.   atenção:   contribuinte deve fazer o download, no sítio da  na internet, no endereço  http://receita.economia.gov.br, do programa relativo ao ano-calendário correspondente e após  preencher e retificar a declaração de acordo com as instruções vigentes para aquele ano, deve  apresentá-la pela internet, mediante utilização do programa de transmissão receitanet ou, a partir  do exercício de 2017, no respectivo programa  na opção “entregar declaração”, ou, ainda, em  mídia removível, nas unidades da . ',
 'como proceder quando a declaração retificadora do contribuinte implicar modificações na  declaração do cônjuge ou companheiro?    cônjuge ou companheiro também deve apresentar declaração retificadora. ',
 'como proceder quanto ao pagamento do imposto após a declaração retificadora?   1 - quando a retificação resultar em redução do imposto declarado, observar o seguinte procedimento:   a) calcular o novo valor de cada quota, mantendo-se o número de quotas em que o imposto foi parcelado na  declaração retificada, desde que respeitado o valor mínimo;   b) os valores pagos a maior relativos às quotas vencidas, bem assim os acréscimos legais referentes a esses  valores, podem ser compensados nas quotas vincendas, ou ser objeto de pedido de restituição;   c) sobre o montante a ser compensado ou restituído incidem juros equivalentes à taxa selic, tendo como termo  inicial o mês subsequente ao do pagamento a maior e como termo final o mês anterior ao da restituição ou da  compensação, adicionado de 1% no mês da restituição ou compensação.   2 - quando a retificação resultar em aumento do imposto declarado, observar o seguinte  procedimento:   a) calcular o novo valor de cada quota, mantendo-se o número de quotas em que o imposto foi parcelado na  declaração retificada;   b) sobre a diferença correspondente a cada quota vencida incidem acréscimos legais, calculados de acordo  com a legislação vigente.',
 'contribuinte que tenha optado pelo pagamento do imposto à vista deve retificar a declaração,  caso pretenda efetuar o pagamento em quotas?   sim.  pessoa física que tenha optado pelo pagamento do imposto em quota única deve retificar a declaração  para assim poder recolher o imposto parceladamente, até o limite de oito quotas.  pessoa física pode,  também, fazer tal alteração, mediante acesso ao sítio da  na internet, opção “onde encontro”; “extrato da  ”. ',
 'contribuinte, com 65 anos ou mais, que não utilizou na declaração a parcela de isenção  mensal relativa aos proventos de aposentadoria ou pensão a que tem direito, pode retificar a sua  declaração para se utilizar desse benefício?   sim.  contribuinte pode retificar a declaração a fim de se beneficiar da isenção legal sobre os proventos de  aposentadoria ou pensão, até o valor permitido na legislação.   atenção:   parcela isenta na declaração está limitada a até $ 1.903,98, por mês, no ano-calendário de  2019, a partir do mês em que o contribuinte completar 65 anos.  portanto, se o declarante com 65 anos ou mais receber, referente a proventos de aposentadoria,  reserva remunerada, reforma ou pensão, valor superior a $ 1.903,98 ao mês, a parcela que  ultrapassar esse limite deverá ser informada como rendimento tributável, na ficha “rend. trib.  receb. de pessoa jurídica”. ',
 'como deve proceder o contribuinte que sofreu retenção na fonte sobre verbas especiais  indenizatórias pagas em decorrência de incentivo à adesão a programa de desligamento voluntário   ?   salvo na hipótese de a fonte pagadora ter efetuado a dedução do imposto retido a maior no mesmo ano- calendário ou subsequente ao da ocorrência da retenção indevida, o contribuinte, ainda que desobrigado,  pode pleitear a devolução do valor pago a maior exclusivamente por meio da declaração de ajuste anual  relativa ao ano-calendário da retenção.   as verbas especiais indenizatórias recebidas a título de  devem ser informadas na ficha “rendimentos  isentos e não tributáveis”, no item “outros” e o imposto retido na fonte sobre essas verbas deve ser informado  na ficha “rend. trib. receb. de pessoa jurídica”.   atenção:    1 - não se incluem no conceito de verbas especiais indenizatórias recebidas a título de adesão ao  :   a) as verbas rescisórias previstas na legislação trabalhista em casos de rescisão de contrato de  trabalho, tais como: décimo terceiro salário, saldo de salário, salário vencido, férias proporcionais  ou vencidas, abono e gratificação de férias, gratificações e demais remunerações provenientes do  trabalho prestado, remuneração indireta, aviso prévio trabalhado, participação dos empregados nos  lucros ou resultados da empresa; e   b) os valores recebidos em função de direitos adquiridos anteriormente à adesão ao , em  decorrência do vínculo empregatício, a exemplo do resgate de contribuições efetuadas a entidades  de previdência complementar em virtude de desligamento do plano de previdência.   2 - com relação à tributação de férias indenizadas, consulte a pergunta 162.',
 'erros na declaração de bens e direitos ou na declaração de dívidas e ônus reais, que não  influenciem no saldo de imposto a pagar ou a restituir, precisam ser retificados ou podem ser  corrigidos na próxima declaração?   os erros na declaração de bens e direitos ou na declaração de dívidas e ônus reais devem ser retificados  mediante a apresentação de declaração retificadora relativa ao ano-calendário correspondente.',
 'qual é o procedimento a ser adotado pela pessoa física quando a fonte pagadora não lhe  fornecer o comprovante de rendimentos ou fornecê-lo com inexatidão?    fonte pagadora, pessoa física ou jurídica, deve fornecer à pessoa física beneficiária, até o último dia útil do  mês de fevereiro do ano subsequente àquele a que se referirem os rendimentos ou por ocasião da rescisão  do contrato de trabalho, se esta ocorrer antes da referida data, documentos comprobatórios, em uma via, com  indicação da natureza e do montante do pagamento, das deduções e do imposto retido no ano-calendário de  2019, conforme modelo oficial.   no caso de retenção na fonte e não fornecimento do comprovante, o contribuinte deve comunicar o fato à  unidade de atendimento da secretaria especial da receita federal do brasil   de sua jurisdição, para as  medidas legais cabíveis.   ocorrendo inexatidão nas informações, tais como salários que não foram pagos nem creditados no ano- calendário ou rendimentos tributáveis e isentos computados em conjunto, o interessado deve solicitar à fonte  pagadora outro comprovante preenchido corretamente.   na impossibilidade de correção, por motivo de força maior, o contribuinte pode utilizar os comprovantes de  pagamentos mensais, ficando sujeito à comprovação de suas alegações, a critério da autoridade lançadora.      39   é permitida a disponibilização, por meio da internet, do comprovante para a pessoa física que possua  endereço eletrônico e, nesse caso, fica dispensado o fornecimento da via impressa.',
 'contribuinte que auferiu rendimentos diversos, mas que não possui comprovantes de todas  as fontes pagadoras, declara somente os rendimentos comprovados por documentos?    contribuinte deve oferecer à tributação todos os rendimentos tributáveis percebidos no ano-calendário, de  pessoas físicas ou jurídicas, mesmo que não tenha recebido comprovante das fontes pagadoras, ou que este  tenha se extraviado.   se o contribuinte não tem o comprovante do desconto na fonte ou do rendimento percebido, deve solicitar à  fonte pagadora uma via original, a fim de guardá-la para futura comprovação. se a fonte pagadora se recusar  a fornecer o documento pedido, o contribuinte deve comunicar o fato à unidade de atendimento da secretaria  especial da receita federal do brasil   de sua jurisdição, para que a autoridade competente tome as  medidas legais que se fizerem necessárias.',
 'quais as penalidades a que estão sujeitas as fontes pagadoras que deixarem de fornecer ou  fornecerem com inexatidão o comprovante de rendimentos?    fonte pagadora que deixar de fornecer aos beneficiários, dentro do prazo, ou fornecer com inexatidão o  informe de rendimentos e de retenção do imposto, fica sujeita ao pagamento de multa equivalente a $ 41,43  por documento.    fonte pagadora que prestar informação falsa sobre rendimentos pagos, deduções ou imposto retido na fonte,  está sujeita à multa de 300% sobre o valor que for indevidamente utilizado como redução do imposto sobre a  renda devido, independentemente de outras penalidades administrativas ou criminais. na mesma penalidade  incorre aquele que se beneficiar de informação sabendo ou devendo saber da falsidade.',
 'pessoa física não residente no brasil está obrigada à inscrição no cadastro de pessoas  físicas  ?   está obrigada a inscrever-se no  a pessoa física não residente que possua no brasil bens e direitos  sujeitos a registro público, inclusive imóveis, veículos, embarcações, aeronaves, participações societárias,  contas-correntes bancárias, aplicações no mercado financeiro e aplicações no mercado de capitais.',
 'é obrigatória a inscrição no cadastro de pessoas físicas de dependente relacionado  em declaração de ajuste anual?   é obrigatória a inscrição no cadastro de pessoas físicas   das pessoas físicas que constem como  dependente para fins do imposto sobre a renda da pessoa física. ',
 'como deve proceder o contribuinte que perdeu a cópia da declaração de ajuste anual do ano  anterior e não tem dados para preencher a declaração de bens e direitos?    contribuinte pode obter a cópia da declaração de ajuste anual mediante acesso ao sítio da  na internet,  utilizando a opção “e-”, por intermédio de certificação digital, ou solicitá-la, por escrito, ao titular da unidade  da secretaria especial da receita federal do brasil de sua jurisdição fiscal. os pedidos de cópias de  quaisquer documentos estão sujeitos ao recolhimento prévio de taxa específica para ressarcimento de  despesas, que deve ser recolhida por meio de darf, utilizando-se o código 3292.',
 'qual é a tabela a ser aplicada para o cálculo do imposto sobre a renda na declaração de  ajuste anual do exercício de 2020, ano-calendário de 2019?    tabela progressiva anual para o cálculo do imposto é a seguinte:   base de cálculo em $ alíquota (%) parcela a deduzir do  imposto em $   até 22.847,76 - -   de 22.847,77 até 33.919,80 7,5 1.713,58   de 33.919,81 até 45.012,60 15 4.257,57   de 45.012,61 até 55.976,16 22,5 7.633,51   acima de 55.976,16 27,5 10.432,32 ',
 'que se considera base de cálculo do imposto sobre a renda a ser apurado na declaração?    base de cálculo do imposto devido é a diferença entre a soma dos rendimentos recebidos durante o ano- calendário (exceto os isentos, não tributáveis, tributáveis exclusivamente na fonte ou sujeitos à tributação  definitiva) e as deduções permitidas pela legislação. ',
 'imposto apurado na declaração de ajuste anual do exercício de 2020, ano-calendário de  2019 pode ser pago em quotas?    saldo do imposto pode ser pago em até 8 (oito) quotas, mensais e sucessivas, observado o seguinte:   a) nenhuma quota deve ser inferior a $ 50,00 (cinquenta reais);   b) o imposto de valor inferior a $ 100,00 (cem reais) deve ser pago em quota única.   c) a primeira quota ou quota única vence em 30 de junho de 2020, sem acréscimo de juros, se recolhida até  essa data.   d) as demais quotas vencem no último dia útil de cada mês subsequente ao da apresentação, e seu valor  sofre acréscimo de juros equivalentes à taxa referencial do sistema especial de liquidação e de custódia  (selic), para títulos federais, acumulada mensalmente, a partir do primeiro dia do mês subsequente ao previsto  para a apresentação da declaração até o mês anterior ao do pagamento, e de 1% referente ao mês do  pagamento, ainda que as quotas sejam pagas até as respectivas datas de vencimento.   caso o pagamento venha a ser efetuado posteriormente ao prazo legal, incide a multa de mora de 0,33% ao  dia, limitada a 20%.    saldo do imposto a pagar que resultar inferior a $ 10,00 não deve ser pago, devendo ser adicionado ao  imposto correspondente aos exercícios subsequentes, até que o total seja igual ou superior a $ 10,00,  quando, então, deve ser pago no prazo estabelecido na legislação para este último exercício.   atenção:    1) é facultado ao contribuinte:    - antecipar, total ou parcialmente, o pagamento do imposto ou das quotas, não sendo necessário,  nesse caso, apresentar declaração de ajuste anual retificadora com a nova opção de pagamento;    - ampliar o número de quotas do imposto inicialmente previsto na declaração de ajuste anual,  até a data de vencimento da última quota pretendida, observado os itens de “a” a “d” da presente  resposta, mediante a apresentação de declaração retificadora ou o acesso ao sítio da  na  internet, opção “onde encontro”; “extrato da ”.   2) tendo em vista a mudança do prazo de apresentação da declaração em função da covid-19, o  contribuinte que, antes dessa mudança, já havia pagado a primeira quota ou quota única do imposto  não precisa fazer nada em relação a esa quota   para as demais, a partir da segunda quota, caso existam, deverá atualizar a versão do programa  gerador da declaração ( 2020) e gerar os documentos de arrecadação de receitas federais  (darf) em cada mês de vencimento, a partir do mês de julho de 2020.   caso o contribuinte já tenha gerado o darf para o pagamento da 1ª quota ou quota única com  vencimento original para 30/04/2020, basta atualizar a versão do  2020 e gerar um novo darf  com vencimento para 30/06/2020.',
 'como efetuar o cálculo do pagamento das quotas de ?   quando pagas dentro do prazo legal, o valor a recolher é calculado da seguinte maneira:   1ª quota ou quota única (vencimento em 30/06/2020): o valor apurado na declaração;   2ª quota (vencimento em 31/07/2020): valor apurado, mais 1%;   3ª quota (vencimento em 31/08/2020): valor apurado, mais juros à taxa selic de julho, mais 1%;   4ª quota (vencimento em 30/09/2020): valor apurado, mais juros à taxa selic acumulada (julho e agosto), mais  1%;   5ª quota (vencimento em 30/10/2020): valor apurado, mais juros à taxa selic acumulada (julho a setembro),  mais 1%;   6ª quota (vencimento em 30/11/2020): valor apurado, mais juros à taxa selic acumulada (julho a outubro),  mais 1%;      42   7ª quota (vencimento em 30/12/2020): valor apurado, mais juros à taxa selic acumulada (julho a novembro),  mais 1%;   8ª quota (vencimento em 29/01/2021): valor apurado, mais juros à taxa selic acumulada (julho a dezembro),  mais 1%.   caso o pagamento de alguma quota venha a ser efetuado posteriormente ao prazo legal, incide multa de mora  de 0,33%, por dia de atraso, limitada a 20%, mais juros à taxa selic acumulada até o mês anterior ao do  pagamento, mais 1% no mês do pagamento.',
 'contribuinte residente em um estado pode efetuar o pagamento do imposto sobre a renda em  qualquer outro estado?   sim.  pagamento pode ser efetuado em qualquer agência bancária integrante da rede arrecadadora de  receitas federais, independentemente do domicílio fiscal do contribuinte.    pagamento integral do imposto, ou de suas quotas, e de seus respectivos acréscimos legais pode ser  efetuado mediante:    - transferência eletrônica de fundos por meio de sistemas eletrônicos das instituições financeiras autorizadas  pela  a operar com essa modalidade de arrecadação;    - documento de arrecadação de receitas federais (darf), em qualquer agência bancária integrante da rede  arrecadadora de receitas federais, no caso de pagamento efetuado no brasil.',
 'como pagar o imposto e seus respectivos acréscimos legais?    pagamento integral do imposto ou de suas quotas e de seus respectivos acréscimos legais pode ser feito  mediante:    – contribuinte residente no brasil:   a) transferência eletrônica de fundos por meio de sistemas eletrônicos das instituições financeiras  autorizadas pela secretaria especial da receita federal do brasil   a operar com essa modalidade  de arrecadação;   b) em qualquer agência bancária integrante da rede arrecadadora de receitas federais, mediante  documento de arrecadação de receitas federais (darf), no caso de pagamento efetuado no brasil; ou   c) débito automático em conta-corrente bancária (consulte item 3 do tópico “atenção”);    -  contribuinte ausente, no exterior, a serviço do brasil:   no caso de pessoa física que receba rendimentos do trabalho assalariado de autarquias ou repartições do  governo brasileiro situadas no exterior, além do previsto no item , o pagamento integral do imposto ou de  suas quotas e de seus respectivos acréscimos legais pode ser efetuado mediante remessa de ordem de  pagamento com todos os dados exigidos no darf, no respectivo valor em reais ou em moeda estrangeira,  a favor da secretaria especial da receita federal do brasil  , por meio do banco do brasil ..,  gerência regional de apoio ao comércio exterior – brasília- (gecex brasília - ), prefixo 1608-.    atenção:    1 -  pagamento da 1ª quota ou quota única deve ser efetuado até 30/06/2020;   2 –  partir do exercício de 2018, o programa da declaração de ajuste anual do imposto sobre a  renda da pessoa física permite a impressão do darf para o pagamento de todas as quotas,  inclusive em atraso, sendo necessário que esteja conectado à internet.      43    contribuinte também  pode obter o darf para pagamento de todas as quotas do imposto sobre  a renda da pessoa física, no sítio da  na internet, no endereço  http://receita.economia.gov.br, das seguintes formas:   2.1 - pelo acesso ao centro virtual de atendimento (e-), por meio do código de acesso,  previamente criado, ou por certificado digital. na opção “pagamentos e parcelamentos”; “meu  imposto de renda (extrato da ”; selecionar em “pagamento”, consultar “consultar débitos,  emitir  e alterar quotas” para saber o quantitativo de quotas solicitadas e a situação de  cada uma delas, e clicar na impressora para impressão do darf da quota desejada;   ou   2.2 – a partir da barra de menu na opção “onde encontro”; “pagamentos”; “pagamento do  imposto de renda de pessoa física”; “pagamento das quotas do imposto de renda pessoa  física  ”; “programa para cálculo e emissão do darf das quotas do ”; “cálculo”;  preencher os dados referentes ao município de domicílio fiscal do contribuinte, exercício,  quota(s), data de pagamento e o valor original da quota.   3 –  débito automático em conta-corrente bancária:   3.1 - somente é permitido para declaração original ou retificadora apresentada:   a) até 10 de junho de 2020, para quota única ou a partir da 1ª quota;   b) entre 11 e 30 de junho de 2020, para débitos a partir da 2ª quota;   3.2 - é autorizado mediante a utilização do  ou do serviço “meu imposto de renda (extrato da  ” ou do aplicativo “meu imposto de renda” (consulte os itens “i”, “ii” e “iii” do “atenção” da  pergunta 001) e formalizado no recibo de entrega da declaração de ajuste anual;   3.3 - é automaticamente cancelado:   a) quando da apresentação de declaração retificadora depois do prazo previsto para a  apresentação da declaração original – 30 de junho de 2020;   b) na hipótese de envio de informações bancárias com dados inexatos;   c) quando o número de inscrição no cadastro de pessoas físicas   informado na declaração  for diferente daquele vinculado à conta-corrente bancária; ou   d) quando os dados bancários informados na declaração referirem-se a conta-corrente do tipo não  solidária;   3.4 - está sujeito a estorno, a pedido do contribuinte titular da conta-corrente, caso fique comprovada  a existência de dolo, fraude ou simulação;   3.5 – pode ser incluído, cancelado ou modificado, após a apresentação da declaração, mediante o  acesso ao sítio da  na internet, opção “onde encontro”; “extrato da  (meu imposto de  renda)”:   a) até as 23h59min59s (vinte e três horas, cinquenta e nove minutos e cinquenta e nove segundos),  horário de brasília, do dia 14 de cada mês, produzindo efeitos no próprio mês;   b) após o prazo de que trata a alínea “a”, produzindo efeitos no mês seguinte.   4 – tendo em vista a mudança da data de apresentação da  de 30 de abril para 30 de junho  de 2020, por conta da pandemia de covid-19, caso o contribuinte já tenha apresentado a  declaração, esclarece-se abaixo como ficam os débitos das quotas do imposto:   4.1 - se o contribuinte entregou a declaração e optou pelo débito automático, não será necessário  fazer nada, pois a secretaria especial da receita federal do brasil   reprogramará os débitos  para só ocorrerem nas novas datas, sendo que a primeira quota ou quota única será debitada no  dia 30/06/2020 e as demais nos últimos dias úteis dos meses subsequentes;   4.2 - se não houve a opção pelo agendamento do débito automático, o contribuinte deverá atualizar  a versão do programa gerador da declaração ( 2020) e gerar os documentos de arrecadação  de receitas federais (darf) em cada mês de vencimento, a partir de junho de 2020.   5 – caso o contribuinte já tenha apresentado declaração sem opção pelo débito automático e queira  fazer essa opção, desde que obedecidos os prazos citados no item 3 deste atenção, basta  apresentar uma declaração retificadora e assinalar a opção pelo débito automático na aba  parcelamento da ficha resumo da declaração/cálculo do imposto.',
 'como deve proceder o contribuinte que perdeu o darf de recolhimento?    contribuinte pode solicitar confirmação do pagamento na unidade de atendimento da secretaria especial da  receita federal do brasil   de sua jurisdição fiscal.',
 'há correção monetária na restituição ou compensação de imposto pago a maior ou  indevidamente?   não. sobre o valor a ser utilizado na compensação ou na restituição incidem juros equivalentes à taxa selic,  para títulos federais, acumulada mensalmente a partir do mês subsequente ao do pagamento indevido ou a  maior até o mês anterior ao da compensação ou da restituição, e de 1% relativamente ao mês em que estiver  sendo efetuada. ',
 'qual é o prazo para pleitear a restituição do imposto sobre a renda pago indevidamente?    prazo para que o contribuinte possa pleitear a restituição do imposto pago indevidamente ou em valor maior  que o devido, inclusive na hipótese de o pagamento ter sido efetuado com base em lei posteriormente  declarada inconstitucional pelo supremo tribunal federal em ação declaratória ou em recurso extraordinário,  extingue-se após o transcurso do prazo de 5 anos, contados da data da extinção do crédito tributário, tratando- se de rendimentos sujeitos à tributação exclusiva na fonte, não tributáveis ou isentos.   em se tratando de rendimentos recebidos ao longo do ano-calendário sujeitos ao ajuste anual, e tendo havido  antecipação do pagamento do imposto mediante retenção pela fonte pagadora, o termo inicial da contagem  do prazo de 5 anos é o dia 31 de dezembro do ano-calendário correspondente.   esse mesmo prazo aplica-se também à restituição do imposto sobre a renda na fonte incidente sobre os  rendimentos recebidos como verbas indenizatórias a título de incentivo à adesão a programas de  desligamento voluntário.',
 'restituição só pode ser creditada em conta bancária?    crédito da restituição só pode ser efetuado em conta-corrente ou de poupança de titularidade do contribuinte. ',
 'no caso de conta conjunta, ambos os contribuintes podem indicá-la para o recebimento da  restituição?   sim. ambos os contribuintes podem indicá-la para o recebimento da restituição.',
 'é possível autorizar o crédito da restituição em conta de terceiros?   não.  restituição só é creditada em conta se o declarante for seu titular ou utilizar conta conjunta.',
 'é possível alterar a conta indicada para o crédito da restituição?   antes da inclusão do contribuinte em um dos lotes de restituição, a conta bancária indicada para crédito da  restituição pode ser alterada pelas seguintes formas:   1) por meio do centro virtual de atendimento (e-)”, acessar “restituição e compensação”; em  “restituição do imposto de renda pessoa física – ”, acessar “meu imposto de renda (extrato da  ”; em “serviços”, procurar por “restituição” e acessar “consultar e alterar conta para crédito de  restituição”; ou   2) mediante apresentação de declaração retificadora.   se a restituição foi liberada, mas não creditada, entrar em contato com a central de atendimento do banco  do brasil, por meio dos telefones 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088  (exclusivo para deficientes auditivos), das 8h às 21h, ou compareça pessoalmente a uma agência do banco  do brasil.      46    restituição não resgatada no banco do brasil no prazo de um ano é devolvida para a secretaria especial da  receita federal do brasil   e poderá ser requerida pelo contribuinte no prazo de 5 (cinco) anos contado  a partir da data em que o pagamento foi disponibilizado na instituição financeira.    requisição será efetuada por meio do formulário eletrônico “pedido de pagamento de restituição  ”  disponível no sítio da , no endereço http://receita.economia.gov.br, a partir da barra de menu na opção  “onde encontro”; “restituição ”; em “consulta restituições ” informar os dados solicitados em  relação à declaração com restituição não resgatada; acesse, então, o formulário “pedido de pagamento de  restituição – ” e preencha com as informações solicitadas. também poderá ser efetuada mediante  acesso ao serviço “meu imposto de renda (extrato da )”, disponível no centro virtual de atendimento  (e-) no sítio da  na internet (“restituição”; “solicitar restituição não resgatada na rede bancária”). ',
 'como é feita a restituição para os declarantes no exterior?    declarante no exterior deve indicar a conta bancária de sua titularidade, em qualquer banco no brasil  autorizado pela  a efetuar a restituição.    caso o contribuinte não possua conta bancária no brasil, deve nomear um procurador no brasil para receber  a sua restituição.  procurador, munido de procuração pública, deve comparecer a uma agência do banco do  brasil e indicar uma conta de sua titularidade, em qualquer banco, para que seja feito o respectivo crédito.    as restituições não resgatadas no prazo de um ano ficam à disposição dos beneficiários nas unidades da  secretaria especial da receita federal do brasil  , e serão pagas mediante ordem bancária do sistema  de administração financeira do governo federal (siafi) para crédito em conta bancária no brasil. ',
 'como declara o contribuinte casado?    contribuinte casado apresenta declaração em separado ou, opcionalmente, em conjunto com o cônjuge.   declaração em separado   a) cada cônjuge deve incluir na sua declaração o total dos rendimentos próprios e 50% dos rendimentos  produzidos pelos bens comuns, compensando 50% do imposto pago ou retido sobre esses rendimentos,  independentemente de qual dos cônjuges tenha sofrido a retenção ou efetuado o recolhimento; ou   b) um dos cônjuges inclui na sua declaração seus rendimentos próprios e o total dos rendimentos produzidos  pelos bens comuns, compensando o valor do imposto pago ou retido na fonte, independentemente de qual  dos cônjuges tenha sofrido a retenção ou efetuado o recolhimento.   os dependentes comuns não podem constar simultaneamente nas declarações de ambos os cônjuges.   verifique as instruções de preenchimento da declaração de bens e direitos, relativamente aos bens privativos  e bens comuns, constantes do ajuda do programa  2020.   declaração em conjunto   é apresentada em nome de um dos cônjuges, abrangendo todos os rendimentos, inclusive os provenientes  de bens gravados com cláusula de incomunicabilidade ou inalienabilidade, e das pensões de gozo privativo.    declaração em conjunto supre a obrigatoriedade da apresentação da declaração de ajuste anual a que  porventura estiver sujeito o outro cônjuge.',
 'qual é o tratamento tributário dos rendimentos produzidos pelos bens adquiridos em  condomínio por contribuintes que venham a se casar posteriormente pelo regime de comunhão  parcial de bens?   neste caso, os bens adquiridos antes do casamento mantêm a forma de tributação estabelecida para bens  adquiridos em condomínio, ou seja, cada cônjuge deve incluir em sua declaração 50% dos rendimentos  produzidos pelos bens em condomínio, salvo estipulação contrária em contrato escrito, quando deve ser  adotado o percentual nele previsto.    atenção:    os demais rendimentos de cada cônjuge (rendimentos próprios e rendimentos produzidos por bens  comuns) seguem as regras para contribuinte casado.   na união estável, salvo contrato escrito entre os companheiros, aplica-se às relações patrimoniais,  no que couber, o regime da comunhão parcial de bens.',
 'como deve declarar o contribuinte que tenha companheiro(a)?   conceito de companheiro(a) – lei nº 8.971, de 29 de dezembro de 1994.   apresenta declaração em separado ou, opcionalmente, em conjunto com o(a) companheiro(a).   declaração em separado   cada companheiro deve incluir na sua declaração os rendimentos próprios e 50% dos rendimentos produzidos  pelos bens em condomínio, salvo estipulação contrária em contrato escrito, quando deve ser adotado o  percentual nele previsto.  imposto pago ou retido é compensado na mesma proporção dos rendimentos  tributáveis produzidos pelos bens em condomínio.   declaração em conjunto   é apresentada em nome de um dos companheiros, abrangendo todos os rendimentos, inclusive os  provenientes de bens gravados com cláusula de incomunicabilidade ou inalienabilidade, e das pensões de  gozo privativo.    declaração em conjunto supre a obrigatoriedade da apresentação da declaração de ajuste anual a que  porventura estiver sujeito o outro companheiro(a).   atenção:     contribuinte pode incluir o companheiro, abrangendo também as relações homoafetivas, como  dependente para efeito de dedução do imposto sobre a renda da pessoa física, desde que tenha  vida em comum por mais de 5 (cinco) anos, ou por período menor se da união resultou filho   (parecer / nº 1.503, de 19 de julho de 2010, aprovado pelo ministro de estado da  fazenda em 26 de julho de 2010; e instrução normativa  nº 1.500, de 29 de outubro de 2014,  art. 90, § 8º).   na união estável, salvo contrato escrito entre os companheiros, aplica-se às relações patrimoniais,  no que couber, o regime da comunhão parcial de bens (lei nº 10.406, de 10 de janeiro de 2002, art.   1.725). ',
 'como deve declarar o contribuinte divorciado que tenha se casado novamente?   apresenta declaração na condição de casado, em separado ou, opcionalmente, em conjunto com o cônjuge.',
 'como deve declarar o contribuinte separado de fato?   apresenta declaração de acordo com as instruções para contribuinte casado.',
 'como deve declarar o contribuinte divorciado ou separado judicialmente ou por escritura  pública?   apresenta declaração na condição de solteiro, caso não estivesse casado ou vivendo em união estável em  31/12/2019, podendo incluir dependente (nesse caso, devem ser somados os rendimentos recebidos pelo  dependente) do qual detenha a guarda judicial ou deduzir pensão alimentícia paga em face das normas do  direito de família, quando em cumprimento de decisão judicial, inclusive os alimentos provisionais, de acordo  homologado judicialmente ou de escritura pública, a que se refere o art. 733 da lei nº 13.105, de 16 de março  de 2015 - código de processo civil.    atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.',
 'como deve declarar o contribuinte viúvo no decorrer do inventário?   no curso do inventário, apresenta declaração com o número de inscrição no cadastro de pessoas físicas    próprio, abrangendo bens e rendimentos próprios e os provenientes de bens não integrantes do  inventário do cônjuge falecido.  viúvo pode optar por tributar 50% dos rendimentos produzidos pelos bens  comuns na sua declaração ou integralmente na declaração do espólio.',
 'como deve declarar o contribuinte menor?   apresenta declaração da seguinte maneira:   a) em separado: os rendimentos recebidos pelo menor são tributados em seu nome com número de inscrição  no cadastro de pessoas físicas   próprio; ou   b) em conjunto: os rendimentos recebidos pelo menor devem ser tributados em conjunto com um dos pais.   no caso de menor sob a responsabilidade de um dos pais, em virtude de sentença ou acordo judicial, a  declaração em conjunto só pode ser feita com aquele que detém a guarda judicial do menor.    declaração em conjunto supre a obrigatoriedade da apresentação da declaração a que porventura estiver  sujeito o menor.',
 'como deve declarar o contribuinte menor emancipado?   apresenta declaração em seu nome com número de inscrição no cadastro de pessoas físicas   próprio.  se preencher os requisitos para permanecer como dependente, pode apresentá-la em conjunto com um dos  pais.',
 'como deve declarar o contribuinte incapaz?    declaração é feita em nome do incapaz pelo tutor, curador ou responsável por sua guarda, usando o número  de inscrição no cadastro de pessoas físicas   do incapaz.   opcionalmente, o incapaz pode ser considerado dependente do tutor, curador ou responsável por sua guarda  judicial, desde que o declarante inclua os rendimentos do incapaz em sua declaração. ',
 'quem é considerado declarante em conjunto?   somente é considerado declarante em conjunto o cônjuge, companheiro ou dependente cujos rendimentos  sujeitos ao ajuste anual estejam sendo oferecidos à tributação na declaração apresentada pelo contribuinte  titular.',
 'cônjuge e filho podem apresentar a declaração de rendimentos em conjunto ou, sem  apresentá-la, ficar na condição de dependente do declarante?   sim. porém, somente é considerada declaração em conjunto aquela em que estejam sendo oferecidos à  tributação rendimentos sujeitos ao ajuste anual do cônjuge ou filho, desde que este se enquadre como  dependente, nos termos da legislação do imposto sobre a renda.     declaração em conjunto supre a obrigatoriedade da apresentação da declaração a que porventura estiver  sujeito o cônjuge ou filho dependente para fins do imposto sobre a renda.   atenção:     cônjuge ou filho que se enquadrar em qualquer das hipóteses de obrigatoriedade de entrega de  declaração e não estiver declarando em conjunto fica dispensado de apresentá-la, caso conste  como dependente na declaração apresentada por outro cônjuge ou por um dos pais, na qual sejam  informados seus rendimentos, bens e direitos.    pessoa física que se enquadrar apenas na hipótese de obrigatoriedade relativa à posse ou à  propriedade de bens e direitos, inclusive terra nua, de valor total superior a $ 300.000,00 (trezentos  mil reais), em 31 de dezembro, e que, na constância da sociedade conjugal ou da união estável,  tenha os bens comuns declarados pelo outro cônjuge ou companheiro, fica dispensada da  apresentação da declaração, desde que o valor total dos seus bens privativos não exceda esse  limite. ',
 'imposto referente a aluguel de imóvel possuído em condomínio ou em decorrência da  sociedade conjugal foi pago em nome de um dos proprietários. pode o outro proprietário compensar  em sua declaração o valor do imposto pago sobre sua parte?   no caso de propriedade em condomínio: cada condômino tributa a parcela do rendimento que lhe cabe, mas  somente aquele em cujo nome foi efetuado o recolhimento pode compensar o imposto em sua declaração, a  não ser que seja retificado o darf (carnê-leão) ou a dirf (no caso de fonte).   no caso de propriedade em comum em decorrência da sociedade conjugal/união estável: o imposto pago por  um dos cônjuges/companheiros ou retido na fonte pode ser compensado meio a meio, independentemente  de quem o tenha pagado ou suportado a retenção; opcionalmente, o imposto pode ser compensado pelo total  na declaração de um deles, desde que tribute a totalidade dos rendimentos comuns.',
 'que é espólio?   espólio é o conjunto de bens, direitos e obrigações da pessoa falecida. é contribuinte distinto do meeiro, dos  herdeiros e dos legatários.',
 'é devido imposto sobre a renda de contribuinte que faleceu após a apresentação da  declaração do exercício?   se houver bens a inventariar, o imposto deve ser pago pelo espólio.   inexistindo bens a inventariar, o cônjuge/companheiro sobrevivente ou os dependentes não respondem pelos  tributos devidos pela pessoa falecida.   (instrução normativa  nº 81, de 11 de outubro de 2001, arts. 18 a 21. ',
 'é dispensável o alvará judicial na restituição, ao cônjuge viúvo ou aos herdeiros do falecido,  do imposto sobre a renda não recebido em vida pelo titular, quando já tenha sido encerrado o  inventário?   existindo bens sujeitos a inventário ou a arrolamento, e tendo sido encerrado o inventário sem a inclusão do  imposto sobre a renda não recebido em vida pelo titular, a restituição depende:   a) de alvará judicial, caso o inventário tenha sido feito por processo judicial de inventário; ou    b) de escritura pública de inventário e partilha, na hipótese de o inventário ter sido feito dessa forma.   não havendo bens sujeitos a inventário e existindo dependentes habilitados na forma da legislação  previdenciária ou militar, a restituição é liberada mediante requerimento dirigido ao delegado da delegacia da  receita federal do brasil da jurisdição do último endereço do de cujus.    requerimento deve ser formulado pelo cônjuge viúvo, convivente ou por herdeiro capaz, ou pelo tutor ou  curador, conforme o caso, devendo nele constar os nomes completos e os demais dados civis de todos os  beneficiários habilitados à restituição, inclusive número de inscrição no cadastro de pessoas físicas    de quem estiver inscrito.    pedido deve ser entregue acompanhado de:   - cópia da certidão de óbito;      51   - comprovação dos dependentes habilitados emitida pelo órgão de previdência (regime geral de previdência  social, regimes de servidores públicos) ou militar;   - declaração de inexistência de outros bens a inventariar ou arrolar, bem assim autenticidade dos documentos  e dados apresentados, devendo ser usado como modelo os termos da declaração do anexo  da instrução  normativa  nº 81, de 11 de outubro de 2001.   na hipótese de não haver bens ou direitos sujeitos a inventário ou arrolamento e o contribuinte não possuir  dependentes habilitados, na forma da legislação previdenciária ou militar, é obrigatória a apresentação de  alvará judicial ou de escritura pública extrajudicial que defina o direito do sucessor e o percentual a ser pago.',
 'qual é o procedimento a ser adotado no caso de falecimento, no ano-calendário de 2019, de  contribuinte que deixou bens a inventariar?   embora a lei civil disponha que “aberta a sucessão, a herança transmite-se, desde logo, aos herdeiros  legítimos e testamentários” é indispensável o processamento do inventário, com a emissão do formal de  partilha ou carta de adjudicação e a transcrição desse instrumento no registro competente, a fim de que o  meeiro, herdeiros e legatários possam usar, gozar e dispor, de forma plena e legal, dos bens e direitos  transmitidos causa mortis (lei nº 10.406, de 10 de janeiro de 2002 - código civil, arts. 1.784, 1.991, 2.013 a  2.022; lei nº 6.015 de 31 de dezembro de 1973, art. 167, inciso , itens 24 e 25).   para a legislação tributária, a pessoa física do contribuinte não se extingue imediatamente após sua morte,  prolongando-se por meio do seu espólio (regulamento do imposto sobre a renda – /2018, art. 9º,  aprovado pelo decreto nº 9.580, de 22 de novembro de 2018).  espólio é considerado uma universalidade  de bens e direitos, responsável pelas obrigações tributárias da pessoa falecida, sendo contribuinte distinto do  meeiro, herdeiros e legatários.   para os efeitos fiscais, somente com a decisão judicial ou por escritura pública de inventário e partilha,  extingue-se a responsabilidade da pessoa falecida, dissolvendo-se, então, a universalidade de bens e direitos.   com relação à obrigatoriedade de apresentação das declarações de espólio, aplicam-se as mesmas normas  previstas para os contribuintes pessoas físicas. assim, caso haja obrigatoriedade de apresentação, a  declaração de rendimentos, a partir do exercício correspondente ao ano-calendário do falecimento e até a  data da decisão judicial da partilha ou da adjudicação dos bens ou da lavratura da escritura pública de  inventário e partilha, é apresentada em nome do espólio, classificando-se em inicial, intermediária e final.   havendo bens a inventariar, a apresentação da declaração final de espólio é obrigatória, independentemente  de outras condições de obrigatoriedade de apresentação.   atenção:    caso a pessoa falecida não tenha apresentado as declarações anteriores às quais estivesse  obrigada, essas declarações devem ser apresentadas em nome da pessoa falecida. se essas  declarações foram apresentadas, porém constatou-se que ocorreram erros, omissões ou  inexatidões, elas devem ser retificadas (verificar as orientações contidas na pergunta 098).    responsabilidade pelo imposto devido pela pessoa falecida, até a data do falecimento, é do  espólio.   encerrada a partilha, a responsabilidade pelo imposto devido pela pessoa falecida, até aquela data,  é do sucessor a qualquer título e do cônjuge meeiro, limitando-se ao montante dos bens e direitos  a eles atribuídos.   (lei nº 10.406, de 10 de janeiro de 2002 - código civil, art. 1.997.',
 'que se considera declaração inicial, intermediária e final de espólio?   declaração inicial   é a que corresponde ao ano-calendário do falecimento.   declarações intermediárias   referem-se aos anos-calendário seguintes ao do falecimento, até o ano-calendário anterior ao da decisão  judicial da partilha, sobrepartilha ou adjudicação ou da lavratura da escritura pública de inventário e partilha  dos bens.   atenção:    aplicam-se, quanto à obrigatoriedade de apresentação das declarações de espólio inicial e  intermediárias, as mesmas normas previstas para os contribuintes pessoas físicas. opcionalmente,  as referidas declarações poderão ser apresentadas pelo inventariante, em nome do espólio, em  conjunto com o cônjuge, companheiro ou dependente cujos rendimentos sujeitos ao ajuste anual  estejam sendo oferecidos à tributação nestas declarações.   declaração final   é a que corresponde ao ano-calendário da decisão judicial da partilha, sobrepartilha ou adjudicação ou da  lavratura da escritura pública de inventário e partilha dos bens. essa declaração corresponde ao período de  1º de janeiro à data da decisão judicial ou da lavratura de escritura pública de inventário e partilha.   é obrigatória a apresentação da declaração final de espólio elaborada em computador mediante a utilização  do programa gerador declaração  2020, sempre que houver bens a inventariar.    declaração final de espólio deve ser enviada pela internet ou entregue em mídia removível, nas unidades  da secretaria especial da receita federal do brasil  .   atenção:    ocorrendo o falecimento a partir de 1º de janeiro do ano seguinte ao do recebimento dos  rendimentos, porém antes da apresentação da declaração de ajuste anual, esta não se caracteriza  como de espólio, devendo, se obrigatória, ser apresentada em nome da pessoa falecida pelo  inventariante, cônjuge meeiro, sucessor a qualquer título ou por representante desses.',
 'se o espólio não estava obrigado a apresentar as declarações inicial e intermediárias,  havendo bens a inventariar, é obrigatória a apresentação da declaração final de espólio?   sim, a entrega da declaração final de espólio é obrigatória sempre que houver bens a inventariar.',
 'como o espólio declara os rendimentos referentes aos bens comuns?   na declaração de espólio, devem ser incluídos os rendimentos próprios, 50% dos produzidos pelos bens  comuns recebidos no ano-calendário, os bens e direitos que constem do inventário e as obrigações do espólio.  opcionalmente, os rendimentos produzidos pelos bens comuns podem ser tributados, em sua totalidade, em  nome do espólio, o qual pode compensar o total do imposto pago ou retido na fonte sobre esses rendimentos.  caso haja a alienação de algum bem ou direito no curso do inventário, o espólio deverá apurar o ganho de  capital.   atenção:    responsabilidade dos sucessores e do inventariante   são pessoalmente responsáveis:    - o espólio, pelos tributos devidos pelo de cujus até a data da abertura da sucessão;      53    - o sucessor a qualquer título e o cônjuge meeiro, pelos tributos devidos pelo de cujus até a data  da partilha ou adjudicação, limitada essa responsabilidade ao montante do quinhão, do legado, da  herança, ou da meação;    - o inventariante, pelo cumprimento da obrigação tributária do espólio resultante dos atos  praticados com excesso de poderes ou infração de lei.',
 'que bens são passíveis de sobrepartilha?   são passíveis de sobrepartilha os bens:    - sonegados;    - da herança, de que se tiver ciência após a partilha;    - litigiosos, assim como os de liquidação difícil ou morosa;    - situados em lugar remoto da sede do juízo onde se processa o inventário.   nas situações de que tratam os itens  e , a partilha dos demais bens integrantes do espólio, embora  implique sua baixa na declaração de bens e direitos do espólio, observadas as instruções quanto ao valor de  transmissão, não obriga a entrega da declaração final, que somente é exigida quando do trânsito em julgado  da sobrepartilha. se a sobrepartilha não for homologada no mesmo ano-calendário, a baixa dos bens  resultantes da partilha será efetuada na declaração intermediária, informando-se no campo “discriminação”  da declaração de bens e direitos os dados referentes aos beneficiários e ao valor de transmissão. caso o  valor de transmissão seja superior ao constante no campo do ano anterior, deve ser apurado o ganho de  capital, observadas as instruções específicas.',
 'que deduções são permitidas nas declarações do espólio?   são permitidas as mesmas deduções facultadas à pessoa física, exceto na declaração final de espólio, em  que não é permitido o uso do desconto simplificado. ',
 'os mesmos dependentes podem constar na declaração final de espólio e também na  declaração de ajuste anual do meeiro?   nas declarações de espólio, inclusive na final, podem ser considerados dependentes o cônjuge ou convivente  sobrevivente e demais dependentes, desde que não tenham recebido rendimentos ou, caso os tenham  recebido, sejam os mesmos incluídos nas declarações de espólio.   no caso de encerramento de espólio, a relação de dependência entre os dependentes e o espólio termina  com a entrega da declaração final de espólio. os dependentes nessa declaração podem ser, nesse ano,  dependentes também do meeiro, desde que preencham os requisitos legais para tanto. ',
 'são dedutíveis nas declarações de espólio os gastos com funeral, taxas, emolumentos e  honorários advocatícios relativos ao inventário?   não. nas declarações de espólio, inclusive na final, são permitidas apenas as deduções previstas na  legislação tributária.',
 'quem deve apresentar as declarações de espólio?   as declarações de espólio devem ser apresentadas em nome da pessoa falecida, com a indicação de seu  número de inscrição no cadastro de pessoas físicas  , utilizando o código de natureza de ocupação  relativo a espólio (81) deixando em branco o código de ocupação principal, devendo tal apresentação ser  efetuada pelo inventariante, indicando seu nome, o número de inscrição no  e o endereço.   enquanto não houver iniciado o inventário, as declarações são apresentadas pelo cônjuge meeiro, sucessor  a qualquer título ou por representante do de cujus.',
 'devem ser apresentadas declarações de espólio de pessoa não residente no brasil?   não, porém os rendimentos produzidos no brasil e recebidos pelo espólio estão sujeitos à incidência do  imposto de forma definitiva ou exclusiva na fonte, que deve ser recolhido em nome do espólio, a partir do  falecimento até a data do trânsito em julgado da decisão judicial da partilha, sobrepartilha, adjudicação ou da  lavratura da escritura pública de inventário e partilha dos bens, nos prazos previstos na legislação vigente.',
 'incidem acréscimos legais quando a responsabilidade pela obrigação tributária do espólio  for transferida aos herdeiros?   sim. incidem acréscimos legais sobre o valor do imposto devido pelo espólio, limitada essa responsabilidade  ao montante do quinhão, legado ou da meação.',
 'como devem ser apresentadas as declarações de espólio no caso de ocorrência de morte de  ambos os cônjuges?   no caso de regime de comunhão parcial ou total de bens e ocorrendo morte conjunta, deve ser apresentada,  se obrigatória, em relação a cada exercício, uma única declaração de rendimentos em nome de um dos  cônjuges, abrangendo todos os bens, direitos, rendimentos e obrigações, informando o falecimento do  cônjuge, seu nome e o número de inscrição no cadastro de pessoas físicas  . caso tenham ocorrido  mortes em datas diferentes e antes de encerrado o inventário do pré-morto, deve ser apresentada uma única      55   declaração de rendimentos para cada exercício, em nome deste, a partir do exercício correspondente ao ano- calendário de seu falecimento.   no regime de separação de bens, quer a morte seja conjunta ou em datas diferentes, deve ser apresentada  uma única declaração ou duas, segundo a sucessão seja processada em um único inventário ou dois.',
 'como proceder no caso de serem trazidos novos bens ao inventário antes do trânsito em  julgado da decisão judicial da partilha, sobrepartilha ou adjudicação ou lavratura da escritura?   as declarações de espólio continuam a ser apresentadas normalmente, nelas incluindo-se os bens e direitos,  a partir do ano-calendário em que forem trazidos aos autos, bem como os rendimentos por eles produzidos.   se os referidos bens e direitos produziram rendimentos em anos anteriores, não abrangidos pela decadência,  deve ser apresentada retificadora das declarações anteriormente apresentadas, desde a abertura da  sucessão, para que nelas sejam incluídos esses bens e os rendimentos por eles produzidos. é admitida a  opção pela tributação em conjunto ou em separado dos rendimentos produzidos pelos bens e direitos  possuídos em comum.',
 'como apurar o imposto na declaração final de espólio?    declaração final deve conter os rendimentos recebidos no período compreendido entre 1º de janeiro e a  data da decisão judicial da partilha, sobrepartilha ou adjudicação, ou a data da lavratura da escritura pública  de inventário e partilha, aplicando-se as normas previstas para o ano-calendário em que esta ocorrer, devendo  ser elaborada em computador mediante a utilização de programa gerador próprio.     imposto sobre a renda é calculado mediante a utilização dos valores correspondentes à soma das tabelas  progressivas mensais relativas aos meses do período abrangido pela tributação, no ano-calendário a que  corresponder a declaração final, até a data da decisão judicial transitada em julgado ou da lavratura da  escritura pública de inventário e partilha, ainda que os rendimentos correspondam a apenas um ou alguns  meses desse período.',
 'como deve ser preenchida a declaração de bens e direitos da declaração final de espólio?   na declaração de bens e direitos correspondente à declaração final deve ser informada, discriminadamente,  em relação a cada bem ou direito, a parcela que corresponder a cada beneficiário, identificado por nome e  número de inscrição no cadastro de pessoas físicas  .   no item “situação na data da partilha”, os bens ou direitos devem ser informados pelo valor, observada a  legislação pertinente, constante na última declaração apresentada pelo de cujus, atualizado até 31/12/1995,  ou pelo valor de aquisição, se adquiridos após essa data.   no item “valor de transferência”, deve ser informado o valor pelo qual o bem ou direito, ou cada parte deste,  será incluído na declaração de bens e direitos do respectivo beneficiário. ',
 'qual é o tratamento tributário aplicável à transferência de bens e direitos a herdeiros ou  legatários?   estão sujeitas à apuração do ganho de capital as operações que importem transferência de propriedade de  bens e direitos, por sucessão causa mortis, a herdeiros e legatários, quando a transferência dos referidos bens  e direitos for efetuada por valor de mercado, desde que este seja superior ao valor, observada a legislação  pertinente, constante da última declaração do de cujus.   nesse caso, a opção é informada na declaração final de espólio, sendo este o contribuinte do imposto.   imposto deverá ser pago pelo inventariante até a data prevista para a entrega da declaração final de espólio.   no caso de transferência pelo valor constante na última declaração de bens do de cujus, não há ganho de  capital a ser apurado.   atenção:    na hipótese de a propriedade de um bem ser adquirida parte por meação e parte por herança,  torna-se necessário conhecer as datas de aquisição de cada parte do bem para fins de apuração  do ganho de capital numa alienação futura.    parte recebida por herança tem como data de aquisição aquela da abertura da sucessão.   na parcela havida por meação, entretanto, considera-se data de aquisição:   a) a do instrumento original, se se tratar de bens ou direitos preexistentes à sociedade conjugal ou  união estável, se pertencentes ao alienante;   b) a do casamento, se pertencentes ao outro cônjuge e o regime for de comunhão de bens; e   c) a da aquisição, se adquiridos na constância da sociedade conjugal ou união estável. ',
 'os bens doados em vida respondem pelas dívidas fiscais do espólio?   os bens doados em estrita observância à lei, bem como os bens e rendimentos privativos do cônjuge  sobrevivente e dos herdeiros e legatários, não respondem pelas dívidas do espólio.    somente na hipótese de haver meação, herança ou legado haverá incidência tributária, limitada essa  responsabilidade ao montante do quinhão, do legado ou da meação. ',
 'qual é o tratamento tributário dos proventos de aposentadoria ou reforma e valores a título  de pensão de pessoa com moléstia grave recebidos pelo espólio ou por seus herdeiros?   caso os proventos de aposentadoria ou reforma e valores a título de pensão de pessoa com moléstia grave  (falecido) sejam recebidos pelo espólio ou por seus herdeiros, independentemente de situações de caráter  pessoal, devem ser tributados na fonte e na declaração de ajuste anual ou na declaração final de espólio.',
 'qual é o conceito de residente no brasil para fins tributários?   considera-se residente no brasil, para fins tributários, a pessoa física:    - que resida no brasil em caráter permanente;    - que se ausente para prestar serviços como assalariada a autarquias ou repartições do governo brasileiro  situadas no exterior;    - que ingresse no brasil:         a) com visto permanente, na data da chegada;         b) com visto temporário:   1. para trabalhar com vínculo empregatício ou atuar como médico bolsista no âmbito do programa  mais médicos de que trata a medida provisória nº 621, de 8 de julho de 2013, convertida na lei nº  12.871, de 22 de outubro de 2013, na data da chegada;    2. na data em que complete 184 dias, consecutivos ou não, de permanência no brasil, dentro de um  período de até doze meses;   3. na data da obtenção de visto permanente ou de vínculo empregatício, se ocorrida antes de  completar 184 dias, consecutivos ou não, de permanência no brasil, dentro de um período de até  doze meses;    - brasileira que adquiriu a condição de não residente no brasil e retorne ao país com ânimo definitivo, na  data da chegada, ainda que tenha ingressado para prestar serviços como funcionária de órgão de governo  estrangeiro situado no país;    - que se ausente do brasil em caráter temporário, ou se retire em caráter permanente do território nacional  sem entregar a comunicação de saída definitiva do país, durante os primeiros doze meses consecutivos de  ausência. ',
 'quando se inicia nova contagem para estabelecer a condição de residente no brasil da pessoa  física que entrou no país com visto temporário e permaneceu no brasil por menos de 184 dias,  dentro de um intervalo de até 12 meses?   novo período de até doze meses será contado da data do ingresso seguinte àquele em que se iniciou a  contagem anterior. ',
 'quem é considerado não residente no brasil para fins tributários?   considera-se não residente no brasil,para fins tributários, a pessoa física:    - que não resida no brasil em caráter permanente e não se enquadre nas hipóteses previstas na pergunta  106;    - que se retire em caráter permanente do território nacional, na data da saída, com a entrega da declaração  de saída definitiva do país ou da comunicação de saída definitiva do país;    - que, na condição de não residente, ingresse no brasil para prestar serviços como funcionária de órgão de  governo estrangeiro situado no país;    - que ingresse no brasil com visto temporário:                a) e permaneça até 183 dias, consecutivos ou não, em um período de até doze meses;      59   b) até o dia anterior ao da obtenção de visto permanente ou de vínculo empregatício, se ocorrida antes  de completar 184 dias, consecutivos ou não, de permanência no brasil, dentro de um período de até  doze meses;    - que se ausente do brasil em caráter temporário, a partir do dia seguinte àquele em que complete doze  meses consecutivos de ausência.   (lei nº 9.718, de 27 de novembro de 1998, art. 12;',
 'quando se inicia nova contagem para estabelecer a condição de não residente de pessoa  física que se ausentou do brasil em caráter temporário, ou em caráter permanente sem entregar a  declaração de saída definitiva do país, e ficou fora do brasil menos de 12 meses consecutivos?   novo período de 12 meses consecutivos será contado da data da saída seguinte.',
 'como deve proceder a pessoa física que se ausentar do brasil em caráter temporário e  permanecer ausente por mais de 12 meses consecutivos?    pessoa física que se ausente do território nacional em caráter temporário e permaneça no exterior por mais  de doze meses consecutivos, deve:    - apresentar a comunicação de saída definitiva do país a partir da data da caracterização da condição de  não residente e até o último dia do mês de fevereiro do ano-calendário subsequente. os dependentes,  inscritos no cadastro de pessoas físicas  , que se retirem do território nacional na mesma data do titular  da comunicação devem constar desta;    - apresentar a declaração de saída definitiva do país, relativa ao período em que tenha permanecido na  condição de residente no brasil no ano-calendário da caracterização da condição de não residente, até o  último dia útil do mês de abril do ano-calendário subsequente ao da caracterização;    atenção:    se a caracterização da condição de não residente tiver ocorrido em 2019, o prazo originalmente  fixado para 30 de abril de 2020 (último dia do mês de abril do ano-calendário subsequente ao da  caracterização) fica excepcionalmente prorrogado para 30 de junho de 2020, tendo em vista a  pandemia de covid-19.    - recolher em quota única, até a data prevista para a entrega da declaração de que trata o inciso   (observado o seu “atenção”), o imposto nela apurado e os demais créditos tributários ainda não quitados,  cujos prazos para pagamento são considerados vencidos naquela data, se prazo menor não estiver estipulado  na legislação tributária.    atenção:     declaração de saída definitiva do país deve ser elaborada em computador mediante a utilização  de programa gerador próprio e ser apresentada pela internet ou entregue em mídia removível, nas  unidades da secretaria especial da receita federal do brasil  .     aplicativo da comunicação de saída definitiva do país encontra-se disponível no sítio da   na internet e a sua apresentação não dispensa a declaração de saída definitiva do país. ',
 'como são tributados os rendimentos recebidos pela pessoa física que se ausentar do brasil  em caráter temporário e permanecer ausente por mais de 12 meses consecutivos?   os rendimentos recebidos nos primeiros doze meses de ausência são tributados como os rendimentos  recebidos pelos demais residentes no brasil.   os rendimentos recebidos a partir do primeiro dia seguinte ao que completar doze meses consecutivos de  ausência sujeitam-se à tributação exclusiva na fonte ou definitiva, nos termos previstos nos arts. 26 a 45 da  instrução normativa  nº 208, de 27 de setembro de 2002.',
 'como deve proceder a pessoa física que se retirar em caráter permanente do brasil?    pessoa física residente no brasil que se retirar em caráter permanente do território nacional no curso do  ano-calendário deve:    – apresentar a comunicação de saída definitiva do país, a partir da data de saída e até o último dia do mês  de fevereiro do ano-calendário subsequente;    - apresentar a declaração de saída definitiva do país, relativa ao período em que tenha permanecido na  condição de residente no brasil no ano-calendário da saída, até o último dia útil do mês de abril do ano- calendário subsequente ao da saída definitiva, bem como as declarações correspondentes a anos-calendário  anteriores, se obrigatórias e ainda não entregues;    atenção:    se a saída definitiva tiver ocorrido em 2019, o prazo originalmente fixado para 30 de abril de 2020  (último dia do mês de abril do ano-calendário subsequente ao da saída definitiva) fica  excepcionalmente prorrogado para 30 de junho de 2020, tendo em vista a pandemia de covid-19.    - recolher em quota única, até a data prevista para a entrega das declarações de que trata o item   (observado o seu “atenção”), o imposto nelas apurado e os demais créditos tributários ainda não quitados,  cujos prazos para pagamento são considerados vencidos nesta data, se prazo menor não estiver estipulado  na legislação tributária, sendo consideradas sem efeito suspensivo da cobrança as reclamações contra  imposto sobre a renda lançado ou arrecadado na fonte, permitidos, todavia, depósitos, em dinheiro,  relativamente à parte objeto de reclamação.      atenção:     comunicação de saída definitiva do país está disponível no sítio da secretaria especial da  receita federal do brasil  : a partir da barra de menu, na opção “onde encontro”, selecione  “saida definitiva”.    declaração de saída definitiva do país deve ser elaborada em computador mediante a utilização  de programa gerador, apresentada pela internet.',
 'como é apurado o imposto sobre a renda na declaração de saída definitiva do país?   na declaração de saída definitiva do país, o imposto devido é calculado mediante a utilização dos valores  correspondentes à soma das tabelas progressivas mensais relativas aos meses em que o contribuinte tenha  permanecido na condição de residente no brasil, referentes ao período abrangido pela tributação no respectivo  ano-calendário.   na hipótese de pleitear a restituição de imposto por meio da declaração, deve ser indicado o banco, a agência  e o número da conta-corrente ou de poupança de sua titularidade em que pretende seja efetuado o crédito.      61   determinação da base de cálculo   na determinação da base de cálculo na declaração de saída definitiva do país podem ser deduzidos,  observados os limites e condições fixados na legislação pertinente:    - as importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais;   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.     - o valor correspondente à dedução anual por dependente. para o ano-calendário de 2019 esse valor está  fixado em $ 2.275,08  por dependente;    - as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios,  cujo ônus tenha sido do próprio contribuinte e desde que destinadas ao seu próprio benefício;    - as contribuições para as entidades de previdência complementar domiciliadas no brasil, destinadas a  custear benefícios complementares assemelhados aos da previdência social e para os fundos de  aposentadoria programada individual (fapi), cujo ônus tenha sido do próprio contribuinte e desde que  destinadas a seu próprio benefício bem assim de seus dependentes, condicionadas, entretanto, ao  recolhimento, também, de contribuições para o regime geral de previdência social ou, quando for o caso, para  regime próprio de previdência social dos servidores titulares de cargo efetivo da união, dos estados, do distrito  federal ou dos municípios (para contribuições feitas a partir de 1º de janeiro de 2005, veja o tópico “atenção”  da pergunta 316);    – as contribuições para as entidades fechadas de previdência complementar de natureza pública de que  trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a custear  benefícios complementares assemelhados aos da previdência social;    - as despesas médicas e as despesas com instrução, próprias, de seus dependentes e de seus  alimentandos;     - as despesas escrituradas em livro-caixa.   relativamente à dedução a que se refere o item , deve ser observado que:   a) excetuam-se da condição nele previsto os beneficiários de aposentadoria ou pensão concedidas por regime  próprio de previdência ou pelo regime geral de previdência social, mantido, entretanto, o limite de 12% (doze  por cento) do total dos rendimentos computados na determinação da base de cálculo do imposto devido na  declaração de saída definitiva do país;   b) as contribuições para planos de previdência complementar e para fapi, cujo titular ou quotista seja  dependente, para fins fiscais, do declarante, podem ser deduzidas desde que o declarante seja contribuinte  do regime geral de previdência social ou, quando for o caso, para regime próprio de previdência social dos  servidores titulares de cargo efetivo da união, dos estados, do distrito federal ou dos municípios;   c) na hipótese do item “b”, a dedução de contribuições efetuadas em benefício de dependente com mais de  16 anos fica condicionada, ainda, ao recolhimento, em seu nome, de contribuições para o regime geral de  previdência social, observada a contribuição mínima, ou, quando for o caso, para regime próprio de  previdência social dos servidores titulares de cargo efetivo da união, dos estados, do distrito federal ou dos  municípios.   relativamente à dedução a que se refere o item , deve ser observado que:   a) desde que limitada à alíquota de contribuição do ente público patrocinador, não se sujeita ao limite de 12%  (doze por cento) do total dos rendimentos computados na determinação da base de cálculo do imposto devido  na declaração de saída definitiva do país;      62   b) os valores de contribuição excedentes ao disposto no item “a” poderão ser deduzidos desde que seja  observado o limite conjunto de dedução previsto no caput do art. 11 da lei nº 9.532, de 10 de dezembro de  1997.   consulte a pergunta 316   as despesas médicas e com instrução de alimentandos, quando realizadas pelo alimentante em virtude de  cumprimento de decisão judicial ou acordo homologado judicialmente, podem ser deduzidas a tais títulos na  determinação da base de cálculo do imposto na declaração do alimentante, observados os limites e condições  fixados na legislação pertinente.   pagamentos efetuados em moeda estrangeira   as deduções referentes a pagamentos efetuados em moeda estrangeira são convertidas em dólares dos  estados unidos da américa, pelo valor fixado pela autoridade monetária do país no qual as despesas foram  realizadas para a data do pagamento e, em seguida, em reais pela cotação do dólar fixada, para venda, pelo  banco central do brasil para o último dia útil da primeira quinzena do mês anterior ao do pagamento.',
 'como é considerada a pessoa física que se retire em caráter temporário do brasil ou, se em  caráter permanente, sem a entrega da comunicação de saída definitiva do país?    pessoa física que se retire do brasil em caráter temporário ou, se em caráter permanente, sem a entrega  da comunicação de saída definitiva do país, é considerada:    - como residente no brasil, durante os primeiros 12 meses consecutivos de ausência; - como não residente, a partir do 13º mês consecutivo de ausência.',
 'qual é o regime de tributação do imposto sobre a renda aplicável à pessoa física não residente  portadora de visto temporário que entra e sai várias vezes do brasil?   se não adquirir a condição de residente, os rendimentos recebidos no brasil serão tributados de forma  definitiva ou exclusiva na fonte. caso adquira a condição de residente no país, a partir dessa data, os  rendimentos recebidos de fontes situadas no território nacional ou no exterior serão tributados de acordo com  as mesmas normas aplicáveis aos residentes no brasil.',
 'brasileiro, não residente, que retorna ao território nacional readquire de imediato a condição  de residente ou deve permanecer 184 dias para readquirir essa condição?    brasileiro que retorna ao brasil com ânimo definitivo de aqui residir passa a ser residente a partir da data da  chegada.   ',
 'como são tributados os rendimentos recebidos no brasil por não residente?   os rendimentos recebidos de fontes situadas no brasil, por não residente, estão sujeitos à tributação exclusiva  na fonte ou definitiva, conforme descrito a seguir.   alienação de bens e direitos    alienação de bens e direitos situados no brasil realizada por não residente está sujeita à tributação definitiva  sob a forma de ganho de capital, com alíquotas que podem variar de 15% a 22,5%.   na apuração do ganho de capital de não residente não se aplicam as isenções e reduções do imposto  previstas para os residentes no brasil.   consulte as perguntas 544 e 607   operações financeiras    - ressalvado o disposto nos itens  e , o não residente se sujeita às mesmas normas de tributação pelo  imposto sobre a renda previstas para os residentes no brasil, em relação aos:   a) rendimentos decorrentes de aplicações financeiras de renda fixa e em fundos de investimento;   b) ganhos líquidos auferidos em operações realizadas em bolsas de valores, de mercadorias, de futuros e  assemelhadas;   c) ganhos líquidos auferidos na alienação de ouro, ativo financeiro, e em operações realizadas nos mercados  de liquidação futura, fora de bolsa;   d) rendimentos auferidos nas operações de swap.    - os rendimentos auferidos por investidor não residente, que realizar operações financeiras no brasil de  acordo com as normas e condições estabelecidas pelo conselho monetário nacional  , sujeita-se à  incidência do imposto sobre a renda às seguintes alíquotas:   a) 10% em aplicações nos fundos de investimento em ações, swap, registradas ou não em bolsa, e em  operações nos mercados de liquidação futura, fora de bolsa;   b) 15% nos demais casos, inclusive em operações de renda fixa, realizadas no mercado de balcão ou em  bolsa.    - não estão sujeitos à incidência do imposto os ganhos de capital apurados pelo investidor não residente  de que trata o item , assim entendidos os resultados positivos auferidos:   a) nas operações realizadas em bolsas de valores, de mercadorias, de futuros e assemelhadas, ressalvado o  disposto no item ;   b) nas operações com ouro, ativo financeiro, fora de bolsa.    -  não incidência prevista na alínea "a" do item  não se aplica aos resultados positivos auferidos por não  residente nas operações conjugadas que permitam a obtenção de rendimentos predeterminados, tais como  as realizadas:   a) nos mercados de opções de compra e de venda em bolsas de valores, de mercadorias e de futuros (box);   b) no mercado a termo nas bolsas de que trata a alínea "a" deste item, em operações de venda coberta e sem  ajustes diários;   c) no mercado de balcão.   consulte a pergunta 687   remuneração do trabalho, de aposentadoria, de pensão e de prestação de serviços   os rendimentos do trabalho, com ou sem vínculo empregatício, de aposentadoria, de pensão e os da  prestação de serviços, exceto serviços técnicos e de assistência técnica e administrativas, pagos, creditados,  entregues, empregados ou remetidos a não residente sujeitam-se à incidência do imposto na fonte à alíquota  de 25%.   atenção:    também se sujeitam à alíquota de 25%, os rendimentos de pensões e de aposentadorias recebidos  por não residente de forma acumulada, tendo em vista que, nesse caso, não se aplica a forma de  tributação de que trata o art. 12- da lei nº 7.713, de 22 de dezembro de 1988, uma vez que esses  rendimentos não se sujeitam à tabela progressiva.   consulte a pergunta 121      64   royalties e serviços técnicos e de assistência técnica e administrativa   as importâncias pagas, creditadas, entregues, empregadas ou remetidas a não residente a título de royalties  de qualquer natureza e de remuneração de serviços técnicos e de assistência técnica, administrativa e  semelhantes sujeitam-se à incidência do imposto na fonte à alíquota de 15%, ou se recebidos por residente  em país com tributação favorecida, à alíquota de 25%.    - classificam-se como royalties os rendimentos de qualquer espécie decorrentes do uso, fruição, exploração  de direitos, tais como:   a) direito de colher ou extrair recursos vegetais, inclusive florestais;   b) direito de pesquisar e extrair recursos minerais;   c) uso ou exploração de invenções, processos e fórmulas de fabricação e de marcas de indústria e comércio;   d) exploração de direitos autorais, salvo quando recebidos pelo autor ou criador do bem ou obra.    - considera-se:   a) serviço técnico o trabalho, obra ou empreendimento cuja execução dependa de conhecimentos técnicos  especializados, prestados por profissionais liberais ou de artes e ofícios;   b) assistência técnica a assessoria permanente prestada pela cedente de processo ou fórmula secreta à  concessionária, mediante técnicos, desenhos, estudos, instruções enviadas ao brasil e outros serviços  semelhantes, os quais possibilitem a efetiva utilização do processo ou fórmula cedido.   os juros de mora e quaisquer outras compensações decorrentes do pagamento em atraso dos royalties e da  remuneração de serviços técnicos e de assistência técnica, administrativa e semelhantes sujeitam-se à  incidência de imposto na fonte nas mesmas condições dos valores principais a que se refiram.   instalação e manutenção de escritórios comerciais e de representação, de armazéns, depósitos ou  entrepostos   os rendimentos recebidos de fontes situadas no brasil, por não residente, relativos a pagamento, crédito,  emprego, entrega ou remessa para o exterior de despesas com instalação e manutenção de escritórios  comerciais e de representação, de armazéns, depósitos ou entrepostos sujeitam-se à incidência do imposto  na fonte à alíquota de 15%.   juros, comissões, despesas e descontos decorrentes de colocações no exterior de títulos de crédito  internacionais   os juros, comissões, despesas e descontos decorrentes de colocações no exterior, previamente autorizadas  pelo banco central do brasil, de títulos de crédito internacionais, inclusive commercial papers, pagos,  creditados, empregados, entregues ou remetidos a não residente sujeitam-se à incidência do imposto na fonte  à alíquota de 15%.   comissões pagas por exportadores, fretes, afretamentos, aluguéis ou arrendamento de embarcações  ou aeronaves, aluguel de containers, sobrestadia e demais serviços de instalações portuárias   os rendimentos recebidos de fontes situadas no brasil, por não residente, relativos a pagamento, crédito,  emprego, entrega ou remessa de comissões por exportadores a seus agentes no exterior, de receitas de  fretes, afretamentos, aluguéis ou arrendamentos de embarcações marítimas ou fluviais ou de aeronaves  estrangeiras, feitos por empresas, desde que tenham sido aprovados pelas autoridades competentes, bem  assim de aluguel de containers, sobrestadia e outros relativos ao uso de serviços de instalações portuárias,  sujeitam-se à incidência do imposto na fonte à alíquota de 0%, ou se recebidos por residente em país com  tributação favorecida, à alíquota de 25%.   despesas com promoção, propaganda e pesquisas de mercado, aluguéis e arrendamentos   os rendimentos recebidos de fontes situadas no brasil, por não residente, relativos a pagamento, crédito,  emprego, entrega ou remessa para o exterior de despesas relacionadas com pesquisa de mercado para  produtos brasileiros de exportação, bem assim aquelas decorrentes de participação em exposições, feiras e  eventos semelhantes, inclusive aluguéis e arrendamentos de estandes e locais de exposição, vinculadas à  promoção de produtos brasileiros, e de despesas com propaganda realizadas no âmbito desses eventos,  sujeitam-se à incidência do imposto na fonte à alíquota de 0%, ou se recebidos por residente em país com  tributação favorecida, à alíquota de 25%.   demais rendimentos   os demais rendimentos pagos, creditados, empregados, entregues ou remetidos a não residente por fontes  situadas no brasil, inclusive a título de juros sobre o capital próprio, bem assim os decorrentes de cessão de  direitos de atleta profissional, solicitação, obtenção e manutenção de direitos de propriedades industriais no  exterior, aquisição ou remuneração, a qualquer título, de qualquer forma de direito, e os relativos a comissões  e despesas incorridas nas operações de colocação, no exterior, de ações de companhias abertas,      65   domiciliadas no brasil, desde que aprovadas pelo banco central do brasil e pela comissão de valores  mobiliários, sujeitam-se à incidência do imposto na fonte à alíquota de 15%, quando não tiverem tributação  específica prevista em lei, ou se recebidos por residente em país com tributação favorecida, à alíquota de  25%.   atenção:    1. rendimentos de não residentes recebidos de fonte no exterior não são tributáveis no brasil;   2. considera-se país com tributação favorecida aquele que não tribute a renda ou que a tribute à  alíquota inferior a 20%, conforme lista constante no art. 1º da instrução normativa  nº 1.037,  de 4 de junho de 2010;   3. caso o brasil tenha celebrado tratado para evitar a dupla tributação com o país de residência  do contribuinte, o tratado deve ser consultado para verificar a existência de tratamento  diferenciado daquele previsto na legislação brasileira;   4. consulte, também, as perguntas 191, 162, 493 e 699.',
 'como são tributados os rendimentos recebidos por residente no brasil de fontes no exterior?   os rendimentos recebidos de fontes situadas no exterior, por residente no brasil, transferidos ou não para o  país, inclusive de representações diplomáticas e organismos internacionais, observados os acordos,  convenções e tratados internacionais firmados entre o brasil e o país de origem dos rendimentos, têm o  seguinte tratamento:   1 - ganhos de capital    alienação de bens ou direitos e a liquidação ou resgate de aplicações financeiras, inclusive depósito  remunerado, adquiridos, a qualquer título, em moeda estrangeira, bem assim a alienação de moeda  estrangeira mantida em espécie, de propriedade de pessoa física, estão sujeitos à apuração de ganho de  capital sujeito à tributação de forma definitiva (ganho de capital sobre bens e direitos adquiridos em moeda  estrangeira e ganho de capital sobre alienação de moeda estrangeira em espécie);   consulte as perguntas 601, 602, 603, 604 e 605   2 - resultado da atividade rural    resultado da atividade rural exercida no exterior, quando positivo, integra a base de cálculo do imposto  devido na declaração de ajuste anual.   consulte as perguntas 491, 492 e 493   3 - demais rendimentos recebidos   os demais rendimentos recebidos de fontes situadas no exterior por residente no brasil, transferidos ou não  para o país, estão sujeitos à tributação sob a forma de recolhimento mensal obrigatório (carnê-leão), no mês  do recebimento, e na declaração de ajuste anual.   3.1 - base de cálculo sujeita à incidência mensal do imposto   na determinação da base de cálculo sujeita à incidência mensal do imposto, quando não utilizados para fins  de retenção na fonte, podem ser deduzidos, observados os limites e condições fixados na legislação  pertinente:    - as importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais, ou de escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de março de  2015 - código de processo civil;      66   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.     – o valor mensal, por dependente, para o ano-calendário de 2019, é de $ 189,59;    - as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios,  cujo ônus tenha sido do próprio contribuinte e desde que destinadas a seu próprio benefício;    - as despesas escrituradas em livro-caixa, quando permitidas (consulte a pergunta 400);    – as contribuições para as entidades fechadas de previdência complementar de natureza pública de que  trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a custear  benefícios complementares assemelhados aos da previdência social.   3.1.1 - carnê-leão   a) o imposto relativo ao carnê-leão deve ser calculado mediante utilização da tabela progressiva mensal  vigente no mês do recebimento do rendimento e recolhido até o último dia útil do mês subsequente ao do  recebimento do rendimento;   b) o imposto pago no país de origem dos rendimentos pode ser compensado no mês do pagamento com o  imposto relativo ao carnê-leão e com o apurado na declaração de ajuste anual, até o valor correspondente à  diferença entre o imposto calculado com a inclusão dos rendimentos de fontes no exterior e o imposto  calculado sem a inclusão desses rendimentos, observado os acordos, tratados e convenções internacionais  firmados pelo brasil ou da existência de reciprocidade de tratamento;   c) se o imposto pago no exterior ocorrer em ano-calendário posterior ao do recebimento do rendimento, a  pessoa física pode compensá-lo com o imposto relativo ao carnê-leão do mês do seu efetivo pagamento e  com o apurado na declaração de ajuste anual do ano-calendário do pagamento do imposto, observado o  limite de compensação de que trata a alínea "b" relativamente à declaração de ajuste anual do ano-calendário  do recebimento do rendimento;   d) caso o imposto pago no exterior seja maior do que o imposto relativo ao carnê-leão no mês do pagamento,  a diferença pode ser compensada nos meses subsequentes até dezembro do ano-calendário e na declaração  de ajuste anual, observado o limite de que trata a alínea "b".   3.2 - base de cálculo na declaração de ajuste anual   na determinação da base de cálculo na declaração de ajuste anual podem ser deduzidos, observados os  limites e condições fixados na legislação pertinente:    - as importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais, ou de escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de março de  2015 - código de processo civil;   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.    - o valor de $ 2.275,08, por dependente;      67    - as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios,  cujo ônus tenha sido do próprio contribuinte e desde que destinadas a seu próprio benefício;    - as contribuições para as entidades de previdência complementar domiciliadas no brasil, destinadas a  custear benefícios complementares assemelhados aos da previdência social e para os fundos de  aposentadoria programada individual (fapi), cujo ônus tenha sido do próprio contribuinte e desde que  destinadas a seu próprio benefício bem assim de seus dependentes (esta dedução é limitada a 12% do total  dos rendimentos tributáveis computados na determinação da base de cálculo do imposto devido na  declaração) - para contribuições feitas a partir de 1º de janeiro de 2005, veja “atenção” da pergunta 316;    - as despesas médicas e as despesas com instrução, próprias, de seus dependentes e de seus  alimentandos;    - as despesas escrituradas em livro-caixa, quando permitidas (consulte a pergunta 400);    – as contribuições para as entidades fechadas de previdência complementar de natureza pública de que  trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a custear  benefícios complementares assemelhados aos da previdência social.   3.2.1 - declaração de ajuste anual   outros rendimentos recebidos de fontes situadas no brasil ou no exterior pelas pessoas físicas residentes no  brasil devem ser declarados segundo as mesmas normas aplicáveis às demais pessoas físicas residentes no  brasil.    declaração de ajuste anual deve ser transmitida pela internet até o último dia útil do mês de junho do ano- calendário subsequente ao do recebimento dos rendimentos.    saldo do imposto apurado na declaração deve ser recolhido de acordo com as normas aplicáveis às demais  pessoas físicas residentes no brasil.   atenção:   sobre acordos internacionais e legislação de reciprocidade, consulte a pergunta 122;   sobre 13º salário recebido de governo estrangeiro, consulte a pergunta 251.',
 'qual é o tratamento tributário dos rendimentos recebidos do exterior quando inexistir acordo  ou lei que preveja a reciprocidade?   nesse caso, os rendimentos do exterior recebidos por residente no brasil, transferidos ou não para o país,  submetem-se às disposições da legislação tributária brasileira vigente, não podendo ser compensado o valor  do imposto porventura pago no país de origem.',
 'como são tributados os benefícios ou resgates de previdência complementar e fundo de  aposentadoria programada individual (fapi) recebidos por não residente no brasil, mesmo que o  beneficiário tenha 65 anos ou mais?   sujeita-se à tributação exclusiva na fonte, à alíquota de 25%, a totalidade dos valores pagos por fonte no brasil  a residentes no exterior a título de benefício ou resgate de previdência complementar e de fundo de  aposentadoria programada individual (fapi). não se aplica aos rendimentos desse contribuinte a tabela progressiva, nem a isenção sobre os rendimentos  de aposentadoria que usufruem os residentes tributários no brasil com 65 anos ou mais.',
 'como são tributados os rendimentos recebidos, de fonte pagadora situada no brasil, por não  residente a título de pensão alimentícia, pensão por morte ou invalidez permanente ou de  aposentadoria?   os rendimentos de beneficiário não residente estão sujeitos à tributação exclusiva na fonte, à alíquota de 25%,   prevista no art. 746 do regulamento do imposto sobre a renda – /2018, aprovado pelo decreto nº 9.580,  de 22 de novembro de 2018, não fazendo jus à isenção de que trata o art. 35 do referido diploma legal,  ressalvada a existência de tratado ou acordo internacional.   cabe ao beneficiário comunicar à fonte pagadora tal condição.   atenção:    também se sujeitam à alíquota de 25%, os rendimentos de pensões e de aposentadorias recebidos  por não residente de forma acumulada, tendo em vista que, nesse caso, não se aplica a forma de  tributação de que trata o art. 12- da lei nº 7.713, de 22 de dezembro de 1988, uma vez que esses  rendimentos não se sujeitam à tabela progressiva. ',
 'como são tributados os rendimentos de residente no brasil recebidos do exterior na  existência de acordo internacional?    tratamento fiscal, inclusive quanto à compensação do imposto pago no exterior (desde que não seja  compensado ou restituído no exterior), é aquele pactuado entre o brasil e o país contratante do acordo.    compensação do imposto pago no exterior (desde que não seja compensado ou restituído no exterior) é  também possível no caso de o imposto ter sido pago em país cuja legislação permita a reciprocidade de  tratamento.    teor e a vigência da lei estrangeira concessiva de reciprocidade devem ser comprovados pelo contribuinte,  com cópia da lei publicada em órgão da imprensa oficial do país de origem do rendimento, traduzida por  tradutor juramentado e autenticada pela representação diplomática do brasil naquele país, ou mediante  declaração desse órgão atestando a reciprocidade de tratamento tributário.   não é necessária a prova de reciprocidade para a alemanha, o reino unido e os estados unidos da américa  (consulte a pergunta 128).   os países com os quais o brasil tem acordo vigente são os seguintes:   áfrica do sul coreia do sul  hungria  noruega  suécia   argentina dinamarca  índia  países baixos (holanda) trinidad e tobago   áustria equador  israel  peru  turquia    bélgica espanha  itália  portugal  ucrânia   canadá filipinas  japão  república eslovaca  venezuela   chile finlândia  luxemburgo  república tcheca     china  frança  méxico rússia          69   atenção:   os acordos internacionais podem ser consultados na página da secretaria especial da receita  federal do brasil  , na internet, no endereço http://receita.economia.gov.br. na página inicial,  na barra de menu lateral, procure por “ rápido” e selecione “legislação”; selecione  “ ” e, em seguida, selecione “ ”; por fim, selecione  “acordos para evitar a dupla tributação e prevenir a evasão fiscal”. ',
 'qual é o tratamento tributário dos rendimentos recebidos no brasil e no exterior por  estrangeiro que transfira residência para o brasil?   existindo acordo ou tratado para evitar a dupla tributação firmado entre o brasil e o país de onde provenham  esses rendimentos, a tributação é a prevista nesses atos.     tratamento tributário previsto na legislação brasileira é o seguinte:    - quando caracterizada a condição de residente no brasil, os rendimentos são tributados como os de  residente no país. os rendimentos oriundos de fontes do exterior estão sujeitos ao recolhimento mensal  obrigatório (carnê-leão) e na declaração de ajuste anual;    - os rendimentos recebidos do exterior antes de caracterizada a condição de residente no país não estão  sujeitos à tributação no brasil;    - não cabe a compensação na declaração de ajuste de imposto pago sob as formas de tributação exclusiva  de fonte ou tributação definitiva;    - enquanto não caracterizada a condição de residência no brasil, caso a pessoa física aufira rendimentos  no país, esses valores são tributados conforme descrito na pergunta 118, até a data do implemento da  condição de residente. ',
 'qual é o tratamento tributário dos lucros e dividendos recebidos por residente no brasil de  empresa domiciliada no exterior?   os lucros e dividendos recebidos de empresa domiciliada no exterior, transferidos ou não para o brasil, estão  sujeitos ao recolhimento mensal obrigatório (carnê-leão) e ao ajuste na declaração anual de rendimentos,  observados os acordos, convenções e tratados internacionais firmados entre o brasil e o país de origem dos  rendimentos. ',
 'como devem ser convertido em reais os pagamentos dedutíveis realizados em moeda  estrangeira?   os pagamentos efetuados em moeda estrangeira devem ser convertidos em dólares dos estados unidos da  américa com base na sua cotação para o dia em que forem realizados, e de dólares para reais com base no  valor fixado para venda pelo banco central do brasil para o último dia útil da primeira quinzena do mês anterior  ao do pagamento das despesas no exterior. ',
 'imposto sobre a renda cobrado pelo país de origem pode ser compensado no brasil quando  da existência de ato internacional ou de reciprocidade de tratamento?    imposto sobre a renda pago em país com o qual o brasil tenha firmado acordo, tratado ou convenção  internacional prevendo a compensação, ou naquele em que haja reciprocidade de tratamento, pode ser  considerado como redução do imposto devido no brasil desde que não seja compensado ou restituído no  exterior.    imposto pago no país de origem dos rendimentos pode ser compensado na apuração do valor mensal a  recolher (carnê-leão) e na declaração de rendimentos até o valor correspondente à diferença entre o imposto  calculado com a inclusão dos rendimentos de fonte no exterior e o imposto calculado sem a inclusão desses  rendimentos.   se o pagamento do imposto no exterior for posterior ao recebimento do rendimento, mas ocorrer no mesmo  ano-calendário, a pessoa física pode compensá-lo no carnê-leão do mês do efetivo pagamento do imposto no  exterior e na declaração de ajuste anual relativa a esse ano-calendário.   se o pagamento do imposto no exterior for em ano-calendário posterior ao do recebimento do rendimento, a  pessoa física pode compensá-lo no carnê-leão do mês do efetivo pagamento do imposto no exterior e na  declaração de ajuste anual do ano-calendário do pagamento do imposto, observado o limite de compensação  apurado na declaração de ajuste anual do ano-calendário do recebimento do rendimento.   se o valor a compensar do imposto pago no exterior for maior do que o valor mensal a recolher (carnê-leão),  a diferença pode ser compensada nos meses seguintes até dezembro do ano-calendário e na declaração,  observado o limite de compensação.    imposto pago no exterior deve ser convertido em dólares dos estados unidos da américa pelo seu valor  fixado pela autoridade monetária do país no qual o pagamento foi realizado, na data do pagamento e, em  seguida, em reais mediante utilização do valor do dólar dos estados unidos da américa, fixado para compra  pelo banco central do brasil para o último dia útil da primeira quinzena do mês anterior ao recebimento do  rendimento.    compensação mensal pode ser efetuada conforme demonstrado nos exemplos a seguir:   exemplo 1: (as cotações constantes dos exemplos desta pergunta são fictícias)   rendimentos produzidos na república federal da alemanha,   recebidos pelo contribuinte , em 17/06/2019   $ 10,000.00   imposto sobre a renda pago na alemanha $ 1,000.00   taxa de câmbio para compra vigente em 15/05/2019 $ 4,0025   conversão em reais   valor dos rendimentos em reais   imposto correspondente pago em reais      $ 40.025,00   $   4.002,50   cálculo do imposto anual     imposto devido antes da inclusão dos rendimentos produzidos na  alemanha   imposto devido após a inclusão desses rendimentos   limite para a compensação do imposto: diferença ( - )      $ 19.000,00      $ 32.100,00   $ 13.100,00      : neste exemplo, o imposto correspondente aos rendimentos produzidos no exterior pode ser  compensado integralmente ($ 4.002,50), uma vez que se enquadra no limite permitido para a  compensação.   exemplo 2: (as cotações constantes dos exemplos desta pergunta são fictícias)   rendimentos produzidos na frança, recebidos pelo    contribuinte , em 24/09/2019   $ 4,400.00   imposto sobre a renda pago na frança $ 1,500.00   taxa de câmbio para compra vigente em 15/08/2019 $ 4,0182   conversão em reais   valor dos rendimentos em reais   imposto correspondente pago em reais      $ 17.680,08   $   6.027,30   cálculo do imposto anual   imposto devido antes da inclusão dos rendimentos produzidos na  frança   imposto devido após a inclusão desses rendimentos   limite para a compensação do imposto: diferença ( — )      $   2.000,00      $   5.900,00   $   3.900,00   : nesta hipótese, o imposto correspondente aos rendimentos produzidos no exterior pode ser  compensado até o limite de $ 3.900,00.   atenção:  contribuinte deve providenciar a tradução para o português, por tradutor juramentado, do teor dos  comprovantes dos rendimentos e do imposto pago, para que este possa ser compensado na  declaração de ajuste anual.',
 'imposto pago no exterior pode ser compensado apenas com o imposto apurado na  declaração de ajuste anual?   não.  imposto sobre a renda pago no exterior pode ser compensado tanto na apuração mensal do imposto  quanto na declaração de ajuste anual, desde que haja acordo ou tratado firmado entre o país de origem dos  rendimentos e o brasil ou tratamento de reciprocidade no país de origem e o imposto não seja compensado  ou restituído no exterior. ',
 'é compensável no brasil o imposto pago sobre rendimentos recebidos na alemanha, nos  estados unidos da américa e no reino unido?   sim.  reciprocidade de tratamento permite a compensação no brasil do imposto pago, observado o limite de  compensação.   atenção:     valor pago nos estados unidos da américa é compensável apenas quando se tratar de imposto  federal. ',
 'qual é o tratamento tributário dos rendimentos recebidos por servidor diplomático, consular,  oficial de representações diplomáticas estrangeiras?   os rendimentos recebidos por esse servidor, pagos ou creditados por fontes estrangeiras, inclusive os  correspondentes às funções oficiais exercidas no brasil, não sofrem incidência do imposto sobre a renda.   os rendimentos recebidos de pessoa física ou jurídica domiciliada no brasil são tributados como os de não  residente no brasil.   atenção:    incluem-se no conceito de servidor diplomático o chefe de representação ou missão e os demais  funcionários que tenham a qualidade de diplomata ou estejam no exercício das funções consulares  ou oficiais.   os rendimentos do trabalho com ou sem vínculo empregatício, de aposentadoria, de pensão e os  da prestação de serviços, pagos, creditados, entregues, empregados ou remetidos por fonte  pagadora situada no brasil a não residente no brasil, estão sujeitos à tributação exclusiva à alíquota  de 25%.',
 'qual é o tratamento tributário dos rendimentos recebidos por funcionário administrativo ou  técnico estrangeiro de representações diplomáticas?   1 - não residente no brasil   somente estão sujeitos à tributação no brasil, na condição de não residente no país, os rendimentos pagos  ou creditados a esse servidor por fonte pagadora situada no brasil. todos os demais rendimentos por ele  percebidos, pagos ou creditados por fontes estrangeiras, inclusive os correspondentes às funções oficiais  exercidas no brasil, não sofrem incidência, desde que haja reciprocidade de tratamento a brasileiros que  exerçam funções idênticas no país estrangeiro.   obs.: os rendimentos do trabalho com ou sem vínculo empregatício, de aposentadoria, de pensão e os da  prestação de serviços, pagos, creditados, entregues, empregados ou remetidos por pessoa física ou jurídica  residente ou domiciliada no brasil para residente no exterior, estão sujeitos à tributação exclusiva à alíquota  de 25%.   (regulamento do imposto sobre a renda - /2018, art. 20, inciso , aprovado pelo decreto nº  9.580, de 22 de novembro de 2018; instrução normativa  nº 208, de 27 de setembro de 2002,  art. 23, inciso ; e parecer normativo nº 129, de 13 de setembro de 1973)      73   2 - residente no brasil   ainda que o estado representado seja signatário da convenção de viena, os rendimentos pagos ou creditados  a esse servidor por fontes situadas no brasil ou no exterior, inclusive os decorrentes de suas funções  específicas, sujeitam-se à tributação nas mesmas condições estabelecidas para os demais residentes no  brasil, sendo irrelevante o fato de ser servidor de representação oficial estrangeira.',
 'qual é o tratamento tributário dos rendimentos auferidos por empregado particular  estrangeiro de membros de missão diplomática?   1 - não residente no brasil   se o estado representado for signatário da convenção de viena e o empregado não tiver residência  permanente no brasil, os rendimentos do trabalho decorrentes de suas funções estão isentos do imposto  sobre a renda brasileiro.   obs.: os rendimentos do trabalho com ou sem vínculo empregatício, de aposentadoria, de pensão e os da  prestação de serviços, pagos, creditados, entregues, empregados ou remetidos por fonte pagadora situada  no brasil a não residente, estão sujeitos à tributação exclusiva à alíquota de 25%.   2 - residente no brasil   ainda que o estado representado seja signatário da convenção de viena, os rendimentos pagos ou creditados  a esse empregado por membro de missão diplomática, inclusive os decorrentes de suas funções específicas,  sujeitam-se à tributação nas mesmas condições estabelecidas para os demais residentes no brasil. assim, os  rendimentos são tributáveis no recolhimento mensal obrigatório (carnê-leão) e na declaração de ajuste anual  correspondente ao ano-calendário de seu recebimento, sendo irrelevante o fato de ser empregado particular  de membro de representação oficial estrangeira.',
 'qual é o tratamento tributário dos rendimentos auferidos por residente no brasil que seja  servidor administrativo, técnico de missão diplomática ou empregado particular de seus membros?   os rendimentos do trabalho de residente no brasil, decorrentes de suas funções específicas, prestadas a  organismos internacionais ou a representações de estados estrangeiros ou a seus membros, bem como sobre  quaisquer outros que aufira, sujeitam-se à tributação nas mesmas condições estabelecidas para os demais  residentes no brasil, sendo irrelevante o fato de ser servidor de representação oficial estrangeira ou  empregado particular de seus membros, ainda que esta seja signatária da convenção de viena. assim, os  rendimentos sujeitam-se ao carnê-leão e ao ajuste anual. ',
 'qual é o tratamento tributário dos rendimentos recebidos por empregado de empresas  estatais estrangeiras situadas no brasil?   os empregados de empresas estatais estrangeiras situadas no brasil não fazem jus ao tratamento  diferenciado concedido àqueles vinculados a embaixadas, consulados e repartições oficiais de outros países.  assim, submetem-se à tributação pelas mesmas regras aplicáveis aos demais contribuintes, de acordo com  a sua condição de residente ou de não residente no brasil. ',
 'religiosos, missionários, pesquisadores e professores estrangeiros, trabalhando no brasil,  e recebendo proventos diretamente de entidades e empresas sediadas no exterior, são contribuintes  do imposto sobre a renda no brasil?    tratamento tributário é o previsto na legislação brasileira para o residente ou não residente no brasil,  conforme a condição da pessoa física. se a pessoa física passar à condição de residente no país, deve-se  verificar se há acordo ou tratado para evitar a dupla tributação entre o brasil e o país de origem dos  rendimentos.  ',
 'qual é o tratamento tributário dos rendimentos auferidos por cientistas, professores,  pesquisadores, técnicos estrangeiros que ingressam no brasil para o fim específico de prestar  serviços ou ministrar aulas e conferências?   deve-se verificar se existe ato internacional firmado entre o brasil e o país de origem dessas pessoas físicas  ou com organismos internacionais que elas representem, no sentido de evitar a dupla tributação internacional  da renda, inclusive por meio de ato internacional de cooperação científica, cultural ou técnica.    existindo o ato e estando expressamente definida a situação fiscal dessas pessoas físicas, o tratamento  tributário é aquele nele determinado.    não havendo tal ato ou se a situação fiscal não estiver nele expressamente determinada, os rendimentos  recebidos sujeitam-se à tributação pelas mesmas regras aplicáveis aos demais contribuintes, de acordo com  a sua condição de residente ou não residente no brasil.    instrução normativa  nº 1.226, de 23 de dezembro de 2011 disciplina o fornecimento de informações  fiscais com o país com o qual o brasil tenha firmado acordo para evitar a dupla tributação.',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário do programa das  nações unidas para o desenvolvimento no brasil ? os rendimentos do funcionário do , da , têm o seguinte tratamento:   1 - funcionário estrangeiro   sobre os rendimentos do trabalho oriundos de suas funções específicas nesse organismo, bem como os  produzidos no exterior, não incide o imposto sobre a renda.   é contribuinte do imposto sobre a renda, na condição de não residente no brasil, quanto aos rendimentos que  tenham sido produzidos no brasil, tais como remuneração por serviços aqui prestados e por aplicação de  capital em imóveis no país, pagos ou creditados por qualquer pessoa física ou jurídica, quer sejam estas  residentes no brasil ou não.   caracteriza-se a condição de residente, se receber de fonte brasileira rendimentos do trabalho com vínculo  empregatício.   2 - funcionário brasileiro   os rendimentos do trabalho oriundos de suas funções específicas nesse organismo não se sujeitam ao  imposto sobre a renda, desde que o nome do funcionário conste da relação entregue à secretaria especial da  receita federal do brasil   na forma do anexo  da instrução normativa  nº 208, de 27 de setembro  de 2002.  quaisquer outros rendimentos percebidos, quer sejam pagos ou creditados por fontes nacionais ou  estrangeiras, no brasil ou no exterior, sujeitam-se à tributação como os demais residentes no brasil.   3 - pessoa física não pertencente ao quadro efetivo   os rendimentos de técnico que presta serviços a esses organismos, sem vínculo empregatício, na condição  de perito de assistência técnica com contrato temporário com período pré-fixado ou por meio de empreitada  a ser realizada (apresentação ou execução de projeto e/ou consultoria) não se sujeitam ao imposto sobre a  renda. //   atenção:    os proventos da aposentadoria, bem como as pensões, qualquer que seja a forma de pagamento,  pagos pelas nações unidas aos seus funcionários aposentados ou aos seus dependentes, não  estão sujeitos à tributação pelo imposto sobre a renda no brasil.   para que os rendimentos do trabalho oriundos do exercício de funções específicas no programa  das nações unidas para o desenvolvimento no brasil  , nas agências especializadas da  organização das nações unidas  , na organização dos estados americanos   e na  associação latino-americana de integração (aladi), situadas no brasil, recebidos por funcionários  aqui residentes, sejam considerados isentos, é necessário que seus nomes sejam relacionados e  informados à secretaria especial da receita federal do brasil   por tais organismos, como  integrantes de suas categorias por elas especificadas, em formulário específico conforme modelo  constante no anexo  da instrução normativa  nº 208, de 27 de setembro de 2002, e enviado  à coordenação-geral de fiscalização (cofis) da  até o último dia útil do mês de fevereiro do  ano-calendário subsequente ao do pagamento dos rendimentos.',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário da organização das  nações unidas   e de suas agências especializadas?   os funcionários da  possuem isenção do imposto sobre a renda por força da convenção sobre privilégios  e imunidades das nações unidas.   os funcionários das agências especializadas da  também possuem isenção desse imposto, por força da  convenção sobre privilégios e imunidades das agências especializadas.   aplica-se a convenção sobre privilégios e imunidades das nações unidas ou a convenção sobre privilégios  e imunidades das agências especializadas:     à agência internacional de energia atômica   decreto nº 59.308, de 23 de setembro de 1966,  decreto nº 59.309, de 23 de setembro de 1966 - international atomic energy agency  ;     à associação internacional de desenvolvimento   decreto nº 63.151, de 22 de agosto de 1968 -  international development association  ;     ao banco internacional para a reconstrução e o desenvolvimento - banco mundial - decreto nº  21.177, de 27 de maio de 1946; decreto nº 63.151, de 22 de agosto de 1968 - international bank for  reconstruction and development  ;     à corporação financeira internacional   decreto nº 63.151, de 22 de agosto de 1968 -  international finance corporation  ;     ao fundo das nações unidas para a infância (fisi) decreto nº 62.125, de 16 de janeiro de 1968 -  united nations children fund (unicef);      ao fundo monetário internacional   decreto nº 21.177, de 27 de maio de 1946; decreto nº 63.151,  de 22 de agosto de 1968 - international monetary fund  ;     à organização da aviação civil internacional (oaci) decreto nº 63.151, de 22 de agosto de 1968 -  international civil aviation organization (icao);     à organização das nações unidas para a alimentação e agricultura   decreto nº 63.151, de 22  de agosto de 1968; decreto nº 86.006, de 14 de maio de 1981 - food and agriculture organization of  the united nations  ;      76     à organização das nações unidas para educação, ciência e cultura (unesco) decreto nº 63.151, de  22 de agosto de 1968 - united nations educational, scientific and cultural organization (unesco);     à organização internacional de refugiados   decreto nº 63.151, de 22 de agosto de 1968 - office  of the united nations high comissioner for refugies  ;     à organização internacional do trabalho   decreto nº 63.151, de 22 de agosto de 1968 -  international labour organization  ;     à organização marítima consultiva intergovernamental   decreto nº 63.151, de 22 de agosto  de 1968 - inter-governamental maritime consultative organization  ;     à organização meteorológica mundial   decreto nº 63.151, de 22 de agosto de 1968 - world  meteorological organization  ;     à organização mundial da saúde   decreto nº 63.151, de 22 de agosto de 1968 - world health  organization  ;     à organização mundial do turismo   decreto nº 75.102, de 20 de dezembro de 1974; (segundo  o sitio da  é agencia especializada da );      à união internacional de telecomunicações   decreto nº 63.151, de 22 de agosto de 1968 -  international telecomunication union  ;     à união postal universal decreto nº 63.151, de 22 de agosto de 1968 - universal postal union . ',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário da associação  latino-americana de integração (aladi)?    funcionário da aladi, seja ele nacional ou estrangeiro, tem seus rendimentos tributados da mesma forma e  condições dos funcionários do , da , exceto no que concerne a proventos de aposentadoria e  pensões pagos pela aladi, que se submetem ao imposto sobre a renda brasileiro, quando residente no brasil. ',
 'qual é o tratamento dos rendimentos recebidos por funcionário da organização dos estados  americanos?    funcionário da , seja ele nacional ou estrangeiro, tem seus rendimentos tributados da mesma forma e  condições dos funcionários do , da , exceto no que concerne a proventos de aposentadoria e  pensões pagos pela , que se submetem ao imposto sobre a renda brasileiro, quando residente no brasil.',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário dos organismos  especializados na organização dos estados americanos  ?      os rendimentos recebidos por funcionários dos organismos especializados na organização dos estados  americanos  , bem como por profissionais que lhes prestem serviço sem vínculo empregatício, que  sejam residentes no brasil, estão sujeitos à tributação sob a forma de recolhimento mensal obrigatório (carnê- leão) no mês do recebimento e na declaração de ajuste anual, exceto se houver sido firmado acordo dispondo  de forma diversa.   atenção:    os funcionários do quadro de pessoal internacional do instituto interamericano de cooperação para  a agricultura   estão isentos do imposto sobre a renda em relação aos salários e vencimentos  pagos  pelo , uma vez que esse organismo especializado possui acordo dispondo  expressamente sobre a isenção do imposto sobre a renda relativo a esses rendimentos. ',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário da agência espacial  europeia  ?   não estão sujeitos ao imposto sobre a renda no brasil os rendimentos pagos pela  a seus funcionários,  desde que estes não sejam residentes no brasil ou aqui não permaneçam por mais de 183 dias em cada  exercício financeiro. ',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário da rede de  informação tecnológica latino-americana (ritla) do sistema econômico latino-americano (sela)?    funcionário da secretaria do comitê de ação para o estabelecimento da rede de informação tecnológica  latino-americana (ritla), do sistema econômico latino-americano (sela), em nível de direção, técnico e  administrativo, que não seja brasileiro, nem residente no brasil, está isento do imposto sobre a renda quanto  aos salários e vencimentos a ele pago pelo sela, em decorrência de suas funções específicas. ',
'qual é o tratamento tributário dos rendimentos auferidos por funcionário da organização  internacional de telecomunicações por satélite (intelsat)?   os salários e vencimentos auferidos por membros do quadro de funcionários da intelsat não estão sujeitos ao  imposto sobre a renda, excluindo pensões e outros benefícios similares.  ',
 'qual é o tratamento tributário relativo aos salários e emolumentos pagos pela organização  dos estados ibero-americanos para a educação, a ciência e a cultura  ?   os salários e emolumentos pagos pela  aos seus funcionários e especialistas estão sujeitos ao seguinte  tratamento tributário:    1 – se estrangeiro e residente no exterior:   não incide o imposto sobre a renda.   2 – se estrangeiro com residência permanente no brasil ou se brasileiro:   tributação sob a forma de recolhimento mensal obrigatório ("carnê-leão") no mês do recebimento e na  declaração de ajuste anual. ',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário do conselho de  cooperação aduaneira?   os salários e emolumentos auferidos por funcionário do conselho de cooperação aduaneira não estão  sujeitos ao imposto sobre a renda no brasil. esse organismo determina expressamente as categorias de  funcionários beneficiários dessa isenção. ',
 'qual é o tratamento tributário dos rendimentos auferidos por funcionário da corporação  interamericana de investimentos?   os salários e honorários auferidos por funcionário e empregado da corporação, que não sejam brasileiros,   estão isentos do imposto sobre a renda. ',
 'qual é o tratamento tributário aplicado ao funcionário de organismo internacional a partir do  momento em que deixa a instituição?    partir do momento em que a pessoa física deixa o organismo internacional passa a ser tributada como os  demais residentes no brasil, desde que se enquadre como tal, conforme perguntas 104 e 106, salvo  disposição expressa em acordo ou tratados internacionais em relação a proventos de aposentadoria.',
 'qual é o tratamento tributário estabelecido à pessoa física ausente no exterior a serviço do  brasil, em relação aos rendimentos recebidos de autarquias ou repartições do governo brasileiro  situados no exterior?   os rendimentos do trabalho assalariado, pagos por autarquias ou repartições do governo brasileiro situados  no exterior, em dólares norte-americanos, ou em moeda estrangeira equivalente ao valor que deveriam  receber em dólar, são tributados da mesma forma que os rendimentos recebidos de fonte brasileira pelos  residentes no brasil, porém são considerados tributáveis apenas 25% do valor total de tais rendimentos (os  75% restantes são informados como rendimentos não tributáveis).   atenção:    1 - as deduções legais, referentes a pagamentos efetuados em moeda estrangeira, são convertidas  em dólares dos estados unidos da américa pelo valor fixado pela autoridade monetária do país no  qual as despesas foram realizadas para a data do pagamento e, em seguida, em reais pela cotação  do dólar para venda fixada pelo banco central do brasil para o último dia útil da primeira quinzena  do mês anterior ao do pagamento;   2 - os rendimentos e o imposto pago no exterior devem ser convertidos em dólares dos estados  unidos da américa, pelo seu valor fixado pela autoridade monetária do país de origem do  rendimento na data do recebimento e, em seguida, em reais mediante utilização do valor do dólar  fixado para compra pelo banco central do brasil para o último dia útil da primeira quinzena do mês  anterior ao do recebimento do rendimento.',
 'quem pode se enquadrar na condição de "a serviço do país no exterior" para usufruir dos  benefícios fiscais próprios dessa situação particular?   considera-se pessoa física ausente no exterior a serviço do país o servidor público ou o militar das forças  armadas a serviço no exterior, assim considerado aquele que se encontra em missão fora do brasil, por ter  sido nomeado ou designado para o desempenho ou exercício de cargo, função ou atividade no exterior,  inclusive:   a) o funcionário da administração federal direta regido pela legislação trabalhista, da administração federal  indireta e das fundações sob supervisão ministerial;   b) o funcionário do poder legislativo, do poder judiciário e do tribunal de contas da união;   c) no que couber, o funcionário do distrito federal, dos estados e dos municípios, bem como a pessoa sem  vínculo com o serviço público designada pelo presidente da república.   essas pessoas estão obrigadas a apresentar a declaração de ajuste anual nas condições previstas no ajuda  do programa  2020.   atenção:     empregado de empresa pública ou de sociedade de economia mista, bem assim o contratado  local de representações diplomáticas, não são considerados ausentes "a serviço do país no  exterior" quando se encontrarem a serviço específico dessas entidades fora do brasil. ',
 'como devem ser declarados os rendimentos do trabalho assalariado de servidor público, civil  ou militar, em missão no exterior durante o ano-calendário?    tratamento fiscal dos rendimentos recebidos depende da situação particular de cada servidor, de acordo  com a natureza do serviço desempenhado no exterior, da forma abaixo explicitada.   1 - residente no brasil ausente no exterior a serviço do país   os rendimentos recebidos antes da saída ou após o regresso, de autarquias ou repartições do governo, pagos  no brasil, são declarados e tributados normalmente, como de residente no brasil, na forma da legislação  vigente. já os rendimentos recebidos durante a permanência no exterior são tributados conforme resposta à  pergunta 148.   2 - residente no brasil em missão temporária, transitória ou eventual no exterior   os rendimentos são declarados como rendimentos tributáveis pelo seu total.   os valores recebidos em dólares são convertidos em reais pela taxa cambial para compra, fixada pelo banco  central do brasil, para o último dia útil da primeira quinzena do mês anterior ao de cada recebimento.   atenção:    os rendimentos referentes a diárias e ajudas de custo não estão sujeitos a tributação, devendo os  valores recebidos em moeda estrangeira, convertidos em reais na forma do item 2, serem  informados na ficha “rendimentos isentos e não tributáveis”.',
 'pessoa física que se ausenta do país pode optar pela manutenção da condição de residente  no brasil para fins tributários, ainda que preencha os requisitos para ser considerada residente no  exterior?   não.  contribuinte tem que verificar em que condição se enquadra, ou seja, residente ou não residente no  brasil, de acordo com as condições descritas nas perguntas 106 e 108.',
 'como são tributados os rendimentos do trabalho assalariado auferidos no exterior por  brasileiro que saiu do brasil para prestar serviço a empresa estrangeira?    pessoa física que se ausentou do brasil, exceto a serviço do país, tem seus rendimentos tributados  conforme se enquadre como residente ou não residente no brasil.',
 'como deve informar os bens na declaração de ajuste anual a pessoa física que passou à  condição de residente no brasil em 2019?     pessoa física que passou à condição de residente no brasil, ou readquiriu tal condição, está sujeita às  normas vigentes na legislação tributária aplicáveis aos demais residentes, a partir da data em que se  caracterizar a condição de residente, estando obrigada a apresentar a declaração de ajuste anual   do  exercício de 2020, ano-calendário de 2019.   na declaração de bens e direitos da  devem ser relacionados, pormenorizadamente, os bens móveis,  imóveis, direitos e obrigações que, no brasil e no exterior, constituíam o patrimônio da pessoa física e o de  seus dependentes, observadas as mesmas regras de preenchimento da declaração de bens e direitos  aplicáveis a quem tenha mantido a condição de residente no país durante todo o ano, e ainda:    -  campo “situação em 31/12/2018 ($)”: deverá conter as informações referentes à data em que  caracterizou a condição de residente no ano de 2019;      81    -  custo dos bens e direitos situados no exterior, adquiridos no período em que o contribuinte se encontrava  na situação de não residente no brasil é o valor de aquisição convertido:   a) em reais pela cotação cambial de venda da moeda em que o bem foi adquirido, fixada pelo banco central  do brasil para a data da aquisição, no caso de bens adquiridos até 31 de dezembro de 1999;   b) em dólares dos estados unidos da américa e, em seguida, em reais pela cotação do dólar fixada, para  venda, pelo banco central do brasil, para a data da aquisição, no caso de bens adquiridos a partir de 1º de  janeiro de 2000. ',
 'como são tributados os rendimentos do trabalho assalariado, auferidos no japão, por  brasileiro que saiu do brasil temporariamente para prestar serviços com vínculo empregatício  naquele país?   desde que não adquira a condição de residente no exterior (consulte as perguntas 106 a 109), o brasileiro  que se encontre temporariamente no japão prestando serviços com vínculo empregatício tem seus  rendimentos tributados no brasil e isentos no japão se ocorrerem essas três condições cumulativas:   a) período de permanência no japão não superior a 183 dias no ano fiscal;   b) remuneração paga por um empregador ou em nome de um empregador que não seja residente no japão;   c) o encargo da remuneração não seja suportado por um estabelecimento permanente ou por uma instalação  fixa que o empregador tiver no japão.   não atendidas todas essas condições, os rendimentos são tributados em ambos os países, e o imposto pago  no japão pode ser compensado no carnê-leão e na declaração de ajuste anual, desde que não exceda a  diferença entre o imposto calculado antes da inclusão dos rendimentos produzidos no japão e o imposto  devido após a inclusão desses rendimentos.   caso o brasileiro adquira a condição de residente no exterior, os rendimentos auferidos no japão serão  tributados no brasil até a data em que se caracterizar a condição de não residente.   atenção:    não incide o imposto sobre a renda sobre a remessa desses rendimentos para o brasil. entretanto,  os valores devem ser remetidos por intermédio de instituições financeiras autorizadas a operar com  câmbio ou declarados à alfândega no momento do desembarque, para que possam dar cobertura  a acréscimos patrimoniais.',
 'qual é o regime de tributação dos rendimentos do trabalho pagos por fontes situadas no  brasil a funcionários ou empregados brasileiros que estejam no exterior por motivo de estudos em  estabelecimento de ensino superior, técnico ou equivalente?   não há tratamento tributário especial ao contribuinte nesta situação. enquanto perdurar a condição de  residente no brasil, os rendimentos provenientes do vínculo de emprego são tributados na fonte, normalmente,  às alíquotas aplicáveis aos rendimentos do trabalho assalariado, e incluídos pelo seu total na declaração de  ajuste anual. os rendimentos recebidos de outras fontes, situadas ou não no brasil, têm o mesmo tratamento  de rendimentos auferidos por residente no país.   após a perda da condição de residente no brasil, os rendimentos recebidos de fontes situadas no brasil são  tributados como os de não residente. ',
 'como converter para reais os valores recebidos ou pagos em moeda estrangeira?   os rendimentos e o imposto pago no exterior devem ser convertidos em dólares dos estados unidos da  américa, pelo valor fixado pela autoridade monetária do país de origem dos rendimentos para a data do  recebimento ou pagamento e, em seguida, em reais mediante utilização do valor do dólar fixado para compra  pelo banco central do brasil para o último dia útil da primeira quinzena do mês anterior ao do recebimento do  rendimento.   as deduções referentes aos pagamentos efetuados em moeda estrangeira devem ser convertidas em dólar  dos estados unidos da américa seu valor fixado pela autoridade monetária do país no qual as despesas foram  realizadas para a data do pagamento e, em seguida, em reais mediante utilização do valor do dólar fixado  para venda pelo banco central do brasil para o último dia útil da primeira quinzena do mês anterior ao do  pagamento. ',
 'quais são as deduções cabíveis na determinação da base de cálculo relativa a rendimentos  recebidos de fontes situadas no exterior?   1 - na determinação da base de cálculo sujeita à incidência mensal do imposto podem ser deduzidas:     as importâncias pagas a título de pensão alimentícia em face das normas do direito de família, quando  em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais, ou de escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de março  de 2015 - código de processo civil – a seguir transcrito (tratando-se de pensão paga em cumprimento de  sentença estrangeira, consulte a pergunta 339);   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.      o valor mensal, por dependente, para o ano-calendário 2019, de $ 189,59;     contribuição à previdência social no brasil; e     despesas escrituradas no livro-caixa, desde que relativas a rendimentos recebidos por profissional  autônomo.   as deduções podem ser consideradas apenas quando não utilizadas para fins de retenção na fonte.    imposto deve ser calculado mediante utilização da tabela progressiva mensal vigente no mês do  recebimento do rendimento.   2 - na determinação da base de cálculo sujeita à incidência anual do imposto para o ano-calendário de  2019 podem ser deduzidas:     as importâncias pagas a título de pensão alimentícia em face das normas do direito de família, quando  em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais, ou de escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de março  de 2015 - código de processo civil;      83   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.     $ 2.275,08 por dependente;     despesas escrituradas no livro-caixa, desde que relativas a rendimentos recebidos por profissional  autônomo;     contribuição à previdência social no brasil;     contribuições a entidades de previdência complementar domiciliadas no brasil destinadas a custear  benefícios complementares assemelhados aos da previdência social e para os fundos de aposentadoria  programada individual (fapi), cujo ônus tenha sido do próprio contribuinte e desde que destinadas a seu  próprio benefício ou a de seus dependentes, limitadas a 12% do total dos rendimentos tributáveis na  declaração (para contribuições feitas a partir de 1º de janeiro de 2005, veja o tópico “atenção” da pergunta  316);     contribuições para as entidades fechadas de previdência complementar de natureza pública de que trata  o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a custear  benefícios complementares assemelhados aos da previdência social, observados os limites e condições  fixados na legislação pertinentes;     despesas médicas, próprias, de seus dependentes e de seus alimentandos, inclusive quando pagas no  exterior, observadas as condições previstas na legislação; e     despesas com instrução própria, de seus dependentes e de seus alimentandos, até o limite anual individual  de $ 3.561,50.   atenção:    as deduções referentes aos pagamentos efetuados em moeda estrangeira são convertidas em  dólares dos estados unidos da américa, pelo seu valor fixado pela autoridade monetária do país  no qual as despesas foram realizadas, na data do pagamento e, em seguida, em reais mediante  utilização do valor do dólar dos estados unidos da américa, fixado para venda pelo banco central  do brasil para o último dia útil da primeira quinzena do mês anterior ao do pagamento.',
 'pessoa física com 65 anos ou mais, não residente, tem direito à parcela de isenção referente  a rendimentos de aposentadoria ou pensão recebidos de fonte no brasil?   não. apenas o residente no brasil tem direito a essa isenção.  ',
 'não residente no brasil, ainda que se enquadre em qualquer das hipóteses de obrigatoriedade  de apresentação da declaração de ajuste anual para o residente no brasil está obrigado a apresentá- la?    pessoa física não residente no brasil não pode apresentar declaração de ajuste anual no brasil. ',
 'são tributáveis os lucros e dividendos pagos por pessoa jurídica domiciliada no brasil a sócio  não residente?   não estão sujeitos à incidência do imposto sobre a renda os lucros e dividendos calculados com base nos  resultados apurados a partir de 1º de janeiro de 1996 e pagos, creditados, entregues, empregados ou  remetidos pela pessoa jurídica a seus sócios ou acionistas domiciliados no exterior.   atenção:    observar as alterações trazidas pela medida provisória nº 627, de 11 de novembro de 2013, aos  que não optarem pelo fim do regime tributário de transição. ',
 'como devem ser tributados os rendimentos correspondentes a férias?    valor pago a título de férias, acrescido do adicional previsto no inciso  do art. 7º da constituição federal  (terço constitucional), deve ser tributado no mês de seu pagamento e em separado de qualquer outro  rendimento pago no mês.   para determinação da base de cálculo mensal podem ser efetuadas as deduções dos valores  correspondentes a pensão alimentícia, dependentes e contribuições à previdência oficial, às entidades de  previdência complementar domiciliadas no brasil e aos fundos de aposentadoria programada individual  (fapi).   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.      85   na declaração de ajuste anual, as férias são tributadas em conjunto com os demais rendimentos.   hipóteses de não tributação   em decorrência do disposto no art. 19 da lei nº 10.522, de 19 de julho de 2002, com a redação dada pelo art.  21 da lei nº 11.033, de 21 de dezembro de 2004, não são tributados pelo imposto sobre a renda na fonte,  nem na declaração de ajuste anual, os pagamentos efetuados sob as rubricas de férias não gozadas -  integrais, proporcionais ou em dobro - convertidas em pecúnia, e de adicional de um terço constitucional  quando agregado a pagamento de férias, por ocasião da rescisão do contrato de trabalho, aposentadoria, ou  exoneração, observados os termos dos atos declaratórios editados pelo procurador-geral da fazenda  nacional em relação a essas matérias, indicados abaixo.   pela mesma razão, não são tributados os pagamentos efetuados sob as rubricas de abono pecuniário relativo  à conversão de 1/3 do período de férias, de que trata o art. 143 do decreto-lei nº 5.452, de 1º de maio de  1943 – consolidação das leis do trabalho  , com a redação dada pelo decreto-lei nº 1.535, de 13 de  abril de 1977.  pessoa física que recebeu tais rendimentos com desconto do imposto sobre a renda na fonte  e que incluiu tais rendimentos na declaração de ajuste anual como tributáveis, para pleitear a restituição da  retenção indevida, deve apresentar declaração retificadora do respectivo exercício da retenção, excluindo o  valor recebido a título de abono pecuniário de férias do campo "rendimentos tributáveis" e informando-o no  campo "outros" da ficha "rendimentos isentos e não tributáveis", com especificação da natureza do  rendimento.   atenção:     procurador-geral da fazenda nacional editou atos declaratórios   declarando que,  relativamente às hipóteses neles previstas, fica autorizada a dispensa de interposição de recursos  e a desistência dos já interpostos, desde que inexista outro fundamento relevante.    secretaria especial da receita federal do brasil  , em decorrência do disposto no art. 19,  inciso , e § 4º, da lei nº 10.522, de 19 de julho de 2002, não constituirá os créditos tributários  relativos às matérias de que tratam os  do procurador-geral da fazenda nacional mencionados  abaixo.   são os seguintes os atos declaratórios relacionados a férias:     ato declaratório  nº 4, de 12 de agosto de 2002, em relação ao pagamento (in  pecunia) de férias não gozadas - por necessidade do serviço - pelo servidor público;     ato declaratório  nº 1, de 18 de fevereiro de 2005, em relação às verbas recebidas  por trabalhadores em geral a título de férias e licença-prêmio não gozadas por necessidade  do serviço;     ato declaratório  nº 5, de 16  de novembro de 2006, em relação às férias  proporcionais convertidas em pecúnia;     ato declaratório  nº 6, de 16  de novembro de 2006, em relação ao abono pecuniário  de férias de que trata o art. 143 da consolidação das leis do trabalho - , aprovada  pelo decreto-lei nº 5.452, de 1º de maio de 1943;     ato declaratório  nº 6, de 1º de dezembro de 2008, em relação ao adicional de um  terço previsto no art. 7º, inciso , da constituição federal, quando agregado a  pagamento de férias – simples ou proporcionais – vencidas e não gozadas, convertidas em  pecúnia, em razão de rescisão do contrato de trabalho;     ato declaratório  nº 14, de 1º de dezembro de 2008, em relação às férias em dobro  pagas ao empregado na rescisão contratual;     ato declaratório interpretativo  nº 5, de 27 de abril de 2005, em relação às verbas  recebidas em face da conversão em pecúnia de licença-prêmio e férias não gozadas por  necessidade do serviço, por trabalhadores em geral ou por servidores públicos; e     ato declaratório interpretativo  nº 14, de 1º de dezembro de 2005, esclarecendo que o    nº 5, de 2005, tratou somente das hipóteses de pagamento de valores a título de  férias integrais e de licença-prêmio não gozadas por necessidade do serviço quando da  aposentadoria, rescisão de contrato de trabalho ou exoneração, previstas nas súmulas nºs  125 e 136 do superior tribunal de justiça  , a trabalhadores em geral ou a servidores  públicos. ',
 'qual é o tratamento tributário da participação dos empregados nos lucros das empresas?   até 31/12/2012 a participação dos empregados nos lucros das empresas era tributada na fonte, em separado  dos demais rendimentos recebidos no mês, como antecipação do imposto sobre a renda devido na declaração  de rendimentos da pessoa física, competindo à pessoa jurídica a responsabilidade pela retenção e  recolhimento do imposto (lei nº 10.101, de 19 de dezembro de 2000).   regra vigente a partir de 1º de janeiro de 2013, diante do novo tratamento tributário introduzido pela lei nº  12.832, de 20 de junho de 2013:    participação dos trabalhadores nos lucros ou resultados da empresa é tributada pelo imposto sobre a renda  exclusivamente na fonte, em separado dos demais rendimentos recebidos, no ano do recebimento ou crédito,  com base na tabela progressiva anual abaixo indicada e não integrará a base de cálculo do imposto devido  pelo beneficiário na declaração de ajuste anual.   na determinação da base de cálculo da referida participação, poderão ser deduzidas as importâncias pagas  em dinheiro a título de pensão alimentícia em face das normas do direito de família, quando em cumprimento  de decisão judicial, de acordo homologado judicialmente ou de separação ou divórcio consensual realizado  por escritura pública, desde que correspondentes a esse rendimento, não podendo ser utilizada a mesma  parcela para a determinação da base de cálculo dos demais rendimentos.',
 'são tributáveis as importâncias recebidas por parlamentares a título de remuneração,  inclusive por motivo de convocação extraordinária da casa legislativa?   as importâncias recebidas por parlamentares a título de remuneração são tributáveis na fonte e na declaração  de ajuste anual.   entretanto, em decorrência do disposto no art. 19 da lei nº 10.522, de 19 de julho de 2002, e do ato  declaratório    nº 3, de 18 de setembro de 2008, não são tributados os pagamentos efetuados sob  as rubricas de parcela indenizatória devida aos parlamentares em face de convocação para sessão legislativa  extraordinária, observados os termos do .',
 'os rendimentos recebidos a título de bolsa por pessoa física que realiza pesquisa acadêmica  e atua como orientador de trabalhos de conclusão são tributáveis?   sim. os valores recebidos para proceder a estudos ou pesquisas que importem em contraprestação de  serviços, ou que, de alguma forma, representem vantagem para o doador em função dos resultados obtidos  na pesquisa, são considerados rendimentos tributáveis e estão sujeitos à retenção de imposto na fonte e ao  ajuste anual, ainda que o concedente desenvolva atividades sem fins lucrativos.',
 'qual é o tratamento aplicável às importâncias recebidas a título de residência médica ou por  estágio remunerado em hospitais, laboratórios, centro de pesquisa e universidades, para  complementação de estudo ou treinamento e aperfeiçoamento?   essas importâncias são consideradas rendimentos do trabalho, ainda que não haja vínculo empregatício e  obrigatoriedade de desconto para o , devendo compor a base de cálculo na apuração da renda mensal  sujeita à retenção na fonte e ao ajuste anual.    no caso específico das bolsas de estudo recebidas pelos médicos-residentes, em função do disposto no art.  26 da lei nº 9.250, de 26 de dezembro de 1995 (a seguir transcrito), há previsão de isenção do imposto sobre  a renda, desde que atendidas às condições impostas nos citados dispositivos legais:   “art. 26. ficam isentas do imposto de renda as bolsas de estudo e de pesquisa caracterizadas como  doação, quando recebidas exclusivamente para proceder a estudos ou pesquisas e desde que os  resultados dessas atividades não representem vantagem para o doador, nem importem  contraprestação de serviços.   parágrafo único.  não caracterizam contraprestação de serviços nem vantagem para o doador, para   efeito da isenção referida no caput, as bolsas de estudo recebidas pelos médicos-residentes, nem as   bolsas recebidas pelos servidores das redes públicas de educação profissional, científica e   tecnológica que participem das atividades do pronatec, nos termos do § 1º do art. 9º da lei nº 12.513,   de 26 de outubro de 2011”.',
 'como são tributados os rendimentos de sócios ou titular de empresa optante pelo regime  especial unificado de arrecadação de tributos contribuições devidos pelas microempresas e  empresas de pequeno porte - simples nacional?   são considerados isentos do imposto sobre a renda, na fonte e na declaração de ajuste do beneficiário, os  valores efetivamente pagos ou distribuídos ao titular ou sócio da microempresa ou empresa de pequeno porte  optante pelo simples nacional, salvo os que corresponderem a pro labore, aluguéis ou serviços prestados.    isenção fica limitada ao valor resultante da aplicação dos percentuais de presunção, de que trata o art. 15  da lei nº 9.249, de 26 de dezembro de 1995, do imposto sobre a renda das pessoas jurídicas   sobre  a receita bruta mensal, no caso de antecipação de fonte, ou da receita bruta total anual, tratando-se de  declaração de ajuste, subtraído do valor devido na forma do simples nacional no período, relativo ao .    limite não se aplica na hipótese de a microempresa ou empresa de pequeno porte manter escrituração  contábil e evidenciar lucro superior àquele limite.   exemplo:    empresa irmãos santiago ltda, exerce a atividade de comércio de calçados e possui como sócios josé  santiago e maria santiago (cada um possui 50% das cotas).       88   esteve em todo o ano-calendário de 2019 submetida ao regime do simples nacional na condição de  microempresa. enquanto empresa comercial, fica sujeita ao anexo  da resolução  nº 140, de 22 de  maio de 2018, abrangendo os seguintes tributos: ,  cofins, /pasep,  e .    microempresa obteve receita bruta anual de $ 80.000,00. ficou sujeita, portanto, à alíquota de 4%. sendo  que desses 4%, conforme anexo  da resolução citada anteriormente, 5,5% correspondem ao .   considere que teve $ 20.000,00 de despesas, aí incluído o valor anual do pro labore no valor de  $ 12.000,00 para maria (josé não retirou pro labore).   seu lucro, portanto, foi de $ 60.000,00. considerando que não possui escrituração contábil, será necessário  calcular qual o seu lucro que pode ser distribuído de forma isenta do imposto sobre a renda:   lucro passível de distribuição isenta = [(receita bruta anual da atividade x percentual de presunção do lucro  presumido) – ( no simples nacional) = [($ 80.000,00 x 8%) – (4% x 5,5% x $ 80.000,00)] =  [$ 6.400,00 – $ 176,00] = $ 6.224,00.   assim sendo, caso maria apresente declaração de ajuste anual (só por esses rendimentos não está  obrigada), deverá informar o valor de $ 6.224,00 x 50% (das cotas) = $ 3.112,00 no campo “tipo de  rendimento” da ficha “rendimentos isentos e não tributáveis”, no código “09 – lucros e dividendos  recebidos”. josé deve proceder da mesma maneira.   caso os sócios optem por distribuir também a parcela não isenta, ou seja, $ 60.000,00 – $ 6.224,00 =  $ 53.776,00, como não possui escrituração contábil, esse valor não é isento e deve ser informado na ficha  “rendimentos tributáveis recebidos de  pelo titular” (50% para cada um dos sócios).   em relação aos pro labores de maria cuja soma no ano totalizou o valor de $ 12.000,00, também deve ser  informado na ficha “rendimentos tributáveis recebidos de  pelo titular”. ',
 'como são tributados os rendimentos de titular de empresa optante pelo regime especial  unificado de arrecadação de tributos contribuições devidos pelas microempresas e empresas de  pequeno porte - simples nacional, na condição de microempreendedor individual  ?   considera-se isento do imposto sobre a renda, na fonte e na declaração de ajuste do beneficiário, o lucro do  titular de empresa optante pelo regime especial unificado de arrecadação de tributos e contribuições  devidos pelas microempresas e empresas de pequeno porte (simples nacional), na condição de  microempreendedor individual  .    isenção fica limitada ao valor resultante da aplicação, sobre a receita bruta mensal, no caso de antecipação  de fonte, ou da receita bruta total anual, tratando-se de declaração de ajuste anual, dos percentuais de  apuração do lucro presumido, mencionados no artigo 15, da lei nº 9.249, de 26 de dezembro de 1995.    limite acima não se aplica na hipótese de o microempreendedor individual manter escrituração contábil que  evidencie lucro superior àquele limite.   não são considerados isentos os valores pagos ao  correspondentes a pro labore, aluguéis ou serviços  prestados.   exemplo:    contribuinte joão pedro esteve em todo o ano-calendário de 2019 submetido ao regime do simples nacional  na condição de . exerceu atividade comercial, ficando, portanto, sujeito somente ao recolhimento mensal  de  e de contribuição para a seguridade social relativa à pessoa do empresário, na qualidade de  contribuinte individual.   suponha que tenha obtido $ 80.000,00 de receita de suas atividades. considere que teve $ 20.000,00 de  despesas, aí incluído o valor anual de seus pro labores no valor de $ 12.000,00.    seu lucro, portanto, foi de $ 60.000,00. considerando que não possui escrituração contábil, será necessário  calcular qual o seu lucro que pode ser distribuído de forma isenta do imposto sobre a renda:   lucro passível de distribuição isenta = receita bruta anual da atividade x percentual de presunção do lucro  presumido (lei nº 9.249, de 26 de dezembro de 1995, art. 15, caput) = $ 80.000,00 x 8% = $ 6.400,00.   assim sendo, caso apresente declaração de ajuste anual (só por esses rendimentos não está obrigado),  deverá informar o valor de $ 6.400,00 no campo “tipo de rendimento” da ficha “rendimentos isentos e não  tributáveis”, no código “09 – lucros e dividendos recebidos”.      89   caso opte por distribuir também a parcela não isenta, ou seja, $ 60.000,00 – $ 6.400,00 = $ 53.600,00,  como não possui escrituração contábil, esse valor não é isento e deve ser informado na ficha “rendimentos  tributáveis recebidos de  pelo titular”.   em relação ao pro labore cuja soma no ano totalizou o valor de $ 12.000,00, também deve ser informado na  ficha “rendimentos tributáveis recebidos de  pelo titular”. ',
 'qual é o tratamento tributário dos benefícios indiretos concedidos pela empresa a  administradores, diretores, gerentes e assessores?   são computados, para fins de apuração do imposto sobre a renda na fonte, todos os pagamentos efetuados  em caráter de remuneração, inclusive as despesas de representação e os benefícios e vantagens concedidos  pela empresa a título de salários indiretos, tais como despesas de supermercado e cartões de crédito,  pagamento de anuidades escolares, clubes, associações etc.    integram ainda a remuneração desses beneficiários, como salário indireto, as despesas pagas ou incorridas  com o aluguel de imóveis e com os veículos utilizados para o seu transporte, quando de uso particular,  computando-se, também, a manutenção, conservação, consumo de combustíveis, encargos de depreciação  e respectiva correção monetária, valor do aluguel ou do arrendamento dos veículos.   atenção:    se o beneficiário não for identificado, a tributação é definitiva e à alíquota de 35% sobre o  rendimento reajustado. ',
 'qual é o tratamento a ser dado às parcelas relativas a horas extras pagas em virtude de acordo  coletivo mediado por sindicato?   pagamentos de horas extras, mesmo que realizados em virtude de acordo coletivo mediado por sindicato,  configuram-se como rendimentos do trabalho, sujeitando-se à retenção na fonte e à tributação na declaração  de ajuste anual. ',
 'como são tributados os valores pagos pelas entidades de previdência complementar aos  participantes de planos de benefícios?   no caso de contribuintes não optantes pelo regime de tributação de que trata o art. 1º da lei nº 11.053,  de 2004:   os benefícios pagos por essas entidades sujeitam-se à incidência do imposto sobre a renda na fonte,  aplicando a tabela mensal, e na declaração de ajuste anual.   os resgates de contribuições, parciais ou totais, em virtude de desligamento do participante do plano de  benefícios da entidade, sujeitam-se à incidência de imposto sobre a renda na fonte à alíquota de 15%,  calculado sobre os valores de resgate, no caso de planos de previdência, inclusive fapi, e na declaração de  ajuste anual, com exceção do resgate de recursos efetuado em plano estruturado na modalidade de benefício  definido, que permanece submetido à tributação com base na tabela progressiva mensal e na declaração de  ajuste anual.      90   na hipótese de optantes pelo regime de tributação de que trata o art. 1º da lei nº 11.053, de 2004:    pagamento de valores a título de benefício ou resgate de contribuições aos participantes ou assistidos por  planos de caráter previdenciário, estruturados nas modalidades de contribuição definida ou contribuição  variável e mantidos por entidade de previdência complementar, sociedade seguradora ou fundo de  aposentadoria programada individual (fapi), estão sujeitos à tributação exclusiva na fonte, com as seguintes  alíquotas:    - 35%, para recursos com prazo de acumulação inferior ou igual a 2 anos;   - 30%, para recursos com prazo de acumulação superior a 2 anos e inferior ou igual a 4 anos;   - 25%, para recursos com prazo de acumulação superior a 4 anos e inferior ou igual a 6 anos;   - 20%, para recursos com prazo de acumulação superior a 6 anos e inferior ou igual a 8 anos;   - 15%, para recursos com prazo de acumulação superior a 8 anos e inferior ou igual a 10 anos; e   - 10%, para recursos com prazo de acumulação superior a 10 anos.   tratamento tributário aplicável ao plano gerador de benefício livre  , ao fundo de aposentadoria  programada individual (fapi) e ao vida gerador de benefício livre – , na declaração de ajuste anual:   a) no plano gerador de benefício livre   e no fundo de aposentadoria programada individual (fapi),  planos de caráter previdenciário, o valor das contribuições são dedutíveis na declaração de ajuste anual,  limitado a 12% do rendimento tributável incluído na base de cálculo do imposto sobre a renda na declaração.  quando do pagamento/benefício ou crédito, tributa-se a totalidade do rendimento, sendo adotado o regime de  tributação, conforme a opção do contribuinte. informar na ficha “pagamentos efetuados”, de acordo com a  natureza do tipo de previdência complementar, no código 36 - previdência complementar, no código 37 –  contribuições para as entidades de previdência complementar fechadas de natureza pública, ou no código 38  – fapi – fundo de aposentadoria programada individual, o valor das contribuições pagas no ano-calendário;   b) no vida gerador de benefício livre  , plano de seguro de vida com cláusula de cobertura por  sobrevivência, o valor das contribuições não é dedutível na declaração de ajuste anual. quando do  recebimento, tributa-se a diferença entre o valor recebido e o valor aplicado, sendo adotado o regime de  tributação, conforme a opção do contribuinte. informar na ficha “bens e direitos” no código 97 –  – vida  gerador de benefício livre, a discriminação do  contratado e os saldos acumulados referentes aos  valores históricos dos prêmios de  em 31 de dezembro do ano-calendário anterior e em 31 de dezembro  do ano-calendário, independentemente do valor atual (com correção).     cabe esclarecer que as importâncias pagas pelas entidades fechadas de previdência complementar a título  de reversão de valores não se enquadram no conceito de benefício previdenciário ou resgate, sujeitando-se  à incidência do imposto sobre a renda na fonte e na declaração de ajuste anual. consulte a pergunta 285.   atenção:     instrução normativa  nº 1.343, de 5 de abril de 2013, estabelece normas e procedimentos  relativos ao tratamento tributário a ser aplicado na apuração do imposto sobre a renda da pessoa  física   sobre os valores pagos ou creditados por entidade de previdência complementar a  título de complementação de aposentadoria, resgate e rateio de patrimônio em caso de extinção da  entidade de previdência complementar, correspondentes às contribuições efetuadas  exclusivamente pelo beneficiário no período de 1º de janeiro de 1989 a 31 de dezembro de 1995.   para os beneficiários que se aposentarem a partir de 1º de janeiro de 2013, a entidade de  previdência complementar (fonte pagadora) fica desobrigada da retenção do imposto na fonte  relativamente à complementação de aposentadoria recebida de entidade de previdência  complementar, inclusive a relativa ao abono anual pago a título de décimo terceiro salário, no limite  que corresponda aos valores das contribuições efetuadas exclusivamente pelo beneficiário no  período de 1º de janeiro de 1989 a 31 de dezembro de 1995.   os valores das contribuições, nas hipóteses descritas, devem ser abatidos da complementação de  aposentadoria recebida de previdência complementar, mês a mês, até se exaurirem.    fonte pagadora deverá fornecer ao beneficiário comprovante de rendimentos, com a informação  dos referidos valores abatidos, no quadro correspondente aos rendimentos isentos e não  tributáveis.   para os beneficiários que se aposentaram entre os anos de 2008 e 2012, consultar disciplinamento  contido na referida instrução normativa  nº 1.343, de 5 de abril de 2013, disponível no sítio da  secretaria especial da receita federal do brasil   na internet, no endereço  http://receita.economia.gov.br.   ',
 'qual é o tratamento tributário das importâncias pagas a título de seguros aos beneficiários  de participantes de planos previdenciários pelas entidades de previdência complementar?   são isentos do imposto sobre a renda os seguros recebidos de entidade de previdência complementar  decorrente de morte ou invalidez permanente do participante.  palavra “seguros” tem o significado de pecúlio  recebido de uma só vez.   entende-se por pecúlio, apenas, o benefício pago em parcela única, por entidade de previdência  complementar, em virtude de morte ou invalidez permanente do participante de plano de previdência, assim  entendido como benefício de risco, com característica de seguro, previsto expressamente no plano de  benefício contratado.    importância paga em prestação única, em razão de morte ou invalidez permanente do participante,  correspondente à reversão (devolução) de contribuições efetuadas ao plano, acrescida ou não de rendimentos  financeiros, não caracteriza pagamento de pecúlio (seguro), sendo portanto, tributável na fonte, como  antecipação do imposto devido na declaração de ajuste anual da pessoa física, ou de forma exclusiva, nos  casos em que houve opção por aquele regime de tributação.   atenção:    pecúlio não se confunde com resgate de contribuições. as importâncias pagas a entidades de  previdência complementar a título de pecúlio (seguro) não são dedutíveis para fins de apuração do  imposto sobre a renda devido na declaração de ajuste anual da pessoa física.',
 'é tributável a pensão especial paga a ex-combatente?   é tributável a pensão especial concedida a ex-combatente ou a seus dependentes, por força da lei nº 8.059,  de 4 de julho de 1990.   somente as pensões e os proventos concedidos aos ex-combatentes da força expedicionária brasileira    e seus herdeiros, de acordo com o decreto-lei nº 8.794, de 23 de janeiro de 1946, e decreto-lei nº  8.795, de 23 de janeiro de 1946, lei nº 2.579, de 23 de agosto de 1955, e lei nº 4.242, de 1963, art. 30  (mantido pelo art. 17 da lei nº 8.059, de 4 de julho de 1990), são isentos do imposto sobre a renda. essa  isenção não substitui nem impede o contribuinte maior de 65 anos de usufruir a parcela de isenção sobre  rendimentos recebidos de outra fonte pagadora a título de aposentadoria ou pensão.',
 'é tributável a pensão especial concedida à viúva ou ao dependente de funcionário civil ou  militar que faleceu por motivo de doença grave?   sim. os rendimentos decorrentes de pensão especial estabelecida pela lei nº 3.738, de 4 de abril de 1960,  assegurada à viúva ou a dependentes de funcionário civil ou militar falecido em decorrência de doença grave,  caracterizam-se como rendimentos tributáveis e devem compor a base de cálculo para apurar a renda mensal  sujeita à incidência na fonte e na declaração de ajuste, pelo seu total. síndico  condomínio',
 'são tributáveis os rendimentos auferidos pelo síndico de condomínio?   sim. esses rendimentos são considerados prestação de serviços e devem compor a base de cálculo para  apuração do recolhimento mensal obrigatório (carnê-leão) e do ajuste anual, ainda que havidos como dispensa  do pagamento do condomínio. ',
 'como devem ser tributados os rendimentos oriundos da prestação de serviços efetuados  com a utilização de veículos, abrangendo transporte de passageiros e de cargas (inclusive por  aplicativos)?   esses rendimentos, bem como aqueles referentes a fretes e carretos, aos prestados com tratores, máquinas  de terraplenagem, colheitadeiras e semelhantes, barcos, chatas, carros, camionetas, caminhões, aviões etc.,  podem ser considerados como de pessoa física ou jurídica.   são considerados rendimentos de pessoa física se observadas, cumulativamente, as condições descritas  abaixo (caso contrário, são considerados rendimentos de pessoa jurídica):   a) se executados apenas pelo locatário ou proprietário do veículo (ainda que este tenha sido adquirido com  reserva de domínio ou esteja sob alienação fiduciária);   b) se para auxiliá-lo na execução do serviço for necessária a participação remunerada de outras pessoas, com  ou sem vínculo empregatício, estas não podem ser profissionais qualificados, mas sim meros auxiliares ou  ajudantes;   c) se o veículo for de propriedade ou estiver na posse de duas ou mais pessoas, estas não podem explorar o  serviço em conjunto, por meio de sociedade regular ou não;   d) se houver a posse ou a propriedade de dois ou mais veículos, estes não podem ser utilizados ao mesmo  tempo na prestação de um determinado serviço.   por força das disposições da lei nº 7.713, de 22 de dezembro de 1988, tais rendimentos sujeitam-se ao  recolhimento mensal (carnê-leão), se recebidos de pessoa física ou, na fonte, se pagos por pessoa jurídica,  devendo, na segunda hipótese, a fonte pagadora fornecer ao beneficiário documento autenticado  comprobatório da retenção na fonte efetuada.  rendimento bruto dessas atividades é o correspondente a, no  mínimo, 10% do valor total dos fretes e carretos recebidos, bem como da prestação de serviços com tratores,  máquinas de terraplenagem, colheitadeiras e assemelhados, ou, no mínimo, 60% no caso de transporte de  passageiros.    valor relativo a 90% dos fretes e 40% do transporte de passageiros deve ser informado na linha específica  da ficha rendimentos isentos e não tributáveis. esses valores não justificam acréscimo patrimonial, e a  pessoa física que desejar fazê-lo, deve incluir como tributável na declaração de ajuste e no recolhimento do  carnê-leão percentual superior aos referidos acima. ',
 'como devem ser tributados os rendimentos percebidos por transportador autônomo  residente na republica do paraguai?   os valores pagos, creditados, entregues, empregados ou remetidos por contratante pessoa jurídica  domiciliada no país, autorizada a operar transporte rodoviário internacional de carga, a beneficiário  transportador autônomo pessoa física residente na república do paraguai, considerado como sociedade  unipessoal naquele país, quando decorrentes da prestação de serviços de transporte rodoviário internacional  de carga, estão sujeitos à incidência do imposto sobre a renda na fonte, calculado mediante a utilização da  tabela progressiva mensal.    imposto incidirá sobre 10% (dez por cento) do rendimento bruto decorrente do transporte rodoviário  internacional de carga e deve ser retido na fonte por ocasião de cada pagamento, crédito, entrega, emprego      93   ou remessa, aplicando-se, se houver mais de um desses eventos efetuados pela mesma fonte pagadora no  mês de apuração, a alíquota correspondente à base de cálculo apurada após a soma dos rendimentos,  compensando-se o imposto retido anteriormente.    imposto sobre a renda apurado deve ser recolhido até o último dia útil do primeiro decêndio do mês  subsequente ao da ocorrência dos fatos geradores, mediante a utilização do código de receita 0610. ',
 'qual é o tratamento tributário dos rendimentos recebidos por representante comercial  autônomo?   os rendimentos recebidos por representante comercial autônomo que exerce exclusivamente a mediação  para a realização de negócios mercantis, nos termos do art. 1º da lei nº 4.886, de 9 de dezembro de 1965,  quando praticada por conta de terceiros, são tributados na pessoa física. é irrelevante, para os efeitos do  imposto sobre a renda, a existência de registro, como firma individual, na junta comercial e no cadastro  nacional da pessoa jurídica  .   alerte-se que, no caso de o representante comercial executar os negócios mercantis por conta própria, ele  adquire a condição de comerciante, independentemente de qualquer requisito formal, ocorrendo neste caso,  para efeitos tributários, equiparação da empresa individual a pessoa jurídica, por força do disposto no art. 162  do regulamento do imposto sobre a renda – /2018, aprovado pelo decreto nº 9.580, de 22 de novembro  de 2018, sendo seus rendimentos tributados nessa condição. ',
 'qual é o tratamento tributário dos rendimentos recebidos por pessoa física autora de obras  de arte, tais como escultura, pintura?   os rendimentos obtidos por pessoa física com a criação de objeto artístico configuram-se como rendimentos  do trabalho. não descaracteriza esse entendimento a utilização de mão de obra de terceiros para tarefas  auxiliares. as despesas podem ser deduzidas desde que decorrentes do exercício da atividade e devidamente  registradas em livro-caixa.   no caso de obra adquirida por pessoa jurídica, esta deve efetuar retenção na fonte, por ocasião do crédito ou  pagamento, na forma do art. 7º da lei nº 7.713, de 22 de dezembro de 1988. quando adquirida por pessoa física, o contribuinte sujeita-se ao recolhimento mensal obrigatório (carnê-leão),  na forma do art. 8º da lei nº 7.713, de 22 de dezembro de 1988, c/c o art. 6º da lei nº 8.134, de 27 de  dezembro de 1990, e à declaração de ajuste.',
 'contribuinte que recebe da fonte pagadora prêmios em bens, a título de produtividade,  promoção de vendas, eficiência, não ocorrência de acidentes etc., deve oferecer à tributação o valor  correspondente?    valor do prêmio em bens ou direitos avaliados em dinheiro na data de sua percepção assume o aspecto de  remuneração do trabalho assalariado ou não assalariado, conforme haja ou não vínculo empregatício entre a  pessoa física e a fonte pagadora.  valor de tal prêmio sujeita-se à tributação no carnê-leão e na declaração  de ajuste anual, se recebido de pessoa física sem vínculo empregatício ou, na fonte e na declaração de ajuste,  se distribuído por pessoa jurídica ou empregador pessoa física. ',
 'os prêmios obtidos em concursos e competições artísticas, científicas, desportivas e  literárias são tributáveis?   sim. na hipótese da ocorrência de concursos artísticos, desportivos, científicos, literários ou a outros títulos  assemelhados, com distribuição de prêmios efetuada por pessoa jurídica a pessoa física, quando houver  vinculação quanto à avaliação do desempenho dos participantes, hipótese na qual tais prêmios assumem o  aspecto de remuneração do trabalho, independentemente se distribuídos em dinheiro ou sob a forma de bens  e serviços, o imposto sobre a renda incide na fonte, calculado de acordo com a tabela progressiva mensal, a  título de antecipação do devido na declaração de ajuste anual.   no caso de não ocorrência da vinculação quanto à avaliação do desempenho dos participantes.',
 'os valores das gratificações, prêmios, participações etc., pagos a atleta profissional, em  decorrência dos resultados obtidos em competições esportivas, são tributáveis?   sim. os valores pagos pelo empregador a título de luvas, prêmios, bichos, direito de arena, publicidade em  camisas etc., como retribuição pelo contrato de serviços profissionais, por vitórias, empates, títulos e troféus  conquistados, possuem caráter remuneratório e, como tal, são considerados rendimentos do trabalho  assalariado devendo compor, juntamente com os salários pagos ou creditados em cada mês, a base de  cálculo para apurar a renda mensal sujeita à incidência na fonte e na declaração de ajuste.',
 'como deve proceder a fonte pagadora que assume o ônus do imposto sobre a renda devido  pelo beneficiário?    valor do rendimento deve ser reajustado mediante a seguinte fórmula:     =         -                1 - ( / 100)   sendo:    = rendimento reajustado;    = rendimento pago (corresponde à base de cálculo antes do reajustamento);    = parcela a deduzir da classe de rendimentos a que pertence o , observados os itens do atenção;    = alíquota da classe de rendimentos a que pertence o , observados os itens do atenção.   exemplo:    lúcia emília, residente em portugal, possui imóvel no brasil alugado por $ 3.000,00 mensais. seu  procurador, josé alexandre, é quem recebe o aluguel e o remete para ela em portugal.   josé alexandre deveria ter efetuado a retenção na fonte do imposto sobre a renda referente ao rendimento  de aluguel do mês de março de 2019 no valor de $ 3.000,00 (deveria ter gerado um carnê-leão no valor de  $ 95,20).   josé alexandre não efetuou a retenção devida, e os $ 3.000,00 enviados a lúcia emília são considerados  rendimentos líquidos do imposto, devendo ser feito, portanto, o reajustamento.   observe que como se trata de rendimentos de aluguel auferidos por residente no exterior, a alíquota é fixa e  igual a 15% (ver item 1 do atenção).   nesse caso, o rendimento reajustado será:    = ( – ) / [1 – (/100)]    = (3.000 – 0) / [1 – (15/100)] = 3000 / 0,85 = $ 3.529,41      95   atenção:    1 - se a alíquota aplicável for fixa, o valor da parcela a deduzir é zero e  é a própria alíquota;   2 - se alíquota aplicável integrar a tabela progressiva, observar se o  obtido pertence ou não à  classe de renda do . se  pertencer à classe de renda seguinte, o cálculo deve ser refeito,  utilizando-se a dedução e a alíquota da classe a que pertencer o  apurado;   3 - o valor reajustado deve ser informado no comprovante de rendimentos e na declaração de  imposto sobre a renda retido na fonte (dirf). ',
 'quitação de honorários ou rendimentos do trabalho assalariado em notas promissórias é  tributável?   sim.  nota promissória é um título de crédito que se basta a si mesmo, ou seja, tem característica de  independência, não se ligando ao ato originário de onde proveio.   assim, a quitação de direitos mediante recebimento em notas promissórias ou título de crédito caracteriza a  disponibilidade jurídica referida no § 4º do art. 3º da lei nº 7.713, de 22 de dezembro de 1988, devendo o valor  a elas correspondente ser tributado no mês do recebimento do respectivo título e na declaração de ajuste.',
 'valor da dívida perdoada em troca de serviços prestados é tributável?   sim.  importância com que for beneficiado o devedor nos casos de perdão ou cancelamento de dívida,  inclusive correção monetária e juros vencidos, se houver, em troca de serviços prestados, bem como qualquer  hipótese que resulte em acréscimo patrimonial, é rendimento sujeito à tributação no mês em que os serviços  forem prestados e na declaração de ajuste.   observe-se que se o perdão ou cancelamento da dívida não corresponder à contraprestação de serviços ao  credor, tal valor será considerado rendimento isento.   ',
 'qual é o tratamento tributário aplicável no caso de pagamento efetuado em bens?    pagamento efetuado em bens móveis ou imóveis, títulos de crédito ou valores mobiliários etc., é tributável,  devendo os bens serem avaliados em dinheiro pelo valor que tiverem na data de sua percepção, sujeitando- se ao recolhimento mensal (carnê-leão), se recebido de pessoa física sem vínculo empregatício ou, na fonte,  se pago por pessoa jurídica, ou pessoa física com vínculo empregatício e na declaração de ajuste.',
 'quando devem ser tributados os rendimentos (comissões, honorários, salários etc.)  correspondentes a um mês e recebidos no mês seguinte?   os rendimentos são tributados no mês em que forem recebidos, considerado como tal o da entrega de  recursos pela fonte pagadora, mesmo mediante depósito em instituição financeira em favor do beneficiário.   no caso de comissões recebidas por trabalhador assalariado, essas devem ser somadas ao salário do mês  do recebimento.  ',
 'valor recebido em virtude de acidente de trabalho é tributável?    indenização e os proventos de aposentadoria ou reforma recebidos em decorrência de acidente de trabalho  são isentos.   atenção:     pensão paga aos dependentes em decorrência do falecimento da pessoa acidentada é tributável. ',
 'qual é o tratamento tributário dos rendimentos recebidos a título de vantagem pecuniária  individual, instituída pela lei nº 10.698, de 2 de julho de 2003, e de abono de permanência?   são tributados como rendimentos do trabalho com vínculo empregatício, na fonte e na declaração de ajuste  anual, os rendimentos recebidos a título de vantagem pecuniária individual, instituída pela lei nº 10.698, de  2 de julho de 2003, e de abono de permanência, a que se referem o § 19 do art. 40 da constituição federal,  o § 5º do art. 2º e o § 1º do art. 3º da emenda constitucional nº 41, de 19 de dezembro de 2003, e o art. 7º da  lei nº 10.887, de 18 de junho de 2004. ',
 'qual é o tratamento tributário do valor da restituição da contribuição previdenciária retida ou  recolhida indevidamente?    valor da restituição de contribuição previdenciária, incluindo a incidente sobre o 13º salário, que fora utilizado  como dedução da base de cálculo do imposto sobre a renda da pessoa física, deverá ser oferecido à  tributação do imposto na declaração de ajuste anual relativa ao ano-calendário do recebimento.   consideram-se sujeitos à tributação do imposto sobre a renda, também, os juros de mora decorrentes da  restituição da contribuição previdenciária, exceto se estes corresponderem a rendimentos isentos ou não  tributáveis. ',
 'qual o tratamento tributário dos rendimentos de aluguel de imóvel localizado no brasil  recebido por residente no brasil?   as quantias recebidas por pessoa física pela locação de imóvel, sujeitam-se ao recolhimento mensal  obrigatório (carnê-leão) se recebidas de pessoa física ou de fonte no exterior, ou à retenção na fonte se pagas  por pessoa jurídica, e ao ajuste na delaração de ajuste anual.   exemplo 1:   alexandre aluga loja localizada no bairro do catete, no rio de janeiro, para a empresa biscoitos sublimes  ltda.  valor mensal do aluguel é de $ 6.000,00.   no dia 05.10.2019 a empresa comercial efetuou o pagamento do aluguel referente ao mês de setembro de  2019 para a imobiliária contratada por alexandre, que cobra uma taxa de administração de 10% ($ 600,00).    pagamento, entretanto, foi somente no valor de $ 5.219,36, pois a empresa efetuou, também, o pagamento  de um darf (no  da empresa) no valor de $ 780,64 referentes à retenção na fonte (pagamento de  aluguel feito por pessoa jurídica).   alexandre receberá da imobiliária, portanto, o valor de $ 4.619,36 ($ 6.000,00 – 780,64 – $ 600,00).   alexandre deverá, na declaração de ajuste anual   informar os valores recebidos de aluguel na ficha  “rendimenos tributáveis recebidos de pessoa jurídica”. na ocasião do preenchimento da , alexandre  poderá descontar do valor dos aluguéis recebidos, a comissão paga à imobiliária. esse valor pago deverá ser  informado na ficha “pagamentos efetuados”, no código “71 – administrador de imóveis”.   exemplo 2:   telma aluga apartamento localizado no bairro da tijuca, no rio de janeiro, para élder.  valor mensal do  aluguel é de $ 3.000,00.   no dia 17.10.2019 élder efetuou o pagamento do aluguel referente a setembo de 2019 para a imobiliária  contratada por telma, que cobra taxa de administração de 10%. como o pagamento foi efetuado com atraso,  houve o pagamento de uma multa de 5%.  total pago por élder foi de $ 3.150,00.   telma deverá gerar o darf relativo ao carnê-leão, uma vez que se trata de aluguel pago por pessoa física.    considerando o valor recebido de $ 3.150,00 (aluguel + multa) e descontando-se a taxa de administração  de $ 315,00, o valor que servirá de base de cálculo para o imposto sobre a renda será de $ 2.835,00.   valor do darf, que pode ser gerado por meio do programa carnê-leão 2019, será de $ 70,45.   telma deverá, na declaração de ajuste anual, informar os valores recebidos de aluguel (acrescidos de multa)  na ficha “rendimentos tributáveis recebidos de pessoa física e do exterior pelo titular”, na aba “outras  informações”, na coluna relativa a “aluguéis”. os valores a serem informados podem ser os valores líquidos,  ou seja, já descontados os valores pagos de comissão à imobiliária. esses valores pagos à imobiliária,  entretanto, deverão ser informados na ficha “pagamentos efetuados”, no código “71 – administrador de  imóveis”.   atenção:    para determinação da base de cálculo sujeita ao recolhimento mensal obrigatório (carnê-leão), no  caso de rendimentos de aluguéis de imóveis pagos por pessoa física, e no caso de aluguéis de  imóveis pagos por pessoa jurídica, não integrarão a base de cálculo para efeito de incidência do  imposto sobre a renda:   1)  valor dos impostos, taxas e emolumentos incidentes sobre o bem que produzir o rendimento;  2)  aluguel pago pela locação do imóvel sublocado;  3) as despesas pagas para sua cobrança ou recebimento; e  4) as despesas de condomínio.   os encargos citados acima somente poderão reduzir o valor do aluguel quando o ônus tenha sido  do locador.   compõem a base de cálculo os juros de mora, atualização monetária, multas por rescisão de  contrato de locação, indenização por rescisão antecipada ou término do contrato e quaisquer  acréscimos ou compensações pelo atraso no pagamento do aluguel, bem como as benfeitorias  realizadas no imóvel pelo locatário não reembolsadas pelo locador e as luvas pagas ao locador,  ainda que cedido o direito de exploração.   no caso de rendimentos de aluguéis de imóveis recebidos por residentes ou domiciliados no  exterior, compete ao procurador a retenção do imposto mediante aplicação da alíquota de 15%. ',
 'qual é o tratamento tributário dos rendimentos de aluguel de imóvel localizado no brasil  recebidos por não residente no brasil?   preliminarmente, deve-se verificar se há acordo ou tratado entre o brasil e o país de origem do residente no  exterior. existindo tais instrumentos, o tratamento fiscal será aquele neles previsto. não havendo acordo o  rendimento é tributado exclusivamente na fonte à alíquota de 15%.   atenção:     imposto deve ser recolhido na data da ocorrência do fato gerador, sendo responsável pelo  recolhimento o procurador do residente no exterior.  procurador deve efetuar o recolhimento de  darf, com código de receita 9478, em seu próprio , posteriormente, na dirf, informará o  beneficiário dos respectivos rendimentos.  ',
 'quando ocorre o fato gerador no caso de o aluguel de imóvel ser pago ao locador em mês  subsequente ao do recebimento pela imobiliária?    fato gerador ocorre no mês em que o locatário efetuar o pagamento do aluguel à imobiliária,  independentemente de quando o mesmo tenha sido repassado para o locador.',
 'pai deu ao filho o usufruto de rendimentos de aluguel de imóvel. como tributar esses  rendimentos?   se o usufruto constar de escritura pública averbada no registro de imóveis, o pai, ao relacionar o imóvel em  sua declaração de bens e direitos, informa a constituição do usufruto em favor do filho. os rendimentos do  aluguel são tributáveis em nome do filho.   se não houver escritura averbada, o pai, ao relacionar o imóvel em sua declaração de bens e direitos, informa  que os rendimentos respectivos foram doados ao filho. os rendimentos do aluguel estão sujeitos ao  recolhimento mensal (carnê-leão), se recebidos de pessoa física ou, na fonte, se pagos por pessoa jurídica e  devem ser incluídos, como rendimentos tributáveis, na declaração de ajuste do pai. para o filho, os  rendimentos são não tributáveis, como doação em espécie. ',
 'como tratar os rendimentos produzidos por imóvel cujo direito de exploração tenha sido  cedido, por meio de contrato, a terceiros?   esses rendimentos são tributáveis em nome de quem explora o imóvel, ou seja, o cessionário ou arrendatário.   por sua vez, o proprietário do imóvel deve tributar o valor recebido pela cessão de direitos, como rendimentos  equiparados a aluguéis, por meio do recolhimento mensal (carnê-leão), se recebidos de pessoa física ou, na  fonte, se pagos por pessoa jurídica e na declaração de ajuste. ',
 'como deve ser tributada a quantia recebida por locação de espaço físico em imóveis ou  condomínios edilícios?   as quantias recebidas por pessoa física pela locação de espaço físico sujeitam-se ao recolhimento mensal  obrigatório (carnê-leão) se recebidas de pessoa física ou de fonte no exterior, ou à retenção na fonte se pagas  por pessoa jurídica, e ao ajuste na declaração de ajuste anual.   ressalte-se que, diante da inexistência de personalidade jurídica do condomínio edilício, as receitas de  locação por este auferidas, na realidade, constituem-se em rendimentos dos próprios condôminos, devendo  ser tributados por cada condômino, na proporção do quinhão que lhe for atribuído, na forma explicada no  primeiro parágrafo. ainda que os condôminos não tenham recebido os pagamentos em espécie, são eles os  beneficiários dessa quantia, observando-se isso, por exemplo, quando o valor recebido se incorpora ao fundo  para o qual contribuem, ou quando diminui o montante do condomínio cobrado, ou, ainda, quando utilizado  para qualquer outro fim.   desde 14 de maio de 2014, são isentos do imposto sobre a renda das pessoas físicas os rendimentos  recebidos pelos condomínios residenciais constituídos nos termos da lei nº 4.591, de 16 de dezembro de  1964, observado o limite de $ 24.000,00 (vinte e quatro mil reais) por ano-calendário, e desde que sejam  revertidos em benefício do condomínio para cobertura de despesas de custeio e de despesas extraordinárias,  estejam previstos e autorizados na convenção condominial, não sejam distribuídos aos condôminos e  decorram:   1 - de uso, aluguel ou locação de partes comuns do condomínio;   2 - de multas e penalidades aplicadas em decorrência de inobservância das regras previstas na convenção  condominial; ou   3 - de alienação de ativos detidos pelo condomínio. ',
 'como apurar o rendimento tributável de aluguel, inclusive quando o contrato de locação  contenha cláusula que admita a sua compensação com as despesas efetuadas com benfeitorias  pelo locatário?   tributa-se o valor recebido de aluguel subtraído, quando o encargo tenha sido exclusivamente do locador,  somente das quantias relativas a:   a) impostos, taxas e emolumentos incidentes sobre o bem que produzir o rendimento;   b) aluguel pago pela locação de imóvel sublocado;   c) despesas pagas para cobrança ou recebimento do rendimento;   d) despesas de condomínio.    valor das benfeitorias efetuadas, compensadas em determinado mês com o valor total ou parcial do aluguel  de imóvel, tem natureza de rendimento de aluguel para o proprietário e sofre incidência do imposto sobre a  renda, juntamente com valores recebidos no mês a título de aluguel. ',
 'os rendimentos oriundos de contrato de arrendamento de imóvel rural são tributáveis?   os rendimentos provenientes de arrendamento de imóvel rural, ainda que o contrato celebrado refira-se a  parceria rural, se o cedente perceber quantia fixa sem partilhar os riscos do negócio, que é da essência do  contrato de parceria rural, estão sujeitos ao imposto sobre a renda. esses rendimentos são tributados, como  rendimentos equiparados a aluguéis, por meio do recolhimento mensal (carnê-leão), se recebidos de pessoa  física ou, na fonte, se pagos por pessoa jurídica e na declaração de ajuste.   quando o contrato celebrado referir-se a parceria rural e o cedente não receber quantia fixa e participar dos  riscos do negócio, a tributação desses rendimentos é efetuada como atividade rural. ',
 'qual é o tratamento tributário de rendimentos de aluguel depositados judicialmente?   no caso de o locatário efetuar em juízo o depósito desses rendimentos, tal fato não configura a aquisição da  disponibilidade econômica ou jurídica da renda ou proventos para o seu legítimo titular, não estando este  obrigado a tributar os rendimentos no mês do depósito. esses rendimentos são tributados somente quando  liberados pela autoridade judicial.',
 'as luvas e gratificações pagas ao locador são rendimentos tributáveis?   sim. as luvas, prêmios, gratificações ou quaisquer outras importâncias pagas ao locador ou cedente do direito,  pelo contrato celebrado para uso, ocupação, fruição ou exploração de bens corpóreos e direitos são  rendimentos tributáveis, como aluguéis, por meio do recolhimento mensal (carnê-leão), se recebidos de  pessoa física ou, na fonte, se pagos por pessoa jurídica, e na declaração de ajuste.  ',
 'importância recebida pelo locatário a título de indenização para desocupação do imóvel  locado é tributável?   sim.  valor da indenização recebida pelo locatário para desocupar o imóvel locado é rendimento tributável,  quer tenha sido pago pelo locador, pelo novo proprietário ou por terceiro. esse valor é tributável na fonte, se  pago por pessoa jurídica, ou sujeito ao recolhimento mensal (carnê-leão), se recebido de pessoa física e na  declaração de ajuste.  ',
 'qual é o tratamento tributário do valor locativo de imóvel cedido gratuitamente?    valor locativo de imóvel cedido a terceiro é tributado na declaração de ajuste anual, devendo ser informado  em rendimentos tributáveis recebidos de pessoas jurídicas, não se sujeitando, portanto, ao recolhimento  mensal (carnê-leão).    valor tributável corresponde a 10% do valor venal do imóvel, podendo ser adotado o constante da guia do   do ano-calendário da declaração de ajuste anual. se a cessão de uso não abrangeu todo o ano- calendário, o valor tributável é apurado proporcionalmente ao período de cessão de uso de imóvel.   não há incidência do imposto quando o imóvel for ocupado por seu proprietário ou cedido gratuitamente para  uso do cônjuge ou de parentes de 1º grau (pais e filhos).   do valor tributável podem ser subtraídas as seguintes despesas, quando o ônus tenha sido do proprietário:   a) impostos, taxas e emolumentos incidentes sobre o bem que produzir o rendimento;   b) aluguel pago pela locação de imóvel sublocado;   c) despesas pagas para cobrança ou recebimento do rendimento;   d) despesas de condomínio. ',
 'os rendimentos oriundos da sublocação de imóvel são tributáveis?   sim. os rendimentos recebidos pelo sublocador estão sujeitos ao recolhimento mensal (carnê-leão), se  recebidos de pessoa física ou, à retenção na fonte, se pagos por pessoa jurídica e na declaração de ajuste  anual. é dedutível do valor do rendimento bruto recebido pela sublocação o aluguel pago ao proprietário do  imóvel sublocado.',
 'como proceder quando o imóvel locado pertencer a mais de uma pessoa física?   quando o imóvel locado pertencer a mais de uma pessoa física, em condomínio, o contrato de locação deve  discriminar a percentagem do aluguel que cabe a cada condômino. caso não conste no contrato essa  cláusula, recomenda-se fazer um aditivo ao mesmo.   quando o locatário for pessoa jurídica, esta deve efetuar a retenção na fonte aplicando a tabela mensal em  relação ao valor pago individualmente a cada condômino. anualmente, a pessoa jurídica locatária deve  fornecer comprovante do rendimento que couber a cada um, com indicação do respectivo valor retido na fonte.   em se tratando de bens comuns, em decorrência do regime de casamento, inclusive no caso de contribuinte  separado de fato, os rendimentos são tributados na proporção de 50% em nome de cada cônjuge ou,  opcionalmente, podem ser tributados pelo total em nome de um dos cônjuges. na união estável, adota-se  idêntico tratamento, salvo contrato escrito entre os companheiros (neste caso, será fixado o percentual nele  previsto), aplicando-se o regime da comunhão parcial de bens (bens comuns, em decorrência do regime de  casamento).',
 'podem os contribuintes casados no regime de comunhão de bens optar por tributar 100%  dos rendimentos produzidos por um dos bens comuns na declaração de um dos cônjuges e 50%  dos rendimentos produzidos pelos demais bens comuns na declaração de cada um dos cônjuges?   não, pois a opção, efetuada por ocasião da entrega da declaração de ajuste anual, é pela tributação em  nome de um dos cônjuges da totalidade dos rendimentos produzidos pelos bens comuns durante o ano- calendário.   portanto, no caso de contribuintes casados pelo regime de comunhão de bens, ou segue-se a regra geral e  tributa-se 50% dos rendimentos produzidos pelos bens comuns na declaração de cada um dos cônjuges ou  opta-se pela tributação da totalidade dos rendimentos produzidos pelos bens comuns na declaração de um  dos cônjuges.',
 'qual é o tratamento tributário aplicável à pensão alimentícia recebida mensalmente?    rendimento recebido a título de pensão está sujeito ao recolhimento mensal (carnê-leão) e à tributação na  declaração de ajuste anual.  contribuinte do imposto é o beneficiário da pensão, ainda que esta tenha sido  paga a seu representante legal.  beneficiário deve efetuar o recolhimento do carnê-leão até o último dia útil  do mês seguinte ao do recebimento.    atenção:    se um contribuinte informar em sua declaração de ajuste um dependente que receba pensão  alimentícia, deve incluir tais rendimentos como tributáveis, independentemente do valor. pode ainda  o beneficiário da pensão apresentar declaração em nome próprio, tributando os rendimentos de  pensão em separado.  ',
 'qual é a forma de tributação no caso de pensão alimentícia recebida acumuladamente em  cumprimento de decisão judicial, acordo homologado judicialmente ou por escritura pública?   quando correspondentes a anos-calendário anteriores ao do recebimento:   essas pensões são tributadas exclusivamente na fonte, no mês do recebimento ou crédito, em separado dos  demais rendimentos no mês.  imposto será retido pela pessoa física ou jurídica obrigada ao pagamento ou  pela instituição financeira depositária do crédito, sendo calculado sobre o montante dos rendimentos pagos  ou creditados, mediante a utilização de tabela progressiva resultante da multiplicação da quantidade de meses  a que se referem os rendimentos pelos valores constantes da tabela progressiva mensal correspondente ao  mês do recebimento ou crédito. poderão ser excluídas as despesas, relativas ao montante dos rendimentos  tributáveis, com ação judicial necessárias ao seu recebimento, inclusive de advogados, se tiverem sido pagas  pelo contribuinte, sem indenização.   quando correspondentes ao ano-calendário em curso:   serão tributadas, no mês do recebimento ou crédito, sobre o total dos rendimentos, diminuídos do valor das  despesas com ação judicial necessárias ao seu recebimento, inclusive de advogados, se tiverem sido pagas  pelo contribuinte, sem indenização.  ',
 'qual é o tratamento tributário de pensão alimentícia paga por meio de bens e direitos?    pensão alimentícia paga em bens e direitos não está sujeita à tributação sob a forma de carnê-leão, pelo  beneficiário, por não ter sido efetuada em dinheiro.    alimentando que recebeu os bens e direitos deve incluí-los na declaração de ajuste considerando como  custo de aquisição o valor relativo à pensão alimentícia.    alimentante deve apurar o ganho de capital relativo aos bens e direitos dados em pagamento, quando  tributáveis, considerando como valor de alienação o valor da pensão alimentícia. ',
 'qual é o tratamento tributário da indenização recebida em decorrência de constituição de  servidão de passagem?   na servidão o proprietário do imóvel suporta limitações em seu domínio, mas não perde o direito de  propriedade, portanto, não ocorre a alienação do bem. assim, o valor recebido a título de indenização  decorrente de desvalorização de área de terras, para instituição de servidão de passagem (ex.: linha de  transmissão de energia elétrica), bem como a correção monetária incidente sobre a indenização, é tributável  na fonte, no caso de fonte pagadora pessoa jurídica, ou como recolhimento mensal (carnê-leão), no caso de  pagamento efetuado por pessoa física, e, em ambas as situações, na declaração de ajuste.',
 'valor do laudêmio recebido por pessoa física é tributável?   sim. é tributável o valor do laudêmio e do foro anual como carnê-leão, se recebidos de pessoa física, ou na  fonte, se recebidos de pessoa jurídica, conforme o caso, e na declaração de ajuste.   ',
 'qual é o tratamento tributário dos juros produzidos por quantia correspondente à indenização  por morte, cujos valores ficam bloqueados até a maioridade do beneficiário?    crédito dos juros em conta de depósito configura a disponibilidade jurídica, para os efeitos do imposto sobre  a renda, devendo ser tributados exclusivamente na fonte, exceto quando se tratar de caderneta de poupança,  cujos juros estão isentos.',
 'qual é o tratamento tributário da indenização recebida por danos morais?   em decorrência do disposto no art. 19 da lei nº 10.522, de 19 de julho de 2002, não é tributada pelo imposto  sobre a renda na fonte, nem na declaração de ajuste anual, a indenização - de verba percebida a título de  dano moral por pessoa física - paga por pessoa física ou jurídica, em virtude de acordo ou decisão judicial.    fonte pagadora está desobrigada de reter o tributo devido pelo contribuinte e a secretaria especial da  receita federal do brasil   não constituirá os respectivos créditos tributários, tendo em vista a vigência  do ato declaratório  nº 9, de 20 de dezembro de 2011.    atenção:   para declarar esses rendimentos, a pessoa física deverá informá-los no campo “outros” da ficha  “rendimentos isentos e não tributáveis”, com especificação da natureza do rendimento. ',
 'os valores recebidos em razão do encargo estipulado em doação modal de bens ou direitos  são tributáveis?   sim.  doação modal ou onerosa é aquela que traz consigo um encargo para o donatário. os valores recebidos  em função desse encargo estão sujeitos ao recolhimento mensal (carnê-leão), se recebidos de pessoa física  ou, na fonte, se pagos por pessoa jurídica, e na declaração de ajuste. ',
 'incide o imposto sobre a renda sobre os rendimentos recebidos por pessoa física decorrentes  de empréstimos?   sim. os juros decorrentes de empréstimos concedidos a pessoa jurídica estão sujeitos à incidência do imposto  sobre a renda, exclusivamente na fonte, às alíquotas de:    a) 22,5%, com prazo de até seis meses;   b) 20%, com prazo de seis meses e um dia até doze meses;   c) 17,5%, com prazo de doze meses e um dia até vinte e quatro meses;   d) 15%, com prazo acima de vinte e quatro meses.   se recebidos de pessoa física, os juros recebidos estão sujeitos ao recolhimento mensal (carnê-leão) e à  tributação na declaração de ajuste anual. ',
 'como são tributados os rendimentos pagos em cumprimento de decisão judicial?   os rendimentos pagos em cumprimento de decisão judicial estão sujeitos ao imposto sobre a renda na fonte  mediante aplicação da tabela progressiva mensal.  retenção, de responsabilidade da pessoa física ou jurídica  obrigada ao pagamento, dar-se-á no momento em que, por qualquer forma, os rendimentos se tornem  disponíveis ao beneficiário (observadas as orientações contidas na pergunta 235).   fica dispensada a soma dos rendimentos pagos no mês, para aplicação da alíquota correspondente, nos  casos de:    - juros e indenizações por lucros cessantes; e    - honorários advocatícios e remuneração pela prestação de serviços no curso do processo judicial, tais  como: serviços de engenharia, médico, contador, perito, assistente técnico, avaliador, leiloeiro, síndico,  testamenteiro, liquidante.   atenção:    1 - decisão da justiça federal   desde 1º de fevereiro de 2004, os rendimentos pagos, em cumprimento de decisão da justiça  federal, mediante precatório ou requisição de pequeno valor, estão sujeitos à retenção do imposto  sobre a renda na fonte, pela instituição financeira responsável pelo pagamento, à alíquota de 3%  sobre o montante pago, sem quaisquer deduções, no momento em que, por qualquer forma, o  rendimento se torne disponível para o beneficiário (lei nº 10.833, de 29 de dezembro de 2003, arts.  27 e 93, inciso ; e lei nº 10.865, de 30 de abril de 2004, art. 21).   fica dispensada a retenção do imposto quando o beneficiário declarar à instituição financeira  responsável pelo pagamento que os rendimentos recebidos são isentos ou não tributáveis.    imposto retido na fonte é considerado antecipação do imposto apurado na declaração de ajuste  anual do imposto sobre a renda das pessoas físicas.   deve ser indicado como fonte pagadora o número do cadastro nacional da pessoa jurídica    da instituição financeira depositária do crédito.   2 - decisão da justiça do trabalho   cabe à fonte pagadora, no prazo de 15 (quinze) dias da data da retenção de que trata o caput do  art. 46 da lei nº 8.541, de 23 de dezembro de 1992, comprovar, nos respectivos autos, o  recolhimento do imposto sobre a renda na fonte incidente sobre os rendimentos pagos em  cumprimento de decisão da justiça do trabalho (lei nº 10.833, de 29 de dezembro de 2003, art.  28).   na hipótese de omissão da fonte pagadora relativamente à referida comprovação, e nos  pagamentos de honorários periciais, competirá ao juízo do trabalho calcular o imposto sobre a  renda na fonte e determinar o seu recolhimento à instituição financeira depositária do crédito.    não indicação pela fonte pagadora da natureza jurídica das parcelas objeto de acordo  homologado perante a justiça do trabalho acarretará a incidência do imposto sobre a renda na  fonte sobre o valor total da avença.   3 - decisão da justiça estadual   deve ser apurado e recolhido utilizando o código de receita próprio, conforme a natureza do  rendimento.    no caso de rendimentos recebidos acumuladamente, deve-se observar o seguinte:    partir de 11 de março de 2015, os rendimentos submetidos à incidência do imposto sobre a renda com base  na tabela progressiva, quando correspondentes a anos-calendário anteriores ao do recebimento, são  tributados exclusivamente na fonte, no mês do recebimento ou crédito, em separado dos demais rendimentos  recebidos no mês, inclusive os decorrentes de decisões das justiças do trabalho, federal, estaduais e do  distrito federal.    mesma regra aplica-se, desde 28 de julho de 2010, aos rendimentos provenientes de aposentadoria,  pensão, transferência para a reserva remunerada ou reforma, pagos pela previdência social da união, dos  estados, do distrito federal e dos municípios, e aos rendimentos do trabalho.    disposto acima aplica-se ao décimo terceiro salário e a quaisquer acréscimos e juros referentes aos mesmos  rendimentos.    imposto será retido pela pessoa física ou jurídica obrigada ao pagamento ou pela instituição financeira  depositária do crédito, sendo calculado sobre o montante dos rendimentos pagos ou creditados, mediante a  utilização de tabela progressiva resultante da multiplicação da quantidade de meses a que se referem os      106   rendimentos pelos valores constantes da tabela progressiva mensal correspondente ao mês do recebimento  ou crédito.   do montante recebido poderão ser excluídas despesas, relativas aos rendimentos tributáveis, com ação  judicial necessária ao seu recebimento, inclusive de advogados, se tiverem sido pagas pelo contribuinte, sem  indenização, e deduzidas as seguintes despesas:   a) importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de  família, quando em cumprimento de decisão judicial, de acordo homologado judicialmente ou de  separação ou divórcio consensual realizado por escritura pública; e   para efeitos da aplicação da referida dedução, observe-se que:    – as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família,  serão aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título  de pensão alimentícia;    – tratando-se de sociedade conjugal, a dedução somente se aplica quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;    – o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta  321 que trata da dedução de dependentes;    – não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei  nº 9.307, de 23 de setembro de 1996.   b) contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios.    inclusão dos rendimentos recebidos acumuladamente e respectivos dados, na declaração de ajuste anual   , será feita mediante acesso ao menu “fichas da declaração” no programa  e seleção da ficha  “rendimentos recebidos acumuladamente”, para fins de preenchimento.',
 'qual é o tratamento tributário de precatórios recebidos por pessoas físicas?    imposto sobre a renda relativo a rendimentos pagos, em cumprimento de decisão da justiça federal,  mediante precatório ou requisição de pequeno valor, será retido na fonte pela instituição financeira  responsável pelo pagamento e incidirá à alíquota de 3% (três por cento) sobre o montante pago, sem  quaisquer deduções, no momento do pagamento ao beneficiário ou seu representante legal (verificar as  orientações contidas na pergunta 215 e, em especial, o conteúdo a respeito da tributação de rendimentos  recebidos acumuladamente, constante da pergunta 235)   vale ressaltar que o valor retido na fonte (3%) não é definitivo.  imposto retido será considerado antecipação  do imposto apurado, ou seja, o contribuinte deverá informar por ocasião da declaração de ajuste anual, o valor  dos rendimentos recebidos pelo precatório e respectiva antecipação, para fins de apuração do imposto sobre  a renda.    retenção do imposto é dispensada, quando o beneficiário declarar à instituição financeira responsável pelo  pagamento que os rendimentos recebidos são isentos ou não tributáveis.   atenção:    ato declaratório  nº 2, de 10 de março de 2016, revogou o ato declaratório  nº 1, de  27 de março de 2009, que considerava que o cálculo do imposto sobre a renda incidente sobre  rendimentos pagos acumuladamente deveria ser realizado levando-se em consideração as tabelas  e alíquotas das épocas próprias a que se referiram tais rendimentos, devendo o calculo ser mensal  e não global. ',
 'qual é o tratamento tributário do pecúlio recebido por pessoas físicas?    valor relativo ao pecúlio recebido é tributável quando pago, na demissão ou retirada, por ex-empregador,  institutos, caixas de aposentadoria ou entidades governamentais em decorrência de emprego, cargo ou função  exercido no passado, independentemente da denominação empregada, tal como pecúlio resgate, pecúlio  restituição, pecúlio patrimônio, pecúlio reserva de poupança, pecúlio devolução.   é isento quando pago por intermédio:   a) de companhia de seguro, por morte do segurado;   (lei nº 7.713, de 22 de dezembro de 1988, art. 6º, inciso )   b) do , correspondente às contribuições pagas ou descontadas dos aposentados que tenham voltado a  trabalhar até 15/04/1994, quer seja o pecúlio recebido pelo segurado ou por seus dependentes, após sua  morte;   (regulamento do imposto sobre a renda - /2018, art. 35, inciso , alínea “g”, aprovado pelo  decreto nº 9.580, de 22 de novembro de 2018; e instrução normativa  nº 1.500, de 29 de  outubro de 2014, art. 6º, inciso )   c) de entidade de previdência complementar, quando se tratar de benefício de risco, com característica de  seguro, previsto expressamente no plano de benefício contratado, pago em prestação única em razão de  morte ou invalidez permanente do participante de plano de previdência complementar. as parcelas pagas a  entidade de previdência complementar originadoras deste direito não podem ter sido utilizadas para fins de  dedução do imposto devido na declaração de ajuste anual. ',
 'qual é o tratamento tributário dos valores pagos por companhia seguradora a autônomo que  faz seguro para ter garantido o seu rendimento mensal em caso de inatividade temporária, em razão  de acidente pessoal?   tal rendimento é tributável, não se enquadrando entre as isenções previstas no art. 35 do regulamento do  imposto sobre a renda – /2018, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018, nem em  qualquer outro dispositivo legal de isenção.',
 'são tributáveis os rendimentos recebidos por pessoa física com doença grave?   são isentos apenas os rendimentos recebidos por pessoa física residente no brasil, com doença grave,  relativos a proventos de aposentadoria, reforma ou pensão, e suas respectivas complementações, ainda que  pagas por fonte situada no exterior. tributam-se os demais rendimentos de outra natureza recebidos pelo  contribuinte. ',
 'laudo pericial expedido por entidade privada vinculada ao sistema único de saúde   é  documento comprobatório de doença grave?   não. somente podem ser aceitos laudos periciais expedidos por instituições públicas, independentemente da   vinculação destas ao sistema único de saúde  . os laudos periciais expedidos por entidades privadas   não atendem à exigência legal e, portanto, não podem ser aceitos, ainda que o atendimento decorra de   convênio referente ao .   entende-se por laudo pericial o documento emitido por médico legalmente habilitado ao exercício da profissão  de medicina, integrante de serviço médico oficial da união, dos estados, do distrito federal ou dos municípios,  independentemente de ser emitido por médico investido ou não na função de perito, observadas a legislação  e as normas internas especificas de cada ente. ',
 'qual é o tratamento dos rendimentos recebidos acumuladamente, por pessoa com doença  grave após o seu reconhecimento por laudo pericial oficial?   para definir qual o tratamento tributário, deve-se verificar a natureza dos rendimentos recebidos; tratando-se  de rendimentos do trabalho assalariado, são tributáveis; se se tratarem de proventos de aposentadoria,  reforma ou pensão, são isentos, ainda que se refiram a período anterior à data em que foi contraída a moléstia,  desde que recebidos após a data da emissão do laudo ou após a data constante do laudo que confirme a  partir de que data foi contraída a doença.   sobre laudo pericial consultar a pergunta 221.   no caso de rendimentos recebidos acumuladamente provenientes do trabalho assalariado, a pessoa com  doença grave deve verificar as orientações contidas nas perguntas 234 e 235. ',
 'os proventos recebidos por militar integrante de reserva remunerada com doença grave são  isentos?   tendo em vista o disposto no ato declaratório  nº 1, de 12 de março de 2018, o benefício da isenção  prevista no inciso  do art. 6º da lei nº 7.713, de 22 de dezembro de 1988, estende-se aos proventos  recebidos por militares transferidos para a reserva remunerada. ',
 'é tributável o salário recebido em conjunto com as indenizações previstas no decreto-lei nº  5.452, de 1º de maio de 1943 - consolidação das leis do trabalho -  e o montante referente ao  fundo de garantia do tempo de serviço -  decorrentes da rescisão de contrato de trabalho?    valor referente ao salário é tributável na fonte e na declaração de ajuste. são isentas as indenizações pagas  por despedida ou rescisão de contrato de trabalho até o limite garantido pela lei trabalhista (consolidação das  leis do trabalho - ) ou por dissídio coletivo e convenções trabalhistas homologados pela justiça do  trabalho, bem como o montante recebido pelos empregados e diretores, ou respectivos beneficiários,  referente aos depósitos, juros e correção monetária creditados em contas vinculadas, nos termos da legislação  do fundo de garantia do tempo de serviço. ',
 'é isento o valor referente ao  pago pelo ex-empregador em virtude de decisão judicial?   sim.  isenção alcança os depósitos, juros, correção monetária e multa, pagos nos limites e termos da  legislação do fundo de garantia do tempo de serviço.',
 'qual é o tratamento tributário das indenizações pagas a título de incentivo à adesão a  programa de desligamento voluntário  ?   as verbas especiais pagas a título de  por pessoa jurídica de direito público a servidor público civil são  isentas do imposto sobre a renda na fonte e na declaração de ajuste.    partir de 31/12/1998, a instrução normativa  nº 165, de 31 de dezembro de 1998, dispensou a  constituição de créditos da fazenda nacional relativos à incidência do imposto sobre a renda na fonte sobre  as verbas indenizatórias pagas em decorrência de programas de desligamento voluntário  ,  independentemente de a fonte pagadora ser pessoa jurídica de direito público. assim sendo, os valores pagos  por pessoa jurídica a seus empregados a título de  não se sujeitam à incidência do imposto sobre a renda  na fonte nem na declaração de ajuste anual.   não se incluem no conceito de verbas especiais indenizatórias recebidas a título de adesão ao :   a) as verbas rescisórias previstas na legislação trabalhista em casos de rescisão de contrato de trabalho, tais  como: décimo terceiro salário, saldo de salário, salário vencido, férias proporcionais ou vencidas, abono e  gratificação de férias, gratificações e demais remunerações provenientes de trabalho prestado, remuneração  indireta, aviso prévio trabalhado, participação dos empregados nos lucros ou resultados da empresa; e   b) os valores recebidos em função de direitos adquiridos, anteriormente à adesão ao , em decorrência do  vínculo empregatício, a exemplo do resgate de contribuições efetuadas à previdência complementar em  virtude de desligamento do plano de previdência.   com relação à tributação de férias indenizadas, consulte a pergunta 162. ',
 'contribuinte que adere a programa de demissão voluntária   e, na mesma ocasião,  resgata valores pagos a entidade de previdência complementar da empresa da qual fazia parte, pode  considerar os valores do resgate da previdência também isentos?   ainda que recebidos por ocasião de adesão a programa de demissão voluntária  , os resgates de  previdência complementar, sujeitam-se à incidência do imposto sobre a renda na fonte e na declaração de  ajuste anual, quando não optante pela tributação exclusiva.    no entanto, exclui-se da incidência o valor do resgate das contribuições, cujo ônus tenha sido da pessoa física,  recebido por ocasião de seu desligamento do plano de benefícios da entidade de previdência complementar,  que corresponder às parcelas de contribuições efetuadas no período de 01/01/1989 a 31/12/1995. ',
 'qual é o tratamento tributário dos rendimentos de sócios ou titular de empresa tributada com  base no lucro real, presumido ou arbitrado?   os lucros ou dividendos calculados com base nos resultados apurados a partir de 01/01/1996, pagos ou  creditados pelas pessoas jurídicas tributadas com base no lucro real, presumido ou arbitrado, não estão  sujeitos à incidência do imposto sobre a renda na fonte nem integram a base de cálculo do imposto do  beneficiário. são tributáveis os valores que ultrapassarem o resultado contábil e os lucros acumulados e  reservas de lucros de anos anteriores, observada a legislação vigente à época da formação dos lucros. ',
 'qual é o tratamento tributário dos rendimentos pagos ao sócio de serviço a título de pro-  labore?   incide imposto sobre a renda, na fonte e na declaração de ajuste anual, sobre os valores pagos ao sócio de  serviço (consulte a solução de consulta interna cosit nº 12, de 15 de maio de 2013, disponível no sítio da   na internet: http://receita.economia.gov.br => legislação => soluções de consultas e de divergências  => sistema padrão de pesquisas da legislação da receita federal), a título de pro labore (rendimentos de  trabalho). no entanto, não incide imposto sobre a renda sobre valores pagos a título de distribuição de lucros  pelas pessoas jurídicas.',
 'qual é a diferença entre pro labore e lucro distribuído? como são tributados?    pro labore refere-se à remuneração paga aos sócios pela prestação de serviços à empresa e sujeita-se à  incidência do imposto sobre a renda na fonte e na declaração de ajuste.    lucro distribuído refere-se à remuneração do capital integralizado pelo sócio. quanto à forma de tributação,  depende da legislação vigente no período de formação do lucro que está sendo distribuído. ',
 'qual é o tratamento tributário dos rendimentos recebidos pela pessoa física em decorrência  da locação de unidade imobiliária em sistema denominado de pool hoteleiro?   no sistema de locação conjunta de unidades imobiliárias denominado de pool hoteleiro, constitui-se,  independentemente de qualquer formalidade, sociedade em conta de participação   com o objetivo de  lucro comum, onde a administradora (empresa hoteleira) é a sócia ostensiva e os proprietários das unidades  imobiliárias integrantes do pool são os sócios ocultos.    as  são equiparadas às pessoas jurídicas pela legislação do imposto sobre a renda, e, como tais, são  contribuintes do imposto sobre a renda das pessoas jurídicas , da contribuição social sobre o lucro  líquido , da contribuição para o /pasep e da contribuição para o financiamento da seguridade  social (cofins).   portanto, os rendimentos recebidos pelas pessoas físicas em decorrência da locação de unidade imobiliária  em sistema denominado de pool hoteleiro seguem as normas gerais aplicáveis aos pagamentos efetuados  por pessoa jurídica aos seus sócios. ',
 'qual é o tratamento tributário dos lucros distribuídos apurados na liquidação da massa falida  de pessoa jurídica?   os lucros distribuídos durante a liquidação sujeitam-se às mesmas normas de tributação de distribuição de  lucros aplicáveis às demais pessoas jurídicas. ',
 'são tributáveis os valores ressarcidos ou pagos pelas empresas a título de complementação  de rendimento, tais como seguro-desemprego, auxílio-creche, auxílio-doença, auxílio-funeral,  auxílio pré-escolar, prêmio assiduidade, gratificações por quebra de caixa, indenização adicional  por acidente de trabalho etc.?   em regra geral, esses valores são tributáveis na fonte e na declaração de ajuste.   são isentos, entretanto, os rendimentos pagos pela previdência oficial da união, dos estados, do distrito  federal e dos municípios, ainda que pagos pelo empregador por força de convênios com órgãos  previdenciários, e pelas entidades de previdência complementar, decorrentes de seguro-desemprego, auxílio- natalidade, auxílio-doença, auxílio-funeral e auxílio-acidente.   hipóteses de não tributação   em decorrência do disposto no art. 19 da lei nº 10.522, de 19 de julho de 2002, no caso de verbas recebidas  a título de auxílio-creche e auxílio pré-escolar pelos trabalhadores até o limite de cinco anos de idade de seus  filhos, a fonte pagadora está desobrigada de reter o tributo devido pelo contribuinte e a  não constituirá  os respectivos créditos tributários, tendo em vista o parecer //nº 2.118, de 10 de novembro de 2011,  e o ato declaratório  nº 13, de 20 de dezembro de 2011.    da mesma forma, não incide imposto sobre a renda nas verbas recebidas a título de reembolso-babá pelos  trabalhadores, ficando a fonte pagadora desobrigada de reter o tributo devido pelo contribuinte e a  não  constituirá os respectivos créditos tributários, tendo em vista o ato declaratório  nº 1, de 2 de janeiro de  2014.   atenção:   para declarar as verbas alcançadas pelos atos declaratórios acima citados, a pessoa física deverá  informá-las no campo “outros” da ficha “rendimentos isentos e não tributáveis”, com especificação  da natureza do rendimento. ',
 'como são tributados os rendimentos recebidos acumuladamente em 2019 e correspondentes  a esse mesmo ano-calendário?   desde o ano-calendário de 1989, por força da edição do art. 12 da lei nº 7.713, de 22 de dezembro de 1988,  os rendimentos recebidos acumuladamente, independente do período a que correspondiam, eram tributáveis  no mês de seu recebimento na fonte e na declaração de ajuste, utilizando-se da tabela progressiva vigente no  momento do pagamento (regime de caixa).   no entanto, essa norma foi declarada inconstitucional pelo supremo tribunal federal  , do que decorreu  sua revogação e alterações legislativas nos arts. 12- e 12- desse ato normativo, trazidas pelas medidas  provisórias nºs 497, de 27 de julho de 2010, e 670, de 10 de março de 2015, bem como pela lei nº 13.149,      113   de 21 de julho de 2015. entendeu a corte suprema pela aplicação da tabela progressiva mensal vigente à  época de referência do pagamento dos rendimentos, quando estes se referirem a anos-calendário anteriores  ao do recebimento efetivo (regime de competência).   assim sendo, os rendimentos recebidos acumuladamente, quando correspondentes ao ano-calendário em  curso, no caso 2019, continuam tributados no mês do recebimento ou crédito, sobre o total dos rendimentos,  diminuído apenas do valor das despesas com ação judicial necessárias à sua percepção, inclusive honorários  de advogados, se tiverem sido suportadas pelo contribuinte.   ressalta-se que se a percepção da renda ou provento acumulado, quando correspondentes ao ano-calendário  em curso, se der em cumprimento de decisão da justiça federal ou trabalhista, cabível será a aplicação da  retenção na fonte segundo os arts. 27 e 28 da lei nº 10.833, de 29 de dezembro de 2003.  ',
 'qual é o tratamento tributário de diferenças salariais recebidas acumuladamente relativas a  anos-calendário anteriores ao do recebimento?   os rendimentos recebidos acumuladamente submetidos à tabela progressiva, quando correspondentes a  anos-calendário anteriores ao do recebimento, serão tributados exclusivamente na fonte, no mês do  recebimento ou crédito, em separado dos demais rendimentos recebidos no mês, por força da alteração do  art. 12- da lei nº 7.713, de 22 de dezembro de 1988, promovida pela lei nº 13.149, de 21 de julho de 2015.   destaca-se que este tratamento já era conferido, desde 28 de julho de 2010, aos rendimentos recebidos  acumuladamente relativos a anos-calendário anteriores ao do recebimento, quando decorrentes de:   a) aposentadoria, pensão, transferência para a reserva remunerada ou reforma, pagos pela previdência social  da união, dos estados, do distrito federal e dos municípios; e   b) rendimentos do trabalho.   aplica-se a referida tributação, inclusive, aos rendimentos decorrentes de decisões das justiças do trabalho,  federal, estaduais e do distrito federal; devendo abranger tais rendimentos o décimo terceiro salário e  quaisquer acréscimos e juros deles decorrentes.   atenção:   não se aplica a retenção na fonte no percentual de 3%, de que trata o art. 27 da lei nº 10.833, de  29 de dezembro de 2003, a rendimento recebido acumuladamente relativo a ano-calendário anterior  decorrente do cumprimento de decisão de justiça federal e pago mediante precatório ou requisição  de pequeno valor.    imposto será retido, pela pessoa física ou jurídica obrigada ao pagamento ou pela instituição  financeira depositária do crédito, e calculado sobre o montante dos rendimentos pagos, mediante a  utilização de tabela progressiva resultante da multiplicação da quantidade de meses a que se  referem os rendimentos pelos valores constantes da tabela progressiva mensal correspondente ao  mês do recebimento ou crédito.   do montante recebido poderão ser excluídas despesas, relativas aos rendimentos tributáveis, com ação  judicial necessária ao seu recebimento, inclusive de advogados, se tiverem sido pagas pelo contribuinte, sem  indenização; e deduzidas as seguintes despesas:   a) importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial, de acordo homologado judicialmente ou de separação ou divórcio  consensual realizado por escritura pública; e   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;      114   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.   b) contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios.    inclusão dos rendimentos recebidos acumuladamente e respectivos dados, na , será feita mediante  acesso ao menu “fichas da declaração” no programa  e seleção da ficha “rendimentos recebidos  acumuladamente”, para fins de preenchimento.   no caso de opção pela forma de tributação “ajuste anual”, à opção irretratável do contribuinte, os valores  relativos aos  integrarão a base de cálculo do imposto sobre a renda na  do ano-calendário do  recebimento. nesse caso, o imposto decorrente da tributação na fonte efetuada durante o ano-calendário pela  fonte pagadora é considerado antecipação do imposto devido apurado na referida .   deve marcar essa opção o contribuinte cuja tributação dos  na fonte ocorreu:   a) de forma exclusiva e ele quer alterar a forma de tributação para ajuste anual; e   b) pelo ajuste anual e ele quer confirmar a opção por essa forma de tributação.    contribuinte somente pode alterar a forma de tributação dos rendimentos recebidos acumuladamente para  ajuste anual até 30/04/2020.   atenção:   na hipótese em que a pessoa responsável pela retenção não a tenha feito em conformidade com  o disposto no capítulo  da instrução normativa  nº 1.500, de 29 de outubro de 2014, ou que  tenha promovido retenção indevida ou a maior, a pessoa beneficiária poderá alterar a forma de  tributação dos rendimentos recebidos acumuladamente para exclusiva ainda que após a data de  30/04/2020.  ',
 'os rendimentos recebidos acumuladamente relativos a aposentadoria, pensão, transferência  para a reserva remunerada ou reforma pagos pela previdência social da união, dos estados, do  distrito federal e dos municípios ou por qualquer pessoa jurídica de direito público interno, ou   ainda, por entidade de previdência complementar, estão sujeitos à isenção prevista no inciso  do  art. 6º da lei nº 7.713, de 22 de dezembro de 1988?    benefício da isenção limita-se apenas aos rendimentos sujeitos à tabela progressiva mensal e ao ajuste  anual. dessa forma, esse benefício somente será aplicado no caso de o contribuinte optar por incluir o total  desses rendimentos na base de cálculo do imposto sobre a renda na declaração de ajuste anual do ano- calendário do recebimento, isto é, se optar pela forma de tributação “ajuste anual” na ficha “rendimentos  tributáveis de pessoa jurídica recebidos acumuladamente pelo titular”.   ressalte-se que a referida isenção está limitada a $ 1.903,98 por mês, no ano-calendário de 2019, a partir  do mês em que o contribuinte completar 65 anos. ',
 'qual é o tratamento tributário de diferenças salariais recebidas acumuladamente, de  rendimentos de anos anteriores, por força de decisão judicial, quando o beneficiário da ação é a  pessoa falecida?   1 - se recebidas no curso do inventário   as diferenças salariais são tributadas na declaração do espólio, conforme o regime de tributação dos  rendimentos, sejam tributáveis na fonte e na declaração anual de rendimentos, tributáveis exclusivamente na  fonte, isentos ou não tributáveis.   2 - se recebidas após encerrado o inventário   serão tributadas na declaração do(s) herdeiros(s) ou legatários(s), segundo o regime de tributação dos  rendimentos.   em se tratando de rendimentos recebidos acumuladamente  , a quantidade de meses relativa ao valor  dos  transmitidos a cada sucessor será idêntica à quantidade de meses aplicada ao valor dos  do de  cujus.   atenção:     - no caso de rendimentos recebidos acumuladamente provenientes de diferenças salariais  (verificar as orientações contidas na pergunta 215 e, em especial, o conteúdo a respeito da  tributação de rendimentos recebidos acumuladamente, constante das perguntas 234 e 235):    - não se beneficiam da isenção os valores relativos a proventos de aposentadoria, pagos  acumuladamente ao espólio ou diretamente aos herdeiros (mediante alvará judicial), ainda que a  pessoa falecida tivesse alguma moléstia grave no período a que se referem os rendimentos.   (lei nº 5.172, de 25 de outubro de 1966 – código tributário nacional - , art. 144; regulamento  do imposto sobre a renda - /2018, art. 34, parágrafo único, aprovado pelo decreto nº 9.580, de  22 de novembro de 2018.',
 'qual é o tratamento tributário na cessão de direito de precatório referente a rendimentos  recebidos acumuladamente?    fazenda pública por ocasião do pagamento do precatório ao cessionário deve efetuar o cálculo do imposto  com base na tabela progressiva resultante da multiplicação da quantidade de meses a que se referem os  rendimentos pelos valores constantes da tabela progressiva mensal correspondente ao mês do recebimento  ou crédito, nos termos do art. 12- da lei nº 7.713, de 22 de dezembro 1988.   tanto o cedente quanto o cessionário de créditos referentes a rendimentos recebidos acumuladamente    a que se refere a referida legislação, devem apurar o ganho de capital e submetê-lo à incidência do  imposto sobre a renda. ',
 'profissional autônomo que paga a outros profissionais por serviços realizados é considerado  empresa individual?   se a prestação de serviços colegiada é feita apenas eventualmente, sem caráter de habitualidade, tal fato não  caracteriza sociedade.  profissional responsável pelo trabalho deve computar em seu rendimento bruto  mensal o valor total dos honorários recebidos, podendo deduzir os pagamentos efetuados aos outros  profissionais, no caso de escriturar livro-caixa, desde que necessários à percepção da receita e à manutenção  da fonte produtora.   quando a prestação de serviços colegiada for sistemática, habitual, sempre sob a responsabilidade do mesmo  profissional, que recebe em nome próprio o valor total pago pelo cliente e paga os serviços dos demais  profissionais, fica configurada a condição de empresa individual equiparada a pessoa jurídica, nos termos do  inciso  do § 1º do art. 162 do regulamento do imposto sobre a renda – /2028, aprovado pelo decreto nº  9.580, de 22 de novembro de 2018, por se tratar de venda, habitual e profissional, de serviços próprios e de  terceiros.    (parecer normativo  nº 38, de 1975)       equiparação à  jurídica - hipóteses',
 'quais as hipóteses em que a pessoa física é equiparada a pessoa jurídica?    pessoa física equipara-se à pessoa jurídica quando:   a) em nome individual, explore, habitual e profissionalmente, qualquer atividade econômica de natureza civil  ou comercial, com o fim especulativo de lucro, mediante venda a terceiro de bens ou serviços, quer se  encontre, ou não, regularmente inscrita no órgão do registro de comércio ou registro civil, exceto quanto às  profissões de que trata o art. 162, § 2º, do regulamento do imposto sobre a renda – /2018, aprovado  pelo decreto nº 9.580, de 22 de novembro de 2018;   b) promova a incorporação de prédios em condomínio ou loteamento de terrenos. ',
 'quais as atividades exercidas por pessoas físicas que não a equiparam a empresa individual?   não se caracterizam como empresa individual, ainda que, por exigência legal ou contratual, encontrem-se  cadastradas no cadastro nacional da pessoa jurídica ou que tenham seus atos constitutivos registrados em  cartório ou junta comercial, entre outras:   a) a pessoa física que, individualmente, exerça profissões ou explore atividades sem vínculo empregatício,  prestando tais serviços profissionais, mesmo quando possua estabelecimento em que desenvolva suas  atividades e empregue auxiliares (regulamento do imposto sobre a renda – /2018, art. 162, § 2º, inciso  , aprovado pelo decreto nº 9.580, de 22 de novembro de 2018, parecer normativo  nº 38, de 1975);   b) a pessoa física que explore, individualmente, contratos de empreitada unicamente de mão de obra, sem o  concurso de profissionais qualificados ou especializados (regulamento do imposto sobre a renda –  /2018, art. 162, § 2º, inciso , aprovado pelo decreto nº 9.580, de 22 de novembro de 2018; parecer  normativo  nº 25, de 1976);   c) a pessoa física receptora de apostas da loteria esportiva e da loteria de números (quina, lotofácil,  lotomania, mega-sena etc.) credenciada pela caixa econômica federal, ainda que, para atender exigência  do órgão credenciador, esteja registrada como pessoa jurídica, e desde que não explore, no mesmo local,  outra atividade comercial (ato declaratório normativo cosit nº 24, de 14 de setembro de 1999);   d) representante comercial que exerça exclusivamente a mediação para a realização de negócios mercantis,  como definido pelo art. 1º da lei nº 4.886, de 9 de dezembro de 1965, uma vez que não os tenham praticado  por conta própria (regulamento do imposto sobre a renda – /2018, art. 162, § 2º, inciso , aprovado pelo  decreto nº 9.580, de 22 de novembro de 2018, c/c o ato declaratório normativo  nº 25, de 1989);      117   e) a pessoa física que, individualmente, exerça as profissões ou explorem atividades consoante os termos do  art. 162, § 2º, incisos ,  e , do regulamento do imposto sobre a renda – /2018, aprovado pelo  decreto nº 9.580, de 22 de novembro de 2018, como por exemplo: serventuários de justiça, tabeliães,  corretores, leiloeiros, despachantes, artistas e outros;   f) pessoa física que faz o serviço de transporte de carga ou de passageiros em veículo próprio ou locado,  mesmo que ocorra a contratação de empregados, como ajudantes ou auxiliares (regulamento do imposto  sobre a renda – /2018, art. 39, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018). caso haja  a contratação de profissional para dirigir o veículo, descaracteriza-se a exploração individual da atividade,  ficando a pessoa física, que desta forma passa a explorar atividade econômica como firma individual,  equiparada a pessoa jurídica.',
 'em que condições a pessoa física é equiparada a pessoa jurídica pela prática de operações  imobiliárias?   de acordo com a legislação do imposto sobre a renda, somente se considera equiparada a pessoa jurídica,  pela prática de operações imobiliárias, a pessoa física que promove incorporação imobiliária (prédios em  condomínios) ou loteamentos de terrenos urbanos ou rurais, com ou sem construção, inclusive:   a) os proprietários ou titulares de terrenos ou glebas de terra que, efetuando registro dos documentos de  incorporação ou loteamento, outorgarem mandato a construtor ou corretor de imóveis com poderes para  alienação de frações ideais ou lotes de terreno, quando o mandante se beneficiar do produto dessas  alienações, ou assumir a iniciativa ou responsabilidade da incorporação ou loteamento (regulamento do  imposto sobre a renda – /2018, art. 163, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018);   b) os proprietários ou titulares de terrenos ou glebas de terra que, sem efetuar o registro dos documentos de  incorporação ou loteamento, neles promovam a construção de prédio de mais de duas unidades imobiliárias  ou a execução de loteamento, se iniciar a alienação das unidades imobiliárias ou dos lotes de terreno antes  de corrido o prazo de 60 meses contados da data da averbação, no registro de imóveis da construção do  prédio ou da aceitação das obras de loteamento. para os terrenos adquiridos até 30/06/77 o prazo é 36 meses  (regulamento do imposto sobre a renda – /2018, art. 164, aprovado pelo decreto nº 9.580, de 22 de  novembro de 2018);   c) a subdivisão ou desmembramento de imóvel rural, havido após 30/06/1977, em mais de dez lotes, ou a  alienação de mais de dez quinhões ou frações ideais desse imóvel, tendo em vista que tal operação se  equipara a loteamento, salvo se a subdivisão se der por força de partilha amigável ou judicial, em decorrência  de herança, legado, doação como adiantamento da legítima, ou extinção de condomínio (regulamento do  imposto sobre a renda – /2018, art. 165, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018).   atenção:    os condomínios na propriedade de imóveis não são considerados sociedades de fato, ainda que  deles também façam parte pessoas jurídicas. assim, a cada condômino, pessoa física, serão  aplicados os critérios de caracterização da empresa individual e demais dispositivos legais, como  se ele fosse o único titular da operação imobiliária, nos limites da sua participação',
 'quando se considera ocorrido o término da equiparação caso a pessoa jurídica não efetue  nenhuma alienação das unidades imobiliárias ou lotes de terreno?    pessoa física que, após sua equiparação a pessoa jurídica, não promover nenhum dos empreendimentos  nem efetuar nenhuma das alienações de unidades imobiliárias ou lotes de terrenos, durante o prazo de 36  meses consecutivos, deixa de ser considerada equiparada a partir do término deste prazo, salvo quanto aos  efeitos tributários das operações em andamento que terão o tratamento previsto na pergunta 244.',
 'qual é o destino a ser dado aos imóveis integrantes do ativo (patrimônio) da empresa  individual quando, completado o prazo de 36 meses consecutivos sem promover incorporações ou  loteamentos, ocorrer o término da equiparação a pessoa jurídica?   permanecem no ativo da empresa individual, para efeito de tributação como lucro da pessoa jurídica  (regulamento do imposto sobre a renda – /2018, art. 177, aprovado pelo decreto nº 9.580, de 22 de  novembro de 2018):   a) as unidades imobiliárias e os lotes de terrenos integrantes de incorporações ou loteamentos, até sua  alienação e, após esta, o saldo a receber, até o recebimento total do preço;   b) o saldo a receber do preço dos imóveis então alienados, até seu recebimento total.',
 'qual é o tratamento tributário do lucro auferido com a criação e venda de cães, passarinhos,  gatos etc.?    tratamento tributário depende da habitualidade ou não da prática dessa atividade, da forma seguinte:   a) se for exercida de forma eventual, isto é, se configurar prática comercial esporádica, o lucro auferido na  venda desses animais é tributado como ganho de capital da pessoa física;   b) se a atividade for exercida de forma habitual, isto é, se configurar prática comercial contínua, a pessoa física  é considerada empresa individual equiparada a pessoa jurídica, sendo seu lucro tributado nessa condição.',
 'como são tributados os rendimentos recebidos por escritor que assume os encargos da  publicação e venda de livros de sua autoria?   1 - se tais atividades não forem exercidas com habitualidade, os rendimentos decorrentes da venda efetuada  a pessoas físicas sujeitam-se ao recolhimento mensal obrigatório (carnê-leão) e na declaração. tratando-se  de venda a pessoa jurídica, sujeitam-se à retenção na fonte e na declaração.   2 - se houver habitualidade, o autor é considerado empresa individual equiparada a pessoa jurídica e seus  ganhos são tributados nessa condição. ',
 'como são tributados os rendimentos recebidos por pessoa física na venda de artesanato e  de antiguidades em local de atração turística?   por se tratar de venda habitual, a pessoa física é considerada empresa individual equiparada a pessoa jurídica,  sendo seus lucros tributados nessa condição.   ',
 'como são tributados os rendimentos da pessoa física que explora atividade econômica?    pessoa física que, em nome individual, explore, habitual e profissionalmente, qualquer atividade econômica  de natureza civil ou comercial com fim de lucro, mediante venda de bens e serviços, é considerada empresa  individual (firma individual) equiparada a pessoa jurídica.',
 'qual é o tratamento tributário dos rendimentos recebidos por dois ou mais profissionais, que  não constituem sociedade, mas utilizam um mesmo imóvel com despesas em comum?   tais profissionais, tendo despesas comuns, como aluguel, telefone, luz, auxiliares, mas com receitas  totalmente independentes, não perdem a condição de pessoas físicas, conforme entendimento expresso no  parecer normativo  nº 44, de 30 de junho de 1976. nesse caso, devem computar no rendimento bruto  mensal os honorários recebidos em seu nome.   as despesas comuns devem ser escrituradas em livro-caixa da seguinte forma:     aquele que tiver o comprovante da despesa em seu nome deve contabilizar o dispêndio pelo valor total  pago e fornecer aos demais profissionais um recibo mensal devidamente autenticado, correspondente ao  ressarcimento que cabe a cada um, escriturando como receita o valor total dos ressarcimentos recebidos;     os demais devem considerar como despesa mensal o valor do ressarcimento, constante do comprovante  recebido, que servirá como documento comprobatório do dispêndio. ',
 'quem está sujeito ao recolhimento mensal obrigatório (carnê-leão)?   sujeita-se ao recolhimento mensal obrigatório a pessoa física residente no brasil que receber:   1 - rendimentos de outras pessoas físicas que não tenham sido tributados na fonte no brasil, tais como  decorrentes de arrendamento, subarrendamento, locação e sublocação de móveis ou imóveis, e os  decorrentes do trabalho não assalariado, assim compreendidas todas as espécies de remuneração por  serviços ou trabalhos prestados sem vínculo empregatício;   2 - rendimentos ou quaisquer outros valores recebidos de fontes do exterior, tais como, trabalho assalariado  ou não assalariado, uso, exploração ou ocupação de bens móveis ou imóveis, transferidos ou não para o  brasil, lucros e dividendos. deve-se observar o disposto nos acordos, convenções e tratados internacionais  firmados entre o brasil e o país de origem dos rendimentos;   3 - emolumentos e custas dos serventuários da justiça, como tabeliães, notários, oficiais públicos e demais  servidores, independentemente de a fonte pagadora ser pessoa física ou jurídica, exceto quando forem  remunerados exclusivamente pelos cofres públicos;   4 - importâncias em dinheiro a título de pensão alimentícia, em face das normas do direito de família, quando  em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive alimentos provisionais;   5 - rendimentos recebidos por residentes no brasil que prestem serviços a embaixadas, repartições  consulares, missões diplomáticas ou técnicas ou a organismos internacionais de que o brasil faça parte;   6 - rendimento de transporte de carga e de serviços com trator, máquina de terraplenagem, colheitadeira e  assemelhados, considerando-se tributável, no mínimo, 10% do rendimento bruto, a partir de 1º de janeiro de  2013, conforme previsão contida no art. 18 da lei nº 12.794, de 2 de abril de 2013, que alterou o disposto no  inciso  do art. 9º da lei nº 7.713, de 22 de dezembro de 1988; e   7 - rendimento de transporte de passageiros, considerando-se tributável 60%, no mínimo, do rendimento bruto.   atenção:    os médicos, odontólogos, fonoaudiólogos, fisioterapeutas, terapeutas ocupacionais, advogados,  psicólogos, corretores e administradores de imóveis deverão informar na ficha rendimentos  tributáveis recebidos de pessoa física/exterior da declaração de ajuste anual   o número      120   de inscrição no cadastro de pessoas físicas   do titular do pagamento e do beneficiário dos  serviços por eles prestados, bem como o valor dos referidos serviços.   caso o contribuinte tenha preenchido o programa multiplataforma recolhimento mensal obrigatório  (carnê-leão) relativo ao imposto sobre a renda da pessoa física com esses dados, poderá  importá-los por meio da .   os rendimentos em moeda estrangeira devem ser convertidos em dólares dos estados unidos da  américa, pelo seu valor fixado pela autoridade monetária do país de origem dos rendimentos na  data do recebimento e, em seguida, em reais mediante utilização do valor do dólar fixado para  compra pelo banco central do brasil para o último dia útil da primeira quinzena do mês anterior ao  do recebimento do rendimento.   não se sujeitam ao carnê-leão os rendimentos tributados como ganho de capital (moeda  estrangeira) na forma da instrução normativa  nº 118, de 28 de dezembro de 2000.   os rendimentos sujeitos ao carnê-leão estão também sujeitos ao ajuste anual na declaração de  ajuste anual, e o imposto pago será considerado antecipação do apurado nessa declaração.   as importâncias descontadas em folha a título de pensão alimentícia em face das normas do direito  de família, quando em cumprimento de decisão judicial ou acordo homologado judicialmente,  inclusive a prestação de alimentos provisionais, não estão sujeitas à retenção na fonte, devendo o  beneficiário da pensão efetuar o recolhimento mensal obrigatório (carnê-leão), se for o caso. ',
 'como se calcula o recolhimento mensal obrigatório (carnê-leão)?    imposto relativo ao carnê-leão é calculado mediante a aplicação da tabela progressiva mensal, vigente no  mês do recebimento do rendimento, sobre o total recebido no mês, devendo ser recolhido até o último dia útil  do mês subsequente ao do recebimento do rendimento, com o código 0190.   na determinação da base de cálculo sujeita à incidência mensal do imposto, quando não utilizados para fins  de retenção na fonte, podem ser deduzidos, observados os limites e condições fixados na legislação  pertinente:    - as importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais, ou de escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de março de  2015 - código de processo civil;   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.    – a quantia de $ 189,59, por dependente, no ano-calendário de 2019;    - as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios,  cujo ônus tenha sido do próprio contribuinte e desde que destinado a seu próprio benefício;    - as despesas escrituradas em livro-caixa;    –  partir de 28 de agosto de 2009, até o exercício de 2014, ano-calendário de 2013, para fins de  implementação dos serviços de registros públicos, em meio eletrônico, os investimentos e demais gastos  efetuados com informatização, que compreende a aquisição de hardware, aquisição e desenvolvimento de  software e a instalação de redes pelos titulares dos referidos serviços, poderão ser deduzidos da base de      121   cálculo mensal e da anual do imposto sobre a renda da pessoa física, desde que escrituradas em livro-caixa,  e comprovadas com documentação hábil e idônea.    tabela progressiva mensal para fatos geradores ocorridos no ano-calendário de 2019, durante os meses de  janeiro a dezembro, é a seguinte:   base de cálculo ($) alíquota (%) parcela a deduzir do  ($)   até 1.903,98 - -   de 1.903,99 até 2.826,65 7,5 142,80   de 2.826,66 até 3.751,05 15,0 354,80   de 3.751,06 até 4.664,68 22,5 636,13   acima de 4.664,68 27,5 869,36    tributação incide sobre o valor total recebido no mês, independentemente de os valores unitários recebidos  serem inferiores ao limite mensal de isenção.   nos casos de contratos de arrendamento, subarrendamento, locação e sublocação entre pessoas físicas, com  preço e pagamento estipulados para períodos superiores a um mês ou com recebimento acumulado,  antecipado ou não, o rendimento é computado integralmente, para efeito de determinação do cálculo do  imposto, no mês do efetivo recebimento. se o bem produtor dos rendimentos for possuído em condomínio,  cada condômino deve considerar apenas o valor que lhe couber mensalmente para efeito de apurar a base  de cálculo sujeita à incidência da tabela progressiva mensal.   havendo mais de um recebimento no mês, ainda que abaixo do limite da primeira faixa da tabela, e locação  por período menor que um mês, somar-se-ão os rendimentos para apuração do imposto.   atenção:     imposto pago no país de origem dos rendimentos pode ser compensado no mês do pagamento  com o imposto relativo ao carnê-leão e com o apurado na declaração de ajuste anual, até o valor  correspondente à diferença entre o imposto calculado com a inclusão dos rendimentos de fontes  no exterior e o imposto calculado sem a inclusão desses rendimentos, observado os acordos,  tratados e convenções internacionais firmados pelo brasil ou a existência de reciprocidade de  tratamento;   se o pagamento do imposto no país de origem dos rendimentos ocorrer em ano-calendário posterior  ao do recebimento do rendimento, a pessoa física pode compensá-lo com o imposto relativo ao  carnê-leão do mês do seu efetivo pagamento e com o apurado na declaração de ajuste anual do  ano-calendário do pagamento do imposto, observado o limite de compensação de que trata o  parágrafo anterior relativamente à declaração de ajuste anual do ano-calendário do recebimento  do rendimento;   cas',
'qual é o tratamento tributário aplicável à gratificação natalina recebida de órgãos de governo  estrangeiro no país por residente no brasil?   tributa-se em conjunto com os demais rendimentos recebidos no mês pelo beneficiário, sujeitos ao  recolhimento mensal obrigatório (carnê-leão), no mês do recebimento e na declaração de ajuste anual. ',
 'qual é a diferença entre o recolhimento mensal obrigatório (carnê-leão) e o recolhimento  complementar?    pessoa física residente no país que recebe rendimentos de pessoa física ou do exterior, quando não  tributados na fonte no brasil, está obrigada a efetuar o recolhimento mensal (carnê-leão) do imposto sobre a  renda.  código para pagamento do imposto é 0190.  carnê-leão está sujeito a encargos em caso de  pagamento em atraso.    recolhimento complementar é um recolhimento facultativo que pode ser efetuado pelo contribuinte para  antecipar o pagamento do imposto devido na declaração de ajuste anual, no caso de recebimento de  rendimentos tributáveis de fontes pagadoras pessoa física e jurídica, ou de mais de uma pessoa jurídica, ou,  ainda, de apuração de resultado tributável da atividade rural. este recolhimento deve ser efetuado, no curso  do ano-calendário, até o último dia útil do mês de dezembro, sob o código 0246.   por ser facultativo, não há data de vencimento do imposto, sendo incabível multa por atraso no pagamento de  recolhimento complementar.    imposto complementar pode ser retido, mensalmente, por uma das fontes pagadoras, pessoa jurídica,  desde que haja concordância, por escrito, da pessoa física beneficiária, caso em que a pessoa jurídica é  solidariamente responsável com o contribuinte pelo pagamento do imposto correspondente à obrigação  assumida.   atenção:    os rendimentos recebidos decorrentes de ganho de capital e renda variável não estão sujeitos ao  recolhimento por meio do carnê-leão e do recolhimento complementar.  ',
 'como proceder quando o carnê-leão for pago a maior ou indevidamente?   poderá ser compensado o valor do principal pago indevidamente ou a maior em recolhimentos posteriores de  carnê-leão, dentro do mesmo ano-calendário, ou, ainda, na declaração de ajuste anual  .    restituição do indébito poderá ser requerida pela pessoa física à  exclusivamente mediante a  apresentação da . ',
 'os acréscimos legais incidentes no carnê-leão recolhido fora do prazo podem ser  compensados na declaração?   não é permitida a compensação de acréscimos legais pagos por recolhimento em atraso de imposto devido. ',
 'contribuinte sujeito ao carnê-leão, que opte pelo recolhimento complementar do imposto,  pode efetuar o pagamento sob o mesmo código?   não.  carnê-leão, sendo obrigatório e sujeito a encargos, deve ser recolhido sob o código 0190. já o  recolhimento complementar, sendo opcional, deve ser efetuado em outro darf, sob o código 0246 e dentro do  próprio ano-calendário de referência. ',
 'valor do sinal ou arras recebido em virtude de rescisão de contrato é tributável?   "as arras ou sinal constituem a importância em dinheiro ou a coisa dada por um contratante ao outro, com o  escopo de firmar a presunção de acordo final e tornar obrigatório o ajuste; ou ainda, excepcionalmente, com  o propósito de assegurar, para cada um dos contratantes, o direito de arrependimento." (enciclopédia saraiva  do direito, , saraiva, 1978, . 8, p. 19).   no caso de desistência do negócio por parte do promissário comprador, este perde o sinal dado que está  sujeito ao recolhimento mensal obrigatório (carnê-leão), se recebido de pessoa física, ou à retenção na fonte,  se pagos por pessoa jurídica e, em ambos os casos, também ao ajuste na declaração anual, por parte do  promitente vendedor.   no caso de arrependimento por parte do promitente vendedor, este deve restituir ao promissário comprador o  sinal recebido, mais o equivalente.  valor correspondente ao sinal original não sofre incidência de imposto  sobre a renda, porém o valor recebido a maior (inclusive correção monetária e juros) é tributado no carnê- leão.   exemplo:    promissário comprador  pagou a , promitente vendedor, a importância de $ 20.000,00 como sinal pela  aquisição de imóvel. , promissário comprador, arrependeu-se do negócio antes da escritura definitiva,  perdendo para  o sinal pago.     promitente vendedor  deve tributar o valor do sinal recebido ($ 20.000,00) como carnê-leão. tomando- se por base o exemplo acima, considere que a desistência do negócio tenha sido feita pelo promitente  vendedor, .    nesse novo exemplo, em atendimento aos termos do art. 418 do código civil,  deve devolver o sinal recebido  de  mais o equivalente, isto é, $ 40.000,00.  valor relativo ao sinal devolvido ($ 20.000,00) constitui para  , rendimento não tributável. os restantes $ 20.000,00 devem ser tributados como carnê-leão. ',
 'pode ser considerada como rendimento não tributável a parcela de remuneração de  assalariado, que é denominada "indenização", por lei estadual ou municipal?   não.  lei nº 7.713, de 22 de dezembro de 1988, que reformulou o imposto sobre a renda da pessoa física,  em seu art. 3º, declara que "constituem rendimento bruto todo o produto do capital, do trabalho ou da  combinação de ambos, os alimentos e pensões percebidos em dinheiro e demais proventos de qualquer  natureza, assim também entendidos os acréscimos patrimoniais não correspondentes aos rendimentos  declarados”.  art. 6º da citada lei, com alterações posteriores, declara quais rendimentos estão isentos do  imposto sobre a renda, neles não estando incluída tal indenização.   como a base de cálculo do imposto só pode ser fixada por meio de lei emanada do poder competente  (constituição federal, promulgada em 1988, art. 153, inciso ; e lei nº 5.172, de 25 de outubro de 1966 –  código tributário nacional – , art. 97, inciso ), entende-se que qualquer outra lei que não seja federal  não pode instituir ou alterar a base de cálculo do imposto sobre a renda.   ademais, de acordo com o art. 4º combinado com o art. 109, ambos do , são irrelevantes para qualificar  a natureza jurídica específica do tributo a denominação e as demais características formais adotadas pela lei  comum.   assim, os referidos rendimentos estão sujeitos à incidência do imposto sobre a renda, tais como os recebidos  pelos magistrados, militares, prefeitos, vereadores, deputados, senadores. ',
 'são isentas do imposto sobre a renda a indenização paga por despedida ou rescisão do  contrato de trabalho, e as importâncias recebidas pelos empregados e seus dependentes nos limites  e termos da legislação do fundo de garantia do tempo de serviço   e o aviso prévio?    indenização paga por despedida ou rescisão de contrato de trabalho, até o limite garantido pela lei trabalhista  ou por dissídio coletivo e convenções trabalhistas homologados pela justiça do trabalho, bem como o  montante recebido pelos empregados e diretores e seus dependentes ou sucessores, referente aos depósitos,  juros e correção monetária creditados em contas vinculadas, nos termos da legislação do fundo de garantia  do tempo de serviço  , são isentos do imposto sobre a renda.   enquadram-se nesse conceito a indenização do tempo de serviço anterior à opção pelo , nos limites  fixados na legislação trabalhista, quer seja ela percebida pelo próprio empregado ou por seus dependentes  após o falecimento do assalariado.    que exceder às verbas acima descritas será considerado liberalidade do empregador e tributado como  rendimento do trabalho assalariado.   quanto ao aviso prévio, apenas o não trabalhado é isento. ',
 'como são tributados os lucros e dividendos apurados na escrituração comercial em 1993 e  distribuídos aos sócios em 2019 por pessoas jurídicas tributadas com base no lucro real e  presumido?   os rendimentos são isentos. no caso de pessoa jurídica tributada com base no lucro presumido, a isenção  corresponde ao valor distribuído até o limite da base de cálculo do imposto deduzido do imposto sobre a  renda da pessoa jurídica  .   atenção:    considera-se como custo, no caso de quotas e ações recebidas em bonificação por incorporação  de reservas ou lucros, a parcela correspondente ao lucro ou reserva capitalizado. ',
 'valor total recebido a título de pensão e de proventos de aposentadoria, reserva remunerada  ou reforma, por contribuinte maior de 65 anos é isento do imposto sobre a renda?   não. somente estão isentos a pensão e os proventos da inatividade pagos pela previdência social da união,  dos estados, do distrito federal e dos municípios, por pessoa jurídica de direito público interno ou por entidade  de previdência complementar, a partir do mês em que o pensionista ou inativo completar 65 anos de idade,  até o valor de $ 1.903,98 por mês, durante o ano-calendário de 2019, sem prejuízo da parcela isenta prevista  na tabela de incidência mensal do imposto.    valor excedente a esse limite está sujeito à incidência do imposto sobre a renda na fonte e na declaração.   os demais rendimentos recebidos pela pessoa física, inclusive aluguéis, estão sujeitos à tributação pelo   imposto sobre a renda da pessoa física. ',
 'como deve proceder a pessoa física com 65 anos ou mais que recebe proventos de  aposentadoria ou pensão de mais de um órgão público ou previdenciário?   em relação aos rendimentos tributáveis na declaração de ajuste anual o contribuinte deve observar que:   1 – do valor mensal correspondente à soma dos proventos de aposentadoria ou pensão pagos por todas as  fontes pagadoras, somente é considerada isenta a parcela de $ 1.903,98;   2 - na declaração de ajuste anual, somente deve ser informada como rendimento isento a soma dos valores  mensais isentos mencionados no item 1;   3 - compõe os rendimentos tributáveis na declaração de ajuste a diferença positiva entre o total dos proventos  de aposentadoria ou pensão recebidos no ano-calendário e o valor mencionado no item 2. ',
 'qual é a tributação do 13º salário, recebido a título de pensão e de proventos de  aposentadoria, reserva remunerada ou reforma, por contribuinte maior de 65 anos?   é tributada exclusivamente na fonte a gratificação natalina (13º salário) relativa a aposentadoria e pensão,  transferência para a reserva remunerada ou reforma paga pela previdência social da união, dos estados, do  distrito federal e dos municípios, por qualquer pessoa jurídica de direito público interno ou por entidades de  previdência complementar, no caso de contribuinte com idade igual ou superior a 65 anos.    gratificação natalina (13º salário) deve ser integralmente tributada no mês da sua quitação, com base na  tabela progressiva do mês de dezembro, permitidas as seguintes deduções:    - as importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial ou acordo homologado judicialmente, inclusive a prestação de  alimentos provisionais;   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.    – o valor de $ 189,59, por dependente, para os meses de janeiro a dezembro do ano-calendário de 2019;    - as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios,  cujo ônus tenha sido do próprio contribuinte e desde que destinadas a seu próprio benefício;    - as contribuições para as entidades de previdência complementar domiciliadas no brasil, destinadas a  custear benefícios complementares assemelhados aos da previdência social e para os fundos de  aposentadoria programada individual (fapi), cujo ônus tenha sido do próprio contribuinte e desde que  destinadas a seu próprio benefício;     - o valor de $ 1.903,98, se a gratificação natalina tiver sido quitada no ano-calendário de 2019. ',
 'pensionista ou aposentado pela previdência oficial ou complementar, maior de 65 anos,  dependente do declarante, perde direito à isenção de idade por ser dependente?   não.  fato de o pensionista ou aposentado ser incluído como dependente não modifica a natureza do  rendimento, devendo, nesse caso, o declarante incluir os rendimentos recebidos a esse título.',
 'valor inferior à parcela isenta de rendimentos de aposentadoria de maior de 65 anos recebida  em determinado mês, pode ser compensada com valor superior à parcela isenta recebida em outro  mês?   não. caso em um determinado mês o contribuinte maior de 65 anos tenha recebido valor inferior à parcela  isenta e em outro mês valor superior, não pode compensar os valores recebidos para se beneficiar na  declaração de ajuste anual, pois o limite de isenção, por mês, no ano-calendário de 2019 é de $ 1.903,98  em cada mês do mesmo ano. ',
 'qual é o tratamento tributário em relação aos rendimentos recebidos pelas pessoas físicas,  quando provenientes de trabalhos executados por ocasião da realização dos eventos da copa das  confederações 2013 e copa do mundo de 2014?   até 31/12/2015, estavam isentos do imposto sobre a renda:    - os rendimentos pagos, creditados, empregados, entregues ou remetidos pela fifa, sua subsidiária no brasil,  confederações fifa, associações estrangeiras membros da fifa, emissora fonte da fifa, prestadores de  serviços fifa, para pessoas físicas, não residentes no país, empregadas ou de outra forma contratadas para  trabalhar de forma pessoal e direta na organização ou realização dos eventos, que ingressarem no país com  visto temporário.   estas isenções são aplicáveis também aos árbitros, jogadores de futebol e outros membros das delegações,  exclusivamente no que concerne ao pagamento de prêmios relacionados aos eventos, quando efetuados  pelas pessoas jurídicas que usufruem desta isenção.      127    - os valores dos benefícios indiretos e o reembolso de despesas recebidos por voluntário da fifa, da  subsidiária fifa no brasil ou do  (comitê organizador brasileiro), que auxiliar na organização e realização  dos eventos, até o valor de 5 (cinco) salários mínimos por mês, sem prejuízo da aplicação da tabela de  incidência mensal do imposto sobre a renda sobre o valor excedente.     voluntário que receber valor abaixo do limite de isenção em determinado mês não poderá aproveitar a  diferença em meses subsequentes.   nos casos de recebimento de dois ou mais pagamentos em um mesmo mês, a parcela isenta deve ser  considerada em relação à soma desses pagamentos.   não caracteriza residência no país a permanência no brasil durante o período dos eventos, salvo no caso de  obtenção de visto permanente ou de vínculo empregatício com pessoa jurídica distinta da fifa e dos demais  entes referidos acima.   atenção:    sem prejuízo dos acordos, tratados e convenções internacionais firmados pelo brasil, os demais  rendimentos recebidos de fonte no brasil, inclusive o ganho de capital na alienação de bens e  direitos situados no país, pelas pessoas físicas não residentes, são tributados de acordo com  normas específicas aplicáveis aos não residentes no brasil.    tratamento diferenciado por disposição da lei nº 12.350, de 20 de dezembro de 2010, tem  vigência de 1º/01/2011 a 31/12/2015, observado o art. 62, daquela lei.',
 'isenção do imposto sobre a renda dos proventos de aposentadoria, reforma e pensão,  recebidos por pessoas com doença grave, está condicionada à comprovação?   são rendimentos isentos os relativos a aposentadoria, reforma ou pensão (inclusive complementações)  recebidos por pessoas com tuberculose ativa, alienação mental, esclerose múltipla, neoplasia maligna,  cegueira (inclusive monocular), hanseníase, paralisia irreversível e incapacitante, cardiopatia grave, doença  de parkinson, espondiloartrose anquilosante, nefropatia grave, estados avançados da doença de paget  (osteíte deformante), contaminação por radiação, síndrome da imunodeficiência adquirida (aids), hepatopatia  grave e fibrose cística (mucoviscidose).   1) uma vez comprovada a “alienação mental”, a pessoa física portadora do mal de alzheimer faz jus à isenção  do imposto sobre a renda incidente sobre os proventos de aposentadoria e pensão;   2) em relação às pessoas com deficiência física conhecida como "síndrome da talidomida":   a) a partir de 24 de junho de 2008, são isentos do imposto sobre a renda a pensão especial, mensal,  vitalícia e intransferível, e outros valores recebidos em decorrência daquela deficiência física (lei nº 7.070,  de 20 de dezembro de 1982, art. 4º-, com a redação dada pelo art. 20 da lei nº 11.727, de 23 de junho  de 2008);   b) a partir de 1º de janeiro de 2010, não incidirá imposto sobre a renda sobre a indenização por dano moral,  nos termos previstos na lei nº 12.190, de 13 de janeiro de 2010, arts. 1º e 2º.   para efeito de reconhecimento de isenção, a doença deve ser comprovada mediante laudo pericial emitido  por serviço médico oficial da união, dos estados, do distrito federal e dos municípios.   sobre laudo pericial consultar as perguntas 220 e 221  ',
 'são tributáveis os rendimentos de aposentadoria, reforma ou pensão recebidos  acumuladamente por beneficiário com doença grave?   os rendimentos provenientes de aposentadoria, reforma ou pensão, recebidos por pessoa com doença grave,  ainda que acumuladamente, não sofrem tributação, por força do disposto na lei nº 7.713, de 22 de dezembro  de 1988, art. 6º, incisos  e , que isenta os referidos rendimentos recebidos por pessoa com doença  grave.     isenção aplica-se também aos rendimentos de aposentadoria, reforma ou pensão inclusive os recebidos  acumuladamente correspondentes a período anterior à data em que foi contraída a moléstia grave,  reconhecida mediante laudo pericial, desde que sejam provenientes de aposentadoria, reforma ou pensão e  sejam percebidos a partir:     do mês da concessão da pensão, aposentadoria ou reforma, se a doença for preexistente ou a  aposentadoria ou reforma for por ela motivada;     do mês da emissão do laudo pericial que reconhecer a doença, se contraída após a aposentadoria,  reforma ou concessão da pensão;     da data em que a doença for contraída, quando identificada no laudo pericial emitido posteriormente  à concessão da pensão, aposentadoria ou reforma.    comprovação deve ser feita mediante laudo pericial emitido por serviço médico oficial da união, dos estados,  do distrito federal ou dos municípios.   para informações sobre laudo pericial.',
 'qual é o tratamento tributário da complementação de aposentadoria, reforma ou pensão paga  a pessoa com doença grave?   é isenta do imposto sobre a renda a complementação de aposentadoria, reforma ou pensão, recebida de  entidade de previdência complementar, fundo de aposentadoria programada individual (fapi) ou programa  gerador de benefício livre  .    os valores recebidos a título de resgate de entidade de previdência complementar, fapi ou , que só  poderá ocorrer enquanto não cumpridas as condições contratuais para o recebimento do benefício, por não  configurar complemento de aposentadoria, estão sujeitos à incidência do imposto sobre a renda, ainda que  efetuado por pessoa com moléstia grave.   no transcurso do pagamento do benefício inexiste a possibilidade da ocorrência de resgate, nos termos  previstos nas normas previdenciárias em vigor.    isenção também não se aplica aos valores recebidos a título de pensão, inclusive complementações, quando  o beneficiário do rendimento for pessoa com moléstia profissional.    para informações sobre laudo pericial consultar as perguntas 220 e 221   atenção:  para que a complementação de aposentadoria, reforma ou pensão, recebida de entidade de  previdência complementar, fundo de aposentadoria programada individual (fapi) ou programa  gerador de benefício livre   seja isenta é necessário estar aposentado pela previdência  oficial.  ',
 'é tributável a pensão alimentícia judicial ou por escritura pública recebida por pessoa com  doença grave?   não. os valores recebidos a título de pensão em cumprimento de acordo ou decisão judicial, ou ainda por  escritura pública, inclusive a prestação de alimentos provisionais, estão abrangidos pela isenção de pessoas  com moléstia grave. ',
 'são isentos do imposto sobre a renda os rendimentos recebidos a título de bolsa de estudos?   sim, desde que caracterize doação, ou seja, quando recebidos exclusivamente para proceder a estudo ou  pesquisa e o resultado dessas atividades não represente vantagem para o doador e não caracterize  contraprestação de serviços.   os rendimentos isentos recebidos a título de bolsa de estudos não justificam acréscimo patrimonial.   atenção:  não caracterizam contraprestação de serviços nem vantagem para o doador, para efeito desta  isenção, as bolsas de estudo recebidas pelos médicos residentes, nem as bolsas recebidas pelos  servidores das redes públicas de educação profissional, científica e tecnológica que participem das  atividades do pronatec. também recebem o mesmo tratamento os rendimentos relativos às bolsas  de estímulo à inovação de que trata a lei nº 10.973, de 2 de dezembro de 2004.',
 'qual é o tratamento tributário aplicável ao programa de demissão voluntária   recebido  por empregado já aposentado pela previdência oficial ou que possua o tempo necessário para  requerer a aposentadoria?   as verbas especiais indenizatórias recebidas a título de adesão a  não se sujeitam à incidência do imposto  sobre a renda na fonte nem na declaração de ajuste anual, independentemente de o funcionário já estar  aposentado pela previdência oficial, ou possuir o tempo necessário para requerer a aposentadoria pela  previdência oficial ou complementar.',
 'qual é o tratamento tributário aplicável à indenização paga aos beneficiários de  desaparecidos políticos?   esta indenização é não tributável quando paga aos beneficiários diretos.   atenção:  os valores pagos a anistiados políticos a título de indenização, em prestação única ou em prestação  mensal, permanente e continuada, inclusive aposentadorias, pensões ou proventos de qualquer  natureza, são isentos do imposto sobre a renda nos termos do disposto no parágrafo único do art.  9º da lei nº 10.559, de 13 de novembro de 2002, e decreto nº 4.897, de 25 de novembro de 2003. ',
 'indenização de transporte paga a servidor público da união é tributável?   não.  valor da indenização de transporte a que se referem os arts. 60 da lei nº 8.112, de 11 de dezembro  de 1990, e 1º, inciso , alínea "b", da lei nº 8.852, de 4 de fevereiro de 1994, é rendimento isento.  ',
 'que se compreende no conceito de "diárias" e de "ajuda de custo", para fins de isenção do  imposto sobre a renda?   diárias: conceituam-se diárias, para esse efeito, os valores pagos em caráter acidental e transitório, embora  possam estender-se por um mês ou mais, bem como ocorrer em vários meses do ano, destinados a cobrir,  exclusivamente, despesas de alimentação e pousada, em virtude de deslocamento de empregado, funcionário  ou diretor, para município diferente de sua sede profissional, no desempenho de seu emprego, cargo ou  função, para efetuar serviço eventual por conta do empregador.   como as diárias não estão sujeitas a qualquer tipo de acerto quando do retorno do deslocamento, e para  prevenir a hipótese de se tornarem um instrumento de complementação salarial, desviando-se do seu conceito  legal de reembolso de despesas de alimentação e pousada, exclusivamente, além das regras acima  mencionadas, é necessário, para fins de isenção do imposto sobre a renda, que:   a) os valores pagos a esse título guardem critérios de razoabilidade, não só em relação aos preços vigentes  na localidade para a qual se deslocará o servidor, como também em razão da importância que este ocupar na  hierarquia da empresa ou órgão concedente;   b) as diárias não visem indenizar gastos com pessoas sem vínculo com o empregador, como é o caso de  esposa e filhos do empregado, funcionário ou diretor;   c) correspondam a despesas de alimentação, pousada e correlatas no local da prestação do serviço eventual  e temporário; e   d) a qualquer momento, possam ser comprovadas mediante apresentação do bilhete de passagem ou nota  fiscal de serviço e o recibo do estabelecimento hoteleiro, no qual constem o nome do servidor, o efetivo  deslocamento deste, bem como os valores desembolsados pelo empregador.   atenção:    os adiantamentos de recursos para atender às despesas de viagens e estadas, quando sujeitos a  posterior prestação de contas, não se enquadram como diárias; entretanto, não compõem o  rendimento bruto do servidor, desde que devidamente comprovados, o deslocamento e as  despesas efetuadas, conforme acima exposto.    indenização para execução de trabalhos de campo não guarda relação alguma com os institutos  da diária ou ajuda de custo, portanto sujeita-se à tributação pelo imposto sobre a renda na fonte e  na declaração de ajuste anual.   ajuda de custo: conceituam-se ajuda de custo, para fins do disposto no art. 6º, inciso , da lei nº 7.713, de  22 de dezembro de 1988, os valores pagos em caráter indenizatório, destinados a ressarcir os gastos com  transporte, frete e locomoção do beneficiado e seus familiares, em caso de remoção de um município para  outro ou para o exterior.    efetiva remoção está sujeita à comprovação posterior pelo beneficiário, a qualquer momento, por meio de  documentos emitidos pelo empregador.  ',
 'são tributáveis as quotas para uso de serviços postais e telefônicos, bem como as passagens  aéreas atribuídas aos parlamentares no exercício do mandato?   as quotas relativas a direito de uso de serviços postais e telefônicos, bem como a passagens aéreas atribuídas  aos parlamentares no exercício do mandato, nos limites fixados pelo órgão competente, não se sujeitam à  tributação pelo imposto sobre a renda na fonte e na declaração de ajuste anual.   na hipótese de conversão em dinheiro das referidas quotas, os valores recebidos integram o rendimento  tributável do beneficiário. ',
 'estão isentos os valores pagos em espécie pelos estados, distrito federal e municípios,  relativos ao imposto sobre operações relativas à circulação de mercadorias e sobre prestações de  serviços de transporte interestadual e intermunicipal e de comunicação   e ao imposto sobre  serviços de qualquer natureza  , no âmbito de programas de concessão de crédito voltados ao  estímulo à solicitação de documento fiscal (cupons e notas fiscais) na aquisição de mercadorias e  serviços pelo consumidor?   sim, os referidos valores são isentos. entretanto, a mencionada isenção não se aplica aos prêmios recebidos  por meio de sorteios, no âmbito dos referidos programas, os quais quando distribuídos em dinheiro são  tributados exclusivamente na fonte à alíquota de 30%, e quando sob a forma de bens e serviços são tributados  exclusivamente na fonte à alíquota de 20%.   ',
 'os rendimentos correspondentes a indenizações reparatórias em decorrência de ato ilícito  são tributáveis?   os prejuízos físicos ou materiais, em consequência de ato ilícito praticado por terceiros, são indenizáveis na  forma da lei civil. essas indenizações têm por finalidade repor o patrimônio danificado ou destruído, bem como  substituir os rendimentos não percebidos em decorrência da perda do bem, de invalidez temporária,  permanente ou de morte.   as indenizações por ato ilícito podem ser:   1 - indenizações por bem material danificado ou destruído, denominadas "danos emergentes". são valores  que visam exclusivamente a repor o bem destruído ou a reparar o bem danificado, até o limite fixado em  condenação judicial. não sofrem incidência do imposto sobre a renda;   2 - indenização reparatória por invalidez ou morte — o pagamento dessa indenização pode ocorrer das  seguintes maneiras:   a) quantia paga periodicamente, cujo total é indeterminável previamente (desconhecido o termo final da  obrigação), caracteriza-se como pensão civil por ato ilícito, também denominada "lucros cessantes". sob essa  designação, o empregado postula os salários que deixa de perceber; o profissional liberal, os honorários; a  pessoa jurídica, os lucros; o locador, o aluguel; o aplicador, os rendimentos do título (correção monetária,  deságios, juros e outros). tem por finalidade substituir os rendimentos que a vítima deixou de perceber em  razão da invalidez ou morte. tais valores devem ser oferecidos à tributação, no mês do seu recebimento e na  declaração. podem ser deduzidas as despesas judiciais ou extrajudiciais suportadas pelo contribuinte ou por  seu beneficiário para a obtenção dos rendimentos pagos acumuladamente, desde que não ressarcidas;   b) quantia certa paga de uma vez ou dividida em um número certo de parcelas — referindo-se ao  ressarcimento dos danos anteriormente causados e guardando com eles equivalência — caracteriza-se como  indenização. esses valores não sofrem incidência do imposto sobre a renda.   na hipótese do item 2, as quantias recebidas para cobrir despesas médico-hospitalares necessárias ao  restabelecimento da vítima, inclusive próteses de qualquer espécie, estão fora do campo de incidência do  imposto sobre a renda.  ',
 'indenização percebida pelo locador, em decorrência de danos causados no imóvel locado,  é tributável?   não. esta indenização, destinada exclusivamente aos reparos necessários e indispensáveis à recuperação  do imóvel locado, não constitui rendimento tributável.   ',
 'qual é o tratamento tributário a que está sujeito o valor do saldo devedor correspondente a  financiamento para aquisição de casa própria, quitado em virtude de invalidez permanente ou  falecimento do mutuário?    valor correspondente ao saldo devedor quitado por motivo de invalidez permanente ou morte do mutuário  não se sujeita à tributação pelo imposto sobre a renda.',
 'valor recebido em restituição do imposto sobre a renda é tributável?   esse valor não se caracteriza como rendimento tributável, devendo ser informado como rendimento não  tributável na declaração. ',
 'valor do desconto obtido pelo resgate antecipado de notas promissórias é tributável?   esse valor não implica aquisição de disponibilidade econômica ou jurídica para a pessoa física beneficiária.  trata-se, em última análise, de simples redução do preço anteriormente contratado, sem corresponder,  portanto, a um acréscimo patrimonial. assim, o valor do desconto não constitui rendimento e, por conseguinte,  não está sujeito à incidência do imposto sobre a renda. caso se caracterize perdão de dívida, em troca de  serviços prestados, tal importância constitui rendimento tributável.',
 'qual é o tratamento tributário dado ao valor recebido em dinheiro pelo consorciado quando  da falta do bem no mercado?   na declaração de bens e direitos, no código 99, deve ser informada, no campo “discriminação”, essa  circunstância, a soma das parcelas pagas em 2019, e o valor recebido em dinheiro pela falta do bem.    não deve ser preenchida o campo “situação em 31/12/2019 ($)”.   no campo “situação em 31/12/2018 ($)”, o contribuinte deve informar o valor das parcelas pagas até 31 de  dezembro de 2018.   nos anos seguintes, as parcelas pagas no ano serão informadas no campo “discriminação”.    diferença entre o valor recebido em dinheiro e o total das parcelas pagas deve ser informada como  rendimento não tributável.',
 'é tributável a transferência de reserva de contribuições previdenciárias entre entidades de  previdência complementar?   não se configura fato gerador do imposto sobre a renda a transferência direta de reservas entre entidades de  previdência complementar, desde que não haja mudança de titularidade e que os recursos correspondentes,  em nenhuma hipótese, sejam disponibilizados para o participante ou para o beneficiário do plano.  ',
 'é tributável a reversão de valores realizada por entidade de previdência complementar  fechada em favor de participante pessoa física?   considera-se reversão de valores a devolução aos participantes dos resultados positivos (superávit) obtidos  pela entidade fechada de previdência complementar mediante a aplicação das contribuições mensais  aportadas.    reversão de valores não se confunde com o benefício previdenciário complementar tampouco se assemelha  aos resgates de contribuições, guardando natureza própria que lhe sujeita à incidência do imposto sobre a  renda na fonte e na declaração de ajuste anual por ser enquadrada como rendimento pago por pessoa jurídica  a pessoa física. ',
 'condomínio edilício deve efetuar a retenção sobre os pagamentos efetuados a  empregados?   sim. embora não se caracterize como pessoa jurídica, o condomínio é responsável pela retenção e  recolhimento do imposto sobre a renda incidente na fonte, quando se enquadrar como empregador, em face  da legislação trabalhista e previdenciária, devendo reter o imposto sobre os rendimentos pagos aos seus  empregados.  ',
 'multa por rescisão de contrato de aluguel de imóvel está sujeita ao imposto sobre a renda  incidente na fonte?  multa por rescisão de contrato de aluguel é considerada rendimento de aluguel, consequentemente é  tributada como tal.   assim, temos as seguintes situações:   a) se a multa for paga por pessoa jurídica, esse valor é tributável na fonte e na declaração;   b) se a multa for paga por pessoa física, esse valor sujeita-se ao recolhimento mensal obrigatório (carnê-leão)  e à tributação na declaração.   para aluguéis recebidos por não residente, consulte a pergunta 192. ',
 'estão sujeitos à retenção na fonte os benefícios pagos por pessoa jurídica a seus empregados  ou dependentes destes, por conta e ordem de terceiros?   sim. os pagamentos efetuados por conta e ordem de terceiros, em razão de convênio ou contrato, estão  sujeitos à incidência do imposto sobre a renda incidente na fonte, desde que não expressamente isentos.  esses rendimentos devem ser tributados em separado dos demais pagamentos efetuados pela pessoa  jurídica, salvo se houver a anuência do beneficiário do rendimento para a tributação na fonte em conjunto. ',
 'como é tributada a distribuição de vale-brinde?    vale-brinde constitui uma modalidade de distribuição de prêmio que consiste na colocação de pequenos  impressos (vales), dentro de determinado produto ou em seu envoltório, numerados em ordem crescente, a  partir de 1 (um), para distribuição gratuita de prêmios, como meio de propaganda.    vale-brinde difere do sorteio e do concurso, também modalidades de distribuição gratuitas de prêmios, que  têm as seguintes definições:   sorteio - modalidade onde são confeccionados cupons numerados que são entregues pelas empresas aos  clientes que, conforme os resultados de extração de alguma das loterias de números do governo ou de  qualquer extração permitida pela legislação, alguém é sorteado e contemplado com um prêmio, encerrando,  assim, a promoção;   concurso - modalidade em que se visa premiar as habilidades dos participantes, tais como a inteligência, a  memória, a destreza esportiva etc., ou os predicados, como a beleza, a elegância etc.   como o art. 63 da lei nº 8.981, de 20 de janeiro de 1995, abrange somente a distribuição de prêmios por  sorteio e por concurso, depreende-se que seu conteúdo não alcança a distribuição por vale-brinde,  modalidade distinta daquelas, não sofrendo, portanto, tributação. ',
 'é tributável a remessa para o exterior de valor para cobrir despesa de manutenção de cônjuge  e filhos que lá se encontrem?   não ocorre a incidência de imposto sobre a renda nas remessas, independentemente do seu  valor, efetuadas  para cobertura de despesas com a manutenção de cônjuge e filhos no exterior, desde que realizadas em  nome deles, por meio de entidades autorizadas e de acordo com os mecanismos regulares estabelecidos pelo  banco central do brasil. ',
 'são tributáveis os valores pagos, creditados, entregues, empregados ou remetidos para o  exterior, destinados à cobertura de gastos pessoais, no exterior, de pessoas físicas residentes no  país, em viagens?   sim, exceto se destinados a fins educacionais, científicos ou culturais (consulte a pergunta 292).    até 31 de dezembro de 2019, está reduzida a 6% (seis por cento) a alíquota do  sobre valores pagos,  creditados, entregues, empregados ou remetidos para pessoa física ou jurídica residente ou domiciliada no  exterior, destinados à cobertura de gastos pessoais, no exterior, de pessoas físicas residentes no brasil, em  viagens de turismo, negócios, serviço, treinamento ou missões oficiais, até o limite global de $ 20.000,00  (vinte mil reais) ao mês.       são gastos pessoais no exterior, para efeito da redução de alíquota do , as despesas para manutenção  do viajante, tais como despesas com hotéis, transporte, hospedagem, cruzeiros marítimos, aluguel de  automóveis e seguro a viajantes.   atenção:    redução de alíquota somente se aplica às despesas com viagens internacionais de pessoas  físicas residentes no brasil.     redução de alíquota aplica-se às remessas efetuadas por pessoa jurídica, domiciliada no país,  que arque com despesas pessoais de seus empregados e dirigentes residentes no país, registrados  em carteira de trabalho.    as remessas feitas por operadoras e agências de viagem não estão sujeitas ao limite global de  $ 20.000,00 (vinte mil reais).    no caso de beneficiário residente ou domiciliado em país ou dependência com tributação favorecida  ou beneficiado por regime fiscal privilegiado, conforme listas contidas na instrução normativa   nº 1.037, de 4 de junho de 2010, a alíquota aplicável é de 25% (vinte e cinco por cento). contudo,  tais remessas sujeitam-se à alíquota de 6% (seis por cento) até o limite de $ 10.000,00 (dez mil  reais) ao mês por viajante, desde que haja, cumulativamente:     – a identificação do efetivo beneficiário da entidade no exterior, destinatário dos valores pagos,  creditados, entregues empregados ou remetidos;     – a comprovação da capacidade operacional da pessoa física ou entidade no exterior de realizar  a operação; e    – a comprovação documental do pagamento do preço respectivo e do recebimento dos bens e  direitos ou da utilização de serviço.   novas alíquotas a partir de 2020:    medida provisória nº 907, de 26 de novembro de 2019, em seu art. 3º, deu nova redação ao art.  60 da lei nº 12.249, de 11 de junho de 2010, estabelecendo as seguintes alíquotas para remessas  destinada à cobertura de gastos pessoais a partir de 2020:   - 2020: 7,9%   - 2021: 9,8%   - 2022: 11,7%   - 2023: 13,6%   - 2024: 15,5%  ',
 'são tributáveis os valores pagos, creditados, empregados, entregues ou remetidos para o  exterior para fins educacionais, científicos ou culturais?   não, desde que sejam desprovidos de finalidade econômica, destinando-se à manutenção de pessoa física  que esteja cumprindo programa ou participando de evento no exterior de natureza educacional, científica ou  cultural, tais como para pagamento de taxas escolares e de exames de proficiência, material didático,  alojamento, alimentação e outras despesas cobradas por instituições de ensino destinadas à manutenção de  estudantes, taxas de inscrição em congressos, conclaves, seminários ou assemelhados, mesas redondas e  taxas de inscrição em concursos artísticos. ',
 'é tributável a remessa para o exterior de valor para aquisição de publicações?   não. as importâncias remetidas para aquisição de publicações impressas, de qualquer natureza, inclusive por  meio de assinatura, não se sujeitam ao imposto sobre a renda retido na fonte.',
 'são tributáveis os valores pagos, creditados, empregados, entregues ou remetidos para o  exterior, para cobertura de despesas médico-hospitalares com tratamento de saúde, no exterior, do  remetente ou de seus dependentes?   não. as remessas efetuadas por pessoas físicas residentes no país para cobertura de despesas médico- hospitalares com tratamento de saúde, no exterior, do remetente ou de seus dependentes não estão sujeitas  à retenção na fonte do imposto de renda.',
 'é tributável a remessa ao exterior efetuada a não residente a título de doação?   sim. os valores remetidos a título de doação a residente ou domiciliado no exterior, pessoa física ou jurídica,  sujeitam-se à incidência do , à alíquota de 15% (quinze por cento), ou de 25% (vinte e cinco por cento),  na hipótese de o beneficiário ser residente ou domiciliado em país ou dependência com tributação favorecida,  conforme lista contida no art. 1º da instrução normativa  nº 1.037, de 4 de junho de 2010.',
 'são tributáveis os pagamentos de rendimentos para o exterior efetuados por não residente  no brasil?   sim.  pessoa física não residente no brasil que efetue remessa de rendimentos para o exterior é considerada  fonte pagadora situada no brasil, e por isso é sujeito passivo responsável pela retenção do , e não  contribuinte.  incidência e alíquota de  dependerá das regras de tributação aplicáveis ao beneficiário da  remessa e à natureza do pagamento, inclusive considerando-se eventual isenção.  ',
 'como declarar o prêmio distribuído sob a forma de bens ou serviços, no caso de concurso e  sorteio de qualquer espécie?   os prêmios distribuídos sob a forma de bens e serviços, mediante concursos e sorteios de qualquer espécie,  estão sujeitos à incidência do imposto, exclusivamente na fonte, à alíquota de 20%, observando-se que:   a) o imposto incide sobre o valor de mercado do prêmio, na data da distribuição, sem necessidade de  reajustamento da base de cálculo, sendo irrelevante que o seu recebimento, pelo contemplado, ocorra em  outra data;   b) considera-se como custo de aquisição o valor de mercado do bem, acrescido do imposto retido;   c) na hipótese da ocorrência de concursos artísticos, desportivos, científicos, literários ou a outros títulos  assemelhados, com distribuição de prêmios efetuada por pessoa jurídica a pessoa física, quando houver  vinculação quanto à avaliação do desempenho dos participantes, hipótese na qual tais prêmios assumem o  aspecto de remuneração do trabalho, independentemente se distribuídos em dinheiro ou sob a forma de bens  e serviços, o imposto sobre a renda incide na fonte, calculado de acordo com a tabela progressiva mensal, a  título de antecipação do devido na declaração de ajuste anual  .    pagamento do imposto correspondente compete à pessoa jurídica que proceder à distribuição de prêmios.  ',
 'incide o imposto no caso de aposta conjunta em loteria, quando o apostador, em cujo nome  é pago o prêmio, distribui ou doa aos demais apostadores a parte que lhes cabe?   os lucros decorrentes de prêmios em dinheiro obtidos em loterias são rendimentos sujeitos à tributação  exclusiva na fonte, à alíquota de 30%, devendo o valor recebido constar na declaração como rendimento  tributável exclusivamente na fonte.   em consequência, o que o beneficiário dos prêmios recebe é apenas o rendimento líquido, isento de qualquer  outro ônus tributário. assim, o premiado pode distribuir aos outros apostadores a parte do prêmio que couber  a cada um deles, sem que isso configure nova incidência tributária. todavia, todos os beneficiários devem  munir-se de meios idôneos de prova que confirmem a aposta conjunta, de forma a comprovar a origem e a  natureza jurídica dos rendimentos. essas operações (pagamento, distribuição, recebimento etc.) devem ser  informadas nas declarações de bens dos apostadores.    partir de 1º de janeiro de 2008, o imposto sobre a renda sobre prêmios obtidos em loterias incidirá apenas  sobre o valor do prêmio em dinheiro que exceder ao valor da primeira faixa da tabela de incidência mensal do  imposto sobre a renda da pessoa física.',
 'como são tributados os prêmios em dinheiro, distribuídos por loterias, concursos ou  sorteios?   os lucros decorrentes de prêmios em dinheiro obtidos em loterias, concursos desportivos em geral (exceto os  de amortização e resgate das ações das sociedades anônimas), os prêmios em concursos de prognósticos  desportivos e a distribuição, mediante sorteio, de benefícios aos aplicadores em títulos de capitalização, nos  casos em que não há amortização antecipada dos referidos títulos, são tributados exclusivamente na fonte à  alíquota de 30%.      138   os benefícios líquidos pagos aos aplicadores em títulos de capitalização, mediante sorteio, nos casos em que  há amortização antecipada dos referidos títulos, sujeitam-se à incidência do imposto sobre a renda  exclusivamente na fonte à alíquota de 25%.   no caso de concursos, quando não se tratar de concursos de prognósticos desportivos e concursos  desportivos em geral, o imposto sobre a renda incide na fonte, calculado de acordo com a tabela progressiva  mensal, a título de antecipação do devido na declaração de ajuste anual;   na hipótese da ocorrência de concursos artísticos, desportivos, científicos, literários ou a outros títulos  assemelhados, com distribuição de prêmios efetuada por pessoa jurídica a pessoa física, quando houver  vinculação quanto à avaliação do desempenho dos participantes, hipótese na qual tais prêmios assumem o  aspecto de remuneração do trabalho, independentemente se distribuídos em dinheiro ou sob a forma de bens  e serviços, o imposto sobre a renda incide na fonte, calculado de acordo com a tabela progressiva mensal, a  título de antecipação do devido na declaração de ajuste anual.',
 'como são tributados os prêmios em dinheiro distribuídos por meio de jogos de bingo?   os prêmios em dinheiro, distribuídos por intermédio de jogos de bingo sujeitam-se à incidência do imposto  sobre a renda exclusivamente na fonte à alíquota de 30%.  ',
 'os juros sobre capital próprio estão sujeitos à tributação exclusiva na fonte?   sim. os juros pagos ou creditados, a título de remuneração do capital próprio, devem ser tributados  exclusivamente na fonte à alíquota de 15%, na data do pagamento ou crédito.  imposto retido não pode ser  compensado na declaração de ajuste anual.  ',
 'como se tributa o décimo terceiro salário recebido acumuladamente com rendimentos de  outra natureza?   há que ser observado o conteúdo a respeito da tributação de rendimentos recebidos acumuladamente,  constante da pergunta 235.   atenção:    parecer //nº 2.331, de 27 de março de 2011, suspendeu os efeitos do ato declaratório   nº 1, de 27 de março de 2009, que considerava que o cálculo do imposto sobre a renda  incidente sobre rendimentos pagos acumuladamente deveria ser realizado levando-se em  consideração as tabelas e alíquotas das épocas próprias a que se referiram tais rendimentos,  devendo o cálculo ser mensal e não global. ',
 'quem a legislação atribui a responsabilidade pela retenção do imposto sobre a renda sobre  os rendimentos auferidos pelo trabalhador avulso?    responsabilidade pela retenção e recolhimento do imposto sobre a renda devido pelos trabalhadores  portuários avulsos, inclusive os pertencentes à categoria dos “arrumadores”, é do órgão gestor de mão de  obra do trabalho portuário. no caso de trabalhadores avulsos de outros setores, é responsabilidade do  sindicato ou associação de cada categoria profissional do trabalhador avulso.   caberá também ao órgão gestor ou ao sindicato/associação de classe, a responsabilidade de prestar às  autoridades fiscais todos os esclarecimentos ou informações, como representantes das fontes pagadoras. ',
 'como apurar o imposto sobre a renda do trabalhador portuário avulso, quando este presta  serviço a mais de uma empresa?    imposto deve ser apurado utilizando a tabela progressiva mensal, tendo como base de cálculo o total do  valor pago ao trabalhador, independentemente da quantidade de empresas às quais o beneficiário prestou  serviço.  ',
 'qual é o tratamento tributário dos rendimentos pagos a título de honorário de perito?    imposto sobre a renda incidente sobre os rendimentos pagos a título de honorário de perito, em processos  judiciais, deverá ser retido na fonte, pela pessoa física ou jurídica obrigada ao pagamento.    retenção dar-se-á no momento em que o rendimento se torne disponível para o beneficiário.    imposto incidirá sobre a importância total posta à disposição do perito quando do depósito judicial efetuado  para este fim.   as despesas necessárias à percepção da receita e à manutenção da fonte produtora, escrituradas e  relacionadas pelo perito em livro-caixa, inclusive com a contratação de outros profissionais sem vínculo  empregatício, desde que sejam comprovadas com documentação hábil e idônea, poderão ser deduzidas, para  fins de apuração da base de cálculo do imposto sobre a renda, no recolhimento mensal obrigatório (carnê- leão), caso receba rendimentos sujeitos a essa forma de recolhimento, e na declaração de ajuste anual.',
 'como deve agir o beneficiário em relação aos rendimentos sujeitos ao ajuste na declaração  anual, quando a fonte pagadora deveria ter retido o imposto na fonte e não o fez?    beneficiário deve declará-los como rendimentos tributáveis. se já tiver apresentado a respectiva declaração  de ajuste anual sem oferecer tais rendimentos à tributação, deverá apresentar declaração retificadora para  incluí-los como rendimentos tributáveis.   ',
 'de quem é a responsabilidade pelo recolhimento do imposto não retido no caso de  rendimento sujeito ao ajuste na declaração anual?   até o término do prazo fixado para a entrega da declaração de ajuste anual a responsabilidade pelo  recolhimento do imposto é da fonte pagadora; após esse prazo, do beneficiário do rendimento.',
 'de quem é a responsabilidade pelo recolhimento do imposto quando a fonte pagadora fica  impossibilitada de fazer a retenção por força de decisão judicial?   caso a decisão final confirme como devido o imposto em litígio, este deverá ser recolhido, retroagindo os  efeitos da última decisão, como se não tivesse ocorrido a concessão da medida liminar. nesse caso, não há  como retornar a responsabilidade de retenção à fonte pagadora.  pagamento do imposto, com os acréscimos  legais cabíveis, deve ser efetuado pelo próprio contribuinte, tanto em relação aos rendimentos sujeitos à  tributação exclusiva na fonte, quanto aos sujeitos ao ajuste na declaração anual, valendo observar os termos  previstos no parecer normativo cosit nº 1, de 24 de setembro de 2002, em especial ao constante em seus  itens 18 e 19.   consulte as perguntas 306 e 309        -  não',
 'qual é a data de vencimento do imposto, cujo recolhimento é de responsabilidade do  beneficiário do rendimento, caso a decisão final confirme como devido o imposto em litígio?   tratando-se de rendimento sujeito ao ajuste anual, considera-se vencido o imposto na data prevista para a  entrega da declaração. no caso de rendimento sujeito à tributação exclusiva, considera-se vencido o imposto  no prazo originário previsto para o recolhimento do imposto que deveria ter sido retido.  ',
 'quais os acréscimos legais incidentes sobre o imposto não retido por força de decisão  judicial, caso a decisão final confirme como devido o imposto em litígio?    multa de mora fica interrompida desde a concessão da medida judicial até o trigésimo dia de sua cassação.  no caso de pagamento após esse prazo, a contagem da multa de mora será reiniciada a partir do trigésimo  primeiro dia, considerando, inclusive e se for o caso, o período entre o vencimento originário da obrigação e a  data de concessão da medida judicial.   em qualquer hipótese, serão devidos juros de mora sem qualquer interrupção desde o mês seguinte ao  vencimento estabelecido na legislação do imposto.   atenção:    apenas o depósito judicial ou administrativo tempestivo é que interrompe a multa e juros de mora. ',
 'quando a fonte pagadora assumir o ônus do imposto devido pelo beneficiário, este pode  compensar na declaração o imposto retido?   sim, devendo o beneficiário oferecer à tributação o valor reajustado. ',
 'quais as deduções permitidas na legislação para determinação da base de cálculo do imposto  sobre a renda?   1 - na determinação da base de cálculo sujeita à incidência mensal do imposto podem ser deduzidos  do rendimento tributável:   a) no caso de retenção na fonte:     importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de  família, quando em cumprimento de decisão judicial, inclusive a prestação de alimentos provisionais,  de acordo homologado judicialmente, ou por escritura pública a que se refere o art. 733 da lei nº  13.105, de 16 de março de 2015 - código de processo civil – a seguir transcrito (lei nº 9.250, de 26  de dezembro de 1995, art. 4º, inciso ; e regulamento do imposto sobre a renda - /2018, art.  709, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018);   atenção:   para efeitos da aplicação da referida dedução:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.     a quantia mensal, por dependente, de $ 189,59, para os meses de janeiro a dezembro do ano- calendário de 2019 (lei nº 9.250, de 26 de dezembro de 1995, art. 4º, inciso , “h” e “i”).       142     as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios  (lei nº 9.250, de 1995, art. 4º, inciso ; e regulamento do imposto sobre a renda - /2018, art.  710, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018);     as contribuições para as entidades de previdência  complementar domiciliadas no brasil, cujo ônus  tenha sido do contribuinte, destinadas a custear benefícios complementares assemelhados aos da  previdência social, observadas as condições e limite estabelecidos no art. 11 da lei nº 9.532, de 10  de dezembro de 1997 (lei nº 9.250, de 1995, art. 4º, inciso ; e regulamento do imposto sobre a  renda - /2018, art. 710, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018);   atenção:   esta dedução aplica-se exclusivamente à base de cálculo relativa aos seguintes rendimentos,  assegurada, nos demais casos, a dedução dos valores pagos a esse título, por ocasião da apuração  da base de cálculo do imposto devido no ano-calendário (lei nº 9.250, de 1995, art. 4º e parágrafo  único):    – do trabalho com vínculo empregatício ou de administradores; e    – proventos de aposentados e pensionistas, quando a fonte pagadora for responsável pelo  desconto e respectivo pagamento das contribuições previdenciárias.     as contribuições aos fundos de aposentadoria programada individual (fapi), cujo ônus tenha sido do  contribuinte, destinadas a custear benefícios complementares assemelhados aos da previdência  social, cujo titular ou cotista seja trabalhador com vínculo empregatício ou administrador, observadas  as condições e limite estabelecidos no art. 11 da lei nº 9.532, de 10 de dezembro de 1997, com a  redação dada pela lei nº 10.887, de 18 de junho de 2004, art. 13 (instrução normativa  nº 1.500,  de 29 de outubro de 2014, art. 52, inciso );   atenção:   quando a fonte pagadora não for responsável pelo desconto das contribuições para as entidades  de previdência complementar domiciliadas no brasil - e as contribuições para o fapi, os valores  pagos a esse título podem ser considerados para fins de dedução da base de cálculo sujeita ao  imposto mensal, desde que haja anuência da fonte pagadora e que o beneficiário lhe forneça o  original do comprovante de pagamento (instrução normativa  nº 1.500, de 29 de outubro de  2014, art. 52, § 1º).     as contribuições para as entidades de previdência complementar de natureza pública de que trata o  § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a custear  benefícios complementares assemelhados ao da previdência social;   atenção:   as deduções relativas às contribuições para as entidades de previdência complementar de natureza  pública de que trata o § 15 do art. 40 da constituição federal, desde que limitadas à alíquota de  contribuição do ente público patrocinador, não se sujeitam ao limite conjunto de dedução de 12%  do total dos rendimentos computados na determinação da base de cálculo do imposto devido na  declaração de ajuste anual (lei nº 9.532, de 10 de dezembro de 1997, art. 11, caput, e § 6º).     o valor de $ 1.903,98, por mês, para os meses de janeiro a dezembro do ano-calendário de 2019,  relativo à parcela isenta de aposentadoria, pensão, transferência para a reserva remunerada ou  reforma, paga pela previdência oficial, ou complementar, a partir do mês em que o contribuinte  completar 65 anos de idade (lei nº 7.713, de 22 de dezembro de 1988, art. 6º, inciso );   b) no caso de recolhimento mensal (carnê-leão):     as despesas escrituradas em livro-caixa, quando permitidas (consulte a pergunta 400);     importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de  família, quando em cumprimento de decisão judicial, inclusive a prestação de alimentos provisionais,  de acordo homologado judicialmente ou por escritura pública a que se refere o art. 733 da lei nº  13.105, de 16 de março de 2015 - código de processo civil - quando não utilizadas como deduções  para fins de retenção na fonte (lei nº 9.250, de 1995, art. 4º, inciso ; e regulamento do imposto  sobre a renda - /2018, art. 72, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018);   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;      143   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.     a quantia mensal, por dependente, de $ 189,59, para os meses de janeiro a dezembro do ano- calendário de 2019, quando não utilizada essa dedução para fins de retenção na fonte (lei nº 9.250,  de 26 de dezembro de 1995, art. 4º, inciso , “h” e “i”; e instrução normativa  nº 1.500, de 29 de  outubro de 2014, art. 56);      as contribuições para a previdência social da união, dos estados, do distrito federal e dos  municípios, quando não utilizados como deduções para fins de retenção na fonte (lei nº 9.250, de  1995, art. 4º, inciso ; regulamento do imposto sobre a renda - /2018, art. 67, inciso , aprovado  pelo decreto nº 9.580, de 22 de novembro de 2018; e instrução normativa  nº 1.500, de 29 de  outubro de 2014, art. 56).   2 - na declaração de ajuste anual e no caso de recolhimento complementar podem ser deduzidas do  total dos rendimentos recebidos no ano-calendário de 2019:   a) despesas médicas previstas na lei nº 9.250, de 26 de dezembro de 1995, art. 8º, inciso , “a”, pagas para  tratamento do contribuinte, de seus dependentes, e de alimentandos em virtude de decisão judicial, de acordo  homologado judicialmente ou de escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de março  de 2015 – código de processo civil (lei nº 9.250, de 26 de dezembro de 1995; regulamento do imposto  sobre a renda - /2018, art. 73, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018; e instrução  normativa  nº 1.500, de 29 de outubro de 2014, arts. 95 a 99);   atenção:    comprovação da despesa médica deve ser realizada mediante documento fiscal ou outra  documentação hábil e idônea que contenha, no mínimo:    – nome, endereço, número de inscrição no cadastro de pessas físicas   ou  do  prestador do serviço;    - a identificação do responsável pelo pagamento, bem como a do beneficiário caso seja pessoa  diversa daquela;    – data de sua emissão; e     – assinatura do prestador de serviço, salvo na hipótese de emissão de documento fiscal.   na falta de documentação, a comprovação poderá ser feita com a indicação de cheque nominativo  ao prestador do serviço.  juízo da autoridade lançadora ou julgadora, justificativas e outras  comprovações poderão ser solicitadas.   b) soma dos valores mensais relativos:     a despesas escrituradas em livro-caixa, quando permitidas (consulte a pergunta 400);     a importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de  família, quando em cumprimento de decisão judicial, inclusive a prestação de alimentos provisionais,  de acordo homologado judicialmente ou por escritura pública a que se refere o art. 733 da lei nº  13.105, de 16 de março de 2015 - código de processo civil;   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.      144     a contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios  (regulamento do imposto sobre a renda - /2018, art. 67, inciso , aprovado pelo decreto nº 9.580,  de 22 de novembro de 2018);     a contribuições para as entidades fechadas de previdência complementar de natureza pública de que  trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a  custear benefícios complementares assemelhados aos da previdência social (lei nº 9.250, de 1995,  art. 8º, inciso , alínea “i”), observados os limites e condições do art. 11 da lei nº 9.532, de 10 de  dezembro de 1997;     a contribuições para as entidades de previdência complementar domiciliadas no país, desde que o  ônus tenha sido do próprio contribuinte, em beneficio deste ou de seu dependente, destinadas a  custear benefícios complementares assemelhados aos da previdência social, observadas as  condições e limite estabelecidos no art. 11 da lei nº 9.532, de 10 de dezembro de 1997, com a  redação dada pela lei nº 10.887, de 18 de junho de 2004, art. 13 (regulamento do imposto sobre a  renda - /2018, art. 67, inciso , aprovado pelo decreto nº 9.580, de 22 de novembro de 2018; e  medida provisória nº 2.158-35, de 24 de agosto de 2001, art. 61);     a contribuições aos fundos de aposentadoria programada individual (fapi), desde que o ônus tenha  sido do próprio contribuinte, em beneficio deste ou de seu dependente, observadas as condições e  limite estabelecidos no art. 11 da lei nº 9.532, de 10 de dezembro de 1997, com a redação dada pela  lei nº 10.887, de 18 de junho de 2004, art. 13 (regulamento do imposto sobre a renda - /2018,  art. 75, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018; e medida provisória nº 2.158– 35, de 24 de agosto de 2001, art. 61);     ao somatório das parcelas isentas mensais até o valor de $ 1.903,98, nos meses de janeiro a  dezembro de 2019, relativas à aposentadoria, pensão, transferência para a reserva remunerada ou  reforma, pagas pela previdência oficial, ou complementar, a partir do mês em que o contribuinte  completar 65 anos (lei nº 7.713, de 22 de dezembro de 1988, art. 6º, inciso ; regulamento do  imposto sobre a renda - /2018, art. 76, § 1º, aprovado pelo decreto nº 9.580, de 22 de novembro  de 2018);     ao limite anual de $ 2.275,08  por dependente (lei nº 9.250, de 26 de dezembro de 1995, art. 8º,  inciso , “c”); e     a despesas pagas com instrução do contribuinte, de alimentandos em virtude de decisão judicial, de  acordo homologado judicialmente ou de escritura pública a que se refere o art. 733 da lei nº 13.105,  de 16 de março de 2015 – código de processo civil, e de seus dependentes, observadas as  condições previstas na lei nº 9.250, de 26 de dezembro de 1995, art. 8º, inciso , “b”, até o limite  anual individual de $ 3.561,50 (lei nº 9.250, de 1995, art. 8º, inciso , alínea ‘b’, item 10 e § 3º).   atenção:    1 - quando a fonte pagadora não for responsável pelo desconto das contribuições a entidades de  previdência complementar e fundos de aposentadoria programada individual, os valores pagos a  esse título podem ser considerados para fins de dedução da base de cálculo sujeita ao imposto  mensal, desde que haja anuência da fonte pagadora e que o beneficiário lhe forneça o original do  comprovante de pagamento.   2 -  somatório das contribuições a entidades de previdência complementar e aos fundos de  aposentadoria programada individual (fapi) destinadas a custear benefícios complementares,  assemelhados aos da previdência oficial, cujo ônus tenha sido do participante, está limitado a 12%  do total dos rendimentos tributáveis computados na determinação da base de cálculo do imposto  devido na declaração. consulte a pergunta 316.   3 - as deduções referentes às contribuições a entidades de previdência complementar e aos fapi  destinadas a custear benefícios complementares, assemelhados aos da previdência oficial ficam  condicionadas ao recolhimento, também, de contribuições para o regime geral de previdência social  ou, quando for o caso, para o regime próprio de previdência social dos servidores titulares de cargo  efetivo da união, dos estados, do distrito federal ou dos municípios.   4 – excetuam-se da condição estabelecida no item 3 os beneficiários de aposentadoria ou pensão  concedidas por regime próprio de previdência ou pelo regime geral de previdência social, mantido,  entretanto, o limite de 12% do total dos rendimentos computados na determinação da base de  cálculo do imposto devido na declaração de ajuste anual.   5.1 –  dedução das contribuições para as entidades fechadas de previdência complementar de  natureza pública de que trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do  contribuinte, limitada à alíquota de contribuição do ente público patrocinador, não se sujeita ao limite  previsto de 12% do total dos rendimentos computados na determinação da base de cálculo do  imposto devido na declaração de ajuste anual.   5.2 – por sua vez, o valor de contribuição excedente ao limite da aplicação da alíquota de  contribuição do ente público patrocinador, está sujeito ao limite de 12% dos rendimentos      145   computados na determinação da base de cálculo do imposto devido na declaração de rendimentos  conjuntamente com eventuais contribuições a outros planos de previdência complementar. ',
 'quais as deduções permitidas na legislação para determinação da base de cálculo do imposto  sobre a renda incidente na fonte sobre o décimo terceiro salário?    - as importâncias pagas em dinheiro a título de pensão alimentícia em face das normas do direito de família,  quando em cumprimento de decisão judicial, inclusive a prestação de alimentos provisionais, ou acordo  homologado judicialmente, ou ainda por escritura pública a que se refere o art. 733 da lei nº 13.105, de 16 de  março de 2015 - código de processo civil;   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.    – a quantia de $ 189,59, por dependente, nos meses de janeiro a dezembro, do ano-calendário de 2019;    - as contribuições para a previdência social da união, dos estados, do distrito federal e dos municípios;    - as contribuições para as entidades de previdência complementar domiciliadas no brasil, destinadas a  custear benefícios complementares assemelhados aos da previdência social e para os fundos de  aposentadoria programada individual (fapi), cujo ônus tenha sido do próprio contribuinte e desde que  destinadas a seu próprio benefício, observadas as condições e limite estabelecidos no art. 11 da lei nº 9.532,  de 10 de dezembro de 1997;    – as contribuições para as entidades fechadas de previdência complementar de natureza pública de que   trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do contribuinte, destinadas a custear  benefícios complementares assemelhados aos da previdência social, observadas as condições e limites  estabelecidos no art. 11, caput e §§ 6º e 7º, da lei nº 9.532, de 10 de dezembro de 1997;    - o valor de $ 1.903,98, nos meses de janeiro a dezembro, para o ano-calendário de 2019, relativo à  parcela isenta de aposentadoria, pensão, transferência para a reserva remunerada ou reforma, paga pela  previdência oficial, por pessoa jurídica de direito público interno ou por entidade de previdência complementar,  a partir do mês em que o contribuinte completar 65 anos de idade.   atenção:    décimo terceiro salário é tributado exclusivamente na fonte, portanto, as deduções devem ser  correspondentes a esse rendimento e não podem ser utilizadas na declaração de ajuste anual. ',
 'contribuição à previdência oficial, descontada de rendimentos isentos do próprio  contribuinte ou por este recolhida na condição de contribuinte individual (autônomo), é dedutível na  declaração de ajuste nual?   sim, desde que o contribuinte tenha rendimentos tributáveis sujeitos ao ajuste na declaração anual.',
 'contribuição à previdência oficial referente a anos anteriores paga em atraso com  acréscimos legais em 2019 pode ser utilizada como dedução?   sim. as contribuições pagas em 2019 à previdência oficial referentes a anos anteriores (exceto os acréscimos  legais) podem ser consideradas como dedução na declaração de ajuste anual do exercício de 2020. ',
 'qual é o limite para dedução na declaração de ajuste anual das contribuições efetuadas a  entidades de previdência complementar?    dedução relativa às contribuições para entidades de previdência complementar, somadas às contribuições  para o fundo de aposentadoria programada individual (fapi) e a parcela das contribuições para as entidades  fechadas de previdência complementar de natureza pública de que trata o § 15 do art. 40 da constituição  federal efetuada pelo contribuinte que exceder a parcela do ente público patrocinador, destinadas a custear  benefícios complementares, assemelhados aos da previdência oficial, cujo ônus tenha sido do participante,  em beneficio deste ou de seu dependente, fica limitada a 12% do total dos rendimentos computados na  determinação da base de cálculo do imposto sobre a renda devido na declaração.   atenção:    1 - as deduções relativas às contribuições para entidades de previdência complementar e  sociedades seguradoras domiciliadas no país e destinadas a custear benefícios complementares  aos da previdência social, cujo ônus seja da própria pessoa física, ficam condicionadas ao  recolhimento, também, de contribuições para o regime geral de previdência social ou, quando for o  caso, para regime próprio de previdência social dos servidores titulares de cargo efetivo da união,  dos estados, do distrito federal ou dos municípios, observada a contribuição mínima, e limitadas a  12% do total dos rendimentos computados na determinação da base de cálculo do imposto devido  na declaração de ajuste anual.   2 -  disposto no item 1 acima aplica-se, inclusive, às contribuições ao fundo de aposentadoria  programada individual (fapi).   3.1 –  dedução das contribuições para as entidades fechadas de previdência complementar de  natureza pública de que trata o § 15 do art. 40 da constituição federal, cujo ônus tenha sido do  contribuinte, limitada à alíquota de contribuição do ente público patrocinador, não se sujeita ao limite  previsto de 12% do total dos rendimentos computados na determinação da base de cálculo do  imposto devido na declaração de ajuste anual;   3.2 – por sua vez, o valor de contribuição para a previdência complementar pública excedente ao  limite da aplicação da alíquota de contribuição do ente público patrocinador, está sujeito ao limite  de 12% dos rendimentos computados na determinação da base de cálculo do imposto devido na  declaração de rendimentos conjuntamente com eventuais contribuições a outros planos de  previdência complementar.   4 - excetua-se da condição referida no item 1 acima o beneficiário de aposentadoria ou pensão  concedida por regime próprio de previdência ou pelo regime geral de previdência social, mantido,  entretanto, o limite de 12% do total dos rendimentos computados na determinação da base de  cálculo do imposto devido na declaração de ajuste anual.   5 - se o titular ou quotista for dependente do declarante, para a dedução das contribuições aplicam- se ao declarante a condição e o limite acima referidos no item 1.   6 - na hipótese de dependente com mais de 16 anos, a dedução referida fica condicionada, ainda,  ao recolhimento, em seu nome, de contribuições para o regime geral de previdência social,  observada a contribuição mínima, ou, quando for o caso, para regime próprio de previdência social  dos servidores titulares de cargo efetivo da união, dos estados, do distrito federal ou dos  municípios.      147   exemplo 1:   josé maurício recebeu um total de rendimentos tributáveis de $ 100.000,00 e o ente público patrocinador  efetuou contribuição para a entidade fechada de previdência complementar de natureza pública no valor de  $ 8.000,00.    contribuinte josé maurício efetuou as seguintes contribuições:   - para entidades de previdência complementar: $ 7.000,00;   - para o fapi: $ 3.000,00;   - para a entidade fechada de previdência complementar de natureza pública: $ 8.000,00   como a contribuição do patrocinador foi igual à do contribuinte, toda a parcela de contribuição efetuada pelo  contribuinte à previdência complementar ($ 8.000,00) é dedutível.   as demais contribuições somaram $ 10.000,00 ($ 7.000,00 + $ 3.000,00), não atingindo o limite de 12%  e, portanto, são integralmente dedutíveis.    contribuinte pode deduzir, portanto, o valor de $ 18.000,00 ($ 8.000,00 + $ 7.000,00 + $ 3.000,00).   exemplo 2:   regina recebeu um total de rendimentos tributáveis de $ 100.000,00 e o ente público patrocinador efetuou  contribuição para a entidade fechada de previdência complementar de natureza pública no valor de  $ 6.500,00.    contribuinte regina efetuou as seguintes contribuições:   - para entidades de previdência complementar: $ 7.000,00;   - para o fapi: $ 3.000,00;   - para a entidade fechada de previdência complementar de natureza pública: $ 7.500,00    contribuição da participante limitada à do patrocinador ($ 6.500,00) é dedutível.  valor excedente  ($ 1.000,00) será somado às demais contribuições para a previdência complementar para verificação do  limite global de 12%:   $ 11.000,00 ($ 7.000,00 + $ 3.000,00 + $ 1.000,00) é menor do que 12% dos rendimentos tributáveis  ($ 12.000,00), portanto, esse valor de $ 11.000,00 poderá ser deduzido.    contribuinte poderá deduzir, portanto, $ 17.500,00 ($ 6.500,00 + $ 11.000,00).   exemplo 3   susanna recebeu um total de rendimentos tributáveis de $ 100.000,00 e o ente público patrocinador efetuou  contribuição para a entidade fechada de previdência complementar de natureza pública no valor de  $ 6.500,00.    contribuinte susanna efetuou as seguintes contribuições:   - para entidades de previdência complementar: $ 9.000,00;   - para o fapi: $ 3.000,00;   - para a entidade fechada de previdência complementar de natureza pública: $ 7.500,00    contribuição da participante limitada à do patrocinador ($ 6.500,00) é dedutível.  valor excedente  ($ 1.000,00) será somado às demais contribuições para a previdência complementar para verificação do  limite global de 12%:   $ 13.000,00 ($ 9.000,00 + $ 3.000,00 + $ 1.000,00) é maior do que 12% dos rendimentos tributáveis  ($ 12.000,00), portanto, somente parte desse valor ($ 12.000,00) poderá ser deduzido.    contribuinte poderá deduzir, portanto, $ 18.500,00 ($ 6.500,00 + $ 12.000,00).   orientações específicas relativas a contribuições a fundações de previdência complementar do  servidor público dos poderes executivo, legislativo e judiciário:    – se a contribuição para a entidade fechada de previdência complementar de natureza pública de que trata  o § 15 do art. 40 da constituição federal efetuada pelo contribuinte for contribuição obrigatória:   na declaração de ajuste anual  , em pagamentos efetuados, sob o código “37 – contribuições para  as entidades de previdência complementar fechadas de natureza pública”, preencher:      148   a) o campo “valor pago” com o total das contribuições obrigatórias efetuadas no ano-calendário de 2019  ao funpresp pelo contribuinte, exceto a contribuição relativa ao 13º salário (para os servidores que  contribuem para a fundação de previdência complementar do servidor público federal do poder  executivo – funpresp-exe, esse valor encontra-se na linha 03 do quadro 3 do comprovante de  rendimentos pagos e de retenção de imposto de renda na fonte fornecido pela fonte pagadora); e   b) o campo “contribuição do ente público patrocinador” com o mesmo valor do item “a”;    – se a contribuição para a entidade fechada de previdência complementar de natureza pública de que trata  o § 15 do art. 40 da constituição federal efetuada pelo contribuinte for superior à contribuição obrigatória (ou  seja, o contribuinte também efetuou contribuições adicionais facultativas):   na , em pagamentos efetuados, sob o código “37 – contribuições para as entidades de previdência  complementar fechadas de natureza pública”, preencher:   a) o campo “valor pago” com o somatório das contribuições, obrigatórias e facultativas, efetuadas no  ano-calendário de 2019 ao funpresp pelo contribuinte, exceto a contribuição relativa ao 13º salário  (para os servidores que contribuem para a funpresp-exe, deve-se somar o valor da linha 03 do  quadro 3 do comprovante de rendimentos pagos e de retenção de imposto de renda na fonte  fornecido pela fonte pagadora com o valor das contribuições facultativas); e   b) o campo “contribuição do ente público patrocinador” com o total das contribuições obrigatórias feitas  pelo contribuinte, exceto a contribuição relativa ao 13º salário (para os servidores que contribuem para  a funpresp-exe, esse valor encontra-se na linha 03 do quadro 3 do comprovante de rendimentos  pagos e de retenção de imposto de renda na fonte fornecido pela fonte pagadora).   exemplo 4   mirza, servidora pública federal, no ano-calendário de 2019, efetuou mensalmente contribuições obrigatórias  ao funpresp no valor de $ 2.000,00.  ente patrocinador recolheu igual valor. houve, também, recolhimento  no valor de $ 2.000,00 relativo ao 13º, tanto pela servidoraquanto pelo ente patrocinador.   preenchimento na  em pagamentos efetuados, código 37:   campo “valor pago”: $ 2.000,00 x 12 = $ 24.000,00   campo “contribuição do ente público patrocinador”: $ 2.000,00 x 12 = $ 24.000,00   exemplo 5   clarice, servidora pública federal, no ano-calendário de 2019, efetuou mensalmente contribuições obrigatórias  ao funpresp no valor de $ 2.000,00.  ente patrocinador recolheu igual valor. houve, também, recolhimento  no valor de $ 2.000,00 relativo ao 13º, tanto pela servidora quanto pelo ente patrocinador.  servidora  efetuou, ainda, o recolhimento mensal de contribuição facultativa no valor de $ 500,00. em relação a essa  contribuição facultativa, não há recolhimento por parte do ente patrocinador.   preenchimento na  em pagamentos efetuados, código 37:   campo “valor pago”: ($ 2.000,00 x 12) + ($ 500,00 x 12) = $ 30.000,00   campo “contribuição do ente público patrocinador”: $ 2.000,00 x 12 = $ 24.000,00   (lei nº 9.532, de 10 de dezembro de 1997, art. 11 e §§ 5º a 7º; medida provisória nº 2.158-35, de  24 de agosto de 2001, art. 61; regulamento do imposto sobre a renda - /2018, art. 67, aprovado  pelo decreto nº 9.580, de 22 de novembro de 2018; instrução normativa  nº 588, de 21 de  dezembro de 2005, arts. 6º e 7º; e instrução normativa  nº 1.500, de 29 de outubro de 2014,  art. 72, § 1º e arts. 86 e 88)',
 'as contribuições descontadas para o montepio civil ou militar são dedutíveis da base de  cálculo do imposto sobre a renda?   sim. as contribuições, destinadas a custear benefícios complementares aos da previdência social, são  dedutíveis na determinação da base de cálculo para retenção mensal do imposto sobre a renda incidente na  fonte e na declaração de ajuste.   atenção:    as contribuições às entidades de previdência complementar, aos fundos de aposentadoria  programada individual (fapi) e as contribuições para as entidades fechadas de previdência      149   complementar de natureza pública de que trata o § 15 do art. 40 da constituição federal efetuadas  pelo contribuinte acima da alíquota da entidade patrocinadora, destinadas a custear benefícios  complementares, assemelhados aos da previdência oficial, cujo ônus tenha sido do participante,  em beneficio deste ou de seu dependente, estão limitadas a 12% do total dos rendimentos  computados na determinação da base de cálculo do imposto devido na declaração de ajuste. ',
 'contribuinte que, em parte do ano-calendário, recebeu rendimentos do trabalho assalariado  e, durante o período em que ficou desempregado, contribuiu como contribuinte individual  (autônomo), pode deduzir, além da contribuição previdenciária descontada do salário, aquela paga  na condição de contribuinte individual?   sim, em se tratando de contribuição previdenciária oficial do próprio declarante, este pode deduzir na sua  declaração os valores pagos a esse título. ',
 'contribuinte pode deduzir a contribuição previdenciária oficial ou complementar paga em  nome de dependente sem rendimentos próprios?   em relação à previdência oficial somente podem ser deduzidas as contribuições pagas em nome do  dependente que tenha rendimentos próprios tributados em conjunto com os do declarante.    as contribuições a entidades de previdência complementar e aos fundos de aposentadoria programa  individual (fapi) são dedutíveis quando o ônus for do próprio contribuinte, em beneficio deste ou de seu  dependente. ',
 'valor pago a título de previdência complementar em parcela única com resgate em curto prazo  pode ser dedutível na declaração?   sim. as contribuições realizadas a planos de previdência complementar – plano gerador de benefício livre  e fundo de aposentadoria programada individual (fapi) - são passíveis de dedução, desde que o  ônus tenha sido do próprio contribuinte, em beneficio deste ou de seu dependente.    valor resgatado é rendimento tributável e, mesmo que seu valor tenha sido igual ou inferior ao limite da  primeira faixa da tabela progressiva mensal, no valor de $ 1.903,98, para os meses de janeiro a dezembro,  do ano-calendário de 2019, deve ser somado aos rendimentos tributáveis na declaração de ajuste anual, no  ano do seu recebimento.   ',
 'pessoa qualificada como "dependente" pelas leis previdenciárias tem a mesma qualificação  na legislação do imposto sobre a renda?   não.  contribuinte deve observar o disposto nas leis tributárias, no que concerne às condições para a  qualificação de dependência. ',
 'qual é o documento hábil para comprovar a relação de dependência?   para o cônjuge e filhos, a prova desta relação é feita por meio de certidão de casamento e de nascimento. no  que concerne a menor pobre que o contribuinte crie e eduque, esse somente é considerado dependente, para  os efeitos do imposto sobre a renda, se obedecidos os procedimentos estatuídos na lei nº 8.069, de 13 de  julho de 1990 - estatuto da criança e do adolescente - quanto à guarda, tutela ou adoção.',
 'contribuinte residente no brasil pode considerar como dependentes pessoas não residentes  no brasil?    legislação tributária brasileira, em regra geral, não faz distinção em relação à residência dos dependentes.  assim, desde que provadas as condições necessárias para figurarem como tal, essa dedução pode ser  efetuada pelo contribuinte. ',
 'para efeito de dedução, os dependentes próprios de um dos cônjuges ou companheiros  podem ser considerados na declaração do outro cônjuge ou companheiro?   não.  contribuinte pode efetuar apenas as deduções correspondentes a seus dependentes próprios.   assim, somente se um cônjuge ou companheiro apresentar declaração em conjunto onde estejam sendo  tributados rendimentos de ambos os cônjuges ou companheiros, seus dependentes próprios podem ser  incluídos na declaração apresentada em nome do outro cônjuge ou companheiro. contudo, se o cônjuge ou  companheiro apresentar declaração em separado, os seus dependentes próprios só podem constar em sua  declaração de rendimentos.    ressalte-se que os rendimentos dos dependentes também devem ser informados na declaração.   exemplo:   vani e aristóteles são casados, possuem uma filha de 10 anos chamada cristiane.  mãe de vani chama-se  terezinha e tem rendimentos mensais de um salário mínimo.   a) se vani e aristóteles apresentarem declaração em conjunto, terezinha e cristiane poderão constar como  dependentes;   b) se vani e aristóteles apresentarem declaração em separado:   b.1) terezinha só poderá constar como dependente na declaração de vani; e   b.2) cristiane poderá constar como dependente na declaração de vani ou na de aristóteles, mas não nas  duas simultaneamente. ',
 'contribuinte que paga pensão alimentícia judicial a ex-cônjuge e filhos pode considerá-los  dependentes na declaração?   não. entretanto, excepcionalmente, no ano em que se iniciar o pagamento da pensão, o contribuinte pode  efetuar a dedução correspondente ao valor total anual, caso os filhos tenham sido considerados seus  dependentes nos meses que antecederam o pagamento da pensão naquele ano.   atenção:    na ficha pagamentos efetuados, da declaração de ajuste anual, devem ser informados o nome e  o número de inscrição no cadastro de pessoas físicas   de todos os beneficiários da pensão  e o valor total pago no ano, ainda que tenha sido descontado pelo seu empregador em nome de  apenas um dos beneficiários. ',
 'como proceder para a dedução de dependente quando ocorrer mudança de dependência de  um para outro contribuinte, no curso do ano-calendário?   como regra geral, não podem constar dependentes nas declarações de mais de um contribuinte  simultaneamente. todavia, constitui exceção a essa regra a hipótese de ocorrer início ou término, durante o  ano-calendário, da condição de dependência, como, por exemplo, filho dependente do pai ou mãe, que se  casa e passa a ser dependente do cônjuge; ou casal que se separa e, até determinado mês, os filhos eram  dependentes de um dos cônjuges, que depois passa a pagar pensão alimentícia aos filhos.   nesses casos, ambos os contribuintes podem utilizar o valor total anual da dedução correspondente ao  dependente, na declaração de rendimentos relativa a esse ano-calendário, no entanto, as despesas e  rendimentos do dependente, são declarados relativamente ao período de dependência.   no caso do ex-cônjuge ou ex-companheiro que passou a pagar pensão alimentícia judicial, também pode ser  deduzido o valor da pensão no ano-calendário em que se deu a separação.  ',
 'pode ser considerado dependente o filho que nasce e morre no ano-calendário, cônjuge e  outros dependentes que faleçam durante o ano-calendário?   sim. é admissível a dedução pelo valor total anual da dedução de dependente.',
 'contribuinte que passa a ter rendimentos próprios no curso do ano-calendário e que  apresenta declaração em separado pode ser dependente em outra declaração nesse mesmo ano?   não. exceto quando a relação de dependência iniciar ou terminar durante o ano-calendário (ex: filho que  completa 25 anos e passa a declarar), nesse caso, a pessoa física pode, simultaneamente, constar como  dependente em uma declaração e apresentar declaração em separado.   ressalte-se que os rendimentos e despesas devem ser declarados pelo valor correspondente ao período de  dependência, com exceção do valor de dependente, que pode ser deduzido pelo valor total ($ 2.275,08).   observe-se que o cônjuge ou companheiro que passou a ter rendimentos próprios no curso do ano-calendário  e que apresenta declaração em separado não pode constar como dependente na declaração apresentada  pelo outro cônjuge ou companheiro. ',
 'filho universitário que completou 25 anos durante o ano de 2019 pode ser considerado  dependente?   sim. de acordo com a legislação tributária pode ser considerado dependente a filha, o filho, a enteada ou o  enteado, até 21 anos, ou de qualquer idade quando incapacitado física ou mentalmente para o trabalho.  podem ainda ser assim considerados, quando maiores até 24 anos de idade, se ainda estiverem cursando  estabelecimento de ensino superior ou escola técnica de segundo grau.  fato de ter completado 25 anos  durante o ano não ocasiona a perda da condição de dependência.',
 'pessoa que passe a receber o crédito educativo pode ser considerada dependente?    fato de o dependente receber crédito educativo (programa de financiamento estudantil – , por  exemplo) não descaracteriza a condição de dependência. se o beneficiado preencher as condições legais  pode ser considerado dependente para fins de dedução. ',
 'dependente que recebe herança ou doação em bens móveis, imóveis ou direitos perde a  condição de dependente?    simples recebimento de herança ou doação não acarreta a perda da qualidade de dependente, observados  os requisitos legais. os bens ou direitos devem ser incluídos na declaração do responsável.  valor  correspondente deve ser informado como rendimento isento e não tributável e os rendimentos produzidos por  esses bens ou direitos são tributados na declaração do responsável. ',
 'casal sem rendimentos próprios sustentado pelos pais de um deles pode ser considerado  dependente?   apenas o filho ou filha, ainda que casados ou conviventes, podem ser dependentes dos pais desde que se  enquadrem em uma das condições de dependência elencadas na resposta à pergunta 321.   caso o filho ou filha, considerados dependentes na declaração dos pais, estejam declarando em conjunto com  estes, seu cônjuge ou convivente, desde que não esteja declarando em separado, também pode ser  dependente na mesma declaração.',
 'quando podem ser considerados como dependentes o irmão, o neto e o bisneto?   podem ser considerados como dependentes o irmão, o neto ou o bisneto que estiverem em uma das seguintes  situações previstas na lei nº 9.250, de 26 de dezembro de 1995:   a) com idade de até 21 anos, desde que o contribuinte detenha a guarda judicial (art. 35, inciso );   b) com idade de 21 até 24 anos, se ainda estiver cursando estabelecimento de ensino superior ou escola  técnica de segundo grau (art. 35, § 1º). nesse caso, é necessário que o responsável tenha detido a guarda  judicial até a idade de 21 anos;   c) de qualquer idade, quando incapacitado física ou mentalmente para o trabalho (art. 35, inciso ).   guarda judicial só é exigida para aqueles com idade de até 21 anos.  condição de não ter arrimo dos pais,  por outro lado, é necessária para todas as situações acima.',
 'sogra ou sogro podem ser considerados dependentes na declaração do genro ou nora?   de acordo com a lei nº 9.250, de 26 de dezembro de 1995, art. 35, os pais podem ser considerados  dependentes na declaração dos filhos, desde que não aufiram rendimentos, tributáveis ou não, superiores ao  limite de isenção anual ($ 22.847,76). sogro ou a sogra não podem ser dependentes, salvo se seu filho ou filha estiver declarando em conjunto  com o genro ou a nora, e desde que o sogro ou a sogra não aufiram rendimentos, tributáveis ou não, superiores  ao limite de isenção anual ($ 22.847,76), nem estejam declarando em separado. ',
 'menor de 21 anos que se emancipe pode ainda figurar como dependente dos pais ou  responsáveis?    emancipação transforma o menor em plenamente capaz para todos os atos da vida civil (lei nº 10.406, de  10 de janeiro de 2002 - código civil, art. 9º, inciso ). em princípio, o emancipado deve declarar em separado,  com o número de inscrição no cadastro de pessoas físicas   próprio. entretanto, se o emancipado ainda  se enquadrar nas condições que autorizem a dependência para fins de imposto sobre a renda, pode figurar  como tal na declaração de um dos pais.',
 'contribuinte que eduque menor pobre, parente ou não, mas que não viva em sua companhia,  pode considerá-lo dependente?   sim, desde que o contribuinte crie e eduque o menor pobre e detenha a sua guarda judicial, nos termos da  lei nº 8.069, de 13 de julho de 1990, independentemente de que o menor viva em sua companhia ou não. ',
 'quais são as pensões judiciais dedutíveis pela pessoa física?   são dedutíveis da base de cálculo mensal e na declaração de ajuste apenas as importâncias pagas a título  de pensão alimentícia, inclusive a prestação de alimentos provisionais, conforme normas do direito de família,  sempre em decorrência de decisão judicial ou acordo homologado judicialmente ou por escritura pública, a  que se refere o art. 733 da lei nº 13.105, de 16 de março de 2015 - código de processo civil.   não há previsão legal para dedução de importâncias pagas a título de pensão alimentícia decorrentes de  sentença arbitral.   atenção:    as despesas com instrução e as despesas médicas pagas pelo alimentante, em nome do  alimentando, em razão de decisão judicial ou acordo homologado judicialmente ou por escritura  pública, a que se refere o art. 733 da lei nº 13.105, de 16 de março de 2015 - código de processo  civil, podem ser deduzidas somente na declaração de rendimentos, em seus campos próprios,  observado o limite anual relativo às despesas com instrução ($ 3.561,50).    na ficha pagamentos efetuados da declaração de ajuste anual, devem ser informados o nome e  o número de inscrição no cadastro de pessoas físicas   de todos os beneficiários da pensão  e o valor total pago no ano, mesmo que tenha sido descontado pelo empregador em nome de  apenas um dos beneficiários.       155    contribuinte que paga pensão não pode incluir o filho como dependente, observada a resposta à  pergunta 327.   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996.   (lei nº 9.250, de 26 de dezembro de 1995, arts. 4º, inciso , e 8º, inciso , alínea “f”; lei nº 9.307,  de 23 de setembro de 1996, arts 1º e 31; regulamento do imposto sobre a renda - /2018, art.  72, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018.',
 'pensão alimentícia paga por cônjuge ou companheiro(a) a sogro(a) em virtude de acordo  homologado judicialmente é dedutível pelo genro/nora?   somente são dedutíveis as importâncias pagas a título de pensão alimentícia, inclusive a prestação de  alimentos provisionais, conforme normas do direito de família, sempre em decorrência de decisão judicial ou  acordo homologado judicialmente ou por escritura pública para sogro(a) nos casos em que os cônjuges  apresentam declaração em conjunto onde estejam sendo tributados os rendimentos de ambos os cônjuges  ou companheiros. ',
 'são dedutíveis os pagamentos estipulados em sentença judicial que excedam a pensão  alimentícia?   somente é dedutível a título de pensão o valor pago como pensão alimentícia.   as quantias pagas decorrentes de sentença judicial para cobertura de despesas médicas e com instrução,  destacadas da pensão, são dedutíveis sob a forma de despesas médicas e despesas com instrução dos  alimentandos, desde que obedecidos os requisitos e limites legais.   os demais valores estipulados na sentença, tais como aluguéis, condomínio, transporte, previdência  complementar, não são dedutíveis.   atenção:  idêntico tratamento pode ser aplicado quando tais valores forem pagos em decorrência de acordo  homologado judicialmente ou de escritura pública de separação e divórcio consensual. ',
 'como ocorre o cálculo da pensão alimentícia, na hipótese na qual a decisão judicial  estabeleça a determinação de seu valor sobre o rendimento líquido do contribuinte?   no caso de decisão judicial que estabeleça o valor de pensão alimentícia em percentual da remuneração,  aplicado após a subtração do imposto sobre a renda incidente na fonte, de cuja base de cálculo se pode  deduzir a própria pensão, a fonte pagadora empregará fórmula matemática que possibilitará a correta  determinação de ambos os valores interdependentes.   considerando-se a hipótese na qual a contribuição previdenciária será previamente subtraída da base de  cálculo da pensão, bem como as normas relativas à configuração da base de cálculo do imposto sobre a  renda, deve-se adotar o procedimento (fórmula matemática e demais condições) a que se refere a solução  de consulta cosit nº 354, de 17 de dezembro de 2014, disponível no sitio da  na internet  (http://receita.economia.gov.br => legislação => soluções de consultas e de divergências => sistema padrão  de pesquisas da legislação da receita federal => núm. do ato: 354 => ano do ato: 2014 => procurar).    atenção:  no exemplo que consta da solução de consulta cosit nº 354, de 2014, observar que:  - o valor de $ 18.319,40 equivale a $ 18.835,31 – (3 x $ 171,97), considerando haver 3  dependentes;  - $ 171,97 era o valor de dedução por dependente para o ano-calendário de 2013 (o do exemplo). ',
 'é dedutível na declaração de ajuste anual a pensão alimentícia judicial ou por escritura  pública descontada do décimo terceiro salário?    não. tendo em vista que a pensão alimentícia judicial ou por escritura pública descontada do décimo terceiro  salário já constituiu dedução desse rendimento, sujeito à tributação exclusiva na fonte, a utilização da dedução  na declaração de ajuste anual implicaria a duplicação da dedução. no entanto, a pensão alimentícia paga  que foi descontada do décimo terceiro constitui rendimento tributável para o beneficiário da pensão,  sujeitando-se ao carnê-leão e, também, ao ajuste na declaração anual. ',
 'as pensões pagas por liberalidade, ou seja, sem decisão judicial, acordo homologado  judicialmente ou em decorrência de escritura pública são dedutíveis?   as pensões pagas por liberalidade não são dedutíveis por falta de previsão legal. ',
 'pensão alimentícia descontada de rendimentos isentos de aposentadoria é dedutível na  declaração de ajuste anual?   sim.  pensão alimentícia paga em cumprimento de decisão judicial ou acordo homologado judicialmente, ou  por escritura pública, em face das normas do direito de família, a que se refere o art. 733 da lei nº 13.105,  de 16 de março de 2015 - código de processo civil – descontada de rendimentos isentos, pode ser deduzida  dos rendimentos tributáveis na declaração de ajuste anual.   atenção:   para efeitos da aplicação da referida dedução, observe-se que:   1) as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão  aquelas em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão  alimentícia;   2) tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de  alimentos for decorrente da dissolução daquela sociedade;   http://www.rfb.gov.br/    157   3) o beneficiário da pensão não necessita se enquadrar nas condições descritas na pergunta 321,  que trata de dedução de dependentes;   4) não alcança o provimento de alimentos decorrente de sentença arbitral, de que trata a lei nº  9.307, de 23 de setembro de 1996. ',
 'contribuinte pode deduzir a pensão alimentícia paga em cumprimento de sentença  estrangeira?   além do efetivo pagamento da pensão alimentícia, exige a lei que a pensão seja paga em cumprimento de  decisão judicial ou acordo homologado judicialmente. assim, a pensão alimentícia paga em virtude de  sentença proferida no exterior pode ser deduzida do rendimento bruto, desde que o contribuinte faça prova de  sua homologação no brasil pelo superior tribunal de justiça  , conforme determina o art. 105, inciso ,  alínea “i”, da constituição federal, incluído pela emenda constitucional nº 45, de 30 de dezembro de 2004.   assim como a sentença nacional, para efeitos da aplicação da referida dedução da base de cálculo do imposto  sobre a renda de pessoa física  :    - as importâncias pagas relativas ao suprimento de alimentos, em face do direito de família, serão aquelas  em dinheiro e somente a título de prestação de alimentos provisionais ou a título de pensão alimentícia;    - tratando-se de sociedade conjugal, a dedução somente se aplica, quando o provimento de alimentos for  decorrente da dissolução daquela sociedade. ',
 'quais são as despesas médicas dedutíveis na declaração de ajuste anual?   as despesas médicas ou de hospitalização dedutíveis restringem-se aos pagamentos efetuados pelo  contribuinte para o seu próprio tratamento ou o de seus dependentes relacionados na declaração de ajuste  anual, incluindo-se os alimentandos, em razão de decisão judicial ou acordo homologado judicialmente, ou  por escritura pública.   consideram-se despesas médicas ou de hospitalização os pagamentos efetuados a médicos de qualquer  especialidade, dentistas, psicólogos, fisioterapeutas, terapeutas ocupacionais, fonoaudiólogos, hospitais, e as  despesas provenientes de exames laboratoriais, serviços radiológicos, aparelhos ortopédicos e próteses  ortopédicas e dentárias.   no caso de despesas com aparelhos ortopédicos e próteses ortopédicas e dentárias, exige-se a comprovação  com receituário médico ou odontológico e nota fiscal em nome do beneficiário.   consideram-se também despesas médicas ou de hospitalização:     os pagamentos efetuados a empresas domiciliadas no brasil destinados à cobertura de despesas  com hospitalização, médicas e odontológicas, bem como a entidades que assegurem direito de  atendimento ou ressarcimento de despesas da mesma natureza;     as despesas de instrução de deficiente físico ou mental, desde que a deficiência seja atestada em  laudo médico e o pagamento efetuado a entidades destinadas a deficientes físicos ou mentais.     os pagamentos a operadora de plano de saúde ou a administradora de benefícios que cubram as  despesas ou assegurem o direito a atendimento:   1) domiciliar dos serviços de saúde previstos na lei nº 9.250, de 26 de dezembro de 1995;   2) pré-hospitalar de urgência, desde que prestado por meio de  móvel, instalada em ambulância  de suporte avançado (tipo “”) ou em aeronave de suporte médico (tipo “”); ou   3) pré-hospitalar de emergência, realizado por meio de  móvel, instalada em ambulância tipo “”,  “”, “” ou “”, quando necessariamente conte com a presença de um profissional médico e  possua em seu interior equipamentos que possibilitem oferecer ao paciente suporte avançado de  vida.    dedução dessas despesas é condicionada a que os pagamentos sejam especificados, informados na ficha   pagamentos efetuados da declaração de ajuste anual, e comprovados com documentos originais que   indiquem, no mínimo, nome, endereço e número de inscrição no cadastro de pessoas físicas   ou      158   cadastro nacional da pessoa jurídica   de quem prestou o serviço, a identificação do responsável pelo   pagamento, bem como do beneficiário caso seja pessoa diversa daquela, data de sua emissão, e assinatura   do prestador de serviço, caso não seja documento fiscal.   admite-se que, na falta de documentação, a comprovação possa ser feita com a indicação do cheque   nominativo com que foi efetuado o pagamento. conforme previsto no art. 66 do /2018, a juízo da   autoridade fiscal, todas as deduções estarão sujeitas à comprovação ou justificação, e, portanto, poderão ser   exigidos outros elementos necessários à comprovação da despesa médica.   as despesas médicas ou de hospitalização realizadas no exterior também são dedutíveis, desde que  devidamente comprovadas com documentação idônea. os pagamentos efetuados em moeda estrangeira  devem ser convertidos em dólares dos estados unidos da américa, pelo seu valor fixado pela autoridade  monetária do país no qual as despesas foram realizadas, na data do pagamento e, em seguida, em reais  mediante utilização do valor do dólar dos estados unidos da américa, fixado para venda pelo banco central  do brasil para o último dia útil da primeira quinzena do mês anterior ao do pagamento.   atenção:    não são dedutíveis as despesas referentes a acompanhante, inclusive de quarto particular utilizado  por este.   despesas de internação em estabelecimento geriátrico são dedutíveis a título de hospitalização  apenas se o referido estabelecimento se enquadrar nas normas relativas a estabelecimentos  hospitalares editadas pelo ministério da saúde e tiver a licença de funcionamento aprovada pelas  autoridades competentes (municipais, estaduais ou federais).   não são admitidas deduções de despesas médicas ou de hospitalização que estejam cobertas por  apólices de seguro ou quando ressarcidas, por qualquer forma ou meio, por entidades de qualquer  espécie, nacionais ou estrangeiras.   são dedutíveis da base de cálculo do  as despesas médicas comprovadas independentemente  da especialidade, inclusive as relativas à realização de cirurgia plástica, reparadora ou não, com a  finalidade de prevenir, manter ou recuperar a saúde, física ou mental, do paciente.   as despesas com prótese de silicone não são dedutíveis, exceto quando o valor dela integrar a  conta emitida pelo estabelecimento hospitalar relativamente a uma despesa médica dedutível.   as despesas com instrumentador cirúrgico somente poderão ser deduzidas quando o valor integrar  a conta emitida pelo estabelecimento hospitalar, relativamente a uma despesa médica dedutível. ',
 'como deve ser comprovada a despesa médica dedutível?   as despesas médicas devem ser especificadas e comprovadas mediante documentação hábil e idônea.   na hipótese de o comprovante de pagamento do serviço médico prestado ter sido emitido em nome do  contribuinte sem a especificação do beneficiário do serviço, pode-se presumir que esse foi o próprio  contribuinte, exceto quando, a juízo da autoridade fiscal, forem constatados razoáveis indícios de  irregularidades.   no caso de o serviço médico ter sido prestado a dependente do contribuinte, sem a especificação do  beneficiário do serviço no comprovante, essa informação poderá ser prestada por outros meios de prova,  inclusive por declaração do profissional ou da empresa emissora do referido documento comprobatório. ',
 'os valores pagos na prestação dos serviços de coleta, seleção e armazenagem de células- tronco oriundas de cordão umbilical são dedutíveis como despesas medicas na declaração de  ajuste anual?   não. os valores pagos na prestação dos serviços de coleta, seleção e armazenagem de células-tronco  oriundas de cordão umbilical, não se constituem em despesas médicas dedutíveis, uma vez que não se  referem a tratamento de doenças ou recuperação da saúde física e mental.   ',
 'são dedutíveis como despesa médica os gastos com internação hospitalar efetuados na  própria residência do paciente?   é dedutível a despesa com internação hospitalar efetuada em residência, somente se essa despesa integrar  a fatura emitida por estabelecimento hospitalar.       ',
 'que se consideram aparelhos ortopédicos e próteses ortopédicas para fins de dedução  como despesas médicas?   consideram-se aparelhos ortopédicos e próteses ortopédicas pernas e braços mecânicos, cadeiras de rodas,  andadores ortopédicos, palmilhas e calçados ortopédicos, e qualquer outro aparelho ortopédico destinado à  correção de desvio de coluna ou defeitos dos membros ou das articulações.  ',
 'marcapasso é dedutível?   sim, desde que o seu valor esteja incluído na conta hospitalar ou na conta emitida pelo profissional. ',
 'são considerados dedutíveis os gastos com parafusos e placas nas cirurgias ortopédicas ou  odontológicas?   sim, contanto que integrem a conta emitida pelo estabelecimento hospitalar ou pelo profissional.  ',
 'quais os gastos que se enquadram no conceito de prótese dentária?   enquadram-se no conceito de prótese dentária os aparelhos que substituem dentes, tais como dentaduras,  coroas, pontes. ',
 'gasto com colocação e manutenção de aparelho ortodôntico é dedutível como despesa  médica?   sim, desde que comprovado. entretanto, o gasto com a aquisição do aparelho ortodôntico somente é dedutível  se integrar a conta emitida pelo profissional. ',
 'gasto com colocação de lente intraocular em cirurgia de catarata pode ser considerado  como despesa médica?   sim, é considerada despesa médica a cirurgia para a colocação de lente intraocular.  valor referente à lente  é dedutível se integrar a conta emitida pelo profissional ou estabelecimento hospitalar.  ',
 'são dedutíveis os gastos com transfusão de sangue, bem como os pagamentos feitos a  laboratórios de análises clínicas e radiológicas, correspondentes a serviços prestados ao  contribuinte e seus dependentes?   sim, desde que tais serviços sejam prestados por profissionais legalmente habilitados (médicos e dentistas)  ou por empresas especializadas constituídas por esses profissionais.',
 'podem ser deduzidos os pagamentos feitos a assistente social, massagista e enfermeiro?  as despesas efetuadas com esses profissionais são dedutíveis desde que realizadas por motivo de internação  do contribuinte ou de seus dependentes e integrem a fatura emitida pelo estabelecimento hospitalar.',
 'os gastos com  móvel podem ser deduzidos como despesa hospitalar?   podem ser deduzidas da base de cálculo do , desde que comprovadas por meio de documentação hábil  e idônea, as seguintes despesas:   1) atendimento domiciliar dos serviços de saúde previstos na alínea “a” do inciso  do art. 8º da lei nº 9.250,  de 26 de dezembro de 1995;   2) atendimento pré-hospitalar de urgência, desde que prestado por meio de  móvel, instalada em  ambulância de suporte avançado (tipo “”) ou em aeronave de suporte médico (tipo “”); e    3) atendimento pré-hospitalar de emergência, realizado por meio de  móvel, instalada em ambulância tipo  “”, “”, “” ou “”, quando necessariamente conte com a presença de um profissional médico e possua em  seu interior equipamentos que possibilitem oferecer ao paciente suporte avançado de vida.  ',
 'há limite para dedução dos pagamentos efetuados pelo contribuinte a instituições que  oferecem cobertura de despesas médico-hospitalares, comumente denominadas de seguro-saúde?   não. pode ser deduzido o total dos valores das prestações mensais pagas para participação em planos de  saúde que assegurem direitos de atendimento ou ressarcimento de despesas de natureza médica,  odontológica ou hospitalar, operados por empresas domiciliadas no brasil, em benefício próprio ou de seus  dependentes relacionados na declaração de ajuste anual.   essa dedução pode ser usufruída pelo contribuinte pessoa física, quer o contrato de prestação de planos de  saúde seja efetuado diretamente entre o participante e a empresa prestadora ou entre esta e a empresa  empregadora do participante, desde que os pagamentos sejam desembolsados pelo contribuinte.    dedução a esse título é condicionada a que os pagamentos sejam especificados, informados na ficha  pagamentos efetuados da declaração de ajuste anual, e, quando requisitados, comprovados com  documentação contendo o nome, o endereço e o número de inscrição no cadastro nacional da pessoa  jurídica (da empresa, podendo, na sua falta, ser feita indicação do cheque nominativo com que se  efetuou o pagamento.   ',
 'empregados que tenham despesas médicas pagas pelos empregadores  sofrem desconto  parcelado dessas despesas em seus salários podem deduzir os valores descontados?   os desembolsos relativos a despesa médica ou dentária ocorridos no ano podem ser deduzidos pelo  contribuinte que suporta o encargo, desde que os descontos venham devidamente discriminados no  documento da fonte pagadora.',
 'como informar na declaração de rendimentos as despesas médicas pagas pelo contribuinte,  mas reembolsadas pelo empregador ou empresa de seguro-saúde?   se o reembolso for parcial, o valor dedutível como despesa médica é a diferença entre o valor gasto e o  reembolsado. na ficha pagamentos efetuados da declaração no campo “valor pago” deve ser informado o  valor total da despesa paga e no campo “parcela não dedutível/valor reembolsado” o valor reembolsado. ',
 'como declarar o reembolso de despesa médica recebido em ano-calendário posterior ao de  sua dedução?    reembolso deve ser informado na ficha “rendimentos tributáveis recebidos de pessoas jurídicas” da  declaração de ajuste anual correspondente ao ano-calendário de seu recebimento.    (regulamento do imposto sobre a renda - /2018, arts. 33 e 34, aprovado pelo decreto nº 9.580,  de 22 de novembro de 2018.',
 'como declarar as despesas com instrução de pessoa com deficiência física ou mental?   devem ser declaradas como despesas médicas. para a pessoa com deficiência física ou mental, é exigido  laudo médico atestando o estado de deficiência, e os pagamentos devem ser feitos a entidades especializadas  para esse fim.',
 'os gastos com medicamentos, inclusive vacinas, podem ser deduzidos como despesas  médicas?   não, a não ser que integrem a conta emitida pelo estabelecimento hospitalar.',
 'as despesas médico-hospitalares, em decorrência de parto, podem ser deduzidas pelo  marido (ou companheiro) quando a mulher (ou companheira) faz declaração em separado?   via de regra, as despesas médico-hospitalares próprias de um dos cônjuges ou companheiro(a) não podem  ser deduzidas pelo outro quando este apresenta declaração em separado. contudo, como se trata de  despesas necessárias ao parto de filho comum, as importâncias despendidas podem ser deduzidas por  qualquer dos dois. ',
 'pagamentos realizados no âmbito da reprodução assistida, com o emprego do procedimento  maternidade sub-rogada (“barriga de aluguel”), são dedutíveis?   por serem despesas com doadora temporária do útero, não são dedutíveis na declaração de ajuste anual    nem do pai nem da mãe, pois não configura despesa com dependente destes para fins do imposto  sobre a renda, mesmo que efetuados a médicos ou a hospitais, assim como, as despesas com exames  laboratoriais.  ',
 'as despesas médico-hospitalares e referentes a exames laboratoriais realizados no âmbito  de procedimento de reprodução assistida por fertilização in vitro são dedutíveis?   sim. nos casos de declaração em separado, os pagamentos efetuados a médicos e a hospitais, assim como  as despesas com exames laboratoriais, realizados no âmbito de procedimento de reprodução assistida por  fertilização in vitro, devidamente comprovados, são dedutíveis somente na declaração de ajuste anual da  esposa/companheira, que é a paciente do tratamento médico. se a esposa/companheira for dependente do  declarante, a despesa com fertilização in vitro será dedutível na declaração deste.',
 'contribuinte, titular de plano de saúde, pode deduzir o valor integral pago ao plano,  incluindo os valores referentes ao cônjuge e aos filhos quando estes declarem em separado?  a  pessoa física que constou como beneficiário em plano de saúde de outra pode deduzir as suas  despesas?     contribuinte, titular de plano de saúde, não pode deduzir os valores referentes ao cônjuge e aos filhos  quando estes declarem em separado, pois somente são dedutíveis na declaração os valores pagos a planos  de saúde de pessoas físicas consideradas dependentes perante a legislação tributária e incluídas na  declaração do responsável em que forem consideradas dependentes.   na hipótese de apresentação de declaração em separado, são dedutíveis as despesas com instrução ou  médica ou com plano de saúde relativas ao tratamento do declarante e de dependentes incluídos na  declaração, cujo ônus financeiro tenha sido suportado por um terceiro, se este for integrante da entidade  familiar, não havendo, nesse caso, a necessidade de comprovação do ônus. entretanto, se o terceiro não for  integrante da entidade familiar, há que se comprovar a transferência de recursos, para este, de alguém que  faça parte da entidade familiar.    entidade familiar compreende todos os ascendentes e descendentes do declarante, bem como as demais  pessoas físicas consideradas seus dependentes perante a legislação tributária.      163    comprovação do ônus financeiro deve ser feita mediante documentação hábil e idônea, tais como contrato  de prestação de serviço ou declaração do plano de saúde e comprovante da transferência de recursos ao  titular do plano.   aplica-se o conceito de entidade familiar tanto aos valores pagos a empresas operadoras de planos de saúde,  destinados a cobrir planos de saúde, como às despesas pagas diretamente aos profissionais ou prestadores  de serviços de saúde, bem assim aos pagamentos de despesas com instrução, do contribuinte e de seus  dependentes. ',
 'são dedutíveis as despesas médicas e com instrução de cônjuge e filho não incluídos como  dependentes na declaração de ajuste de quem efetuou o pagamento dessas despesas?   não. como regra geral, somente são dedutíveis na declaração as despesas médicas e com instrução de  pessoas físicas consideradas dependentes perante a legislação tributária e incluídas na declaração do  responsável em que for considerado dependente.   contudo, podem ser deduzidas na declaração as despesas médicas e com instrução pagas pelo declarante  referentes a alimentandos, desde que em cumprimento de decisão judicial ou acordo homologado  judicialmente, ou por escritura pública, observados os limites legais. ',
 'são dedutíveis os gastos efetuados com passagem e hospedagem no brasil ou no exterior  para fins de tratamento médico ou hospitalar pelo contribuinte ou dependente?   não, por falta de previsão legal. ',
 'são dedutíveis os gastos efetuados com médico não residente no brasil?   sim, desde que tais despesas sejam comprovadas.   ',
 'podem ser deduzidas as despesas efetuadas com instrução?   sim. são dedutíveis os pagamentos de despesas com instrução do contribuinte e de seus dependentes  relacionados na declaração, inclusive de alimentandos, em decorrência de cumprimento de decisão judicial,  ou de acordo homologado judicialmente ou de escritura pública, efetuados a estabelecimentos de ensino,  relativamente:   1. à educação infantil, compreendendo as creches e as pré-escolas;   2. ao ensino fundamental;   3. ao ensino médio;   4. à educação superior, compreendendo os cursos de graduação e de pós-graduação (mestrado, doutorado  e especialização);   5. à educação profissional, compreendendo o ensino técnico e o tecnológico.   ',
 'as deduções de despesas com instrução estão sujeitas a algum limite?   sim. estão sujeitas ao limite anual individual de $ 3.561,50, para o ano-calendário de 2019.  valor dos  gastos com um dependente que ultrapassar esse limite não pode ser aproveitado nem mesmo para  compensar gastos de valor inferior a $ 3.561,50 efetuados com o próprio contribuinte ou com outro  dependente ou alimentando.  ',
 'limite global para a dedução de despesas com instrução compreende somente o pagamento  de mensalidade e anuidade escolar?   sim. não se enquadram no conceito de despesas com instrução, por exemplo, as efetuadas com uniforme,  transporte, material escolar e didático, com a aquisição de máquina de calcular e microcomputador.  ',
 'que se considera educação infantil?    educação infantil é aquela que precede o ensino fundamental obrigatório, oferecida em creches ou entidades  equivalentes e pré-escolas, compreendendo as despesas efetuadas com a educação de crianças de até cinco  anos de idade. ',
 'que se considera curso de especialização?   considera-se curso de especialização aquele que se realiza após a graduação em curso superior e atende às  exigências de instituições de ensino. nesse conceito enquadram-se, por exemplo, os cursos de pós- graduação lato sensu.',
 'que é curso profissionalizante?    educação profissional compreende os seguintes níveis:    - técnico, destinado a proporcionar habilitação profissional a alunos matriculados ou egressos de ensino  médio, e cuja titulação pressupõe a conclusão da educação básica, formada pela educação infantil, ensino  fundamental e ensino médio;    - tecnológico, corresponde a cursos de nível superior na área tecnológica, destinados a egressos do ensino  médio e técnico.  ',
 'despesas com creche podem ser deduzidas como instrução?   sim. esses gastos são considerados despesas com instrução, obedecidos aos limites e às condições legais.  ',
 'contribuinte pode deduzir despesas de instrução com filho ou enteado dependente?   sim, até 21 anos, ou até 24 anos se o filho ou enteado estiver cursando estabelecimento de nível superior ou  escola técnica de segundo grau. caso o dependente tenha rendimentos próprios, estes devem ser somados  aos do responsável na declaração anual. ',
 'ocorrendo o divórcio ou separação judicial durante o ano-calendário, o contribuinte que não  detém a guarda dos filhos pode deduzir as despesas com instrução com eles efetuadas antes do  divórcio?   sim. as despesas com instrução efetuadas antes do divórcio podem ser deduzidas desde que os filhos figurem  como dependentes na declaração do ano-calendário relativo ao divórcio. admite-se a dedução até o limite  anual individual de $ 3.561,50 para o ano-calendário de 2019.   ',
 'contribuinte que pague instrução de neto, bisneto, irmão, primo ou sobrinho pode deduzir  essas despesas?    laço de parentesco, bem como o efetivo pagamento das despesas com a instrução dessas pessoas, não  são condições suficientes para permitir sua dedução pelo parente que suporta o encargo. esta só é permitida  quando o beneficiado possa ser enquadrado na condição de dependente do contribuinte.   podem ser dedutíveis as despesas com instrução de irmão, neto ou bisneto, sem arrimo dos pais, até 21 anos,  desde que o contribuinte detenha a guarda judicial, ou de qualquer idade, quando incapacitado física ou  mentalmente para o trabalho.   no caso de irmão, neto ou bisneto sem arrimo dos pais, com idade de 21 a 24 anos, a dedução é possível se  o dependente ainda estiver cursando estabelecimento de ensino superior ou escola técnica de 2º grau, desde  que o contribuinte tenha detido sua guarda judicial até os 21 anos.    no caso de primo ou sobrinho, pode ser feita a dedução somente quando esse se enquadrar como menor  pobre e desde que o contribuinte o crie e eduque, até que complete 21 anos e detenha sua guarda judicial.  ',
 'podem ser deduzidas as despesas com instrução de menor pobre?   essas despesas podem ser deduzidas desde que o contribuinte crie e eduque o menor pobre, até que este  complete 21 anos, e detenha a guarda judicial nos termos da lei nº 8.069, de 13 de julho de 1990 (estatuto  da criança e do adolescente).   ',
 'as despesas com instrução pagas em face das normas do direito de família, em virtude de  sentença judicial, estão sujeitas ao limite anual?   sim. estas despesas estão sujeitas ao limite individual anual de $ 3.561,50. ',
 'as despesas relativas à elaboração de dissertação de mestrado ou tese de doutorado podem  ser deduzidas como gastos com instrução?   não. as despesas relativas à elaboração de dissertação de mestrado ou tese de doutorado, tais como:  contratação de estagiários, computação eletrônica de dados, papel, fotocópia, datilografia, digitação, tradução  de textos, impressão de questionários e de tese elaborada, gastos postais e de viagem, não são consideradas  despesas de instrução.  ',
 'as despesas com a aquisição de enciclopédias, livros, publicações e materiais técnicos  podem ser deduzidas?   não.  valor relativo à aquisição dessas publicações não pode ser deduzido na declaração de ajuste anual.',
 'os pagamentos de cursos preparatórios para concursos ou vestibulares, bem como as  respectivas taxas de inscrição, podem ser deduzidos como despesas de instrução?   esses pagamentos não são dedutíveis por falta de previsão legal. ',
 'os pagamentos de aulas de idioma estrangeiro, música, dança, natação, ginástica, dicção,  corte e costura, aulas de trânsito, tênis ou pilotagem podem ser deduzidos como despesas com  instrução?   não são dedutíveis por falta de previsão legal.  ',
 'as despesas efetuadas com instrução de menor internado em instituição que crie e eduque  desvalidos e abandonados podem ser deduzidas?   não são dedutíveis como despesas com instrução os pagamentos feitos a instituição que crie e eduque  desvalidos e abandonados, ainda que sejam destinados ao custeio de sua educação.',
 'as contribuições pagas às associações de pais e mestres são consideradas despesas com  instrução?   não. tais pagamentos não se enquadram no conceito de despesas com instrução.  ',
 'valor dos recursos remetidos para dependente que estude no exterior pode ser deduzido  como despesas de instrução na declaração do contribuinte?   podem ser deduzidos apenas os valores relativos a despesas de instrução, em estabelecimentos de ensino  regular, comprovadas por meio de documentação hábil, realizadas no exterior com dependentes, observados  os requisitos e o limite previstos na legislação.   os pagamentos efetuados em moeda estrangeira devem ser convertidos em dólares dos estados unidos da  américa, pelo seu valor fixado pela autoridade monetária do país no qual as despesas foram realizadas, na  data do pagamento e, em seguida, em reais mediante utilização do valor do dólar dos estados unidos da  américa, fixado para venda pelo banco central do brasil para o último dia útil da primeira quinzena do mês  anterior ao do pagamento.  ',
 'são dedutíveis como instrução os gastos com viagens e estadas feitos pelo contribuinte, com  ele próprio ou com seus dependentes, a fim de estudar ou estagiar no brasil ou no exterior?   não. tais despesas não são dedutíveis por falta de previsão legal.   atenção:     valor do imposto sobre a renda retido sobre a remessa, no caso de estudo ou estágio no exterior,  não pode ser computado como despesa com instrução nem pode ser considerado para fins de  compensação na declaração de rendimentos da pessoa que suporta o encargo. ',
 'filho ou enteado maior de 21 e até 24 anos que trancou matrícula na faculdade pode ser  considerado dependente na declaração de ajuste anual do exercício de 2019?    filho ou enteado pode ser considerado dependente até 24 anos de idade, quando estiver cursando  estabelecimento de ensino superior ou escola técnica de 2º grau. assim, o filho que manteve a matrícula  trancada durante todo o ano-calendário de 2019 não pode ser considerado dependente na declaração.  ',
 'contribuinte assalariado que recebe do empregador ou de entidade a que esteja filiado, auxílio  para pagar despesas com instrução própria e de seus dependentes, pode deduzir o total das  despesas efetivamente realizadas?   sim, mas as importâncias recebidas para esse fim se constituem em rendimento tributável, qualquer que tenha  sido a designação adotada pelo empregador para intitular essas vantagens, inclusive na hipótese de a  empresa optar pelo reembolso diretamente aos empregados e filhos destes dos seus gastos com educação.   ',
 'são dedutíveis os valores ressarcidos pelo empregado, por motivo de rescisão contratual, ao  empregador que patrocinava suas despesas com instrução?   não se consideram despesas com instrução as importâncias pagas a título de indenização por perdas e danos,  por não cumprimento de cláusula contratual, não sendo, portanto, dedutíveis na declaração da pessoa física  que efetua o ressarcimento.',
 'pagamento do valor do crédito educativo pode ser deduzido como despesa com instrução?   não, por falta de previsão legal.  crédito educativo (programa de financiamento estudantil – , por  exemplo) caracteriza-se como empréstimo oneroso, com ônus e encargos próprios desses contratos.  valor  pago à instituição de ensino, ainda que com recursos do crédito educativo, pode ser deduzido como despesa  com instrução, observado os limites previstos na legislação, no ano do efetivo pagamento à instituição de  ensino.',
 'os gastos com construção de escola em imóvel do contribuinte, onde este exerce atividade  rural, assim como os dispêndios com pagamento de professor, para atender ao funcionamento  dessa escola, podem ser deduzidos como despesas de instrução de menores pobres?   não. essas despesas integram os gastos ligados à percepção dos rendimentos da atividade rural e são  consideradas despesas para efeito de apuração do resultado dessa atividade.     ',
 'gasto com matrícula escolar referente ao ano letivo de 2020 efetuada em 2019 é dedutível  na declaração de ajuste anual do exercício de 2020?   sim. uma vez que o pagamento ocorreu já no ano-calendário de 2019, essa despesa poderá ser deduzida na  declaração de ajuste anual   do exercício de 2020, obedecidos aos limites e condições legais. ressalte- se que caso o contribuinte não deduza essa despesa na  do exercício de 2020, não poderá fazê-lo na   do exercício de 2021. ',
 'quem pode deduzir as despesas escrituradas em livro-caixa?    contribuinte que receber rendimentos do trabalho não assalariado, inclusive os titulares de serviços notariais  e de registro e os leiloeiros podem deduzir, da receita decorrente do exercício da respectiva atividade, as  seguintes despesas escrituradas em livro-caixa:   1 - a remuneração paga a terceiros, desde que com vínculo empregatício, e os respectivos encargos  trabalhistas e previdenciários;   2 - os emolumentos pagos a terceiros, assim considerados os valores referentes à retribuição pela execução,  pelos serventuários públicos, de atos cartorários, judiciais e extrajudiciais;   3 - as despesas de custeio pagas, necessárias à percepção da receita e a manutenção da fonte produtora;      170   4 - as importâncias pagas, devidas aos empregados em decorrência das relações de trabalho, ainda que não  integrem a remuneração destes, caso configurem despesas necessárias à percepção da receita e à  manutenção da fonte produtora, observando-se que na hipótese de convenções e acordos coletivos de  trabalho, todas as prestações neles previstas e devidas ao empregado constituem obrigações do empregador  e, portanto, despesas necessárias à percepção da receita e à manutenção da fonte produtora.    excesso de deduções apurado no mês pode ser compensado nos meses seguintes, até dezembro, não  podendo ser transposto para o ano seguinte.   atenção:    não são dedutíveis:    - as quotas de depreciação de instalações, máquinas e equipamentos, bem como as despesas de  arrendamento (leasing);   - as despesas de locomoção e transporte, salvo no caso de representante comercial autônomo,  quando correrem por conta deste;   - as despesas relacionadas à prestação de serviços de transporte e aos rendimentos auferidos  pelos garimpeiros.   as despesas de custeio escrituradas em livro-caixa podem ser deduzidas independentemente de  as receitas serem oriundas de serviços prestados como autônomo a pessoa física ou jurídica. ',
 'que se considera e qual é o limite mensal da despesa de custeio passível de dedução no  livro-caixa?   considera-se despesa de custeio aquela indispensável à percepção da receita e à manutenção da fonte  produtora, como aluguel, água, luz, telefone, material de expediente ou de consumo.    valor das despesas dedutíveis, escrituradas em livro-caixa, está limitado ao valor da receita mensal recebida  de pessoa física ou jurídica.   no caso de as despesas escrituradas no livro-caixa excederem as receitas recebidas por serviços prestados  como autônomo a pessoa física e/ou jurídica em determinado mês, o excesso pode ser somado às despesas  dos meses subsequentes até dezembro do ano-calendário.  excesso de despesas existente em dezembro  não deve ser informado nesse mês nem transposto para o próximo ano-calendário. ',
 'as despesas de custeio escrituradas no livro-caixa podem ser deduzidas independentemente  de as receitas serem oriundas de serviços prestados como autônomo a pessoa física ou jurídica?    profissional autônomo pode escriturar o livro-caixa para deduzir as despesas de custeio, necessárias à  percepção da receita e à manutenção da fonte produtora.  receita e a despesa devem manter correlação  com a atividade, independentemente se a prestação de serviços foi feita para pessoas físicas ou jurídicas.    excesso de deduções apurado no mês pode ser compensado nos meses seguintes, até dezembro.    excedente de que trata o parágrafo anterior, porventura existente no final do ano-calendário, não pode ser  transposto para o ano seguinte. ',
 'qual é o alcance da expressão "serviços notariais e de registro", inserida no caput do art. 11  da lei nº 7.713, de 22 de dezembro de 1988?   esta expressão alcança apenas os titulares dos serviços notariais e de registros em geral, exercidos em  caráter privado, por delegação do poder público, não se estendendo às pessoas que para eles trabalham,  assalariados ou autônomos. ',
 'livro-caixa pode ser escriturado em formulário contínuo?   sim. é permitida a escrituração fiscal do livro-caixa pelo sistema de processamento eletrônico, em formulários  contínuos, com suas subdivisões numeradas em ordem sequencial ou tipograficamente. após o  processamento, os impressos devem ser destacados e encadernados em forma de livro, lavrados os termos  de abertura e de encerramento em que conste, no termo de abertura, o número de folhas já escrituradas, não  contendo intervalo em branco, nem entrelinhas, borraduras, raspaduras ou emendas.    secretaria especial da receita federal do brasil   disponibiliza, no sítio http://receita.economia.gov.br,  o programa carnê-leão que permite a escrituração do livro-caixa pelo sistema de processamento eletrônico. ',
 'podem ser aceitos tíquetes de caixa, recibos não identificados e documentos semelhantes  para comprovar despesas no livro-caixa?   não. para que tais despesas sejam dedutíveis, o documento fiscal deve conter a perfeita identificação do  adquirente e das despesas realizadas, sendo que estas devem ser necessárias e indispensáveis à  manutenção da fonte produtora dos rendimentos. ',
 'as despesas com transporte, locomoção, combustível, estacionamento e manutenção de  veículo próprio são consideradas necessárias à percepção da receita e dedutíveis no livro-caixa?   referidas despesas não são dedutíveis, com exceção das efetuadas por representante comercial autônomo  quando correrem por conta deste.  ',
 'contribuinte autônomo pode utilizar como despesa dedutível no livro-caixa o valor pago na  aquisição de bens ou direitos indispensáveis ao exercício da atividade profissional?   apenas o valor relativo às despesas de consumo é dedutível no livro-caixa. deve-se, portanto, identificar  quando se trata de despesa ou de aplicação de capital.   são despesas dedutíveis as quantias despendidas na aquisição de bens próprios para o consumo, tais como  material de escritório, de conservação, de limpeza e de produtos de qualquer natureza usados e consumidos  nos tratamentos, reparos, conservação.   considera-se aplicação de capital o dispêndio com aquisição de bens necessários à manutenção da fonte  produtora, cuja vida útil ultrapasse o período de um exercício, e que não sejam consumíveis, isto é, não se  extingam com sua mera utilização. por exemplo, os valores despendidos na instalação de escritório ou  consultório, na aquisição e instalação de máquinas, equipamentos, instrumentos, mobiliários etc. tais bens  devem ser informados na ficha “bens e direitos” da declaração de rendimentos pelo preço de aquisição e,  quando alienados, deve-se apurar o ganho de capital.   http://receita.economia.gov.br/',
 
 'podem ser deduzidas as despesas com aluguel, energia, água, gás, taxas, impostos, telefone,  telefone celular, condomínio, quando o imóvel utilizado para a atividade profissional é também  residência?   admite-se como dedução a quinta parte dessas despesas, quando não se possa comprovar quais as oriundas  da atividade profissional exercida. não são dedutíveis os dispêndios com reparos, conservação e recuperação  do imóvel quando este for de propriedade do contribuinte.',
 'qual é o tratamento tributário das despesas com benfeitorias, efetuadas pelo profissional  autônomo em imóvel locado?   as despesas com benfeitorias e melhoramentos efetuadas pelo locatário profissional autônomo, que  contratualmente fizerem parte como compensação pelo uso do imóvel locado, são dedutíveis no mês de seu  dispêndio, como valor locativo, desde que tais gastos estejam comprovados com documentação hábil e idônea  e escriturados em livro-caixa.',
 'profissional autônomo pode deduzir as despesas com aquisição de livros, jornais, revistas,  roupas especiais etc.?   sim, caso o profissional exerça funções e atribuições que o obriguem a comprar roupas especiais e  publicações necessárias ao desempenho de suas funções e desde que os gastos estejam comprovados com  documentação hábil e idônea e escriturados em livro-caixa.',
 'as contribuições a sindicatos de classe, associações científicas e outras associações podem  ser deduzidas?   essas contribuições são dedutíveis desde que a participação nas entidades seja necessária à percepção do  rendimento e as despesas estejam comprovadas com documentação hábil e idônea e escrituradas em livro- caixa.',
 'são dedutíveis os pagamentos efetuados por profissional autônomo a terceiros?   sim.  profissional autônomo pode deduzir no livro-caixa os pagamentos efetuados a terceiros com quem  mantenha vínculo empregatício. podem também ser deduzidos os pagamentos efetuados a terceiros sem  vínculo empregatício, desde que caracterizem despesa de custeio necessária à percepção da receita e à  manutenção da fonte produtora. ',
 'como o autônomo informa a prestação de serviço feita exclusivamente a pessoa jurídica?    autônomo que prestou serviços exclusivamente a pessoa jurídica e escriturou o livro-caixa deve incluir o  rendimento recebido na ficha rendimentos tributáveis recebidos de pessoas jurídicas, da declaração de  ajuste anual. as deduções, limitadas ao valor dos rendimentos de trabalho não assalariado recebidos, devem  ser incluídas na coluna livro-caixa da ficha rendimentos tributáveis recebidos de pessoas físicas e do  exterior, deixando em branco as demais colunas. ',
 'os pagamentos efetuados por profissional autônomo a terceiros correspondentes a serviços  prestados em anos anteriores podem ser deduzidos?   sim. tais pagamentos são dedutíveis no mês de sua quitação, não obstante se referirem a serviços prestados  em anos anteriores, desde que escriturados em livro-caixa.',
 'despesas com propaganda da atividade profissional são dedutíveis?   sim, desde que a propaganda se relacione com a atividade profissional da pessoa física e esses gastos  estejam escriturados em livro-caixa e comprovados com documentação hábil e idônea.  ',
 'gastos relativos a participação em congressos e seminários por profissional autônomo são  dedutíveis?   sim. as despesas efetuadas para comparecimento a encontros científicos, como congressos, seminários etc.,  se necessárias ao desempenho da função desenvolvida pelo contribuinte, observada, ainda, a sua  especialização profissional, podem ser deduzidas, tais como os valores relativos a taxas de inscrição e  comparecimento, aquisição de impressos e livros, materiais de estudo e trabalho, hospedagem, transporte,  desde que esses dispêndios sejam escriturados em livro-caixa, comprovados por documentação hábil e  idônea e não sejam reembolsados ou ressarcidos.  contribuinte deve guardar o certificado de  comparecimento dado pelos organizadores desses encontros. ',
 'quais são os valores passíveis de exclusão dos rendimentos de aluguéis?   podem ser excluídos do valor do aluguel recebido, quando o encargo tenha sido exclusivamente do locador,  as quantias relativas a:     impostos, taxas e emolumentos incidentes sobre o bem que produzir o rendimento;     aluguel pago pela locação de imóvel sublocado;     despesas pagas para cobrança ou recebimento do rendimento; e     despesas de condomínio.',
 'os valores referentes ao pagamento do  são passíveis de exclusão dos rendimentos de  aluguéis?   sim, desde que o encargo tenha sido do locador.  podem ser excluídos do valor do aluguel recebido, quando o encargo tenha sido do locador, as quantias  relativas ao pagamento do imposto sobre a propriedade predial e territorial urbana  relativo ao imóvel  locado, independentemente se a percepção dos rendimentos ocorreu durante todo o ano ou somente em parte  dele, ou ainda, que o imposto tenha sido pago à vista ou parcelado, desde que dentro do ano-calendário em  que o rendimento de aluguel foi recebido.',
 'são dedutíveis dos rendimentos de aluguel as despesas com advogado para retirar inquilino  e a realização de reformas no imóvel para futura locação?   não são dedutíveis as referidas despesas por falta de previsão legal. ',
 'proprietário que aluga seu imóvel e por sua vez paga aluguéis como locatário pode deduzir  dos aluguéis recebidos o valor do aluguel pago?   não. pode ser excluído apenas o aluguel pago pela locação de imóvel sublocado.  ',
 'contribuinte que auferir rendimentos da prestação de serviços de transporte de carga ou  de passageiros, em veículo próprio, pode tributar rendimentos em percentual inferior a 10% e 60%,  respectivamente, utilizando como dedução do livro-caixa o percentual excedente?   não, pois constitui rendimento mínimo a ser tributado 10% do rendimento total decorrente do transporte de  carga e 60% do rendimento total decorrente do transporte de passageiros.    valor não tributado é considerado renda consumida, não podendo justificar acréscimo patrimonial, nem ser  considerado despesa dedutível escriturada em livro-caixa, tendo em vista a vedação legal. ',
 'as doações efetuadas a candidatos a cargos eletivos e a partidos políticos, no ano-calendário  de 2019, podem ser deduzidas?   não, por falta de previsão legal.    no entanto, o doador deverá relacionar na declaração de ajuste anual todas as doações efetuadas a partidos  políticos e a candidatos a cargos eletivos, inclusive os gastos, no ano-calendário de 2019, informando o  número de inscrição no cadastro nacional da pessoa jurídica  e o nome do candidato ou partido  político a quem efetuou doações e o valor doado.   as doações e contribuições em dinheiro ou estimáveis em dinheiro para campanhas eleitorais, efetuadas por  pessoas físicas, ficam limitadas a 10% (dez por cento) dos rendimentos brutos auferidos pelo doador no ano  anterior à eleição.   atenção:    1) os valores doados não constituem dedução do imposto sobre a renda.   2) são considerados gastos eleitorais, sujeitos a registro e aos limites fixados na lei nº 9.504, de 30  de setembro de 1997:   a – confecção de material impresso de qualquer natureza e tamanho, observado que adesivos  poderão ter a dimensão máxima de 50 (cinquenta) centímetros por 40 (quarenta) centímetros;   b – propaganda e publicidade direta ou indireta, por qualquer meio de divulgação, destinada a  conquistar votos;   c – aluguel de locais para a promoção de atos de campanha eleitoral;   d – despesas com transporte ou deslocamento de candidato e de pessoal a serviço das  candidaturas;   e – correspondência e despesas postais;   f – despesas de instalação, organização e funcionamento de comitês e serviços necessários às  eleições;   g – remuneração ou gratificação de qualquer espécie a pessoal que preste serviços às candidaturas  ou aos comitês eleitorais;   h – montagem e operação de carros de som, de propaganda e assemelhados;   i – a realização de comícios ou eventos destinados à promoção de candidatura;   j – produção de programas de rádio, televisão ou vídeo, inclusive os destinados à propaganda  gratuita;   k – realização de pesquisas ou testes pré-eleitorais;   l – custos com a criação e inclusão de sítios na internet;   m – multas aplicadas aos partidos ou candidatos por infração do disposto na legislação eleitoral; e   n – produção de jingles, vinhetas e slogans para propaganda eleitoral. ',
 'honorários advocatícios e despesas judiciais podem ser diminuídos dos valores recebidos  em decorrência de ação judicial?   os honorários advocatícios e as despesas judiciais podem ser diminuídos dos rendimentos tributáveis, desde  que não sejam ressarcidas ou indenizadas sob qualquer forma. da mesma maneira, os gastos efetuados  anteriormente ao recebimento dos rendimentos podem ser diminuídos quando do recebimento dos  rendimentos.   os honorários advocatícios e as despesas judiciais pagos pelo contribuinte devem ser proporcionalizados  conforme a natureza dos rendimentos recebidos em ação judicial, isto é, entre os rendimentos tributáveis, os  sujeitos a tributação exclusiva e os isentos e não tributáveis.    contribuinte deve informar como rendimento tributável o valor recebido, já diminuído do valor pago ao  advogado, independentemente do modelo de formulário utilizado.   na declaração de ajuste anual, deve-se preencher a ficha pagamentos efetuados, informando o nome, o  número de inscrição no cadastro de pessoas físicas  e o valor pago ao beneficiário do pagamento (ex:  advogado).  ',
 'em qual ano–calendário são deduzidos, na declaração de ajuste anual, os honorários  advocatícios pagos em ano–calendário posterior ou anterior àquele em que os rendimentos  decorrentes de decisão judicial foram recebidos?   os honorários advocatícios devem ser deduzidos no ano-calendário em que os rendimentos decorrentes de  decisão judicial foram recebidos, e informados na ficha de pagamentos efetuados da declaração de ajuste  anual nos anos-calendário em que tais honorários forem pagos. ',
 'são dedutíveis as doações efetuadas a entidades filantrópicas, de educação, de pesquisa  científica ou de cultura?   essas doações não são dedutíveis, por falta de previsão legal. ',
 'quais os gastos com incentivos que podem ser deduzidos do valor do imposto apurado?   podem ser deduzidos os seguintes pagamentos, desde que efetuados no ano-calendário de 2019, referentes  a:    - estatuto da criança e do adolescente - contribuições aos fundos controlados pelos  conselhos municipais, estaduais, distrital e nacional dos direitos da criança e do adolescente:     pessoa física poderá deduzir do imposto sobre a renda devido na declaração de ajuste anua ,  observado o limite geral de dedução (ver item 1 do atenção na parte final desta resposta), as doações, em  espécie ou em bens, feitas a esses fundos.   as doações efetuadas em espécie deverão ser depositadas em conta específica, em instituição financeira  pública, vinculada ao fundo dos direitos da criança e do adolescente respectivo.   as doações deverão ser comprovadas por meio de recibo emitido em favor do doador, observado os termos  estabelecidos no atenção deste tópico .       na hipótese de doação em bens, o doador deverá:   1) comprovar a propriedade dos bens, por meio de documentação hábil;   2) baixar os bens doados na declaração de bens e direitos; e   3) considerar como valor dos bens doados o valor constante da última , desde que não exceda o  valor de mercado.   atenção :   doações aos fundos nacional, estaduais, distrital e municipais realizadas diretamente na  declaração de ajuste anual:     na declaração de ajuste anual  do exercício de 2020, ano-calendário de 2019, apresentada  até 30 de junho de 2020, quando utilizado o modelo de  que permite a opção pela utilização  das deduções legais, a pessoa física pode optar pela dedução das doações, em espécie, aos fundos  controlados pelos conselhos dos direitos da criança e do adolescente nacional, distrital, estaduais  ou municipais - observando-se o seguinte:   a) as doações poderão ser deduzidas até o percentual de 3% sobre o imposto sobre a renda devido  apurado na declaração;   b) a dedução está sujeita ainda ao limite global de 6% (seis por cento) do imposto sobre a renda  devido apurado na declaração, juntamente com as demais deduções de incentivo, inclusive quanto  às contribuições efetuadas aos fundos controlados pelos conselhos dos direitos da criança e do  adolescente nacional, distrital, estaduais ou municipais no decorrer do ano-calendário de 2019;    c) o pagamento da doação deve ser efetuado, impreterivelmente, até 30 de junho de 2020, até o  encerramento do horário de expediente bancário das instituições financeiras autorizadas, inclusive  se realizado pela internet ou por terminal de autoatendimento;   d) o não pagamento da doação até 30 de junho de 2020 implica a glosa definitiva dessa parcela de  dedução, e obriga a pessoa física ao recolhimento da diferença de imposto devido apurado na   com os acréscimos legais previstos na legislação.   e) após 30 de junho de 2020, não será admitida a retificação que tenha por objetivo o aumento do  montante dedutível;   f) o programa da  emitirá um darf para o pagamento de cada doação ao fundo beneficiário  indicado, no valor informado pelo declarante e com código de receita 3351, que não se confunde  com o darf emitido para pagamento de eventual saldo de imposto sobre a renda devido;   g) o pagamento da doação informada na  deverá ser realizado mesmo que a pessoa física  tenha direito a restituição ou tenha optado pelo pagamento do saldo de imposto por meio de débito  automático em conta-corrente bancária;   h) uma vez recolhido o montante indicado no darf, a doação efetuada ao fundo nele indicado torna- se irreversível e eventual valor recolhido a maior que o passível de dedução será também  repassado ao fundo indicado, não cabendo devolução, compensação ou dedução desse valor;   i) se o valor recolhido for menor que o informado na declaração, o contribuinte:   i.1) poderá, até 30 de junho de 2020, complementar o recolhimento; ou   i.2) deverá, dentro do prazo decadencial e desde que não esteja sob procedimento de ofício,  retificar a  para corrigir a informação referente ao valor doado;   j) se o valor recolhido for maior que o informado na declaração, o contribuinte:   j.1) poderá, até 30 de junho de 2020, retificar a  para corrigir a informação referente ao valor  doado, respeitados o limite individual de 3% (três por cento) e o limite global de 6% (seis por cento);  ou   j.2) deverá considerar como não dedutível o valor recolhido que ultrapassar o limite individual de  3% (três por cento) e o limite global de 6% (seis por cento), observado que esse valor a maior será  também repassado ao fundo indicado;   k) o pagamento da doação não está sujeito a parcelamento.   consulte a pergunta 430     - fundos controlados pelos conselhos nacional, distrital, estaduais ou municipais do idoso;     pessoa física poderá deduzir do imposto sobre a renda devido na , observado o limite geral de  dedução (ver item 1 do atenção na parte final desta resposta), as contribuições feitas aos fundos do idoso  nacional, distrital, estaduais e municipais.   observar, em especial, o seguinte:   1) as importâncias deduzidas a título de doações sujeitam-se à comprovação, por meio de documentos  emitidos pelos conselhos gestores dos respectivos fundos;      178   2) as doações efetuadas em moeda devem ser depositadas em conta específica, aberta em instituição  financeira pública, vinculada ao respectivo fundo;   3) os conselhos municipais, estaduais ou nacional do idoso, controladores dos fundos beneficiados  pelas doações, devem emitir comprovante em favor do doador;   4) para fins de comprovação, cada fundo deverá registrar em sua escrituração os valores recebidos e  manter em boa guarda a documentação correspondente pelo prazo decadencial.   atenção (fundos idoso):   doações aos fundos nacional, estaduais, distrital e municipais do idoso – nova modalidade  de doação:    partir do exercício de 2020, ano-calendário de 2019, poderá ser realizada, diretamente na  declaração, quando utilizado o modelo de  que permite a opção pela utilização das deduções  legais, a pessoa física pode optar pela dedução das doações, em espécie, observado o seguinte:   a) as doações poderão ser deduzidas até o percentual de 3% sobre o imposto sobre a renda devido  apurado na declaração;   b) a dedução está sujeita ainda ao limite global de 6% do imposto sobre a renda devido apurado na  declaração, juntamente com as demais deduções de incentivo, inclusive quanto às contribuições  efetuadas aos fundos do idoso no decorrer do ano calendário de 2019;   c) o pagamento da doação deve ser efetuado, impreterivelmente, até 30 de junho de 2020, até o  encerramento do horário de expediente bancário das instituições financeiras autorizadas, inclusive  se realizado pela internet ou por terminal de autoatendimento;   d) o não pagamento da doação até 30 de junho de 2020 implica a glosa definitiva dessa parcela de  dedução, e obriga a pessoa física ao recolhimento da diferença de imposto devido apurado na   com os acréscimos legais previstos na legislação.   e) após 30 de junho de 2020, não será admitida a retificação que tenha por objetivo o aumento do  montante dedutível;   f) o programa da  emitirá um darf para o pagamento de cada doação ao fundo beneficiário  indicado, no valor informado pelo declarante e com código de receita 9090, que não se confunde  com o darf emitido para pagamento de eventual saldo de imposto sobre a renda devido;   g) o pagamento da doação informada na  deverá ser realizado mesmo que a pessoa física  tenha direito a restituição ou tenha optado pelo pagamento do saldo de imposto por meio de débito  automático em conta-corrente bancária;   h) uma vez recolhido o montante indicado no darf, a doação efetuada ao fundo nele indicado torna- se irreversível e eventual valor recolhido a maior que o passível de dedução será também  repassado ao fundo indicado, não cabendo devolução, compensação ou dedução desse valor;   i) se o valor recolhido for menor que o informado na declaração, o contribuinte:   i.1) poderá, até 30 de junho de 2020, complementar o recolhimento; ou   i.2) deverá, dentro do prazo decadencial e desde que não esteja sob procedimento de ofício,  retificar a  para corrigir a informação referente ao valor doado;   j) se o valor recolhido for maior que o informado na declaração, o contribuinte:   j.1) poderá, até 30 de junho de 2020, retificar a  para corrigir a informação referente ao valor  doado, respeitados o limite individual de 3% (três por cento) e o limite global de 6% (seis por cento);  ou   j.2) deverá considerar como não dedutível o valor recolhido que ultrapassar o limite individual de  3% (três por cento) e o limite global de 6% (seis por cento), observado que esse valor a maior será  também repassado ao fundo indicado;   k) o pagamento da doação não está sujeito a parcelamento.    - incentivo à cultura - a título de doações ou patrocínios, tanto mediante contribuições ao fundo  nacional de cultura   como em apoio direto, desde que enquadrados nos objetivos do programa  nacional de apoio à cultura, a programas, projetos e ações culturais:   1) em geral, de natureza cultural, com o objetivo de desenvolver as formas de expressão, os modos de  criar e fazer, os processos de preservação e proteção do patrimônio cultural brasileiro, e os estudos e  métodos de interpretação da realidade cultural, bem como contribuir para propiciar meios, à população em  geral, que permitam o conhecimento dos bens de valores artísticos e culturais, compreendendo, entre  outros, os seguintes segmentos (lei nº 8.313, de 1991, art. 25):   a) teatro, dança, circo, ópera, mímica e congêneres;   b) produção cinematográfica, videográfica, fotográfica, discográfica e congêneres;      179   c) literatura, inclusive obras de referência;   d) música;   e) artes plásticas, artes gráficas, gravuras, cartazes, filatelia e outras congêneres;   f) folclore e artesanato;   g) patrimônio cultural, inclusive histórico, arquitetônico, arqueológico, bibliotecas, museus, arquivos e  demais acervos;   h) humanidades; e   i) rádio e televisão, educativas e culturais, de caráter não comercial.   2) exclusivos dos segmentos de (lei nº 8.313, de 1991, art. 18, caput e § 3º):   a) artes cênicas;   b) livros de valor artístico, literário ou humanístico;   c) música erudita ou instrumental;   d) exposições de artes visuais;   e) doações de acervos para bibliotecas públicas, museus, arquivos públicos e cinematecas, bem como  treinamento de pessoal e aquisição de equipamentos para a manutenção desses acervos;   f) produção de obras cinematográficas e videofonográficas de curta e média metragem e preservação  e difusão do acervo audiovisual;   g) preservação do patrimônio cultural material e imaterial; e   h) construção e manutenção de salas de cinema e teatro, que poderão funcionar também como centros  culturais comunitários, em municípios com menos de 100.000 (cem mil) habitantes.    dedutibilidade referente ao incentivo à cultura está condicionada a que:   - os projetos culturais sejam previamente aprovados pelo ministério da cultura (minc) ou, no caso de  projetos relacionados a obras cinematográficas e videofonográficas, pelo minc ou pela agência  nacional do cinema (ancine);   - o doador ou patrocinador obedeça, para suas doações ou patrocínios, o período para a captação de  recursos definido pelas portarias de homologação do minc ou ancine;   - o incentivo em espécie devem ser comprovados mediante recibo de depósito bancário e declaração  de recebimento firmada pelo beneficiário, nos termos estabelecidos pelo minc ou pela ancine; e   - o valor da dedução atenda ao limite mencionado no tópico “atenção”    - incentivo à atividade audiovisual - as quantias  aplicadas em:   1 - investimentos feitos na produção de obras audiovisuais cinematográficas brasileiras de produção  independente, mediante a aquisição de cotas representativas de direitos de comercialização sobre as  referidas obras;   2 - patrocínio feito à produção de obras cinematográficas brasileiras de produção independente;   3 - aquisição de cotas dos fundos de financiamento da indústria cinematográfica nacional (funcines).   4 - investimentos em projetos específicos credenciados pela agência nacional do cinema (ancine);   5 – patrocínios em projetos específicos ou em programas especiais de fomento instituídos pela ancine.    dedutibilidade referente ao incentivo à atividade audiovisual está condicionada a que:   - os investimentos previstos nos itens 1 e 3 sejam realizados no mercado de capitais, em ativos previstos  em lei, e autorizados pela comissão de valores mobiliários  ;   - os projetos ou programas a serem beneficiados pelos incentivos sejam previamente aprovados pela  ancine;   - o incentivo em espécie devem ser comprovados mediante recibo de depósito bancário e declaração  de recebimento firmada pelo beneficiário, nos termos estabelecidos pela ancine; e   - o valor da dedução atenda ao limite mencionado no tópico “atenção”    – incentivo ao desporto - doações ou patrocínios no apoio direto a projetos desportivos e  paradesportivos previamente aprovados pelo ministério do esporte;      180   os projetos desportivos atenderão a pelo menos uma das seguintes manifestações, nos termos, limites e  condições definidas em regulamento:   a) desporto educacional;   b) desporto de participação;    c) desporto de rendimento.    dedutibilidade referente ao incentivo ao desporto está condicionada a que:   - podem receber recursos do incentivo os projetos desportivos destinados a promover a inclusão social  por meio do esporte, preferencialmente em comunidades de vulnerabilidade social.   - é vedada a utilização dos recursos do incentivo para o pagamento de remuneração de atletas  profissionais, em qualquer modalidade desportiva.   - o valor da dedução atenda ao limite mencionado no tópico “atenção”    – incentivo ao programa nacional de apoio à atenção da saúde da pessoa com deficiência (pronas- ):   podem ser deduzidos os valores referentes às doações e aos patrocínios despendidos no ano-calendário  anterior, a que se refere a declaração de ajuste anual, diretamente efetuados em prol de ações e serviços  previamente aprovados pelo ministério da saúde, segundo a forma e o procedimento estabelecidos em  ato do poder executivo, e que estejam em consonância com a política definida para o setor no plano  nacional de saúde e nas diretrizes desse ministério e desenvolvidos por pessoas jurídicas de direito  privado sem fins lucrativos que se destinam ao tratamento de deficiências físicas, motoras, auditivas,  visuais, mentais, intelectuais, múltiplas e de autismo no âmbito do programa nacional de apoio à atenção  da saúde da pessoa com deficiência (pronas/).    – incentivo ao programa nacional de apoio à atenção oncológica (pronon):   podem ser deduzidas as quantias referentes às doações e aos patrocínios despendidos no ano-calendário  anterior, a que se refere a declaração de ajuste anual, diretamente efetuados em prol de ações e serviços,  previamente aprovados pelo ministério da saúde, segundo a forma e o procedimento estabelecidos em  ato do poder executivo, e que estejam em consonância com a política definida para o setor no plano  nacional de saúde e nas diretrizes desse ministério, e desenvolvidos pelas instituições de prevenção e  combate ao câncer que englobam a promoção da informação, a pesquisa, o rastreamento, o diagnóstico,  o tratamento, os cuidados paliativos e a reabilitação referentes às neoplasias malignas e afecções  correlatas destinatárias no âmbito do programa nacional de apoio à atenção oncológica (pronon).   atenção:   1) limites gerais de dedução:   a) o somatório das deduções referidas nos incisos  a  (deduções relativas a estatuto da criança,  fundos do idoso, incentivo à cultura, incentivo à atividade audiovisual e incentivo ao desporto)  está limitado a 6% do imposto devido apurado na declaração de ajuste; esse limite é calculado pelo  próprio programa do imposto e a dedução só se aplica à declaração em que o contribuinte optar  pelas deduções legais.   b) podem ser deduzidos, observado o limite de que trata a alínea “a” deste item, no caso do incentivo  à cultura, a que se refere o inciso :   b.1) 80% (oitenta por cento) do somatório das doações e 60% (sessenta por cento) do somatório  dos patrocínios, na hipótese do item 1; e   b.2) o valor efetivo das doações e patrocínios, na hipótese do item 2;    esse limite é calculado pelo próprio programa do imposto e a dedução só se aplica à declaração  em que o contribuinte optar pelas deduções legais.   c) as deduções previstas nos incisos  e  estão cada uma delas limitada a 1% (um por cento)  do imposto sobre a renda devido apurado na declaração e não estão sujeitas ao limite global de  6% (seis por cento) de que trata a alínea “a” deste item 1; esse limite é calculado pelo próprio  programa do imposto e a dedução só se aplica à declaração em que o contribuinte optar pelas  deduções legais.   2) informações sobre os beneficiários:      181   informe os pagamentos efetuados na ficha de doações efetuadas, em relação aos incisos  a ,  indicando o nome do beneficiário, o número de inscrição no cadastro nacional da pessoa jurídica    ou no cadastro de pessoas físicas  , o código e o valor pago ou doado;   3) em relação às deduções referidas nos incisos  e :   a) considera-se patrocínio a prestação do incentivo com finalidade promocional;   b) o comprovante pode ser emitido anualmente, desde que discrimine os valores doados mês a  mês;   c) no caso de doação em bens, o comprovante deve conter a identificação dos bens, mediante  descrição em campo próprio ou em relação anexa ao comprovante, informando também, se houve  avaliação, o nome, o número de inscrição no  ou no  e o endereço dos avaliadores.  ',
 'quem optou pelo desconto simplificado também pode deduzir os gastos com os fundos da  criança e do adolescente, com os fundos do idoso, com incentivos à cultura e à atividade  audiovisual, incentivo ao desporto, incentivo ao pronas/, ou com incentivo ao pronon?   não.  desconto simplificado substitui todas as deduções a que o contribuinte tem direito na declaração de  rendimentos, inclusive aquelas que são diminuídas diretamente do imposto',
 'podem ser deduzidos os valores referentes à contribuição patronal pagos à previdência  social pelo empregador doméstico?    contribuição patronal paga à previdência social pelo empregador doméstico não tem mais previsão legal  para ser deduzida do valor do imposto sobre a renda apurado.    inciso  do art. 12 da lei nº 9.250, de 26 de dezembro de 1995, previu a vigência dessa dedução somente  até o ano-calendário de 2018 e não houve prorrogação para o ano-calendário de 2019, tornando os valores  pagos a título de contribuição patronal no ano-calendário de 2019 como indedutíveis. ',
 'como são realizadas as doações aos fundos controlados pelos conselhos municipais,  estaduais e nacional dos direitos da criança e do adolescente?   para que o contribuinte possa fazer uso da dedução dos valores relativos a doações na declaração, é  necessário que as doações tenham sido efetuadas diretamente aos fundos de assistência da criança e do  adolescente que são controlados pelos conselhos municipais, estaduais ou nacional dos direitos das      182   crianças e dos adolescentes. as doações realizadas a orfanatos e similares não são equivalentes aos fundos  aqui tratados e, por isso, são indedutíveis.   os fundos de assistência, que estão limitados a um por município, um por estado e um nacional, devem emitir  comprovante em favor do doador, especificando o nome, o número de inscrição no cadastro nacional da  pessoa jurídica   ou no cadastro de pessoas físicas   do doador, a data e o valor efetivamente  recebido em dinheiro, além do número de ordem do comprovante, o nome, o número de inscrição no ,  o endereço do emitente, e ser firmado por pessoa competente para dar a quitação da operação.   as contribuições devem ser depositadas em conta específica por meio de documento de arrecadação próprio. ',
 'qual é o tratamento dado, na declaração de ajuste anual, aos imóveis objeto de transações  de pessoa física que for equiparada a pessoa jurídica pela prática de operações imobiliárias?   se o contribuinte for equiparado a pessoa jurídica pela prática de incorporação ou loteamento, os imóveis  objeto dessas transações serão imediatamente baixados de sua declaração de bens e direitos, historiando- se, no item correspondente à participação societária, sua destinação para formação do capital da pessoa  jurídica equiparada, indicando o valor do capital com ele realizado. ',
 'como declarar os bens adquiridos na constância da união estável?   na união estável, salvo contrato escrito entre os conviventes, aplica-se às relações patrimoniais, no que  couber, o regime da comunhão parcial de bens.   quando os conviventes optarem por apresentar a declaração de ajuste anual em separado, todos os bens  ou direitos comuns devem ser relacionados em apenas uma das declarações, independente do nome de qual  convivente consta na documentação dos referidos bens ou direitos, tais como: imóveis, conta-corrente,  veículos, ações. ',
 'como deve ser relacionada na declaração de bens a informação relativa à conta bancária de  mais de uma titularidade?   cada titular deve informar conforme a sua participação na conta bancária. se não for possível a identificação  do valor atribuído a cada titular, o valor deve ser proporcionalizado igualmente entre os titulares.   atenção:   na constância da união estável observar o contrato escrito, se houver.          183       — declaração',
 'como devem ser relacionados, na declaração de bens dos cônjuges, os bens ou direitos  comuns que estejam em nome de apenas um deles e a opção for pela declaração de ajuste anual  em separado?     quando os cônjuges (ou companheiros) optarem por apresentar a declaração de ajuste anual em separado,  todos os bens ou direitos comuns devem ser relacionados em apenas uma das declarações, independente do  nome de qual cônjuge (ou companheiro) consta na documentação dos referidos bens ou direitos, tais como:  imóveis, conta-corrente, veículos, ações.   consulte a pergunta 435             declaração  cônjuge',
 'que deve ser informado na declaração de bens e direitos do cônjuge que não declara os  bens comuns, relativamente a esses bens?   na declaração do contribuinte em que não constar os bens e direitos, por constarem na declaração do cônjuge  (ou companheiro), deve ser incluída informação no campo “discriminação”, utilizando-se o código 99,  relatando que os bens e direitos comuns estão apostos na declaração do cônjuge (ou companheiro),  informando também o nome e número de inscrição no cadastro de pessoas físicas   do cônjuge (ou  companheiro).   consulte a pergunta 434       doações    móveis  imóveis',
 'como declarar as doações recebidas em bens móveis e imóveis?   aquele que recebe as doações deve declará-las da seguinte forma:   1 - relacionar no campo “discriminação” da declaração de bens e direitos as doações recebidas, com a  indicação do nome e do número de inscrição no cadastro de pessoas físicas   do doador;   2 - informar no campo “situação em 31/12/2019 ($)” o valor do bem ou do direito recebido, conforme  estabelecido pelo instrumento de doação (sujeito à apuração de ganho de capital) ou conforme apresentado  na declaração de bens do doador;   3 - informar o valor correspondente à doação na ficha rendimentos isentos e não tributáveis.   atenção:     doador deve proceder da seguinte forma:   informar no item relativo ao bem doado, no campo “discriminação” da declaração de bens e  direitos, o nome e o número de inscrição no  de quem recebeu a doação; deixar em branco o  campo “situação em 31/12/2019 ($)”; informar ainda o valor na ficha “doações efetuadas”, sob o  código 81.   (regulamento do imposto sobre a renda - /2018, arts. 130 e 132, inciso , aprovado pelo  decreto nº 9.580, de 22 de novembro de 2018)   consulte a pergunta 574       doações   - donatário',
 'como declarar empréstimos efetuados a empresa?   os empréstimos feitos devem ser informados na declaração de bens e direitos, no código 51.    os juros pagos pela pessoa jurídica tomadora do empréstimo são tributados exclusivamente na fonte.   consulte a pergunta 214         - espólio',
 'quando o espólio estiver desobrigado de entregar a declaração de espólio, o (a) viúvo (a)  meeiro (a) está obrigado (a) a declarar os bens e direitos do espólio?   se o espólio estiver desobrigado de apresentar declaração de rendimentos inicial ou intermediária, os bens e  direitos do espólio podem, opcionalmente, ser relacionados na declaração do cônjuge sobrevivente.  relativamente à declaração final, tal opção não é possível, sendo os bens e direitos, obrigatoriamente nela  informados.   (instrução normativa  nº 81, de 11 de outubro de 2001, art. 7º)       depósito não  -',
 'como declarar depósitos não remunerados no exterior?    depósito não remunerado mantido em instituições financeiras no exterior deve ser informado na declaração  de bens e direitos da seguinte forma:   1 – na “discriminação”, pelo valor em moeda estrangeira, o banco e o número da conta;   2 – no campo “situação em 31/12/2018 ($)”, informar o saldo em reais existente em 31/12/2018 constante  na declaração do exercício de 2019, ano-calendário de 2018;   3 – no campo “situação em 31/12/2019”, o saldo existente em 31/12/2019, convertido em reais pela cotação  de compra para essa data, fixada pelo banco do central do brasil.   é isento o acréscimo patrimonial decorrente da variação cambial, o qual deve ser informado em rendimentos  isentos e não tributáveis.   exemplo:   gisele recebeu, em 26/04/2019, uma doação de $ 100.000,00 de seu namorado, luís. ela, nesse mesmo  dia, abriu uma conta não remunerada em um banco de miami e depositou o valor integral da doação.   na ficha “bens e direitos”, sob o código “62 – depósito bancário em conta corrente no exterior ...”, no campo  “discriminação” gisele deverá informar os dados da conta não remunerada no exterior (valor em moeda  estrangeira, banco e número da conta). deverá informar, também, o valor da doação convertido em reais pela  cotação de compra para a data de 26/04/2019, fixada pelo banco central do brasil. admitindo-se, por hipótese,  uma cotação de $ 3,9347, o valor a ser informado seria de $ 393.470,00.   no campo “situação em 31/12/2018 ($)”, informar o valor $ 0,00.   no campo “situação em 31/12/2019 $)”, informar o valor $ 400.370,00 (admitindo-se, por hipótese, o  câmbio do dia 31/12/2019 = $ 4,0037)    valor do acréscimo patrimonial decorrente da variação cambial dos depósitos não remunerados mantidos  em instituições financeiras no exterior ($ 400.370,00 – $ 393.470,00 = $ 6.900,00) é considerado isento  e deve ser informado no código “26 – outros” da ficha “rendimentos isentos e não tributáveis”:      185   a) no campo “descrição”, informar o valor original da doação em dólares ($ 100.000,00), o valor convertido  da doação na data da doação ($ 393.470,00) e o valor na data de 31/12/2019 ($ 400.370,00);    b) no campo “valor”, informar o valor da variação cambial, ou seja, o valor de $ 6.900,00.   gisele, a donatária, deve declarar a doação recebida, no valor de $ 393.470,00, também na ficha  “rendimentos isentos e não tributáveis”, mas no item “14 – transferências patrimoniais – doações e  heranças”.   (lei nº 9.250, de 26 de dezembro de 1995, art. 25, § 4º; e instrução normativa  nº 118, de 28  de dezembro de 2000, art. 11)         condomínio',
 'como declarar bens adquiridos em condomínio?   cada condômino deve informar a parte que lhe cabe. assim, na declaração de bens e direitos, ao descrever  o bem e a transação, deve-se informar que o bem foi adquirido em sociedade e o percentual da propriedade  do declarante no condomínio.        -  declaração',
 'como deve proceder o dependente que apresenta declaração de ajuste anual pela primeira  vez?   os bens e direitos de dependente que passa a apresentar declaração em separado não devem ser incluídos  na declaração de bens e direitos do responsável, que informará tal fato no campo “discriminação”, não sendo  informados nos campos “situação em 31/12/2018 ($)” e ”situação em 31/12/2019 ($)”.   na declaração de bens e direitos do dependente, os bens e direitos devem ser informados, nos campos  “situação em 31/12/2018 ($)” e “situação em 31/12/2019 ($)”, com base nos valores constantes na última  declaração de ajuste do responsável.          espécie -',
 'como declarar os estoques de moeda estrangeira mantida em espécie?    estoque de cada moeda estrangeira mantida em espécie deve ser informado na declaração de bens e  direitos da seguinte forma:   a) no campo “discriminação”, informe o estoque da moeda estrangeira mantida em espécie existente em  31/12/2019;   b) no campo “situação em 31/12/2018 ($)”, repita o valor em reais do saldo de moeda estrangeira existente  em 31/12/2018 ($), informado na declaração de ajuste anual do exercício de 2019, se for o caso;   c) no campo ”situação em 31/12/2019 ($)”, informe o valor em reais do saldo de moeda estrangeira existente  em 31/12/2019, apurado com base no custo médio ponderado, e correspondente ao valor informado no  campo ”situação em 31/12/2018 ($)”, somado ao valor em reais de cada aquisição e diminuído do valor em  reais de cada alienação efetuada no ano-calendário de 2019, observando-se que o custo da moeda adquirida  é:   - no caso de aquisição em reais, o valor pago;   - no caso de aquisição em moeda estrangeira, a quantidade de moeda estrangeira convertida em dólares  dos estados unidos da américa, pelo valor fixado pela autoridade monetária do país emissor da moeda,  para a data da aquisição e, em seguida, em reais pela cotação média mensal do dólar para venda,  divulgada pela secretaria especial da receita federal do brasil  , em seu sítio na internet, no  endereço http://receita.economia.gov.br: no menu “ rápido”, selecione “legislação”; dentro  desse novo menu, selecione “tributos”; em seguida, selecione “ (imposto sobre a renda das pessoas  físicas; clique em “tabelas de conversão para reais do dólar dos estados unidos da américa – ”.   caso o total de alienações de moeda estrangeira mantida em espécie, no ano-calendário de 2019, tenha sido  superior ao equivalente a cinco mil dólares dos estados unidos da américa, preencha o demonstrativo da  apuração dos ganhos de capital - alienação de moeda estrangeira mantida em espécie. ver instruções de  preenchimento. ',
 'as moedas virtuais devem ser declaradas?   sim. as moedas virtuais (bitcoins, por exemplo), muito embora não sejam consideradas como moeda nos  termos do marco regulatório atual, devem ser declaradas na ficha bens e direitos como “outros bens”, uma  vez que podem ser equiparadas a um ativo financeiro. elas devem ser declaradas pelo valor de aquisição.   atenção:    como esse tipo de “moeda” não possui cotação oficial, uma vez que não há um órgão responsável  pelo controle de sua emissão, não há uma regra legal de conversão dos valores para fins tributários.  entretanto, o contribuinte deverá guardar documentação que comprove a autenticidade desses  valores. ',
 'como declarar aplicações financeiras realizadas em moeda estrangeira?   cada aplicação financeira realizada em moeda estrangeira deve ser informada na declaração de bens e  direitos da seguinte forma:   a) no campo “discriminação”, informe o valor em moeda estrangeira da aplicação financeira existente em  31/12/2019;   b) no campo ”situação em 31/12/2018 ($)”, repita o valor em reais da aplicação financeira existente em  31/12/2018 informado na declaração de ajuste anual do exercício de 2019, se for o caso;   c) no campo ”situação em 31/12/2019 ($)”, informe o valor em reais da aplicação financeira existente em  31/12/2019, cujo saldo deve ser ajustado a cada aplicação, liquidação ou resgate realizado no ano-calendário  de 2019.   ver instruções de preenchimento do demonstrativo da apuração dos ganhos de capital - alienação de bens  ou direitos ou liquidação ou resgate de aplicações financeiras adquiridos em moeda estrangeira.  ',
 'como deve declarar proprietário de veículo que sofreu perda total ou foi roubado, recebeu  valor de seguradora e comprou novo veículo?   com relação ao veículo que sofreu perda total ou foi roubado, na ficha declaração de bens e direitos, informar  no campo “discriminação” do veículo o fato e o valor recebido da seguradora. no campo “situação em  31/12/2019 ($)” deixar “em branco”.    na ficha rendimentos isentos e não tributáveis deve ser informada a parcela do valor recebido da seguradora  que exceder ao valor pelo qual o bem acidentado ou roubado esteja declarado.    quanto ao veículo adquirido, informar no campo “discriminação” o valor recebido da seguradora e, no  campo ”situação em 31/12/2019 ($)”, o valor de aquisição. ',
 'como declarar os bens e direitos recebidos em herança no exterior?   no campo “discriminação”, informe os bens e direitos e o valor de aquisição em moeda estrangeira, constantes  nos instrumentos de  transmissão do país onde ocorreu a partilha, os quais devem ser traduzidos por tradutor  juramentado.   no campo “situação em 31/12/2019 ($)”, informe o valor em reais dos bens e direitos recebidos em herança  no exterior.    valor de aquisição dos bens e direitos, quando expresso em moeda estrangeira, deve ser convertido em  dólares dos estados unidos da américa pelo valor fixado pela autoridade monetária do país emissor da moeda  para a data da aquisição e, em seguida, em reais pela cotação do dólar fixada, para venda, pelo banco central  do brasil, para a data da aquisição.  em rendimentos isentos e não tributáveis, informe o valor em reais dos bens e direitos recebidos em herança  no exterior.',
 'como declarar os bens e direitos adquiridos no exterior?   no campo “discriminação”, informe os bens e direitos e o valor de aquisição em moeda estrangeira, constantes  nos instrumentos de transferência de propriedade. informe, ainda, o montante de rendimentos auferidos  originariamente em reais e/ou em moeda estrangeira utilizados na aquisição.   no campo ”situação em 31/12/2019 ($)”, informe o valor em reais dos bens e direitos adquiridos.    valor de aquisição dos bens e direitos, quando expresso em moeda estrangeira, deve ser convertido em  dólares dos estados unidos da américa pelo valor fixado pela autoridade monetária do país emissor da moeda  para a data da aquisição e, em seguida, em reais pela cotação do dólar fixada, para venda, pelo banco central  do brasil, para a data da aquisição.  ',
 'qual é o valor a ser informado na declaração de bens e direitos no caso de imóvel rural?    valor pago na aquisição do imóvel rural, em 2019, deve ser informado no campo ”situação em 31/12/2019  ($)”; o valor da terra nua  constante na declaração do imposto sobre a propriedade territorial rural   de 2019 deve ser informado no campo “discriminação”.  ',
 'como declarar a demolição de residência para a construção de outra no mesmo terreno?    demolição de residência deve ser informada no campo “discriminação” da declaração de bens e direitos,  informando o valor do bem demolido e os gastos referentes à nova construção. esses gastos devem ser  somados ao custo de aquisição informado no campo ”situação em 31/12/2018 ($)” e o resultado declarado  no campo ”situação em 31/12/2019 ($)”.  ',
 'como declarar bem adquirido por meio de consórcio?   caso o bem tenha sido recebido em 2019, informar no código 95, no campo ”situação em 31/12/2018 ($)”,  o valor constante na declaração de ajuste anual do exercício de 2019, ano-calendário de 2018. não preencher  o campo “situação em 31/12/2019 ($)”.   no código específico do bem, informar no campo “discriminação” os dados do bem e do consórcio. deixar em  branco o campo ”situação em 31/12/2018 ($)”. no campo ”situação em 31/12/2019 ($)”, informar o valor  declarado no ano de 2018, no código 95, acrescido dos valores pagos em 2019, inclusive do valor dado em  lance, se for o caso.  ',
 'como declarar consórcio ainda não contemplado?   no caso de consórcio ainda não contemplado, informar o código 95 e os dados do consórcio no campo  “discriminação” da declaração de bens e direitos.   no campo ”situação em 31/12/2018 ($)”, repetir o valor já declarado no exercício de 2019, ano-calendário  de 2018. no campo ”situação em 31/12/2019 ($)”, informar o valor declarado no ano de 2018, acrescido dos  valores pagos em 2019. ',
 'como declarar imóvel adquirido ou quitado com a utilização do ?    contribuinte deve informar o bem na declaração de bens e direitos, e no campo “discriminação” os valores  oriundos do . somar o valor do  aos demais valores pagos pela aquisição e informar o resultado  no campo ”situação em 31/12/2019 ($)”. em rendimentos isentos e não tributáveis informar o valor do   recebido.',
 'como declarar as aquisições efetuadas por meio de contrato particular de compra e venda  ou contrato de gaveta, quando a aquisição ocorre num determinado ano-calendário e a escritura em  cartório em outro ano-calendário?    contrato particular firmado entre construtora/agente financeiro ou pessoa física e o adquirente é instrumento  válido para configurar a aquisição do imóvel, mesmo que o adquirente não tenha desembolsado qualquer  quantia. assim, o adquirente deve informar os dados da aquisição no campo “discriminação” e o valor pago  até 31 de dezembro, no campo da situação referente ao ano-calendário do contrato.  ',
 'como declarar imóvel recebido em doação com cláusula de usufruto?   na declaração de bens e direitos do donatário, no campo “discriminação”, deve ser informada a situação  ocorrida, inclusive o nome e o cadastro de pessoas físicas do usufrutuário. no campo ”situação em  31/12/2019 ($)” e, também, em rendimentos isentos e não tributáveis, o valor correspondente à nua- propriedade.   além disso, na declaração de ajuste anual do doador:   a) se o imóvel doado já era do doador no ano anterior à doação, ele deve ser baixado da sua declaração de  bens e direitos, informando no campo “discriminação” o nome e o  do beneficiário da doação, bem como,  se o usufruto foi instituído para terceiros, o nome e o  do usufrutuário (nesta hipótese, o usufrutuário deve  informar esta situação na sua declaração de bens e direitos, bem como o nome e o  do proprietário da  nua-propriedade);   b) se o imóvel doado foi adquirido pelo doador no ano da doação, ele deve ser incluído em sua declaração de  bens e direitos, informando no campo “discriminação” os dados da aquisição, o nome e o  do beneficiário  da doação, bem como, se o usufruto foi instituído para terceiros, o nome e o  do usufrutuário (nesta  hipótese, o usufrutuário deve informar esta situação na sua declaração de bens e direitos, e, ainda, o nome  e o  do proprietário da nua-propriedade).   em ambos os casos, quando o doador permaneceu com o usufruto, esta situação deve ser informada em  novo item da declaração de bens e direitos, no campo “discriminação”, sem indicação de valor, salvo se foi  atribuído valor ao usufruto no documento de transmissão, correspondente ao valor efetivamente pago como  parte total da aquisição ou que deve ser calculado pela proporção relativa ao usufruto constante deste  documento aplicada sobre o valor total declarado ou de aquisição do imóvel doado.  ',
 'como declarar contrato de leasing de bens móveis?   para leasing realizado:   a) com opção de compra exercida no final do contrato, ocorrida em 2019, utilize o código relativo ao bem, e:     no campo “discriminação”, informe os dados do bem e do contratante;     no campo ”situação em 31/12/2018 ($)”, informe os valores pagos até 31/12/2018, no caso de  leasing contratado em 2019, deixe este campo “em branco”;     no campo ”situação em 31/12/2019 ($)”, informe o valor constante no campo ”situação em  31/12/2018 ($)”, se for o caso, acrescido dos valores pagos em 2019, inclusive o valor residual;      189   b) em 2019, com opção de compra a ser exercida no final do contrato, após 2019, utilize o código 96, e:     no campo “discriminação”, informe os dados do bem, do contratante e o total dos pagamentos  efetuados;     não preencha os campos “situação em 31/12/2018 ($)” e ”situação em 31/12/2019 ($)”;   c) até 2018, com opção de compra exercida no ato do contrato, utilize o código relativo ao bem, e:     no campo “discriminação”, informe os dados do bem e do contratante;     nos campos ”situação em 31/12/2018 ($)” e ”situação em 31/12/2019 ($)”, informe o valor do bem;     em dívidas e ônus reais, informe nos campos ”situação em 31/12/2018 ($)” e ”situação em  31/12/2019 ($)”, respectivamente, o saldo remanescente da dívida em 31/12/2018 e em 31/12/2019.   d) em 2019, com opção de compra exercida no ato do contrato, utilize o código relativo ao bem, e:     no campo “discriminação”, informe os dados do bem e do contratante;     não preencha o campo ”situação em 31/12/2018 ($)”;     no campo ”situação em 31/12/2019 ($)”, informe o valor do bem;     em dívidas e ônus reais, informe o valor da dívida no campo ”situação em 31/12/2019 ($)”. ',
 'como é tributado o resultado da atividade rural?    resultado da atividade rural, quando positivo, integrará a base de cálculo do imposto na declaração de  ajuste anual. para sua apuração, as receitas, as despesas e os investimentos são computados mensalmente  pelo regime de caixa. ',
 'qual é a forma de apuração do resultado da atividade rural da pessoa física?    resultado da exploração da atividade rural exercida pela pessoa física é apurado mediante a escrituração  do livro-caixa, abrangendo as receitas, as despesas, os investimentos e demais valores que integram a  atividade.     escrituração e a apuração devem ser feitas separadamente, por contribuinte e por país, em relação a todas  as unidades rurais exploradas individualmente, em conjunto ou em comunhão em decorrência do regime de  casamento.   quando a receita bruta total auferida no ano-calendário não exceder a $ 56.000,00, é permitida a apuração  mediante prova documental, dispensada a escrituração do livro-caixa, encontrando-se o resultado pela  diferença entre o total das receitas e o das despesas/investimentos.   também é permitido à pessoa física apurar o resultado pela forma contábil. nesse caso, deve efetuar os  lançamentos em livros próprios de contabilidade, necessários para cada tipo de atividade (diário, caixa, razão  etc.), de acordo com as normas contábeis, comerciais e fiscais pertinentes a cada um dos livros utilizados.   ressalte-se que, no caso de exploração de uma unidade rural por mais de uma pessoa física, cada produtor  rural deve escriturar as parcelas da receita, da despesa de custeio, dos investimentos e dos demais valores  que integram a atividade rural que lhe caibam.    secretaria especial da receita federal do brasil   disponibiliza o programa aplicativo livro-caixa da  atividade rural para pessoa física que exerça a atividade rural no brasil ou no exterior, o qual permite a  escrituração pelo sistema de processamento eletrônico, no sítio http://receita.economia.gov.br.   ',
 'como deve proceder o contribuinte que, tendo escriturado as receitas e as despesas (e  investimentos) da atividade rural, encontra insuficiência de caixa em qualquer mês do ano- calendário?    escrituração consiste em assentamentos das receitas, despesas de custeio, investimentos e demais valores  que integram a atividade rural, em livro-caixa. as insuficiências de caixa apuradas devem estar  inequivocamente justificadas pelos rendimentos das demais atividades, rendimentos tributados   exclusivamente na fonte ou isentos e por adiantamentos ou empréstimos, subsídios e subvenções obtidos,  coincidentes em datas e valores, comprovados por documentação hábil e idônea. ',
 'é permitida a escrituração do livro-caixa eletronicamente?   é permitida a escrituração do livro-caixa pelo sistema de processamento eletrônico, com subdivisões  numeradas em ordem sequencial ou tipograficamente.    livro-caixa independe de registro em órgão da secretaria especial da receita federal do brasil   ou  em qualquer repartição pública, deve ser numerado sequencialmente e conter, no início e no encerramento,  anotações em forma de "termos" que identifiquem o contribuinte e a finalidade do livro.     disponibiliza o programa aplicativo do livro-caixa da atividade rural para pessoa física que exerça a  atividade rural no brasil ou no exterior, no sítio http://receita.economia.gov.br. ',
 'quem está obrigado a entregar o arquivo digital com a escrituração do livro caixa digital do  produtor rural?   deve entregar o arquivo digital com a escrituração  a pessoa física que explora a atividade rural e  obteve, no ano-calendário de 2019, individualmente, receita bruta total da atividade rural em valor superior a  $ 7.200.000,00 (sete milhões e duzentos mil reais).  partir do ano-calendário de 2020, esse valor será de  $ 4.800.000,00 (quatro milhões e oitocentos mil reais).    atenção:    escrituração do livro caixa digital do produtor rural deve ser realizada conforme o leiaute vigente  e o manual de preenchimento divulgados pela coordenação-geral de programação e estudos  (copes) da secretaria especial da receita federal do brasil, devendo a entrega do arquivo  digital do  ser realizada até o final do prazo de apresentação da declaração de ajuste anual  do respectivo ano-calendário.    contribuinte que auferir, no ano-calendário, receita bruta total da atividade rural inferior à prevista  para o ano-calendário, mesmo desobrigado, poderá escriturar e entregar o .    produtor rural pessoa física, obrigado à entrega do , que deixar de apresenta-lo até o fim  do prazo de apresentação da  do respectivo ano-calendário, ou o apresentar com incorreções  ou omissões, estará sujeito às multas previstas no art. 57 da medida provisória nº 2.158-35, de 24  de agosto de 2001.   ',
 'como deve ser tributado o resultado da atividade rural no caso de encerramento de espólio  ou saída definitiva do brasil durante o ano-calendário?   nesses casos, o resultado da atividade rural exercida até a data da decisão judicial da partilha, sobrepartilha  ou adjudicação dos bens, ou da lavratura da escritura pública de inventário (encerramento de espólio) ou até  o dia anterior à data da aquisição da condição de não residente (saída definitiva do brasil), quando positivo,  integrará a base de cálculo do imposto devido na declaração final de espólio ou na declaração de saída  definitiva do país. ',
 'todas as pessoas que exploram imóvel rural estão obrigadas a inscrever-se no cadastro de  imóveis rurais do  da secretaria especial da receita federal do brasil ?   não. de acordo com a legislação do , estão obrigados a inscrever-se nesse cadastro apenas o proprietário  do imóvel rural, o possuidor a qualquer título inclusive o usufrutuário e o titular de domínio útil (o  enfiteuta/foreiro). ',
 'como se distinguem os contratos agrários?   os contratos de arrendamento e parceria são basicamente semelhantes no que concerne à natureza jurídica,  pois em todos há cessão de uso e gozo de imóvel ou de área rural, parte ou partes dos mesmos, incluindo,  ou não, outros bens, benfeitorias e facilidades, para ser exercida atividade de exploração agrícola, pecuária,  agroindustrial, extrativa vegetal ou mista. diferem, porém, substancialmente na forma de remuneração do  cedente:   a) no arrendamento ou subarrendamento, o cedente (arrendador ou subarrendador) recebe do arrendatário  ou subarrendatário retribuição certa ou aluguel pelo uso dos bens cedidos (os rendimentos devem ser  tributados como aluguéis, separados da atividade rural);   b) na parceria ou subparceria, o cedente (parceiro-outorgante) partilha com o parceiro-outorgado os riscos de  caso fortuito e força maior, os frutos, produtos ou lucros havidos, nas proporções estipuladas em contrato, e  as variações de preço dos frutos obtidos (apuração na atividade rural – receitas e despesas).   por outro lado, na propriedade em comum, copropriedade ou condomínio, os proprietários, os coproprietários  ou os condôminos partilham os riscos, frutos ou resultados havidos, na proporção da parte que lhes caiba no  total.   ',
 'qual é a conceituação de arrendatário e parceiro?   para fins de exploração da atividade rural, conceitua-se como arrendatário ou subarrendatário (também  conhecido como locatário ou foreiro) e como parceiro-outorgado ou subparceiro-outorgado (também  conhecido como sócio, meeiro, terceiro, quartista ou percentista) a pessoa ou o conjunto familiar representado  pelo seu chefe, que recebe o imóvel ou a unidade rural, parte ou partes dos mesmos, incluindo, ou não, outros  bens, benfeitorias e facilidades, e neles exerce qualquer atividade agrícola, pecuária, agroindustrial, extrativa  vegetal ou mista, sob contrato de arrendamento ou parceria rural.   atenção:    para ter plena validade perante o fisco, esses contratos devem ser comprovados por instrumento  escrito hábil e idôneo.   ',
 'como proceder quando no decorrer do ano-calendário for formalizado mais de um contrato  de parceria para a mesma unidade rural?   os parceiros ou subparceiros devem apurar, separadamente, os lucros ou prejuízos vinculados a cada uma  das situações ocorridas no período, na proporção dos rendimentos e despesas que couberem a cada um,  preenchendo o demonstrativo da atividade rural, quando a isso estiverem obrigados. na declaração, cada  parceiro ou subparceiro outorgado adicionará o resultado, correspondente à parcela que lhes couber na  parceria, incluindo a soma de todos os contratos de que participou, à base de cálculo do imposto na  declaração de ajuste anual.  ',
 'quem deve tributar os rendimentos de propriedade rural objeto de usufruto?    usufrutuário deve tributar os rendimentos de propriedade rural objeto de usufruto de acordo com a natureza  destes, ou seja, deve apurar o resultado da atividade rural, desde que exerça essa atividade no referido imóvel,  caso contrário, o rendimento de qualquer outra natureza sujeita-se ao carnê-leão, se recebido de pessoa física,  ou à retenção na fonte, se pago por pessoa jurídica, e, também, ao ajuste na declaração anual.   ressalte-se que o usufruto deve estar formalizado por escritura pública transcrita no registro de imóvel  competente.',
 'como são tributados os valores recebidos por pessoa física relativos a animais que possua  em propriedade de terceiros?    tratamento tributário é o seguinte:   1 - se essa pessoa explora atividade rural na propriedade de terceiros como arrendatário desta ou como  parceiro (participa dos riscos da atividade), as quantias auferidas e as despesas são incluídas na apuração  da atividade rural (lei nº 8.023, de 12 de abril de 1990; instrução normativa  nº 83, de 11 de outubro de  2001, art. 14).    2 - excetuada a hipótese acima referida:   a) os rendimentos auferidos na venda são tributados como ganho de capital, se essa atividade não for exercida  com habitualidade (lei nº 7.713, de 22 de dezembro de 1988, art. 3º, § 2º); ou   b) se houver habitualidade e fim especulativo de lucro, a pessoa física é considerada empresário (empresa  individual) equiparado a pessoa jurídica, sendo seus lucros tributados nessa condição ',
 'considera-se rendimento da atividade rural o ganho auferido por proprietário de gado que  entrega certa quantidade de animais à parte contratante, para procriação ou terminação?   uma vez que os rendimentos devem ser classificados segundo a sua natureza, somente é considerado  rendimento da atividade rural, neste caso, aquele oriundo de contrato de parceria em que haja partilha dos  riscos, frutos, produtos e resultados havidos, nas proporções estipuladas no contrato.    nos demais casos, trata-se de arrendamento, pois o proprietário dos animais recebe retribuição líquida e certa  pelo prazo estipulado, sem qualquer risco, mesmo que o rendimento seja predeterminado em número de reses  no contrato, e o rendimento sujeita-se ao carnê-leão, se recebido de pessoa física, ou à retenção na fonte, se  pago por pessoa jurídica, e, também, ao ajuste na declaração de ajuste anual.   ressalte-se que o ganho auferido com investimento em títulos representativos de animais (tipo “boi gordo”)  deve ser tributado exclusivamente na fonte conforme as instruções contidas na resposta à pergunta 629.  ',
 'que se considera exploração da atividade rural na captura in natura do pescado?   considera-se atividade rural a captura in natura do pescado realizada por embarcações, inclusive a exploração  realizada em regime de parceria, desde que a exploração se faça com apetrechos semelhantes aos da pesca  artesanal (arrastões de praias, redes de cerca etc.). ',
 'propriedade de mais de um barco pesqueiro, explorado em parceria, descaracteriza a  atividade rural?   não. tal fato não implica equiparação a pessoa jurídica. ',
 'rendimento decorrente da hospedagem de animais em haras é tributado na atividade rural?   não.  valor correspondente a este rendimento não é tributado na atividade rural, devendo ser tributado como  aluguel ou arrendamento, sujeitando-se ao carnê-leão, se recebido de pessoa física, ou à retenção na fonte,  se pago por pessoa jurídica, e, também, ao ajuste na declaração de ajuste anual. ',
 'onsidera-se atividade rural a pasteurização, com acondicionamento do leite produzido por  terceiros?   não.  pasteurização, com acondicionamento, do leite, bem como o acondicionamento e embalagem de  apresentação do mel e do suco de frutas, são considerados atividade rural apenas quando efetuados com  produção própria. ',
 'receita correspondente ao aluguel de pastagens, máquinas e equipamentos é tributada na  atividade rural?   não.  valor correspondente a esse tipo de aluguel não é considerado receita da atividade rural, devendo ser  incluído como rendimento mensal sujeito ao carnê-leão, se recebido de pessoa física, ou submetido à retenção  na fonte, se pago por pessoa jurídica, e, também, ao ajuste na declaração anual.  locador ou arrendatário  deve informar o valor total pago, o nome e o número de inscrição no cadastro de pessoas físicas do  locatário ou arrendador na relação de pagamentos e doações efetuados da sua declaração de rendimentos. ',
 'rendimento proveniente da exploração da avicultura é considerado como sendo de  atividade rural?   sim.  avicultura é considerada pela legislação tributária como atividade rural, nela incluída a produção de  ovos, que não se descaracteriza como tal, em virtude da utilização de máquinas para lavagem, classificação  e embalagem de ovos.   atenção:     fato de o produtor rural entregar, mediante contrato de prestação de serviços, ovos férteis de sua  produção, para que sejam incubados por terceiros, não descaracteriza a atividade rural, uma vez  que o risco da atividade permanece com o produtor.  incubação faz parte da produção dos  "pintinhos de um dia", portanto é irrelevante se a mesma é executada pelo produtor ou por terceiros.  ',
 'turismo rural é considerado atividade rural?   não. as atividades de turismo rural, hotéis-fazenda, locais de passeio etc., não constituem atividade rural.  ',
 'como deve ser declarado o imóvel rural, com bens e benfeitorias, de contribuinte que não  explora a atividade rural?    imóvel rural, seus bens e benfeitorias devem constar na declaração de bens e direitos da declaração de  ajuste anual com a informação de que não é usado na atividade rural. deste modo, quando alienados,  sujeitam-se à apuração do ganho de capital.   (regulamento do imposto sobre a renda – /2018, arts. 128 a 153 e 890, aprovado pelo decreto  nº 9.580, de 22 de novembro de 2018; e instrução normativa  nº 84, de 11 de outubro de 2001,  art. 3º, inciso )         árvores',
 'qual é o tratamento tributário dos rendimentos recebidos na venda de árvores plantadas em  propriedade rural?    os rendimentos recebidos na venda de árvores plantadas têm o seguinte tratamento:   1 - se as árvores plantadas são decorrentes do cultivo de florestas exercido pelo alienante, as quantias  recebidas são incluídas na receita da atividade rural explorada (regulamento do imposto sobre a renda –  /2018, art. 54, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018; instrução normativa  nº  83, de 11 de outubro de 2001, art. 5º; e parecer normativo  nº 90, de 16 de outubro de 1978).   2 - se as árvores plantadas não são decorrentes do cultivo de florestas exercido pelo alienante:   a) os rendimentos auferidos na venda são tributados como ganho de capital, se essa atividade não for exercida  com habitualidade (lei nº 7.713, de 22 de dezembro de 1988, art. 3º, § 2º); ou   b) se houver habitualidade e fim especulativo de lucro, a pessoa física é considerada empresário (empresa  individual) equiparado a pessoa jurídica, sendo seus lucros tributados nessa condição (regulamento do  imposto sobre a renda – /1999, arts. 162, 623 e 624, aprovado pelo decreto nº 9.580, de 22 de novembro  de 2018).   atenção:    considera-se atividade rural o cultivo de florestas que se destinem ao corte para comercialização,  consumo ou industrialização (',
 'como devem ser declarados os rendimentos recebidos por pessoa física que utilize área rural  para extração de madeira ou produção de carvão?    tratamento tributário é o seguinte:   1 - se a pessoa física exerce o cultivo de florestas utilizando-se do setor secundário da economia, porém, por  meio de procedimento que não configure produção, ou pratique a exploração ou extração vegetal, as quantias  recebidas na venda da madeira extraída ou do carvão produzido são incluídas na receita da atividade rural  explorada (lei nº 8.023, de 12 de abril de 1990, art. 2º, inciso , com a redação dada pela lei nº 9.250, de 26  de dezembro de 1995, art. 17; regulamento do imposto sobre a renda – /2018, art. 51, aprovado pelo  decreto nº 9.580, de 22 de novembro de 2018; instrução normativa  nº 83, de 11 de outubro de 2001,  arts. 2º e 5º; parecer normativo  nº 90, de 16 de outubro de 1978).   2 - excetuada a hipótese acima referida:   a) os rendimentos auferidos na venda são tributados como ganho de capital, se essa atividade não for exercida  com habitualidade (lei nº 7.713, de 22 de dezembro de 1988, art. 3º, § 2º); ou   b) se houver habitualidade e fim especulativo de lucro, ou procedimento que configure industrialização, a  pessoa física é considerada empresário (empresa individual) equiparado a pessoa jurídica, sendo seus lucros  tributados nessa condição ',
 'os rendimentos recebidos pela pessoa física com a venda de minério extraído de propriedade  rural são tributados na atividade rural?   não. os rendimentos recebidos na venda de minério extraído de propriedade rural têm o seguinte tratamento:   1 - os rendimentos auferidos na venda são tributados como ganho de capital, se essa atividade não for  exercida com habitualidade (lei nº 7.713, de 22 de dezembro de 1988, art. 3º, § 2º); ou   2 - se houver habitualidade e fim especulativo de lucro, a pessoa física é considerada empresário (empresa  individual) equiparado a pessoa jurídica, sendo seus lucros tributados nessa condição (regulamento do  imposto sobre a renda – /2018, arts. 162, 623 e 624, aprovado pelo decreto nº 9.580, de 22 de novembro  de 2018).   atenção:    quando houver cessão de direitos para pesquisar e extrair os recursos minerais (metal nobre,  pedras preciosas, areia, aterro, pedreiras etc.), os rendimentos, auferidos pelo cedente do direito,  sujeitam-se à apuração do ganho de capital.   (instrução normativa  nº 83, de 11 de outubro de 2001, art. 4º, inciso )',
 'como devem ser considerados os rendimentos da pessoa física que vende produtos de sua  atividade rural diretamente a centros de abastecimento?    produtor rural que em seu nome vender produtos de sua atividade rural diretamente a centros de  abastecimento, fornecendo notas de venda para atender às exigências do fisco estadual, não perde a  condição de pessoa física, devendo os rendimentos derivados dessa atividade compor o resultado da  atividade rural.  ',
 'como deve proceder a pessoa que recebe imóvel rural em herança, antes explorado pela  pessoa falecida, com prejuízos de exercícios anteriores?   considerando que o espólio é uma universalidade de bens e direitos, que há uma sucessão legítima por  transferência causa mortis e que o sucessor recebe o patrimônio na mesma situação em que se encontrava  quando em poder da pessoa falecida, sem solução de continuidade, o saldo de prejuízos não compensado  pelo de cujus pode sê-lo pelo espólio e, após o encerramento do inventário, pelo meeiro ou herdeiro,  proporcionalmente à parcela da unidade rural recebida, desde que nela o meeiro ou herdeiro continue a  explorar atividade rural.   ',
 'como devem ser declarados os rendimentos decorrentes da exploração da atividade rural  enquanto não encerrado o inventário?   os rendimentos próprios do espólio e 50% dos produzidos pelos bens comuns no curso do inventário devem  ser, obrigatoriamente, incluídos na declaração do espólio. opcionalmente, os rendimentos produzidos pelos  bens comuns podem ser tributados em sua totalidade na declaração do espólio.  ',
 'adiantamento de recursos financeiros obtidos especificamente para emprego em atividade  rural pode ser utilizado para aquisição de bens não empregados nessa atividade?    valor recebido por conta de adiantamento de recursos financeiros para aplicação em custeio ou  investimentos, referente a produto rural a ser entregue em ano posterior, informado no campo “apuração do  resultado não tributável” do demonstrativo da atividade rural, deve ser usado pelo contribuinte na forma  prevista no contrato firmado entre as partes, e, nesse caso, os respectivos valores não podem justificar  acréscimo patrimonial.  ',
 'os valores dos empréstimos ou financiamentos obtidos especificamente para emprego em  atividade rural podem justificar acréscimo patrimonial?   preliminarmente, deve-se comprovar se os empréstimos ou financiamentos obtidos especificamente para  emprego em atividade rural, ou seja, aplicação em custeio ou investimentos, foram efetivamente utilizados  nessa atividade, como estabelece o art. 2º do decreto-lei nº 167, de 14 de fevereiro de 1967. em caso  afirmativo, os respectivos valores não podem justificar acréscimo patrimonial, devendo ser informados em  dívidas vinculadas à atividade rural do demonstrativo da atividade rural, o saldo devedor ao final do ano- calendário.',
 'como devem ser informados os produtos da atividade rural exercida pelo contribuinte  estocados e ainda não vendidos?   os produtos da atividade rural exercida pelo contribuinte estocados e ainda não vendidos, inclusive o  produzido no ano-calendário a que se referir a declaração, devem constar na ficha bens da atividade rural,  do demonstrativo da atividade rural, com a discriminação da quantidade e espécie dos produtos existentes  no final do ano, sem indicação do respectivo valor.',
 'produção de alevinos, girinos e embriões de rebanho é considerado atividade rural?   sim. é atividade rural e os rendimentos dela decorrentes são assim tributados, independentemente de sua  destinação: reprodução ou comercialização.',
 'contribuinte que vendeu sua propriedade rural durante o ano-calendário está obrigado a  declarar os rendimentos dessa atividade?   sim, devendo apresentar normalmente o demonstrativo da atividade rural, se a isso estiver obrigado, e  lançar, na declaração de ajuste anual, o resultado positivo apurado até a data da venda, sendo irrelevante o  fato de haver vendido a propriedade em qualquer período do ano-calendário.  mesmo procedimento deve  ser observado no caso de compra no ano-calendário. além disso, deve ser apurado o ganho de capital quanto  à alienação da terra nua, se for o caso, bem como tributada a receita com a venda de bens e benfeitorias,  caso isso tenha ocorrido. ',
 'receita auferida na venda de rebanho bovino anteriormente comprado, com permanência  em poder do contribuinte em prazo inferior a 52 dias, quando em regime de confinamento, ou 138  dias nos demais casos, é tributada na atividade rural?   não.  tratamento tributário é o seguinte:   1 - os rendimentos auferidos, na venda, são tributados como ganho de capital, se essa atividade não for  exercida com habitualidade (lei nº 7.713, de 1988, art. 3º, § 2º); ou   2 - se houver habitualidade e fim especulativo de lucro, a pessoa física é considerada empresário (empresa  individual) equiparado a pessoa jurídica, sendo seus lucros tributados nessa condição.',
 'como devem ser tributados os rendimentos decorrentes de atividade rural exercida, no  exterior, por residente no brasil?    resultado da atividade rural exercida, no exterior, por residente no brasil (que deve ser apurado  separadamente da atividade rural porventura exercida no brasil) quando positivo, integra a base de cálculo do  imposto devido no ano-calendário.   na apuração do resultado, que deve ser feita separadamente para cada país onde a atividade rural foi exercida  (na moeda original e posteriormente convertido para $) e em seguida totalizada pela soma algébrica do  resultado em $ de cada país (posteriormente convertida para $), são aplicadas as mesmas normas  previstas para o contribuinte que exerce a atividade no brasil.   é vedada a compensação de resultado total negativo do exterior com resultado positivo do brasil ou de  resultado negativo do brasil com resultado total positivo do exterior, sendo, porém, permitida a compensação  de resultado total negativo do exterior com resultado total positivo do exterior de anos posteriores, segundo  as mesmas regras previstas para a compensação de prejuízos apurados no brasil.    imposto pago no exterior pode ser compensado na declaração de ajuste anual até o valor correspondente  à diferença entre o imposto calculado com a inclusão do resultado da atividade rural exercida no exterior e o  imposto calculado sem a inclusão desses rendimentos e desde que não seja restituído ou compensado no  país de origem, observados os acordos, tratados e convenções internacionais firmados entre o brasil e o país  de origem dos rendimentos ou a existência de reciprocidade de tratamento.   atenção:    os dados declarados pelo contribuinte ficam sujeitos a comprovação com documentos hábeis e  idôneos, a critério da autoridade lançadora, quando esta o solicitar. à falta destes, o rendimento  proveniente da atividade rural no exterior sujeita-se ao carnê-leão e, também, ao ajuste na  declaração anual. ',
 'como apurar a base de cálculo do imposto devido no caso de o contribuinte residente no  brasil exercer atividade rural no brasil e no exterior?    base de cálculo é apurada da seguinte maneira:   a) o resultado da atividade rural no brasil deve ser calculado em relação a todas as atividades rurais  exploradas individualmente, em conjunto ou em comunhão em decorrência do regime de casamento;   b) o resultado da atividade no exterior deve ser calculado separadamente da atividade rural exercida no brasil  e para cada país onde a atividade rural foi exercida (na moeda original e posteriormente convertido para $)  e em seguida totalizado pela soma algébrica do resultado em $ de cada país (posteriormente convertida  para $), sendo aplicadas as mesmas normas previstas para o contribuinte que exerce a atividade no brasil;   c) é vedada a compensação de resultado total negativo do exterior com resultado positivo do brasil ou de  resultado negativo do brasil com resultado total positivo do exterior, sendo, porém, permitida a compensação  de resultado negativo com o positivo, ambos no brasil, bem como de resultado total negativo do exterior com  resultado total positivo do exterior de anos posteriores, segundo as mesmas regras previstas para a  compensação de prejuízos apurados no brasil;   d) deve ser preenchido o demonstrativo da atividade rural tanto na parte referente à atividade exercida no  brasil, quanto à exercida no exterior;   e) o programa  2020 automaticamente consolida o resultado tributável da atividade rural exercida no  exterior, quando positivo, com o resultado tributável da atividade rural exercida no brasil, quando positivo, para  efeito de apuração da base de cálculo do imposto;   f) o imposto pago no exterior pode ser compensado na declaração de ajuste anual até o valor correspondente  à diferença entre o imposto calculado com a inclusão do resultado da atividade rural exercida no exterior e o  imposto calculado sem a inclusão desses rendimentos e desde que não seja restituído ou compensado no  país de origem, observados os acordos, tratados e convenções internacionais firmados entre o brasil e o país  de origem dos rendimentos ou a existência de reciprocidade de tratamento. ',
 'como devem ser tributados os rendimentos decorrentes de atividade rural exercida no brasil  por não residente no país?    resultado da atividade rural exercida no brasil por não residente no país deve ser apurado por ocasião do  encerramento do ano-calendário, segundo as mesmas normas previstas para quem seja residente no brasil,  constitui a base de cálculo do imposto e é tributado à alíquota de 15%. quando recebido por residente em  país com tributação favorecida, a alíquota é de 25%.    apuração deve ser feita por procurador, a quem compete reter e recolher o imposto devido.    imposto apurado deve ser pago na data da ocorrência do fato gerador.   ocorrendo remessa de valores antes do encerramento do ano-calendário (exceto no caso de devolução de  capital), o imposto deve ser recolhido no ato sobre o valor remetido por ocasião do evento.   na apuração do resultado da atividade rural exercida no brasil por não residente no país não são permitidas:   a) opção pelo limite da base de cálculo à razão de 20% sobre a receita bruta; e    b) compensação de prejuízo apurado. ',
 'como compensar o imposto pago no exterior referente à atividade rural?    imposto pago no exterior pode ser compensado na declaração de ajuste anual até o valor correspondente  à diferença entre o imposto calculado com a inclusão do resultado da atividade rural exercida no exterior e o  imposto calculado sem a inclusão desses rendimentos e desde que não seja restituído ou compensado no  país de origem, observados os acordos, tratados e convenções internacionais firmados entre o brasil e o país  de origem dos rendimentos ou a existência de reciprocidade de tratamento.  ',
 'como deve proceder o contribuinte obrigado à apresentação da declaração de ajuste anual  e desobrigado do preenchimento do demonstrativo da atividade rural, em relação aos bens  destinados à exploração da atividade rural?    contribuinte deve declará-los, neste ano, na declaração de bens e direitos da própria declaração de ajuste  anual, e na descrição informar que se trata de bem destinado à exploração da atividade rural, sem indicação  do respectivo valor, ficando-lhe assegurado o direito de declará-los no demonstrativo da atividade rural, no  primeiro exercício em que estiver novamente obrigado a apresentá-lo.    entretanto, caso pretenda compensar, no ano-calendário de 2019 ou posteriores, resultados negativos  (prejuízos) de anos–calendário anteriores ou do próprio ano-calendário de 2019, deve apresentar a  declaração de ajuste anual por meio do programa  2020, com o preenchimento do demonstrativo da  atividade rural.',
 'como se apura o imposto relativo à alienação de propriedade rural?   inicialmente deve-se destacar que na alienação de propriedade rural pode ocorrer a alienação apenas da terra  nua (sobre a qual se apura o ganho de capital), ou da terra nua mais benfeitorias. caso o instrumento de  transmissão identifique separadamente o valor de alienação da terra nua e das benfeitorias e o custo dessas  benfeitorias (tanto as adquiridas pelo alienante quanto as por este realizadas) tenha sido deduzido como custo  ou despesa da atividade rural, o valor de sua alienação deve ser oferecido à tributação como receita da  atividade rural.    quando o instrumento de transmissão não identificar separadamente o valor de alienação da terra nua e das  benfeitorias, o valor destas deve ser apurado por meio de cálculo específico, conforme regras que constam  nas perguntas a seguir indicadas.   atenção:    caso o custo das benfeitorias (tanto as adquiridas pelo alienante quanto as por este realizadas) não  tenha sido deduzido como custo ou despesa da atividade rural, o seu valor integra o custo de  aquisição para fins de apuração do ganho de capital, sobre o valor total da alienação.   ',
 'qual é o tratamento tributário da transferência dos bens e benfeitorias na herança, dissolução  da sociedade conjugal ou doação, quando recebidos pelos sucessores legítimos, ex-cônjuges ou  donatários?    - caso o custo dos bens e benfeitorias que estejam sendo transferidos tenha sido deduzido como custo ou  despesa da atividade rural, pelo espólio, ex-cônjuge ou doador, o tratamento tributário é o seguinte:   1. espólio, ex-cônjuge ou doador que entrega os bens e benfeitorias:   no mês da partilha, dissolução ou doação, com a entrega efetiva dos bens e benfeitorias, deve ser  incluído como receita da atividade rural o valor correspondente à recuperação de custos (custo histórico)  ou do valor de mercado a eles atribuído;   2. sucessores legítimos, ex-cônjuge ou donatários que recebem os bens e benfeitorias:   a) caso continuem a exploração da atividade rural na unidade recebida, podem deduzir como despesa  da atividade rural no mês da partilha, dissolução ou doação o valor incluído como receita pelo espólio,  ex-cônjuge ou doador e não incluir os bens e benfeitorias na declaração de bens e direitos, devendo  informá-los no campo “discriminação” da declaração de bens do demonstrativo da atividade rural,  identificando-os de forma minuciosa, sem qualquer valor;   b) caso não continuem a exploração da atividade rural na unidade recebida, devem informar, na  declaração de bens e direitos, os bens e benfeitorias recebidos, especificando-os no campo  “discriminação”, deixando em branco o campo ”situação em 31/12/2018 ($)”, e incluindo no  campo ”situação em 31/12/2019 ($)”, o valor incluído como receita pelo espólio, ex-cônjuge ou doador.   em qualquer das hipóteses acima, os sucessores legítimos, os ex-cônjuges ou os donatários que  recebem os bens e benfeitorias informam o valor incluído como receita pelo espólio, ex-cônjuge ou  doador como rendimento isento na declaração de ajuste anual.    - caso o custo dos bens e benfeitorias que estejam sendo transferidos não tenha sido deduzido como custo  ou despesa da atividade rural, pelo espólio, ex-cônjuge ou doador, o seu valor integra o custo de aquisição,  podendo ser somado ao valor da terra nua, para efeito de eventual apuração de ganho de capital, se for o  caso.       202   atenção:    caso o de cujus tenha recebido adiantamento de recursos financeiros por conta de venda para  entrega futura, a qual não foi feita e ficou a cargo do meeiro(a) ou dos sucessores legítimos, a  receita correspondente é tributada como rendimento da atividade rural destes, caso continuem a  exploração da atividade rural na unidade recebida, ou do espólio caso não encerrado o inventário,  no mês da entrega dos produtos (instrução normativa  nº 83, de 11 de outubro de 2001, art.  19). ',
 'como deve o contribuinte tributar a variação ocorrida entre a data da venda e a do efetivo  recebimento, no caso de alienação de imóvel rural com recebimento parcelado atualizado por índice  de cotação de produtos rurais?   na alienação de imóvel rural com valor de alienação recebido parceladamente e atualizado por índice de  cotação de produtos rurais (arroba de boi ou vaca, saca de soja ou milho etc.), o imposto deve ser apurado  da forma a seguir descrita.   1 - contribuinte possui imóvel rural sem benfeitorias ou com benfeitorias que não foram deduzidas  como despesas de custeio ou investimentos da atividade rural em anos anteriores:   nesse caso, apura-se o ganho de capital em relação ao valor total da alienação, podendo o valor das  benfeitorias, se existentes, integrar o custo (regulamento do imposto sobre a renda – /2018, arts. 128 a  153, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018).   os acréscimos auferidos no negócio, correspondentes à variação positiva do índice combinado, devem ser  tributados em separado do ganho de capital, sujeitando-se ao recolhimento mensal obrigatório (carnê-leão),  se recebidos de pessoa física, ou à retenção na fonte, se pagos por pessoa jurídica, e, também, ao ajuste na  declaração anual.   2 - contribuinte possui imóvel rural com benfeitorias que foram deduzidas como despesas de custeio  ou investimentos da atividade rural em anos anteriores:   nesse caso, apura-se o ganho de capital em relação apenas ao valor da terra nua, devendo a receita  correspondente às benfeitorias ser tributada como da atividade rural.   os acréscimos auferidos no negócio, correspondentes à variação positiva do índice combinado, devem ser  tributados:   a) em relação à terra nua, em separado do ganho de capital, sujeitando-se ao recolhimento mensal obrigatório  (carnê-leão), se recebidos de pessoa física, ou à retenção na fonte, se pagos por pessoa jurídica, e, também,  ao ajuste na declaração anual;   b) em relação às benfeitorias, como receita da atividade rural, juntamente com o valor original recebido pela  venda destas.   atenção:    caso a variação do índice contratado resulte em diminuição do valor a pagar em relação ao previsto  originalmente, não haverá redução na base de cálculo do ganho de capital.  ',
 'como deve o contribuinte tributar a variação ocorrida entre a data da venda e a do efetivo  recebimento, no caso de alienação de bens móveis da atividade rural com recebimento parcelado  atualizado por índice de cotação de produtos rurais?   na alienação de bens móveis da atividade rural com valor de alienação recebido parceladamente e atualizado  por índice de cotação de produtos rurais (arroba de boi ou vaca, saca de soja ou milho etc.), o imposto deve  ser apurado da forma a seguir descrita.   1 - contribuinte não deduziu o valor dos bens móveis como despesas de custeio ou investimentos da  atividade rural em anos anteriores:   nesse caso, apura-se o ganho de capital em relação ao valor total da alienação (regulamento do imposto  sobre a renda – /2018, arts. 128 a 153, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018).   os acréscimos auferidos no negócio, correspondentes à variação positiva do índice combinado, devem ser  tributados em separado do ganho de capital, sujeitando-se ao recolhimento mensal obrigatório (carnê-leão),  se recebidos de pessoa física, ou à retenção na fonte, se pagos por pessoa jurídica, e, também, ao ajuste na  declaração anual.   2 - contribuinte deduziu o valor dos bens móveis como despesas de custeio ou investimentos da  atividade rural em anos anteriores:   nesse caso, o valor original recebido pela venda dos bens móveis e os acréscimos auferidos no negócio,  correspondentes à variação positiva do índice combinado, devem ser tributados como receita da atividade  rural.   caso se trate de bens alienados juntamente com imóvel rural, sem que o instrumento de transmissão  identifique separadamente seu valor, deve ser apurada a parcela da receita correspondente à alienação dos  bens, por meio de cálculo específico, conforme regras constantes da pergunta 516.   atenção:    caso a variação do índice contratado resulte em diminuição do valor a pagar em relação ao previsto  originalmente, não haverá redução na base de cálculo do ganho de capital.  ',
 'qual é o tratamento tributário das operações referentes à linha de crédito destinada a financiar  a liquidação de dívidas de produtores rurais de que trata a lei nº 11.524, de 24 de setembro de 2007?   no caso de pessoa física, os valores das participações para constituição do fundo de liquidez de que trata o  artigo 3º da referida lei poderão ser considerados como despesa da atividade rural, enquanto que os bônus  de adimplência de que trata o inciso  do § 1º do mesmo artigo serão classificados como receita da atividade  rural. ',
 'qual é o tratamento tributário da variação cambial apurada quando da liquidação de  empréstimos obtidos em moeda estrangeira?     valor da variação cambial apurada no momento da liquidação de empréstimo obtido em moeda estrangeira,  pela diferença em comparação com o valor contratado, será considerado receita ou despesa da atividade  rural, conforme o caso, na data de pagamento de cada parcela.',
 'como deve proceder o contribuinte que receber, em produtos rurais, o valor correspondente  a determinado arrendamento rural?    valor dos produtos rurais deve ser convertido em moeda pelo preço corrente de mercado, no mês do  recebimento, ou pelo preço mínimo oficial, o maior dos dois, sujeitando-se ao carnê-leão, se recebido de  pessoa física, ou à retenção na fonte, se pago por pessoa jurídica, e, também, ao ajuste na declaração anual.   quando estes bens forem vendidos, o contribuinte deve apurar o ganho de capital considerando como custo  o valor anteriormente a eles atribuído como rendimento (para efeito da apuração da base de cálculo do carnê-  leão ou da retenção na fonte). ',
 'como são considerados os rendimentos recebidos pela pessoa física tripulante de barcos  pesqueiros?   os rendimentos derivados das atividades de captura e venda in natura do pescado são oriundos da exploração  animal e consideram-se da atividade rural.   os rendimentos da pessoa física tripulante de barcos pesqueiros têm o seguinte tratamento:   1 - quando à pessoa física couber parte ou quinhão nos resultados da pesca, equipara-se a parceiro rural para  os efeitos do imposto sobre a renda. não é tributável na fonte ou no carnê-leão o quinhão ou a parte de cada  um, por ocasião do rateio do produto da pesca, pois constitui rendimento da atividade rural;   2 - quando a pessoa física exercer as suas funções mediante contrato de trabalho com vínculo empregatício,  de conformidade com o que prescreve a consolidação das leis do trabalho  , as quantias recebidas são  rendimentos do trabalho e sujeitam-se à tributação na fonte e, também, ao ajuste na declaração anual;    3 - no caso de a pessoa física prestar serviço sem vínculo empregatício, os valores recebidos sujeitam-se ao  carnê-leão, se recebidos de pessoas físicas, ou à retenção na fonte, se pagos por pessoa jurídica, e, também,  ao ajuste na declaração anual. ',
 'como devem ser considerados os valores recebidos em adiantamento relativo à venda de  produto rural a ser entregue em ano posterior ao financiador?   como o contrato de compra e venda de coisa futura configura modalidade de ato jurídico sob condição  suspensiva, ou seja, modalidade em que a eficácia do ato jurídico fica pendente de evento futuro, o fato  gerador da obrigação tributária somente ocorre com o implemento da condição, isto é, com a materialização  da coisa futura (produção rural) e sua venda ao financiador.   dessa forma, a importância paga pela aquisição da produção, referente à parte contratada que o produtor  tenha recebido como antecipação, deve ser computada como receita somente no mês do ano-calendário em  que a condição se implementar, ou seja, no mês em que a venda se concluir com a entrega efetiva dos  produtos.   caso haja devolução de valor antes da efetiva entrega do produto rural, este valor deve ser diminuído da  importância recebida por conta da venda, porém, se houver devolução depois da entrega efetiva do produto  rural, este valor constituirá despesa no mês da devolução.    valor recebido por conta de adiantamento de recursos financeiros, referente à venda de produto rural a ser  entregue em ano posterior, deve ser informado na linha própria da apuração do resultado não tributável do  demonstrativo da atividade rural. ',
 'como tributar os adiantamentos recebidos em anos anteriores relativos a produto rural  entregue no ano-calendário que se está declarando?   os valores que não tiverem sido computados como receita no exercício financeiro correspondente ao seu  recebimento, em função de terem se constituído em adiantamentos por conta de venda para entrega futura  recebidos em anos anteriores, devem ser considerados nos anos posteriores como receita da atividade rural  no mês da efetiva entrega do produto rural.   caso haja devolução de valor antes da efetiva entrega do produto rural, este valor deve ser diminuído da  importância recebida por conta da venda, porém, se houver devolução após a entrega efetiva do produto rural,  este valor constituirá despesa no mês da devolução.   além disso, estes valores devem ser informados na linha própria da apuração do resultado não tributável  do demonstrativo da atividade rural.  ',
 'qual é o tratamento tributário do valor recebido em moeda corrente do fornecedor, decorrente  da devolução de bens e insumos rurais anteriormente adquiridos e pagos?    caso a devolução ocorra no mesmo ano-calendário do pagamento da aquisição, este valor deve ser diminuído  da importância considerada como despesa, porém, se houver a devolução no ano-calendário seguinte (ou  seguintes), este valor constituirá receita da atividade rural no mês do recebimento.  ',
 'considera-se receita da atividade rural a cessão de direito à aquisição, por meio de consórcio,  de bem destinado à atividade rural?   não, tributa-se como ganho de capital a transferência (cessão de direito à aquisição de bem) enquanto não  recebido o bem. entretanto, no caso de bem já contemplado, considera-se receita da atividade rural o valor  recebido pela cessão do próprio bem. ',
 'devem ser consideradas como receita de atividade rural as importâncias recebidas de órgãos  públicos para reembolso do custo operacional?   os valores recebidos de órgãos públicos, tais como auxílios, subvenções, subsídios, aquisições do governo  federal  , prêmio equalizador pago ao produtor (pepro) e as indenizações recebidas do programa de  garantia da atividade agropecuária (pro-agro) são considerados como receita da atividade rural no mês em  que forem recebidos.   atenção:    também classificam-se como receita da atividade rural os valores recebidos a título de crédito  tributário, como, por exemplo, as transferências de crédito de imposto sobre operações relativas à  circulação de mercadorias e sobre prestações de serviços de transporte interestadual e  intermunicipal e de comunicação  , quando previstas nas respectivas legislações.  ',
 'qual é o tratamento tributário do valor recebido por liquidação de sinistro, furto ou roubo de  insumos, bens ou produtos rurais?     valor recebido das companhias seguradoras, nestes casos, deve ser considerado receita da atividade rural,  com o mesmo efeito de ter sido alienado, tendo em vista a anterior apropriação destes itens como custo ou  despesa daquela atividade.',
 'como devem ser consideradas as receitas provenientes de vendas efetuadas a prazo, com  emissão de nota promissória rural a vencer em outro ano-calendário?   os valores correspondentes à venda a prazo, vinculada à emissão de notas promissórias rurais, são  considerados como receita da atividade rural no mês em que o vendedor vier a receber efetivamente o  pagamento garantido pelos títulos.  ',
 'como devem ser tributados as receitas da atividade rural, a atualização monetária e os juros  recebidos em decorrência de decisão judicial, em virtude de mora do devedor?   independentemente do ano a que se refiram, todos esses valores são considerados receita da atividade rural  apenas no mês de seu efetivo recebimento. ',
 'quando se apropria a receita no caso de produto rural vendido por intermédio de cooperativa?   como o ato cooperativo não implica operação de mercado, nem contrato de compra e venda de mercadoria  ou produto, e como a entrega de produto rural do cooperado à sua cooperativa não significa mais do que a  outorga de poderes, o cômputo como receita e a tributação dos rendimentos deve ocorrer no momento do  recebimento do produto da venda de produto agropecuário realizado pela sociedade cooperativa.   ressalte-se que integram também a receita da atividade rural as sobras líquidas decorrentes da  comercialização de produtos agropecuários, apuradas na demonstração de resultado do exercício – sobras  ou perdas líquidas e quando pagas, creditadas ou distribuídas pelas sociedades cooperativas de produção  agropecuária ou agroindustrial aos cooperados produtores rurais.  ',
 'animais, produtos ou bens rurais entregues para integralizar quotas subscritas em sociedade  (empresa rural) configuram receita da atividade rural?   sim, a entrega de animais, produtos ou bens rurais para integralização de capital em sociedade por quotas  implica obtenção de receita e, em consequência, deve compor o resultado da atividade rural.  valor pelo qual  os animais, produtos ou bens rurais forem transferidos deve ser incluído como receita para apuração do  rendimento tributável. ',
 'qual é o valor de alienação a ser atribuído aos bens ou benfeitorias vendidos juntamente com  a terra nua, quando no instrumento de transmissão não constar em separado seu valor?   no caso de alienação de bens ou benfeitorias juntamente com a terra nua, sem que o instrumento de  transmissão identifique separadamente o valor da terra nua, o valor de venda a ser atribuído aos bens ou  benfeitorias é determinado da maneira a seguir descrita.   1 - contribuinte computou como despesa da atividade rural os bens ou benfeitorias:   a) calcula-se o valor dos bens ou benfeitorias que foram deduzidos como custo ou despesa na apuração do  resultado da atividade rural da seguinte forma:     os valores apropriados nos anos-calendário de 1990 e 1991, em moeda, devem ser convertidos em  ufir pelo valor de cr$ 597,06 e reconvertidos para reais pelo valor de $ 0,6767;     os valores apropriados no período de 01/01/1992 a 31/12/1994 em ufir, mediante a divisão pelo seu  valor no mês do pagamento do bem, devem ser reconvertidos para reais pelo valor de $ 0,6767;     os valores apropriados a partir de 01/01/1995 devem ser considerados pelo seu valor original em  reais.   b) determina-se a relação percentual entre o valor dos bens ou benfeitorias computadas como despesa, que  corresponde ao total em reais apurado conforme a alínea "a", e o custo total em reais do patrimônio alienado,  ou seja, do imóvel rural (terra nua e benfeitorias) somado ao dos bens;   c) aplica-se o percentual apurado na alínea "b" sobre o valor de alienação do patrimônio, constante do  instrumento de transmissão. essa quantia deve ser oferecida à tributação como receita da atividade rural e a  diferença entre o valor total de alienação e essa quantia será considerada valor de alienação da terra nua para  cálculo do ganho de capital.   2 - contribuinte não computou como despesa da atividade rural os bens ou benfeitorias:   nesse caso, os rendimentos auferidos na venda dos bens ou benfeitorias são tributados como ganho de  capital, a ser apurado separadamente para os bens, considerando-se:   a) como custo de aquisição, o valor em reais, separadamente dos bens e das benfeitorias, constante na ficha  de bens e direitos de sua declaração de rendimentos, somando-se em seguida o valor das benfeitorias ao da  terra nua, ou zero, caso não tenha sido incluído na ficha de bens e direitos; e   b) como valor de alienação, a parcela do valor recebido na venda do patrimônio correspondente aos bens e  às benfeitorias, a ser calculada da seguinte forma:   b.1) determina-se a relação percentual entre o custo de aquisição em reais dos bens, e posteriormente em  separado das benfeitorias, e o custo total em reais do patrimônio alienado, ou seja, do imóvel rural (terra nua  e benfeitorias) somado ao dos bens. caso os bens ou benfeitorias não tenham sido incluídos na ficha de bens  e direitos, o contribuinte deve providenciar um laudo de avaliação de acordo com as normas da associação  brasileira de normas técnicas  , relativo à data da alienação, para determinar a relação percentual  entre o valor dos bens ou benfeitorias e o do patrimônio alienado;   b.2) aplica-se o percentual apurado conforme o item anterior sobre o valor de alienação do patrimônio,  constante do instrumento de transmissão;   b.3) o valor encontrado relativo às benfeitorias deve ser somado ao valor da terra nua, enquanto que o relativo  aos bens deve ser considerado isoladamente.   ',
 'qual é o documento que comprova a venda de bens e benfeitorias utilizados na atividade  rural?    receita decorrente da venda de bens e benfeitorias deve ser sempre comprovada por documentos  reconhecidos pelas fiscalizações estaduais, por instrumentos de transmissão, quando alienados juntamente  com o imóvel rural, ou por documentos hábeis e idôneos, inclusive recibos, onde necessariamente devem  constar as informações do adquirente, o preço, a data da operação e as condições de pagamento. ',
 'qual é o tratamento tributário do valor obtido na venda de bens da atividade rural em mês  posterior ao da alienação do imóvel rural por pessoa física que nele exercia essa atividade?   os bens (tratores, utilitários e demais equipamentos), quando adquiridos e utilizados na exploração da  atividade rural, são considerados investimentos e deduzidos como despesa na apuração do resultado.   caso isto tenha ocorrido, o resultado da atividade rural deve ser ajustado pela inclusão como receita, no mês  da alienação da propriedade rural, do valor correspondente à recuperação de custos (custo histórico) ou do  valor de mercado a eles atribuído.   quando estes bens forem posteriormente vendidos, o contribuinte deve apurar o ganho de capital  considerando como custo o valor anteriormente a eles atribuído como receita (recuperação de custos ou valor  de mercado) para efeito da apuração do resultado da atividade rural no mês da venda da propriedade.   entretanto se ele voltar a explorar a atividade rural, após alguma interrupção, poderá novamente considerar  como custo dessa atividade o valor anteriormente a eles atribuído como receita (recuperação de custos ou  valor de mercado).    se o contribuinte prosseguir sem interrupção na exploração da atividade rural, transferindo esses bens para  outro imóvel rural, a venda deles será considerada receita da atividade rural no mês do efetivo recebimento.   ',
 'qual é o tratamento tributário do valor obtido na venda de produto rural em data posterior à  da alienação do imóvel rural no qual ele foi produzido?    valor recebido na venda de produto rural deve ser tributado pelo ex-proprietário como receita da atividade  rural, no mês de seu efetivo recebimento, desde que comprovado que tal produto seja resultante de atividade  rural exercida por ele, na propriedade alienada.   excetuada a hipótese acima referida:   a) os rendimentos auferidos na venda são tributados como ganho de capital, se essa atividade não for exercida  com habitualidade (lei nº 7.713, de 22 de dezembro de 1988, art. 3º, § 2º); ou   b) se houver habitualidade e fim especulativo de lucro, a pessoa física será considerada empresário (empresa  individual) equiparado a pessoa jurídica, sendo seus lucros tributados nessa condição (regulamento do  imposto sobre a renda – /2018, arts. 162, 623 e 624, aprovado pelo decreto nº 9.580, de 22 de novembro  de 2018).      209    receita bruta, decorrente da venda de produtos rurais, deve ser comprovada por documentos hábeis e  idôneos usualmente utilizados, tais como: nota fiscal do produtor, nota fiscal de entrada, nota promissória rural  vinculada à nota fiscal do produtor e demais documentos reconhecidos pelas fiscalizações estaduais. ',
 'qual é o tratamento tributário do valor obtido na venda de bens e benfeitorias recebidos em  herança, dissolução da sociedade conjugal ou doação?    tratamento tributário é o seguinte:   1 - se o sucessor legítimo, ex-cônjuge ou donatário explorar atividade rural, ainda que como arrendatário ou  parceiro, o valor obtido será incluído como receita da atividade rural (lei nº 8.023, de 12 de abril de 1990; lei  nº 8.383, de 30 de dezembro de 1991).   2 - excetuada a hipótese acima referida:   a) os rendimentos auferidos na venda são tributados como ganho de capital, se essa atividade não for exercida  com habitualidade ',
 'gastos com assistência à saúde de trabalhador rural podem ser deduzidos como despesa da  atividade rural?   somente podem ser deduzidos os gastos com assistência à saúde do trabalhador rural que se configurem  como permanentes e constantes, e se destinem à melhoria da condição de vida dele, não sendo permitida a  dedução de gastos eventuais. ',
 'quando se apropria a despesa decorrente de aquisição de gado ou de outros bens da  atividade rural com pagamento antecipado do preço?   os valores efetivamente pagos antecipadamente para aquisição de bens da atividade rural são computados  como despesa somente no mês do recebimento dos animais ou bens.   ',
 'podem ser deduzidas na atividade rural as despesas de aluguel, uso e manutenção de  veículos?   sim, desde que as despesas sejam realizadas com veículos utilizados diretamente na atividade rural, tais  como os gastos realizados com aluguel, peças de reposição, manutenção e uso de veículos, combustíveis,  óleos lubrificantes, serviços de mecânico, salários do condutor. ',
 'os gastos com aluguel ou arrendamento de avião somente para deslocamento do contribuinte  visando à compra de animais, insumos etc., podem ser considerados como despesa de custeio?   não. essas despesas são indedutíveis por não estarem diretamente relacionadas com a atividade rural, não  sendo necessárias, assim, à manutenção da fonte produtora dos rendimentos (imóvel rural), nem à percepção  do rendimento tributável.   para a dedutibilidade das despesas devem ser observados alguns princípios, tais como: o da autorização, o  da consumação, o da correlação, o da efetividade, o da necessidade.  ',
 'podem ser deduzidos os gastos com aeronaves?   somente podem ser deduzidos os gastos com:   1 - aquisição de aeronaves próprias para uso agrícola, desde que a utilização seja exclusiva para a atividade  rural, bem assim os gastos realizados com peças de reposição, manutenção e uso da aeronave, combustíveis,  óleos lubrificantes, serviços de mecânico, salários do piloto etc.;   2 - aluguel das aeronaves descritas no item "1" ou a contratação de serviço com o uso delas (pulverização,  semeadura etc.). ',
 'qual é o valor que pode ser considerado como despesa de custeio quando o proprietário rural   desconta do valor total devido ao empregado, que mora em casa situada dentro do imóvel rural, um  percentual a título de habitação por força de contrato de trabalho?    proprietário rural pode considerar como despesa de custeio o valor total devido ao empregado e deve  considerar como rendimento de aluguel, sujeito ao carnê-leão e, também, ao ajuste anual, o valor a título de  habitação descontado do empregado. esse entendimento é válido mesmo na hipótese de não haver contrato  de locação formalizado entre as partes, sendo suficiente, para esse efeito, a cláusula do contrato de trabalho. ',
 'que se considera prestação de serviços da atividade rural, para fins de dedução como  despesa de custeio?   considera-se prestação de serviços da atividade rural, para fins de dedução como despesa de custeio,  aquela  necessária à percepção dos rendimentos e à manutenção da fonte produtora e que contribui normalmente  para a realização da produção rural, como por exemplo:     as operações culturais, de colheita, debulha, enfardação, ceifa e recolha, incluindo operações de  sementeira e de plantação;     as operações de embalagem e de acondicionamento, tais como a secagem, limpeza, trituração,  desinfecção e ensilagem de produtos agrícolas;     a armazenagem de produtos agrícolas;     a guarda, criação ou engorda de animais;     a locação, para fins rurais, dos meios normalmente utilizados nas explorações agrícolas, pecuárias,  silvícolas ou de pesca;     a assistência técnica;     a destruição de plantas e animais nocivos, o tratamento de plantas e de terreno por pulverização;     a exploração de instalações de irrigações e de drenagem; e     a poda de árvores, corte de madeira e outros serviços silvícolas.  ',
 'podem ser deduzidas da receita bruta da atividade rural as despesas ocorridas na utilização  de imóvel fora da área rural?   como regra geral, somente as despesas ocorridas dentro da propriedade rural explorada podem ser  deduzidas das receitas auferidas nessa atividade. assim, em princípio, só podem ser computadas as  despesas ocorridas fora da área de exploração rural quando efetivamente necessárias a essa atividade, como  utilização de mão-de-obra e equipamentos de escritório, uso e ocupação de imóveis etc., quando exista  contrato formal de uso, locação, arrendamento, armazenamento etc.   todavia, quando a pessoa física que explore essa atividade possuir imóveis situados fora da área rural, mas  cuja destinação única e específica seja para seleção, secagem, armazenamento etc., dos produtos oriundos  exclusivamente das propriedades rurais nas quais exerceu essa atividade, percebendo ou não pela cessão  desses bens, as despesas de mão-de-obra correspondentes a essas atividades, bem como as decorrentes  da utilização e da conservação desses bens, podem ser deduzidas da receita bruta, desde que sejam  compatíveis com as práticas usuais e preço de mercado e estejam devidamente comprovadas. ',
 'quando se considera despesa a aquisição de bem da atividade rural por intermédio de  consórcio?   somente após o recebimento do bem, os valores já pagos, acrescidos do valor correspondente ao lance, se  for o caso, podem ser considerados despesa. daí em diante, os demais pagamentos, até o final do contrato,  também serão considerados despesa no mês correspondente.   enquanto o bem não for recebido, os valores pagos em cada ano-calendário devem ser informados apenas  na ficha relativa a bens da atividade rural do demonstrativo da atividade rural, não podendo ser  considerados despesa.   (',
 'como devem ser consideradas as importâncias recebidas a título de financiamento ou  empréstimo para formação e manutenção da atividade rural?   as importâncias correspondentes aos financiamentos ou empréstimos obtidos são consideradas recursos no  ano em que forem recebidas e declaradas pelo saldo em 31 de dezembro de cada ano na ficha dívidas  vinculadas à atividade rural do demonstrativo da atividade rural. os dispêndios com formação e  manutenção da atividade rural são considerados despesas ou investimentos no mês em que forem efetivados  como custeio ou como inversão de capital. os encargos financeiros efetivamente pagos em decorrência de  empréstimos contraídos para o financiamento de custeio e investimentos da atividade rural podem ser  deduzidos como despesa na apuração do resultado.   ressalte-se que as parcelas de amortização do financiamento ou empréstimo, no montante correspondente  ao valor do principal, não podem ser deduzidas como despesa quando de seu pagamento, devendo apenas  ser informadas na ficha dívidas vinculadas à atividade rural do demonstrativo da atividade rural.  ',
 'podem ser deduzidas da receita bruta da atividade rural as multas pagas durante o ano- calendário?    regra geral é que são indedutíveis como despesa da atividade rural os pagamentos de multas efetuados  durante o ano-calendário, especialmente aquelas decorrentes de infrações legais aplicadas por auto de  infração/notificação.   entretanto, as multas moratórias decorrentes do pagamento em atraso de despesas dedutíveis da atividade  rural têm o mesmo tratamento destas despesas e, nesse caso, são também dedutíveis.  também as multas decorrentes do descumprimento de obrigações acessórias, de que não resulte falta ou  insuficiência de pagamento de tributos, são dedutíveis, bem como as decorrentes do descumprimento de  obrigações contratuais que representem despesas dedutíveis da atividade rural.',
 'quais são os gastos que podem ser considerados investimentos?   de forma geral, considera-se investimento a efetiva aplicação de recursos financeiros, durante o ano- calendário, que vise ao desenvolvimento da atividade rural para a expansão da produção e melhoria da  produtividade e seja realizado com:   1 - benfeitorias resultantes de construção, instalações, melhoramentos, culturas permanentes, essências  florestais e pastagens artificiais;   2 - aquisição de tratores, implementos e equipamentos, máquinas, motores, veículos de carga ou utilitários  usados diretamente na atividade rural, utensílios e bens de duração superior a um ano e animais de trabalho,  de produção e de engorda;   3 - serviços técnicos especializados, devidamente contratados, visando elevar a eficiência do uso dos recursos  da propriedade ou exploração rural;   4 - insumos que contribuam destacadamente para a elevação da produtividade, tais como: reprodutores,  sementes e mudas selecionadas, corretivos do solo, fertilizantes, vacinas e defensivos vegetais e animais;   5 - atividades que visem especificamente à elevação socioeconômica do trabalhador rural, tais como: casas  de trabalhadores, prédios e galpões para atividades recreativas, educacionais e de saúde;   6 - estradas que facilitem o acesso ou a circulação na propriedade;   7 - instalação de aparelhagem de comunicação e de energia elétrica;   8 - bolsas de estudo para formação de técnicos em atividades rurais, inclusive gerentes de estabelecimentos  e contabilistas.',
 'custo de aquisição da terra nua constitui investimento da atividade rural?   não. na aquisição de imóvel rural o custo da terra nua não constitui investimento da atividade rural. ',
 'os gastos com a construção de escolas em propriedade rural e com a educação dos filhos  dos empregados podem ser deduzidos das receitas da atividade rural?   sim, esses encargos são considerados investimentos e podem ser deduzidos como despesas para efeito da  apuração do resultado da atividade rural, desde que ocorridos dentro da propriedade rural do contribuinte.  além disso, também podem ser deduzidos os gastos com bolsas para a formação de técnicos em atividades  rurais, inclusive gerentes de estabelecimentos e contabilistas. ',
 'como proceder em relação a adiantamento recebido para aquisição de bem da atividade rural  com pagamento posterior em produtos rurais da atividade exercida pelo contribuinte?    valor do bem adquirido com o adiantamento é considerado investimento, no mês do efetivo recebimento do  bem.    valor da dívida paga com a posterior entrega de produtos da atividade rural exercida pelo contribuinte deve  ser tributado como receita da atividade rural, no mês da entrega. ',
 'que deve ser computado como investimento no caso de financiamento rural para aquisição  de bens?   deve ser computado como investimento o valor total dos bens adquiridos e não o do financiamento.  saldo  em 31 de dezembro de cada ano do financiamento rural deve ser declarado como dívidas vinculadas à  atividade rural do demonstrativo da atividade rural, e o valor dos encargos financeiros efetivamente pagos  em decorrência de empréstimos contraídos para o financiamento dos custeios/investimentos da atividade rural  pode ser deduzido como despesa quando da apuração do resultado. ',
 'que se considera investimento no caso de exploração animal na captura in natura do  pescado?   considera-se investimento na captura in natura do pescado a efetiva aplicação de recursos financeiros,  durante o ano-calendário, que vise ao desenvolvimento da atividade rural para a expansão, captura e melhoria  da produtividade da pesca, tais como reforma/aquisição de motores, de embarcações, de frigoríficos, de redes  de pesca, de botes ou caíques, de rádios de comunicação, de bússolas, de sondas, de radares, de guinchos,  de cordas, de anzóis e de boias.  ',
 'que se considera investimento no caso da exploração da piscicultura?   considera-se investimento na piscicultura a efetiva aplicação de recursos financeiros, durante o ano- calendário, que vise ao desenvolvimento da atividade rural para a expansão e melhoria da produtividade do  cultivo do pescado, tais como: aquisição de matrizes e alevinos, reparo, construção e limpeza de diques,  tanques, comportas e canais.  ',
 'os gastos com desmatamento de terras, para implantação de culturas permanentes, podem  ser considerados investimentos?  os valores pagos a título de desmate, enleiramento, derrubada de árvores, catação de raízes etc., para  implantação de culturas permanentes, essências florestais e pastagens artificiais, constituem investimentos.   alerte-se que, havendo recebimento de valores em virtude da posterior venda de produtos da atividade rural,  tais como madeira, lenha, carvão etc., retirados do imóvel rural, o montante recebido deve ser tributado como  receita da atividade rural.   ',
 'montante despendido no ano-calendário na aquisição de reprodutores ou matrizes pode  ser considerado despesa de custeio?    valor despendido com a compra de reprodutores ou matrizes, inclusive  (puro por origem) ou  (puro  por cruza), bem como de alevinos, girinos, embriões e sêmen, pode ser considerado investimento no ano de  sua aquisição e, como tal, pode ser deduzido no mês em que se efetivarem as despesas.   também pode ser deduzida a parte do valor correspondente a esses reprodutores ou matrizes, quando  adquiridos em sociedade ou condomínio.   ',
 'aquisição de quotas de cooperativa rural ou de participação societária no capital de  empresa rural é considerada investimento na atividade rural?   não, este dispêndio é considerado apenas aquisição de participação societária e deve ser incluído na ficha  de bens e direitos da declaração de ajuste anual.  ',
 'qual é o tratamento tributário do valor das benfeitorias realizadas no imóvel rural durante o  ano-calendário?    valor das benfeitorias realizadas no imóvel rural durante o ano-calendário é considerado investimento e  pode ser deduzido como despesa de custeio e, neste caso, indicado, destacadamente, em bens da atividade  rural do demonstrativo da atividade rural, nos  campos “discriminação” e valores em reais.   caso o contribuinte opte por não considerar o valor das benfeitorias realizadas como despesa da atividade  rural, poderá informá-las na declaração de bens e direitos da declaração de ajuste anual, indicando o valor  dispendido no campo “situação em 31/12/2019 ($)”.  ',
 'quais as operações sujeitas à apuração do ganho de capital?   estão sujeitas à apuração de ganho de capital as operações que importem:    - alienação, a qualquer título, de bens ou direitos ou cessão ou promessa de cessão de direitos à sua  aquisição, tais como as realizadas por compra e venda, permuta, adjudicação, dação em pagamento,  procuração em causa própria, promessa de compra e venda, cessão de direitos ou promessa de cessão de  direitos e contratos afins;    - transferência a herdeiros e legatários na sucessão causa mortis, a donatários na doação, inclusive em  adiantamento da legítima, ou atribuição a ex-cônjuge ou ex-convivente, na dissolução da sociedade conjugal  ou união estável, de bens e direitos por valor superior àquele pelo qual constavam na declaração de ajuste  anual do de cujus, do doador, do ex-cônjuge ou ex-convivente que os tenha transferido;    - alienação de bens ou direitos e liquidação ou resgate de aplicações financeiras, de propriedade de pessoa  física, adquiridos, a qualquer título, em moeda estrangeira.   atenção:   regime especial de regularização cambial e tributária  :   no período de 04/04/2016 a 31/10/2016 foi permitida, por opção, a regularização cambial e tributária  de patrimônio localizado no brasil ou no exterior, em nome do próprio declarante ou de terceiro,  desde que atendidas as seguintes condições:    – ser residente no brasil, para fins tributários, em 31 de dezembro de 2014;    – apresentar, no período de 4 de abril de 2016 a 31 de outubro de 2016, a declaração de  regularização cambial e tributária (dercat);    – informar na  referente ao ano-calendário de 2014 os recursos, bens e direitos de qualquer  natureza constantes na dercat;    – pagar o imposto sobre ganho de capital com alíquota de 15%, tendo como base de cálculo o  valor de mercado, do bem ou direito regularizado, em 31/12/2014;    – pagar a multa de igual valor ao imposto apurado no item ;    – não incorrer nas condições impeditivas aplicáveis ao .   reabertura do :    regularização cambial   foi reaberta no período de 03/04/2017 a 31/07/2017, exigindo  o atendimento das seguintes condições:      216    – ser residente no brasil, para fins tributários, em 30 de junho de 2016;    – apresentar, no período de 3 de abril de 2017 a 31 de julho de 2017, a declaração de  regularização cambial e tributária (dercat);    – informar na  referente ao ano-calendário de 2016 os recursos, bens e direitos de qualquer  natureza constantes na dercat. se necessário retificar a  do exercício de 2017 o prazo para  apresentar a retificadora em virtude da regularização cambial e tributária é 30 de dezembro de  2017.    – pagar o imposto sobre ganho de capital com alíquota de 15%, tendo como base de cálculo  o valor de mercado, do bem ou direito regularizado, em 30/06/2016;    – pagar a multa de valor igual a 135% (cento e trinta e cinco por cento) do imposto apurado no  item ;    – não incorrer nas condições impeditivas aplicáveis ao regime especial de regularização  cambial e tributária.    regime especial de que se trata aplica-se ainda ao espólio cuja sucessão tenha sido aberta  até a data de adesão ao .   é facultado ao contribuinte que aderiu ao  até 31 de outubro de 2016 complementar a  dercat entregue na primeira etapa da regularização, obrigando-se, caso exerça este direito, a  pagar o imposto correlato e multa devidos resultantes desta alteração.  ',
 'quais as alíquotas aplicáveis para efeito de apuração do ganho de capital?    partir de 1º de janeiro de 2017, as operações de alienação de bens e direitos de qualquer natureza passíveis  de apuração de ganho de capital sujeitam-se às seguintes alíquotas:    – 15% sobre a parcela dos ganhos que não ultrapassar $ 5.000.000,00;    – 17,5% sobre a parcela dos ganhos que exceder $ 5.000.000,00 e não ultrapassar $ 10.000.000,00;    – 20% sobre a parcela dos ganhos que exceder $ 10.000.000,00 e não ultrapassar $ 30.000.000,00; e    – 22,5% sobre a parcela dos ganhos que ultrapassar $ 30.000.000,00.   na hipótese de alienação em partes do mesmo bem ou direito, a partir da segunda operação, desde que  realizada até o final do ano-calendário seguinte ao da primeira operação, o ganho de capital deve ser somado  aos ganhos auferidos nas operações anteriores, para fins de definição da alíquota aplicável, deduzindo-se o  montante do imposto pago nas operações anteriores.    apuração do imposto sobre a renda com a utilização de alíquotas progressivas não se aplica:   - ao fundo de investimento imobiliário   - 20% (lei nº 8.668, de 25 de junho de 1993, art. 18);   - aos fundos de investimento em participações - 15% (lei nº 11.312, de 27 de junho de 2006, art. 2º);   - aos fundos de investimentos em cotas de fundos de investimento em participações - 15% (lei nº 11.312,  de 27 de junho de 2006, art. 2º);   - aos fundos de investimentos em empresas emergentes - 15% (lei nº 11.312, de 27 de junho de 2006, art.  2º);   - à diferença a maior entre o valor de mercado e o valor pelo qual o bem ou direito constavam na declaração  de bens do de cujus, do doador ou do ex-cônjuge ou ex-convivente, nas transferências de direito de  propriedade por sucessão, dissolução da sociedade conjugal ou da unidade familiar, nos casos de herança,  legado ou por doação em adiantamento da legítima, nas hipóteses de os bens e direitos serem avaliados a  valor de mercado - 15% (lei nº 9.532, de 10 de dezembro de 1997, art. 23).  ',
 'quais são as isenções relativas ao ganho de capital?   1 - indenização da terra nua por desapropriação para fins de reforma agrária, conforme o disposto no § 5º do  art. 184 da constituição federal de 1988;   atenção:     parcela da indenização, correspondente às benfeitorias, é computada como receita da atividade  rural quando esta tiver sido deduzida como despesa de custeio ou investimento e, quando não  deduzida como despesa de custeio ou investimento, tributável como ganho de capital.    respeito de desapropriação, consulte a pergunta 611   2 - indenização por liquidação de sinistro, furto ou roubo, relativo ao objeto segurado;   3 - alienação, por valor igual ou inferior a $ 440.000,00, do único bem imóvel que o titular possua,  individualmente, em condomínio ou em comunhão, independentemente de se tratar de terreno, terra nua, casa  ou apartamento, ser residencial, comercial, industrial ou de lazer, e estar localizado em zona urbana ou rural,  desde que não tenha efetuado, nos últimos cinco anos, outra alienação de imóvel a qualquer título, tributada  ou não, sendo o limite considerado em relação:     à parte de cada condômino ou coproprietário, no caso de bens possuídos em condomínio;     ao imóvel possuído em comunhão, no caso de sociedade conjugal ou união estável (salvo contrato  escrito entre os companheiros);   consulte as perguntas 589, 611, 634, 635, 636 e 637   4 - ganho apurado na alienação de imóveis adquiridos até 1969;   5 -  valor da redução do ganho de capital para imóveis adquiridos entre 1969 e 1988;   consulte a pergunta 608   6 -  partir de 16/06/2005, o ganho auferido por pessoa física residente no brasil na venda de imóveis  residenciais, desde que o alienante, no prazo de 180 dias contado da celebração do contrato, aplique o produto  da venda na aquisição de imóveis residenciais localizados no país;    opção pela isenção de que trata este item é irretratável e o contribuinte deve informá-la no respectivo  demonstrativo da apuração dos ganhos de capital da declaração de ajuste anual.   atenção:     contagem do prazo de 180 dias inclui a data da celebração do contrato.    no caso de venda de mais de 1 (um) imóvel, o prazo de 180 (cento e oitenta) dias é contado a partir  da data da celebração do contrato relativo à primeira operação.    aplicação parcial do produto da venda implica tributação do ganho proporcionalmente ao valor da  parcela não aplicada.   no caso de aquisição de mais de 1 (um) imóvel, a isenção de que trata este item aplica-se ao ganho  de capital correspondente apenas à parcela empregada na aquisição de imóveis residenciais.    contribuinte somente pode usufruir do benefício de que trata este item 1 (uma) vez a cada 5  (cinco) anos, contados a partir da data da celebração do contrato relativo à operação de venda com  o referido benefício ou, no caso de venda de mais de 1 (um) imóvel residencial, à 1ª (primeira)  operação de venda com o referido benefício.   na hipótese de venda de mais de 1 (um) imóvel, estão isentos somente os ganhos de capital  auferidos nas vendas de imóveis residenciais anteriores à primeira aquisição de imóvel residencial.   relativamente às operações realizadas a prestação, aplica-se a isenção, observada as condições  precedentes:    - nas vendas a prestação e nas aquisições à vista, à soma dos valores recebidos dentro do prazo de  180 (cento e oitenta) dias, contado da data da celebração do primeiro contrato de venda e até a(s) data(s)  da(s) aquisição(ões) do(s) imóvel(is) residencial(is);    - nas vendas à vista e nas aquisições a prestação, aos valores recebidos à vista e utilizados nos  pagamentos dentro do prazo de 180 (cento e oitenta) dias, contado da data da celebração do primeiro  contrato de venda;      218    - nas vendas e aquisições a prestação, à soma dos valores recebidos e utilizados para o pagamento  das prestações, ambos dentro do prazo de 180 (cento e oitenta) dias, contado da data da celebração do  primeiro contrato de venda.   não integram o produto da venda, para efeito do valor a ser utilizado na aquisição de outro imóvel  residencial, as despesas de corretagem pagas pelo alienante.   considera-se imóvel residencial a unidade construída em zona urbana ou rural para fins residenciais,  segundo as normas disciplinadoras das edificações da localidade em que se situar.    isenção aplica-se, inclusive:    - aos contratos de permuta de imóveis residenciais, com torna;    - à venda ou aquisição de imóvel residencial em construção ou na planta.    isenção não se aplica, entre outros:    - à hipótese de venda de imóvel residencial com o objetivo de quitar, total ou parcialmente, débito  remanescente de aquisição a prazo ou à prestação de imóvel residencial já possuído pelo alienante;    - à venda ou aquisição de terreno;    - à aquisição somente de vaga de garagem ou de boxe de estacionamento.   (lei nº 11.196, de 21 de novembro de 2005, art. 39; instrução normativa  nº 599, de 28 de  dezembro de 2005, art. 2º; esolução de consulta cosit nº 211, de 24 de junho de 2019)   7 - alienação de bens ou direitos de pequeno valor, considerado em relação:     ao valor do bem ou do conjunto dos bens ou direitos da mesma natureza, alienados em um mesmo  mês, tais como automóveis e motocicletas, imóvel urbano e terra nua, quadros e esculturas;     à parte de cada condômino ou coproprietário, no caso de bens possuídos em condomínio;     ao valor de cada um dos bens ou direitos possuídos em comunhão e ao valor do conjunto dos bens  ou direitos da mesma natureza, alienados em um mesmo mês, no caso de sociedade conjugal ou  união estável (salvo contrato escrito entre os companheiros).   para alienação de bens ou direitos de pequeno valor, consulte a pergunta 631.   8 - restituição de participação no capital social mediante a entrega à pessoa física, pela pessoa jurídica, de  bens e direitos de seu ativo avaliados por valor de mercado;   9 - transferência a pessoas jurídicas, a título de integralização de capital, de bens ou direitos pelo valor  constante na declaração de rendimentos;   10 - permuta de unidades imobiliárias, sem recebimento de torna (diferença recebida em dinheiro);   atenção:    nas operações de permuta realizadas por contrato particular, somente se configura a permuta se a  escritura pública, quando lavrada, for de permuta.   11 - permuta, caracterizada com a entrega, por valor não superior ao de face, pelo licitante vencedor, de títulos  da dívida pública federal, estadual, do distrito federal ou municipal, ou de outros créditos contra a união, o  estado, o distrito federal ou o município, como contrapartida à aquisição das ações ou quotas leiloadas, no  âmbito dos respectivos programas de desestatização;   12 - alienação de bens localizados no exterior ou representativos de direitos no exterior, bem como a  liquidação ou o resgate de aplicações financeiras, adquiridos a qualquer título, na condição de não residente  (medida provisória nº 2.158-35, de 24 de agosto de 2001, art. 24, § 6º, ; instrução normativa  nº 118, de  27 de dezembro de 2000, art. 14, inciso );   13 -  variação cambial decorrente das alienações de bens ou direitos adquiridos e das liquidações ou  resgates de aplicações financeiras realizadas com rendimentos auferidos originariamente em moeda  estrangeira (instrução normativa  nº 118, de 27 de dezembro de 2000, art. 14, inciso );   atenção:    somente é isenta a variação cambial, sendo tributável o ganho obtido em moeda estrangeira  (consulte a pergunta 601).      219   14 -  variação cambial dos saldos dos depósitos mantidos em instituições financeiras no exterior (instrução  normativa  nº 118, de 27 de dezembro de 2000, art. 11, § 1º);   15 - alienação de moeda estrangeira mantida em espécie, cujo total de alienações, no ano-calendário, seja  igual ou inferior ao equivalente a cinco mil dólares dos estados unidos da américa (medida provisória nº  2.158-35, de 24 de agosto de 2001, art. 24, § 6º, inciso ; instrução normativa  nº 118, de 27 de dezembro  de 2000, art. 14, inciso ). consulte a pergunta 603;   16 -  partir de 01/01/2002, na hipótese de doação de livros, objetos fonográficos ou iconográficos, obras  audiovisuais e obras de arte, para os quais seja atribuído valor de mercado, efetuada por pessoa física a  órgãos públicos, autarquias, fundações públicas ou entidades civis sem fins lucrativos, desde que os bens  doados sejam incorporados ao acervo de museus, bibliotecas ou centros de pesquisa ou ensino, no brasil,  com acesso franqueado ao público em geral:    - o doador deve considerar como valor de alienação o constante em sua declaração de bens;    - o donatário registra os bens recebidos pelo valor atribuído no documento de doação.   atenção:    no caso de alienação de bens recebidos em doação, para efeito de apuração de ganho de capital,  o custo de aquisição é igual a zero ',
 'são isentos os ganhos de capital decorrentes da alienação de um ou mais imóveis  residenciais, cujos recursos são utilizados na construção de outro imóvel?   não.  isenção citada somente se aplica para a aquisição, no prazo de 180 dias, a contar da primeira  alienação, de imóveis residenciais construídos ou em construção, não abrangendo os gastos para a  construção de imóvel, os gastos para a continuidade de obras em imóvel em construção ou ainda os gastos  com benfeitorias ou reformas em imóveis de propriedade do contribuinte.    ',
 'contribuinte que vende imóvel residencial em condomínio e adquire imóvel residencial  privativo fica isento de imposto sobre a renda?    sim. desde que o alienante no prazo de 180 (cento e oitenta) dias contado da celebração do contrato aplique  o produto da venda de sua parte no imóvel em condomínio na aquisição de imóvel privativo, localizado no  brasil.   igualmente isenta será a operação inversa, ou seja, a venda de imóvel residencial privativo e aquisição de  imóvel residencial em condomínio, desde que respeitado o lapso de 180 (cento e oitenta) dias contado da  celebração do contrato para aplicação do recurso.',
 'contribuinte que vende um imóvel residencial e adquire a parte de outro imóvel residencial  pertencente a outro condômino, de um condomínio no qual já é condômino, fica isento de imposto  sobre a renda?   sim. desde que o alienante no prazo de 180 (cento e oitenta) dias contado da celebração do contrato, aplique  o produto da venda na aquisição da parte do imóvel residencial, pertencente ao outro condômino, localizado  no brasil. ',
 'quais são as consequências legais no caso de não observância do prazo de 180 dias para a  compra de imóveis residenciais?    inobservância das condições legais previstas importa em exigência do imposto com base no ganho de  capital, acrescido de:   a) juros de mora, calculados a partir do 2º (segundo) mês subsequente ao do recebimento do valor ou de  parcela de valor do imóvel vendido; e   b) multa, de mora ou de ofício, calculada a partir do 2º (segundo) mês seguinte ao do recebimento do valor  ou de parcela do valor do imóvel vendido, se o imposto não for pago em até 210 (duzentos e dez) dias  contados da data da celebração do contrato.  ',
 'qual é o tratamento tributário aplicável ao valor recebido por ex-cônjuge ou herdeiro referente  a reembolso de diferença no montante de bens e direitos que lhe cabiam em decorrência da meação  ou herança?    valor em moeda corrente recebido por ex-cônjuge ou herdeiro, referente a reembolso de diferença do  montante de bens e direitos que lhe cabiam em decorrência da meação ou herança, corresponde a uma  alienação desses bens e direitos que antes lhe pertenciam e, portanto, está sujeito à apuração do ganho de  capital.',
 'é possível atualizar o valor de imóvel a preço de mercado na declaração de bens e direitos?   não há qualquer previsão legal para atualização do custo de aquisição de imóvel a preço de mercado.  custo  de aquisição do imóvel somente poderá ser alterado caso sejam efetuadas despesas com construção,  ampliação ou reforma no referido imóvel.    cabe destacar, ainda, que essas despesas somente poderão ser incorporadas ao custo de imóvel se  estiverem comprovadas com documentação hábil e idônea (notas fiscais para as despesas com pessoas  jurídicas, recibos para as despesas com pessoas físicas), que deverá ser mantida em poder do contribuinte  por pelo menos cinco anos após a alienação do imóvel.',
 'que se considera como data e custo de aquisição quando o imóvel é adquirido por  usucapião?    data e o custo de aquisição são determinados da forma a seguir:   1 - usucapião extraordinário   neste caso, considera-se como data de aquisição aquela em que tenha se consumado o prazo de prescrição  aquisitiva, ou seja, quinze anos de posse ininterrupta, mansa e pacífica do imóvel, independentemente de  título e boa-fé, podendo-se requerer ao juiz que assim o declare por sentença, a qual servirá de título para o  registro no cartório de registro de imóveis.  prazo estabelecido reduz-se a dez anos se o possuidor houver  estabelecido no imóvel a sua moradia habitual, ou nele realizado obras ou serviços de caráter produtivo.    custo de aquisição será igual a zero, tendo em vista que não houve valor pago pela aquisição do imóvel.  ',
 'integra o custo de aquisição o valor do laudêmio pago na transferência de domínio útil de  bem gravado por enfiteuse?   sim. integra o custo de aquisição o valor de laudêmio pago ao senhorio ou proprietário por desistir do seu  direito de opção na transferência do domínio útil de bem gravado por enfiteuse. ',
 'que se considera como data e valor de aquisição de bem ou direito originariamente  possuído em usufruto e posteriormente em propriedade plena?   nesta hipótese, consideram-se a data e o valor constantes no documento de aquisição da propriedade plena,  sendo irrelevante que o adquirente já possuísse o usufruto do bem ou direito objeto da aquisição. ',
 'que se considera como data e valor de aquisição quando ocorre alienação de imóvel que  havia sido doado em adiantamento da legítima e posteriormente revogada essa doação?    revogação de doação de imóvel ou qualquer outro bem como adiantamento da legítima assinala a data de  aquisição do bem por parte do doador e a de alienação por parte do donatário, para fins de apuração do ganho  de capital.  valor da alienação é aquele da transmissão. na ausência desse, o valor de mercado na data da  operação. ',
 'qual é o custo de aquisição de bens ou direitos adquiridos até 31/12/1991 e os adquiridos  entre 01/01/1992 a 31/12/1995, no caso de contribuinte desobrigado de apresentar a declaração dos  exercícios de 1992, ano-calendário de 1991, e seguintes?   considera-se custo dos bens ou direitos o valor de aquisição expresso em reais.    custo de aquisição dos bens ou direitos adquiridos ou as parcelas pagas até 31/12/1991, avaliados pelo  valor de mercado para essa data e informados na declaração de ajuste anual do exercício de 1992, ano- calendário de 1991, de acordo com o art. 96 da lei nº 8.383, de 30 de dezembro de 1991, é esse valor,  atualizado até 01/01/1996. aplica-se o disposto acima na hipótese de contribuinte desobrigado de apresentar  a declaração do exercício de 1992, ano-calendário de 1991, e seguintes.   no caso de bens ou direitos adquiridos ou de parcelas pagas até 31/12/1991, não avaliados a valor de  mercado, e dos bens ou direitos adquiridos ou das parcelas pagas entre 01/01/1992 e 31/12/1995, o custo  corresponde ao valor de aquisição ou das parcelas pagas até 31/12/1995, atualizado mediante a utilização da  tabela de atualização do custo de bens e direitos, constante no anexo único à instrução normativa  nº  84, de 11 de outubro de 2001.    custo dos bens ou direitos adquiridos ou das parcelas pagas a partir de 01/01/1996 não está sujeito à  atualização. ',
 'qual é o tratamento tributário no caso de cessão do exercício do usufruto?   ocorrendo a cessão do exercício do usufruto:   a) a título oneroso, o cedente (usufrutuário) deve considerar como aluguéis ou arrendamento, e, portanto,  tributadas como tal, as importâncias recebidas periodicamente ou não, fixas ou variáveis, pela celebração do  contrato (art. 41 do regulamento do imposto sobre a renda - /2018, aprovado pelo decreto nº 9.580, de  22 de novembro de 2018);   b) a título gratuito, exceto ao cônjuge ou parente de 1º grau, o cedente (usufrutuário) deve considerar como  rendimento tributável na declaração de rendimentos o equivalente a 10% do valor venal do imóvel, ou do valor  constante na guia do imposto predial e territorial urbano , nos termos do art. 41, § 1º, do regulamento  do imposto sobre a renda - /2018, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018.   (parecer normativo cosit nº 4, de 3 de novembro de 1995).',
 'contribuinte que possui apenas usufruto sobre um imóvel e propriedade de outro, tem direito  a isenção de ganho de capital se alienar o imóvel do qual tem propriedade?   não, porque, para efeitos legais, o contribuinte possui mais de um imóvel.   ',
 'qual é o tratamento tributário na cessão de direito de precatório?   quanto ao cedente:    diferença positiva entre o valor de alienação e o custo de aquisição na cessão de direitos representados por  créditos líquidos e certos contra a fazenda pública (precatórios) está sujeita à apuração do ganho de capital,  pelo cedente.   os ganhos de capital serão apurados, pela pessoa física cedente, no mês em que forem auferidos, e tributados  em separado, mediante aplicação de uma das alíquotas progressivas estabelecidas pela lei nº 13.259, de 16  de março de 2016, não integrando a base de cálculo do imposto na declaração de rendimentos, e o valor do  imposto pago não poderá ser deduzido do devido na declaração.    custo de aquisição na cessão original, ou seja, naquela em que ocorre a primeira cessão de direitos, é igual  a zero, porquanto não existe valor pago pelo direito ao crédito. nas subsequentes, o custo de aquisição será  o valor pago pela aquisição do direito na cessão anterior.   considera-se como valor de alienação o valor recebido do cessionário pela cessão de direitos do precatório.   quanto ao cessionário:    cessionário sub-roga-se no crédito do cedente, que para aquele transfere todos os direitos, inclusive os  acessórios do crédito.   por ocasião do recebimento do precatório, o cessionário apurará o ganho de capital considerando como valor  de alienação o valor líquido passível de compensação, isto é, após excluídas as deduções legais. considera- se como custo de aquisição o valor pago ao cedente, quando da aquisição da cessão de direitos do precatório.      224   os ganhos de capital serão apurados, pela pessoa física cessionária, no mês em que forem auferidos, e  tributados em separado, mediante aplicação de uma das alíquotas progressivas estabelecidas pela lei nº  13.259, de 16 de março de 2016, não integrando a base de cálculo do imposto na declaração de rendimentos,  e o valor do imposto pago não poderá ser deduzido do devido na declaração.   atenção:    crédito líquido e certo, decorrente de ações judiciais, instrumentalizado por meio de precatório,  mantém por toda a sua trajetória a natureza jurídica do fato que lhe deu origem, independendo,  assim, de ele vir a ser transferido a outrem.  acordo de cessão de direitos não pode afastar a  tributação na fonte dos rendimentos tributáveis relativo ao precatório no momento em for quitado  pela união, pelos estados, pelo distrito federal ou pelos municípios.   em função da natureza jurídica do crédito cedido, ocorrerá a incidência de imposto sobre a renda  retido na fonte, quando cabível, no momento do pagamento do precatório, considerado como tal  quando ocorrer a homologação da compensação do precatório com débitos de natureza tributária  do cessionário para com a união, os estados, o distrito federal ou os municípios.   em virtude da transação efetuada, o imposto sobre a renda retido na fonte não constitui ônus do  cessionário nem do cedente, não integrando a base de cálculo do ganho de capital e não sendo  passível de compensação ou dedução na declaração de ajuste anual.   ',
 'como apurar o custo de aquisição de imóvel vendido em partes, em datas diferentes?   para efeito de determinação do ganho de capital tributável, o custo de aquisição de imóvel vendido em partes,  em datas diferentes, deve ser computado na proporção que representar cada parte alienada em relação ao  custo total da área do imóvel.  ',
 'como proceder na hipótese de haver cláusula de promessa de compra e venda prevendo a  rescisão do contrato por falta de pagamento?    promessa de compra e venda de imóvel, desde que contenha todos os requisitos legais que regem este  negócio jurídico, constitui direito entre as partes, sendo instrumento suficientemente válido para configurar a  transmissão dos direitos sobre o imóvel objeto do contrato e a ocorrência do fato gerador da obrigação  tributária.    cláusula que prevê a rescisão do contrato por falta de pagamento do preço ajustado configura modalidade  de ato jurídico sob condição resolutória, ou seja, modalidade em que a eficácia do negócio jurídico não fica  pendente da ocorrência do evento futuro. ela apenas extingue o direito já constituído anteriormente pelo  instrumento de promessa de compra e venda, em virtude da ocorrência do evento futuro previsto no contrato  (falta de pagamento do preço ajustado, no caso).   na hipótese, portanto, por força do art. 117, inciso , do código tributário nacional  , o ato ou negócio  jurídico de alienação do imóvel reputa-se perfeito e acabado, para os efeitos fiscais, a partir da data do  instrumento particular ou público de promessa de compra e venda celebrado entre as partes.   esclareça-se que a cláusula de retrovenda é condição resolutória, não suspensiva do ato. ',
 'como proceder na hipótese de pagamento ao ex-proprietário do imóvel, onde localizada a  jazida, de parcela da receita obtida com a lavra de recursos minerais, em virtude de obrigação  estabelecida no contrato de compra e venda do imóvel?    pagamento ao ex-proprietário do imóvel, onde localizada a jazida, de parcela da receita obtida com a lavra  de recursos minerais, em virtude de obrigação estabelecida no contrato de compra e venda do imóvel, tem  natureza de royalties e constitui rendimento sujeito à tributação pelo  nos termos dos arts. 44, 45 e 688  do regulamento do imposto sobre a de renda (/2018).  ',
 'qual é o tratamento tributário aplicável no caso de rescisão de contrato de promessa de  compra e venda de bens ou direitos?    contrato de promessa de compra e venda, importando em transmissão de bens ou direitos ou na cessão do  direito à sua aquisição, caracteriza alienação para os efeitos da lei nº 7.713, de 22 de dezembro de 1988,  sendo irrelevante, para os efeitos fiscais, a ocorrência de sua rescisão (código tributário nacional  , arts.  116 e 117), ou a existência no instrumento de negociação de cláusula relativa ao desfazimento da transação,  em caso de não pagamento de todas as parcelas na alienação a prestação, bem como ao ressarcimento dos  valores efetivamente pagos. assim, a quantia recebida pelo vendedor e posteriormente ressarcida ao  comprador é considerada pelo vendedor como parte do preço de alienação, devendo o ganho de capital  porventura apurado ser tributado na forma da legislação tributária. ocorrendo posterior alienação dos mesmos  bens ou direitos, é considerado como custo de aquisição:   a) o custo de aquisição original acrescido do ganho de capital apurado na venda cancelada por rescisão, antes  das reduções (ganho de capital 1), no caso de venda à vista; ou   b) o custo de aquisição original acrescido da parcela do ganho de capital apurado na venda cancelada por  rescisão, antes das reduções (ganho de capital 1), correspondente às prestações recebidas, no caso de  venda a prazo. ',
 'como devem ser tributados os resultados obtidos em alienações de participações societárias  quando o preço não pode ser predeterminado?   quando não houver valor determinado, por impossibilidade absoluta de quantificá-lo de imediato (ex.: a  determinação do valor das prestações e do preço depende do faturamento futuro da empresa adquirida, no  curso do período do pagamento das parcelas contratadas), o ganho de capital deve ser tributado na medida  em que o preço for determinado e as parcelas forem pagas.   não obstante ser indeterminado o preço de alienação, toma-se como data de alienação a da concretização  da operação ou a data em que foi cumprida a cláusula preestabelecida nos atos contratados sob condição  suspensiva.   contudo, alerte-se que o tratamento descrito deve ser comprovado pelas partes contratantes sempre que a  autoridade lançadora assim o determinar. ',
 'transferência de bens ou direitos para integralização de capital configura alienação?   sim.  transferência de bens ou direitos a pessoas jurídicas, a título de integralização de capital, configura  alienação.    pessoa física deve lançar, na declaração correspondente ao exercício em que efetuou a transferência, as  ações ou quotas subscritas pelo valor pelos quais os bens ou direitos foram transferidos.    se a transferência dos bens ou direitos tiver sido efetuada por valor superior ao constante para estes na  declaração de bens e direitos, a diferença a maior é tributável como ganho de capital. ',
 'pessoa física que transferiu imóvel a pessoa jurídica, para integralização de capital, deve  apurar ganho de capital quando a empresa aliena o bem?   não, pois nesse caso é a empresa que, sendo proprietária do imóvel, deve apurar ganho de capital quando o  aliena.  ',
 'qual é o tratamento tributário na dissolução parcial de sociedade com transferência de bens  ou direitos para os sócios retirantes, por determinação judicial?   tratando-se de devolução de participação no capital social com a entrega de bens ou direitos do ativo da  pessoa jurídica, estes são informados na declaração de bens e direitos correspondente ao respectivo ano- calendário, pelo valor contábil ou de mercado, conforme avaliado pela pessoa jurídica. se a devolução dos  bens ou direitos à pessoa física for efetuada com base no valor de mercado, a diferença entre o valor de  mercado e o constante na declaração de bens e direitos não se sujeita à incidência do imposto sobre a renda  na pessoa física, sendo tributados na pessoa jurídica. ',
 'qual é o tratamento tributário na transferência de bens e direitos em decorrência de  dissolução da sociedade conjugal ou da união estável?    na transferência do direito de propriedade em decorrência de dissolução de sociedade conjugal ou da união  estável, os bens e direitos podem ser avaliados pelo valor constante na última declaração de bens e direitos  de quem os declarou, atualizado monetariamente até 31/12/1995, ou por valor superior àquele declarado,  observando-se que:   a) se a transferência dos bens ou direitos ao ex-cônjuge ou ex-convivente a quem foram atribuídos os bens  ou direitos foi em valor superior àquele pelo qual constavam na última declaração antes da dissolução da  sociedade conjugal ou união estável, a diferença positiva é tributada à alíquota de 15%. para o cálculo do  ganho de capital em futura alienação deve ser considerada a data desta transferência;   b) se a transferência foi pelo valor constante na última declaração de bens e direitos apresentada antes da  dissolução da sociedade conjugal ou união estável, não há ganho de capital no ato da transferência. ',
 'qual é o prazo para o pagamento do imposto na transferência de bens e direitos em  decorrência de dissolução da sociedade conjugal ou da união estável?    imposto deve ser pago até o último dia útil do mês subsequente ao do trânsito em julgado da decisão judicial  da partilha, sobrepartilha ou da lavratura da escritura pública.    darf do pagamento do imposto deve ser preenchido em nome do ex-cônjuge ou ex-convivente ao qual o  bem ou direito for atribuído e este deve pagar o imposto.  ',
 'qual é o tratamento tributário na transferência de bens ou direitos por herança ou legado?   na transferência do direito de propriedade por sucessão, nos casos de herança ou legado, os bens e direitos  podem ser avaliados pelo valor constante na última declaração de bens e direitos do de cujus, atualizado  monetariamente até 31/12/1995, ou por valor superior àquele declarado, observado o seguinte:    a) se os bens ou direitos forem transferidos por valor superior ao anteriormente declarado, a diferença positiva  entre o valor de transmissão e o valor constante na última declaração de bens e direitos do de cujus ou o  custo de aquisição, é tributada como ganho de capital à alíquota de 15%:      228   a.1) nesta hipótese, o contribuinte do imposto é o espólio, devendo ser preenchido, utilizando-se do  programa gerador específico, o demonstrativo da apuração dos ganhos de capital, exportando o resultado  para a declaração final de espólio;   a.2) o darf do pagamento do imposto deve ser preenchido em nome do espólio;   b) se a transferência for pelo valor constante na última declaração de bens e direitos do de cujus, não há  ganho de capital no ato da transferência;   c) a opção pelo valor constante na última declaração de bens e direitos do de cujus ou por valor superior a  este será feita em relação a cada um dos bens transferidos;   d) o herdeiro ou legatário deve incluir os bens ou direitos, em sua declaração de bens e direitos, pelo valor  de transmissão da parte de que lhe coube, o qual constitui custo para efeito de apuração de ganho de capital  numa eventual alienação futura. considera-se data de aquisição a da abertura da sucessão (falecimento).   atenção:    para efeito de apuração do limite de isenção, na alienação de bens de pequeno valor até  $ 35.000,00, devem ser somados os valores de transferência de todos os bens da mesma  natureza.   ',
 'qual é o prazo para o pagamento do imposto na transferência de bens ou direitos por herança  ou legado?   se a transferência dos bens ou direitos for efetuada por valor superior àquele que vinha sendo declarado, a  diferença a maior sujeitar-se-á à incidência de imposto sobre a renda à alíquota de 15%, em nome do espólio.     imposto devido sobre ganho de capital deve ser pago pelo inventariante até a data prevista para a entrega  da declaração final de espólio.    obs.: preencher, utilizando-se do programa gerador específico, o demonstrativo da apuração dos ganhos de  capital, exportando o seu resultado para a declaração final de espólio. ',
 'qual é o tratamento tributário da doação efetuada em bens ou direitos a terceiros?     doação de bens ou direitos caracteriza alienação e sujeita-se à apuração do ganho de capital, se efetuada  por valor superior ao constante na última declaração de bens e direitos do doador.    doação efetuada em dinheiro (moeda nacional) não é tributada pelo imposto sobre a renda. para fins de  incidência do imposto sobre transmissão causa mortis e doação, consultar a legislação estadual ou do  distrito federal.    doação em espécie está sujeita à comprovação da sua efetivação, bem como da disponibilidade econômico- financeira do doador para exercício de tal liberalidade.   atenção:    na hipótese de doação de livros, objetos fonográficos ou iconográficos, obras audiovisuais e obras  de arte, para os quais seja atribuído valor de mercado, efetuada por pessoa física a órgãos públicos,  autarquias, fundações públicas ou entidades civis sem fins lucrativos, desde que os bens doados      229   sejam incorporados ao acervo de museus, bibliotecas ou centros de pesquisa ou ensino, no brasil,  com acesso franqueado ao público em geral:    - o doador deve considerar como valor de alienação o constante em sua declaração de bens;    - o donatário registra os bens recebidos pelo valor atribuído no documento de doação.   na hipótese aqui prevista, ocorrendo a alienação futura dos bens recebidos em doação, o custo de  aquisição será considerado igual a zero para efeito de apuração de ganho de capital. ',
 'qual é o valor de custo de aquisição que deve ser declarado por quem recebeu um imóvel em  doação?   deve ser informado o valor da doação constante do instrumento de doação (escritura etc). ',
 'qual é o prazo para o pagamento do imposto na transferência de bens ou direitos na doação  em adiantamento da legítima?    imposto deve ser pago até o último dia útil do mês subsequente ao da doação pelo doador e em seu nome  deve ser feito o darf do pagamento do imposto, com a utilização do código 4600. ',
 'como se tributa a alienação de bem imóvel com recebimento de seu valor em bens móveis a  serem entregues parceladamente?   essa operação equipara-se à venda a prazo, devendo o ganho de capital ser apurado no mês da transação e  tributado na medida em que as parcelas (valor dos bens) forem recebidas.   caso o preço efetivo da operação tenha sido contratado pelas partes, considera-se como valor recebido dos  bens móveis no mês de seu efetivo recebimento aquele que foi contratado originalmente.   caso a operação não tenha sido expressa em dinheiro, considera-se como valor recebido dos bens móveis o  seu valor de mercado no mês do efetivo recebimento, sendo que se este for superior ao valor de mercado do  mês da operação, este acréscimo sujeita-se ao recolhimento mensal obrigatório (carnê-leão), se recebido de  pessoa física, ou à retenção na fonte, se pago por pessoa jurídica, e, também, ao ajuste na declaração de  ajuste anual.   atenção:     posterior alienação dos bens móveis recebidos configura uma nova operação de alienação,  devendo o ganho de capital, porventura apurado, ser tributado, atribuindo-se aos bens, como custo  de aquisição, o valor de mercado na data em que foram recebidos.   ',
 'como tributar a venda a prazo com cláusula pro soluto ou pro solvendo?   "pro soluto se diz dos títulos de crédito quando dados com efeito de pagamento, como se dinheiro fossem,  operando a novação do negócio que lhes deu origem. pro solvendo, quando são recebidos em caráter  condicional, sendo puramente representativos ou enunciativos da dívida, não operando novação alguma, só  valendo como pagamento quando efetivamente resgatados" (lei soibelman, dicionário geral de direito,  1974).   se houver venda de bens ou direitos a prazo, com emissão de notas promissórias desvinculadas do contrato  pela cláusula pro soluto, essa operação deve ser considerada como à vista, para todos os efeitos fiscais,  computando-se o valor total da venda no mês da alienação.   se na venda dos bens ou direitos não houver emissão de notas promissórias ou estas forem emitidas  vinculadas ao contrato pela cláusula pro solvendo, essa operação é considerada como venda em prestações,  para todos os efeitos fiscais, computando-se em cada mês o valor efetivamente recebido.',
 'como proceder quanto à tributação de bens ou direitos alienados a prazo sendo a dívida  quitada com desconto?    desconto obtido não constitui rendimento para o beneficiário e, de forma alguma, enseja ao alienante motivo  para restituição de imposto caso tenha havido incidência do imposto sobre a renda sobre o ganho obtido na  alienação.  ',
 'como deve ser tributado o ganho de capital nas alienações de bens ou direitos quando ocorre  emissão de notas promissórias correspondentes às prestações contratadas?    nota promissória é um título de crédito que se basta a si mesmo, ou seja, tem característica de  independência, não se ligando ao ato originário de onde proveio.   assim, só se caracterizam como venda a prazo, e ao abrigo do diferimento previsto no art. 21 da lei nº 7.713,  de 22 de dezembro de 1988, as operações em que as notas promissórias estejam vinculadas ao contrato pela  cláusula pro solvendo.   se as notas promissórias foram emitidas desvinculadas do contrato, pela cláusula pro soluto, esse contrato  está perfeito e acabado, caracterizando a disponibilidade jurídica. em consequência, ainda que a liquidação  seja efetuada em notas promissórias, a apuração do ganho de capital total deve efetuar-se no mês da  alienação, independentemente de serem os títulos quitados ou não. ',
 'como proceder em relação ao ganho de capital apurado quando o contribuinte que alienou  os bens e direitos a prazo, com emissão de notas promissórias doa tais títulos aos filhos?   partindo-se da premissa de que o ato de doação, na hipótese, é legalmente válido, o alienante (genitor) deve  continuar a oferecer à tributação os valores dos ganhos de capital correspondentes às parcelas a serem  recebidas, consoante determina o art. 21 da lei nº 7.713, de 22 de dezembro de 1988, pois as convenções  particulares, relativas à responsabilidade pelo pagamento de tributos, não podem ser opostas à fazenda  pública, para modificar a definição legal do sujeito passivo das obrigações tributárias correspondentes.   no caso de recebimento de acréscimos estipulados no contrato, esses devem ser tributados em separado do  ganho de capital, na fonte ou mediante recolhimento mensal obrigatório (carnê-leão), conforme o caso, e na  declaração de ajuste anual correspondente ao ano-calendário de seu recebimento.   os filhos incluem em suas declarações os valores recebidos, como rendimentos isentos.   para fins de incidência do imposto sobre transmissão causa mortis e doação, consultar a legislação estadual  ou do distrito federal. ',
 'como proceder quando a alienação do imóvel fica condicionada à aprovação de  financiamento por uma instituição financeira, com recursos fornecidos pelo sistema financeiro de  habitação?   esta hipótese configura modalidade de ato jurídico sob condição suspensiva, ou seja, o fato gerador da  obrigação tributária (alienação) somente ocorre com o implemento da condição, isto é, com a aprovação do  financiamento pelo sistema financeiro da habitação   e a celebração do contrato definitivo de compra e  venda, concretizando a transmissão dos direitos sobre o imóvel ( nº 80, de 1979, item 7).   como exemplo, suponha-se que a operação sob essa condição foi acordada em 20/10/2018, sendo o  financiamento liberado e o contrato celebrado em 28/03/2019. assim, somente na data do implemento da  condição (28/03/2019), considera-se consumada a transmissão do imóvel, com todos os efeitos fiscais dela  decorrentes.   na hipótese de ter havido qualquer pagamento inicial em 20/10/2018 (como o sinal para garantia do negócio),  o alienante e o adquirente devem registrar esse fato em suas declarações de bens, historiando a operação  efetuada e o valor pago ou recebido, a fim de evitar a ocorrência de variação patrimonial não justificada no  período correspondente. contudo, para fins de apuração de ganho de capital pelo alienante, considera-se  como data de alienação 28/03/2019 e como valor de alienação, o preço efetivo de venda acordado também  nessa data, incluído aí o sinal recebido.   como custo de aquisição, o adquirente deve considerar o total dos valores pagos até março de 2019, incluindo  o sinal registrado na declaração de bens (ou valor correspondente), as prestações pagas a cada ano- calendário, os juros e demais acréscimos pagos no respectivo financiamento (não se aplica à hipótese de  multa por atraso de pagamento), bem como as despesas permitidas pela legislação do imposto como  integrantes daquele custo (tais como: construção, ampliação, reforma etc).    outro exemplo bastante comum de modalidade de ato jurídico sob condição suspensiva é caso de o  proprietário contratar a alienação da casa onde reside, mas que a transmissão só seria efetivada depois que  ele se mudasse para outro imóvel. assim, somente após a desocupação do imóvel e celebração do contrato  de compra e venda ocorreria o implemento da condição, gerando os efeitos fiscais referidos.    para o novo comprador do imóvel, considera-se data de aquisição 28/03/2019 e o custo de aquisição os  valores efetivamente pagos para efeito de registro da aquisição na declaração de ajuste anual.   ',
 'como proceder quando o mutuário do sistema financeiro da habitação   aliena o imóvel  financiado, por meio de documento particular, sem notificar o agente financeiro e sem haver  transferência do saldo devedor do financiamento?   preliminarmente, esclareça-se que a aquisição de imóvel com financiamento do sistema financeiro da  habitação  , embora formalizado em um único documento público, consubstancia-se em dois atos  jurídicos distintos, sendo um o contrato de compra e venda do bem que se estabelece entre o adquirente e o  proprietário do imóvel e, o outro, um pacto adjeto de hipoteca sobre o imóvel adquirido, para garantia do  empréstimo concedido para sua aquisição, vinculando o mutuário e o agente financiador. este, no caso, pode  executar a hipoteca ou exigir a regularização do financiamento pelo novo adquirente, mas não pode impedir  que o mutuário ceda seus direitos a terceiros.   assim, o contrato particular firmado entre o mutuário e o novo adquirente é instrumento legalmente válido para  configurar a cessão de direitos sobre o imóvel. por outro lado, a legislação tributária não condiciona a eficácia  do ato de alienação à notificação do agente financiador e à transferência do saldo devedor do financiamento.  é, portanto, irrelevante que esses requisitos não tenham sido cumpridos pelos contratantes, no ato de cessão  dos direitos à aquisição do imóvel. com a celebração do contrato particular de cessão de direitos e a      232   percepção da quantia pactuada (ágio), considera-se consumada a alienação, para todos os efeitos fiscais,  estando concretizada a transmissão dos direitos sobre o imóvel decorrente do contrato de compra e venda.   nesse caso, o alienante deve apurar o ganho de capital no mês da assinatura do contrato particular. deve  considerar como custo de aquisição os valores efetivamente pagos pela aquisição do imóvel constantes na  sua declaração de bens e direitos do exercício de 2019, ano-calendário de 2018, acrescido das parcelas  pagas até o mês da alienação.     custo dos bens ou direitos adquiridos ou das parcelas pagas até 31/12/1995 deve ser atualizado conforme  pergunta 558, caso o contribuinte estivesse desobrigado de apresentar a declaração de ajuste anual.     considera-se valor de alienação a importância recebida do adquirente (ágio), que será o valor de aquisição  para este.   esclareça-se que, na hipótese de ocorrer o desfazimento do negócio, não haverá restituição do valor do  imposto pago.  ',
 'qual é o tratamento tributário da doação de bens e direitos privativos entre cônjuges?    doação de bens e direitos privativos de um cônjuge a outro sujeita-se à apuração de ganho de capital na  forma de doação, se efetuada por valor superior ao constante na última declaração de bens e direitos do  doador. ',
 'qual é o custo de aquisição dos bens recebidos por meio de concursos, sorteios e outros?   para fins de apuração do ganho de capital, os bens adquiridos têm como custo o valor de mercado do prêmio,  utilizado como base de cálculo do imposto sobre a renda incidente na fonte, atualizado até 31/12/1995, e  acrescido do correspondente imposto sobre a renda incidente na fonte.   ',
 'transferência da titularidade de quotas de capital entre cônjuges, casados no regime de comunhão universal de bens, constitui alienação para fins de tributação?   essa transferência não constitui alienação, posto o patrimônio comum continuar indiviso. consequentemente,  não se tributa como ganho de capital a referida operação.',
 'como devem ser consideradas as alienações efetuadas pelos cônjuges, para fins de  tributação dos ganhos de capital?   as transações efetuadas na constância da sociedade conjugal em regime de comunhão universal ou parcial  de bens comuns têm o seguinte tratamento, para efeito de tributação: cada cônjuge deve considerar 50% do  ganho de capital. opcionalmente o total do ganho de capital pode ser tributado por um dos cônjuges, exceto  quando se tratar de bens incomunicáveis, caso em que cada um deve tributar o valor que lhe cabe.   atenção:    nas alienações de bens comuns, decorrentes do regime de casamento, o ganho de capital é  apurado em relação ao bem como um todo. apenas a tributação do ganho apurado é que deve ser  feita na razão de 50% para cada cônjuge ou, opcionalmente, 100% em um dos cônjuges. ',
 'como proceder quando os herdeiros desistem, no curso do processo de inventário, em favor  dos outros co-herdeiros, aos bens e direitos que lhes cabiam por direito sucessório?   preliminarmente, é necessário determinar se houve renúncia ou cessão de direitos hereditários.    renúncia é genuinamente abdicativa. nela existe o desejo de recusa ou não aceitação da herança (lei nº  10.406, de 10 de janeiro de 2002 - código civil de 2002, arts. 1.805, § 2º), sendo seu alcance muito mais  amplo, pois se entende que ela retroage ao momento da abertura da sucessão, de modo que o herdeiro  renunciante é considerado como se jamais houvesse sido herdeiro. assim, a renúncia, em rigor e por força de  seu efeito retroativo, não equivale a uma transmissão (ainda que gratuita) de bens. dessa forma, a renúncia  gratuita, pura e simples, feita em benefício dos demais co-herdeiros (ascendentes, descendentes ou  colaterais) não configura a alienação prevista na lei nº 7.713, de 22 de dezembro de 1988.   todavia, se o herdeiro aceitar a herança e ceder seus direitos hereditários, ainda que a título gratuito,  considera-se que houve a alienação prevista na lei, visto a cessão ser caracteristicamente translativa (pois  só se cede o que se possui), equivalente da compra e venda, aplicando-se a ela as mesmas regras desse  contrato.   na declaração de bens e direitos, no caso de renúncia, não há necessidade de registrar esse fato (embora o  renunciante possa fazê-lo se o desejar), cabendo aos herdeiros beneficiados incluir os bens e direitos em suas  declarações, com as informações próprias dessa situação.   havendo cessão de direitos hereditários, o cedente deve registrar esse fato em sua declaração de bens e  direitos e praticar todos os demais atos próprios decorrentes da alienação de bens e direitos, apurando o  ganho de capital de acordo com as disposições legais e normativas previstas para essa operação.',
 'como devem ser tributados os ganhos de capital decorrentes de alienação de bens ou direitos  efetuada ainda no curso do inventário?   como a legislação tributária determina que ao espólio sejam aplicadas as normas a que estão sujeitas as  pessoas físicas (art. 9º do regulamento do imposto sobre a renda - /2018, aprovado pelo decreto nº  9.580, de 22 de novembro de 2018), se no curso do inventário ocorrer a alienação de bens ou direitos no  regime da lei nº 7.713, de 22 de dezembro de 1988, ou nas modalidades prescritas para a equiparação por  loteamento ou incorporação de imóveis (art. 163 e seguintes do /2018), os ganhos porventura auferidos  são tributados em nome do espólio, tomando-se por base o preço de custo e a data de aquisição dos bens,  pelo falecido ou cônjuge sobrevivente, ou por ambos, conforme seja o caso.   ',
 'qual é o tratamento tributário aplicável a pagamento de prestação de serviços mediante dação  em pagamento efetuada com imóvel?    dação em pagamento realizada com imóvel configura uma alienação, devendo o ganho de capital ser  apurado quando da transferência do imóvel.  valor de mercado do bem recebido em pagamento caracteriza,  para o adquirente, rendimento sujeito ao carnê-leão ou ao imposto sobre a renda incidente na fonte, conforme  o imóvel seja recebido de pessoa física ou jurídica, e ao ajuste anual.  ',
 'qual é o custo de aquisição de bens adquiridos por arrendamento mercantil (leasing)?    custo de aquisição de bens adquiridos por arrendamento mercantil (leasing) corresponde aos valores pagos  a esse título obtidos da seguinte forma:   a) com a opção de compra realizada ao final do contrato, pela soma dos valores pagos a título de  arrendamento mercantil acrescido do valor residual pago;   b) com a opção de compra realizada no ato do contrato, pela soma de todos os valores pagos (aluguel mais  valor residual pago a cada parcela).   ',
 'que se considera permuta para efeitos tributários?   para efeitos tributários, considera-se permuta toda e qualquer operação que tenha por objeto a troca de uma  ou mais unidades imobiliárias, prontas ou a construir, por outra ou outras unidades imobiliárias, ainda que  ocorra, por parte de um dos proprietários-contratantes, o pagamento de parcela complementar, em dinheiro,  comumente denominada torna.    expressão "unidade imobiliária ou unidades imobiliárias prontas ou a construir", compreende:   a) o terreno adquirido para venda, com ou sem construção;   b) cada lote oriundo de desmembramento de terreno;   c) cada terreno decorrente de loteamento;   d) cada unidade distinta resultante de incorporação imobiliária;   e) o prédio construído para venda como unidade isolada ou autônoma;   f) cada casa ou apartamento construído ou a construir.   é necessário que a escritura, quando lavrada, seja de permuta.   não se considera permuta a operação que envolva qualquer outro bem ou direito, que não seja bem imóvel,  apurando-se o ganho de capital como dação em pagamento.   exemplos   1) permuta sem torna:   em 04/05/2019, o contribuinte "" permutou uma casa, adquirida em 15/10/1990, declarada por $ 60.000,00,  por um apartamento pertencente ao contribuinte "", adquirido em 10/06/1992, e por ele declarado por  $ 50.000,00.   dados para a declaração (declaração de bens e direitos):      235   contribuinte “”   bem situação em 31/12/2018 situação em 31/12/2019   casa 60.000,00 0,00   apto. 0,00 60.000,00   contribuinte “”   bem  situação em 31/12/2018 situação em 31/12/2019   casa 0,00 50.000,00   apto. 50.000,00 0,00   2) permuta com torna:   mesma operação, com recebimento de torna de $ 10.000,00 por "".   ganho de capital de "":   item cálculos – valores em $   proporção 10.000,00 x 100 / 70.000,00 = 14,2857%   ganho de capital (antes da redução) 14,2857% x 10.000,00 = 1.428,57   ganho de capital (após a redução da lei nº  11.196/2006)    1.428,57/(1,006119x 1,0035150) = 415,0833   custo da torna 10.000,00 - 1.428,57 = 8.571,43   imposto sobre a renda 415,0833 x 15% = 62,26   rendimento tributação exclusiva  415,0833 - 62,26 = 352,82   rendimento isento e não tributável  1.428,57 - 415,0833 = 1.013,48   custo do apto para "" 60.000,00 - 8.571,43 = 51.428,57   dados para a declaração (declaração de bens e direitos):   contribuinte “”   bem  situação em 31/12/2018 situação em 31/12/2019   casa 60.000,00 0,00   apto. 0,00 51.428,57   contribuinte “”   bem  situação em 31/12/2018 situação em 31/12/2019   casa 0,00 60.000,00   apto. 50.000,00 0,00   ',
 'qual é o tratamento tributário aplicável na permuta entre bens móveis e imóveis?   no caso de permuta tendo por objeto bens móveis, não se aplica o tratamento tributário previsto no art. 132  do regulamento do imposto sobre a renda - /2018, aprovado pelo decreto nº 9.580, de 22 de novembro  de 2018, devendo ser apurado o ganho de capital relativo a cada uma das alienações. ',
 'nas operações de permuta de imóvel rural por outro imóvel rural, com ou sem benfeitorias,  aplica-se o tratamento tributário de permuta, disposto no art. 132, inciso , e § 2º do regulamento  do imposto sobre a renda - /2018, aprovado pelo decreto nº 9.580, de 22 de novembro de 2018?   sim, desde que, no caso de imóvel com benfeitorias, o contribuinte não as tenha utilizado como despesa da  atividade rural. ',
 'como proceder no caso de permuta de imóveis com pessoa jurídica à qual a pessoa física  esteja ligada?   as operações de permuta de imóveis realizadas entre a pessoa jurídica e seu sócio, administrador ou titular,  ou com o cônjuge ou parente até o terceiro grau, inclusive afim dessas pessoas físicas, são sempre realizadas  tomando-se por base o valor de mercado das unidades permutadas, apurado com base em laudo de avaliação  dos imóveis permutados. este deve ser elaborado por três peritos ou por entidades ou empresas  especializadas desvinculados dos interesses dos contratantes, com indicação dos critérios de avaliação e dos  elementos de comparação adotados e instruído com os documentos relativos aos bens avaliados.   nesse caso, não há tratamento tributário de permuta e a pessoa física determina o ganho de capital  considerando como preço de alienação o valor de mercado dos bens dados em permuta e registrará os bens  adquiridos pelo valor de mercado a eles atribuídos.    inexistência de laudo implicará arbitramento do valor dos bens pela autoridade fiscal. ',
 'como proceder quando a pessoa física efetua com pessoa jurídica operação de compra e  venda com dação de unidade imobiliária em pagamento?   são aplicáveis às operações quitadas de compra e venda de terreno, seguidas de confissão de dívida e  promessa de dação em pagamento de unidade imobiliária construída ou a construir, todos os procedimentos  e normas da permuta, desde que sejam observadas as condições cumulativas a seguir referidas:   1 - a alienação do terreno e o compromisso de dação em pagamento sejam levados a efeito na mesma data,  mediante instrumento público; e   2 - o terreno objeto da operação de compra e venda seja, até o final do ano-calendário seguinte àquele em  que esta ocorrer, dado em hipoteca para obtenção de financiamento, ou, no caso de loteamento, oferecido  em garantia ao poder público, nos termos da lei nº 6.766, de 19 de dezembro de 1979.    não observância das condições cumulativas estipuladas sujeita a pessoa jurídica, promitente da dação, à  apuração dos resultados da operação e recolhimento do imposto sobre o ganho de capital da pessoa física,  tomando por base, para determinação do preço de alienação dos bens permutados, o valor de mercado ou,  na ausência de laudo de avaliação, o valor que vier a ser arbitrado pela autoridade fiscal. nesta hipótese, a  apuração do resultado da operação reporta-se ao mês em que esta tiver ocorrido, sujeitando-se o promitente  da dação (pessoa jurídica) ao recolhimento do imposto sobre a renda sobre o ganho de capital da pessoa  física como tributo postergado. ',
 'qual é o tratamento tributário dos ganhos de capital auferidos na alienação de bens ou  direitos adquiridos e na liquidação ou resgate de aplicações financeiras realizadas em moeda  estrangeira por pessoa física na condição de residente no brasil?   para fatos geradores ocorridos a partir de 01/01/2000, as operações que importem na alienação, a qualquer  título, de bens ou direitos adquiridos em moeda estrangeira, ações e outros ativos financeiros em bolsa de  valores, de mercadorias, de futuros ou assemelhadas, ou em qualquer mercado do exterior e na liquidação ou  resgate de aplicações financeiras realizadas em moeda estrangeira, por pessoa física na condição de  residente no brasil estão sujeitas à apuração de ganho de capital tributável, segundo alíquotas progressivas  estabelecidas em função do lucro, de acordo com as três situações abaixo:      238   1. bens ou direitos adquiridos e aplicações financeiras realizadas com rendimentos auferidos  originariamente em reais   1.1. operações à vista ou a prazo   na hipótese de bens ou direitos adquiridos e aplicações financeiras realizadas em moeda estrangeira com  rendimentos auferidos originariamente em reais, o ganho de capital corresponde à diferença positiva, em reais,  entre o valor de alienação, liquidação ou resgate e o custo de aquisição do bem ou direito ou o valor original  da aplicação financeira.    valor de alienação, liquidação ou resgate, quando expresso em moeda estrangeira, deve ser convertido em  dólares dos estados unidos da américa e, em seguida, em moeda nacional pela cotação do dólar fixada, para  compra, pelo banco central do brasil, para a data do recebimento.    custo de aquisição de bens ou direitos ou o valor original de aplicações financeiras, quando expresso em  moeda estrangeira, corresponde ao valor de aquisição ou aplicação convertido em dólares e, em seguida, em  reais pela cotação do dólar fixada, para venda, pelo banco central do brasil, para a data do pagamento.    conversão de moeda estrangeira para dólares dos estados unidos da américa é feita pelo valor fixado pela  autoridade monetária do país emissor da moeda, para a data do pagamento, na aquisição, e para a data do  recebimento, na alienação, liquidação ou resgate.   exemplo (cotações fictícias):   alienação à vista em 14/06/2019, por $ 50,000.00, de um bem móvel adquirido em 23/03/1999 por  $ 40,000.00, também à vista.   item cálculo   valor de alienação $ 50,000.00 x $ 2,8500 (*) = $ 142.500,00   custo de aquisição $ 40,000.00 x $ 1,8516 (**) = $ 74.064,00   ganho de capital $ 142.500,00 - $ 74.064,00 = $ 68.436,00   imposto devido (vencimento em 31/07/2019) $ 68.436,00 x 15% = $ 10.265,40   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 14/06/2019 (data do  recebimento);   (**) cotação do dólar fixada para venda, pelo banco central do brasil, para o dia 23/03/1999 (data do  pagamento, na aquisição).   1.2. operações a prestação.   nas operações a prestação, o ganho de capital deve ser apurado, em reais, para cada parcela, na data de  seu recebimento.    valor de alienação, relativo a cada parcela recebida, quando expresso em moeda estrangeira, é convertido  em dólares dos estados unidos da américa e, em seguida, em moeda nacional, pela cotação do dólar fixada,  para compra, pelo banco central do brasil, para a data de cada recebimento.    custo de aquisição, para cada parcela é o resultado da multiplicação do custo de aquisição total, em reais,  pelo quociente do valor de cada parcela recebida pelo valor total de alienação.   no recebimento da última parcela, o ganho de capital total é apurado pela diferença, em reais, entre a soma  de todas as parcelas recebidas e o custo total de aquisição.   do imposto apurado sobre o ganho de capital calculado na forma do parágrafo anterior será diminuído o  imposto pago relativo às parcelas anteriores, obtendo-se o saldo de imposto a pagar referente à última parcela,  ou, caso o saldo seja negativo, o imposto pago a maior poderá ser compensado ou restituído nos termos da  legislação vigente.    conversão de moeda estrangeira para dólares dos estados unidos da américa será feita pelo valor fixado  pela autoridade monetária do país emissor da moeda, para a data do pagamento, na aquisição, e para a data  do recebimento, na alienação, liquidação ou resgate.   exemplo (cotações fictícias):   supondo que, no exemplo anterior, o valor de alienação ($ 50,000.00) tenha sido recebido em três parcelas,  sendo a primeira de $ 20,000.00 em 14/06/2019 e as duas últimas de $ 15,000.00, em 31/07/2019 e em  15/08/2019.      239   portanto:   custo total de aquisição = $ 40,000.00 x $ 1,8516 (*) = $ 74.064,00;   (*) cotação do dólar fixada para venda, pelo banco central do brasil, para o dia 23/03/1999 (data do  pagamento, na aquisição).   fazendo a apuração do ganho de capital para cada parcela:   a) recebimento da 1ª parcela:   item cálculo   valor da parcela recebida em reais $ 20,000.00 x $ 2,8500 (*) = $ 57.000,00   custo de aquisição proporcional   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 74.064,00 x ($ 20,000.00 / $ 50,000.00)  =$ 29.625,60   ganho de capital $ 57.000,00 - $ 29.625,60 = $ 27.374,40   imposto devido (vencimento em 31/07/2019) $ 27.374,40 x 15% = $ 4.106,16   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 14/06/2019 (data do  recebimento da 1ª parcela).   b) recebimento da 2ª parcela:   item cálculo   valor da parcela recebida em reais $ 15,000.00 x $ 2,86690 (*) = $ 43.003,50   custo de aquisição proporcional   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 74.064,00 x ($ 15,000.00 / $ 50,000.00) =  $ 22.219,20   ganho de capital $ 43.003,50 - $ 22.219,20 = $ 20.784,30   imposto devido (vencimento em 30/08/2019) $ 20.784,30 x 15% = $ 3.117,65   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 31/07/2019 (data do  recebimento da 2ª parcela).   c) recebimento da 3ª e última parcela   item cálculo   valor da parcela recebida em reais $ 15,000.00 x $ 2,99220 (*) = $ 44.883,00   valor total de alienação $ 57.000,00 + $ 43.003,50 + $ 44.883,00 =  $ 144.886,50   custo total de aquisição $ 74.064,00   ganho de capital total $ 144.886,50 - $ 74.064,00 = $ 70.822,50   imposto total $ 70.822,50 x 15% = $ 10.623,38   saldo de imposto (vencimento em 30/09/2019) $ 10.623,38 - $ 4.106,16 - $ 3.117,65 =  $ 3.399,57   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 15/08/2019 (data do  recebimento da 3ª parcela).   como o resultado foi positivo, este é o valor do imposto com vencimento em 30/09/2018. se o resultado fosse  negativo, este seria o imposto pago a maior, que poderia ser compensado ou restituído nos termos da  legislação vigente.   2. bens e direitos adquiridos e aplicações financeiras realizadas com rendimentos auferidos  originariamente em moeda estrangeira.   2.1. operações à vista ou a prazo.   na hipótese de bens ou direitos adquiridos e aplicações financeiras realizadas em moeda estrangeira com  rendimentos auferidos originariamente em moeda estrangeira, o ganho de capital corresponde à diferença  positiva, em dólares dos estados unidos da américa, entre o valor de alienação, liquidação ou resgate e o  custo de aquisição do bem ou direito ou o valor original da aplicação, convertida em moeda nacional mediante  a utilização da cotação do dólar fixada, para compra, pelo banco central do brasil, para a data do recebimento.      240    conversão de moeda estrangeira para dólares dos estados unidos da américa é feita pelo valor fixado pela  autoridade monetária do país emissor da moeda, para a data do pagamento, na aquisição, e para a data do  recebimento, na alienação, liquidação ou resgate.   exemplo (cotações fictícias):   alienação à vista em 14/06/2019, por $ 50,000.00, de um bem móvel adquirido em 23/03/1999 com  rendimentos auferidos originariamente em moeda estrangeira, por $ 40,000.00.   item cálculo   ganho de capital em $ $ 50,000.00 - $ 40,000.00 = $ 10,000.00   ganho de capital em reais $ 10,000.00 x $ 2,85000 (*) = $ 28.500,00   imposto devido (vencimento em 31/07/2019) $ 28.500,00 x 15% = $ 4.275,00   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 14/06/2019 (data do  recebimento).   2.2. operações a prestação.   nas operações a prestação, o ganho de capital deve ser apurado, para cada parcela, em dólares dos estados  unidos da américa, e, em seguida, convertido em moeda nacional, pela cotação do dólar fixada, para compra,  pelo banco central do brasil, na data de cada recebimento.    custo de aquisição, para cada parcela, é o resultado da multiplicação do custo de aquisição total, em dólares  dos estados unidos da américa, pelo quociente do valor de cada parcela recebida pelo valor total de  alienação.    conversão de moeda estrangeira para dólares dos estados unidos da américa é feita pelo valor fixado pela  autoridade monetária do país emissor da moeda, para a data do pagamento, na aquisição, e para a data do  recebimento, na alienação, liquidação ou resgate.   exemplo (cotações fictícias):   supondo que, no exemplo anterior, o valor de alienação ($ 50,000.00) tenha sido recebido em três parcelas,  sendo a primeira de $ 20,000.00 em 14/06/2019 e as duas últimas de $ 15,000.00, em 31/07/2019 e em  15/08/2019.   a) recebimento da 1ª parcela:   item cálculo   custo de aquisição proporcional   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 40,000.00 x ($ 20,000.00 / $ 50,000.00) =  $ 16,000.00   ganho de capital em $ $ 20,000.00 - $ 16,000.00 = $ 4,000.00   ganho de capital em reais $ 4,000.00 x $ 2,85000 (*) = $ 11.400,00   imposto devido (vencimento em 31/07/2019) $ 11.400,00 x 15% = $ 1.710,00   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 14/06/2019 (data do  recebimento da 1ª parcela).   b) recebimento da 2ª parcela:   item cálculo   custo de aquisição proporcional   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 40,000.00 x ($ 15,000.00 / $ 50,000.00) =  $ 12,000.00   ganho de capital em $ $ 15,000.00 - $ 12,000.00 = $ 3,000.00   ganho de capital em reais $ 3,000.00 x $ 2,86690 (*) = $ 8.600,70   imposto devido (vencimento em 30/08/2019) $ 8.600,70 x 15% = $ 1.290,11   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 31/07/2019 (data do  recebimento da 2ª parcela).      241   c) recebimento da 3ª e última parcela   item cálculo   custo de aquisição proporcional   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 40,000.00 x ($ 15,000.00 / $ 50,000.00) =  $ 12,000.00   ganho de capital em $ $ 15,000.00 - $ 12,000.00 = $ 3,000.00   ganho de capital em reais $ 3,000.00 x $ 2,99220 (*) = $ 8.976,60   imposto devido (vencimento em 30/09/2019) $ 8.976,60 x 15% = $ 1.346,49   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 15/08/2019 (data do  recebimento da 3ª parcela).   3. bens e direitos adquiridos e aplicações financeiras realizadas com rendimentos auferidos  originariamente parte em reais, parte em moeda estrangeira.   na hipótese de bens ou direitos adquiridos e aplicações financeiras realizadas em moeda estrangeira com  rendimentos auferidos originariamente parte em reais, parte em moeda estrangeira, os valores de alienação,  liquidação ou resgate e os custos de aquisição ou valores originais são determinados, para fins de apuração  do ganho de capital, de forma proporcional à origem do rendimento utilizado na aquisição ou realização.   exemplo (cotações fictícias):    contribuinte adquiriu um automóvel em 23/03/1999 por $ 50,000.00, sendo 80% com rendimentos  auferidos originariamente em reais e 20% com recursos auferidos originariamente em moeda estrangeira.  este automóvel foi alienado em 12/06/2019 por $ 62,500.00, em três parcelas, sendo a primeira de  $ 25,000.00 em 12/06/2019 e as seguintes de $ 18,750.00 em 18/07/2019 e 15/08/2019,  respectivamente.   faz-se o cálculo proporcional, conforme demonstrado abaixo:   a) ganho de capital da parte adquirida com rendimentos auferidos originariamente em reais:   valor de alienação = $ 62,500.00 x 80% = $ 50,000.00;   custo de aquisição = $ 50,000.00 x 80% = $ 40,000.00;   custo de aquisição = $ 40,000.00 x $ 1,8516 (*) = $ 74.064,00;   (*) cotação do dólar fixada para venda, pelo banco central do brasil, para o dia 23/03/1999 (data do  pagamento, na aquisição).   calcula-se o imposto devido correspondente a cada parcela:   a.1) recebimento da 1ª parcela:   item cálculo   valor da parcela recebida em dólares dos     $ 25,000.00 x 80% = $ 20,000.00   valor da parcela recebida em reais $ 20,000.00 x $ 2,85000 (*) = $ 57.000,00   custo de aquisição proporcional (**)   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 74.064,00 x ($ 20,000.00 / $ 50,000.00) =  $ 29.625,60   ganho de capital $ 57.000,00 - $ 29.625,60 = $ 27.374,40   imposto devido (vencimento em 31/07/2019) $ 27.374,40 x 15% = $ 4.106,16   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 12/06/2019 (data do  recebimento da 1ª parcela).   (**) calculado proporcionalmente à parcela cujos rendimentos foram obtidos originariamente em reais.      242   a.2) recebimento da 2ª parcela:   item cálculo   valor da parcela recebida em dólares dos     $ 18,750.00 x 80% = $ 15,000.00   valor da parcela recebida em reais $ 15,000.00 x $ 2,86690 (*) = $ 43.003,50   custo de aquisição proporcional (**)   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 74.064,00 x ($ 15,000.00 / $ 50,000.00) =  $ 22.219,20   ganho de capital $ 43.003,50 - $ 22.219,20 = $ 20.784,30   imposto devido (vencimento em 30/08/2019) $ 20.784,30 x 15% = $ 3.117,66   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 18/07/2019 (data do  recebimento da 2ª parcela).   (**) calculado proporcionalmente à parcela cujos rendimentos foram obtidos originariamente em reais.   a.3) recebimento da 3ª e última parcela:   item cálculo   valor da parcela recebida em dólares dos  $ 18,750.00 x 80% = $ 15,000.00   valor da parcela recebida em reais $ 15,000.00 x $ 2,99220 (*) = $ 44.883,00   valor total de alienação $ 57.000,00 + $ 43.003,50 + $ $ 44.883,00 =  $ 144.886,50   custo total de aquisição $ 74.064,00   ganho de capital total $ 144.886,50 - $ 74.064,00 = $ 70.822,50   imposto total $ 70.822,50 x 15% = $ 10.623,38   saldo de imposto (vencimento em 30/09/2019) $ 10.623,38 - $ 4.106,16 - $ 3.117,65 =  $ 3.399,57   (*) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 15/08/2019 (data do  recebimento da 3ª parcela).   b) ganho de capital da parte adquirida com rendimentos auferidos originariamente em moeda estrangeira:   valor de alienação proporcional = $ 62,500.00 x 20% = $ 12,500.00;   custo de aquisição proporcional = $ 50,000.00 x 20% = $ 10,000.00;   b.1) recebimento da 1ª parcela:   item cálculo   valor da parcela recebida em dólares dos  $ 25,000.00 x 20% = $ 5,000.00   custo de aquisição proporcional (*)   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 10,000.00 x ($ 5,000.00 / $ 12,500.00)  = $ 4,000.00   ganho de capital $ 5,000.00 - $ 4,000.00 = $ 1,000.00   ganho de capital em reais $ 1,000.00 x $ 2,85000 (**) = $ 2.850,00   imposto devido (vencimento em 31/07/2018) $ 2.850,00 x 15% = $ 427,50   (*) calculado proporcionalmente à parcela cujos rendimentos foram obtidos originariamente em moeda  estrangeira.   (**) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 12/06/2019 (data  do recebimento da 1ª parcela).      243   b.2) recebimento da 2ª parcela:   item cálculo   valor da parcela recebida em dólares dos  $ 18,750.00 x 20% = $ 3,750.00   custo de aquisição proporcional (*)   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 10,000.00 x ($ 3,750.00 / $ 12,500.00)  = $ 3,000.00      ganho de capital $ 3,750.00 - $ 3,000.00 = $ 750.00   ganho de capital em reais $ 750.00 x $ 2,86690 (**) = $ 2.150,18   imposto devido (vencimento em 30/08/2019) $ 2.150,18 x 15% = $ 322,53   (*) calculado proporcionalmente à parcela cujos rendimentos foram obtidos originariamente em moeda  estrangeira.   (**) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 18/07/2019 (data  do recebimento da 2ª parcela).   b.3) recebimento da 3ª e última parcela   item cálculo   valor da parcela recebida em dólares dos  $ 18,750.00 x 20% = $ 3,750.00   custo de aquisição proporcional (*)   [custo total de aquisição x (valor da parcela  recebida / valor total de alienação)]   $ 10,000.00 x ($ 3,750.00 / $ 12,500.00)  = $ 3,000.00      ganho de capital $ 3,750.00 - $ 3,000.00 = $ 750.00   ganho de capital em reais $ 750.00 x $ 2,99220 (**) = $ 2.244,15   imposto devido (vencimento em 30/09/2019) $ 2.244,15 x 15% = $ 336,62   (*) calculado proporcionalmente à parcela cujos rendimentos foram obtidos originariamente em moeda  estrangeira.   (**) cotação do dólar fixada para compra, pelo banco central do brasil, para o dia 15/08/2019 (data  do recebimento da 3ª parcela).   portanto, o imposto devido pelo contribuinte a cada parcela recebida é:   vencimento imposto devido   31/07/2019 $ 4.106,16 + $ 427,50 = $ 4.533,66   30/08/2019 $ 3.117,65 + $ 322,53 = $ 3.440,18   30/09/2019 $ 3.399,57 + $ 336,62 = $ 3.736,19   atenção:    as cotações constantes dos exemplos desta pergunta são fictícias.    imposto sobre a renda pago em país com o qual o brasil tenha firmado acordos, tratados ou  convenções internacionais prevendo a compensação, ou naquele em que haja reciprocidade de  tratamento, pode ser considerado como redução do imposto devido no brasil, desde que não seja  compensado ou restituído no exterior.    imposto pago no exterior deve ser convertido em dólares dos estados unidos da américa, pelo  valor fixado pela autoridade monetária do país de origem dos rendimentos na data do pagamento  e, em seguida, em moeda nacional pela cotação do dólar fixada, para compra, pelo banco central  do brasil, para o último dia útil da primeira quinzena do mês anterior ao do recebimento do  rendimento.   efetuada a alienação em 2019, o contribuinte deve dar baixa do bem na declaração de bens e  direitos, incluindo-se o valor de alienação expresso em moeda do país estrangeiro no campo  “discriminação”, bem assim o valor expresso em reais, após efetuada a conversão na forma do art.  2º, § 1º, da instrução normativa  nº 118, de 27 de dezembro de 2000, deixando em branco o  campo “situação em 31/12/2019 ($)”.       244   na hipótese dos bens e direitos estarem situados em país com o qual o brasil mantém acordo,  tratado ou convenção para evitar a dupla tributação internacional de renda, o tratamento fiscal é  aquele previsto no ato internacional.   não incide imposto sobre a renda sobre o ganho de capital auferido na alienação de bens  localizados no exterior ou representativos de direitos no exterior, e na liquidação ou resgate de  aplicações financeiras, adquiridos, a qualquer título, pela pessoa física na condição de não  residente no brasil (instrução normativa  nº 118, de 27 de dezembro de 2000, art. 14, inciso ).  ',
 'qual é o tratamento tributário dos juros recebidos em conta remunerada no exterior?    crédito de rendimentos relativos a depósito remunerado realizado em moeda estrangeira, por pessoa física  residente no brasil, implica a apuração de ganho de capital tributável, desde que o valor creditado seja passível  de saque pelo beneficiário.    tributação da variação cambial (ganho de capital) nas aplicações financeiras realizadas em moeda  estrangeira com rendimentos auferidos originariamente em reais só ocorrerá no momento da liquidação ou  resgate (parcial ou total) da aplicação financeira.    sobre o valor dos juros creditados, desde que este valor seja passível de saque pelo beneficiário, incide o  imposto sobre a renda sobre o ganho de capital, sendo o custo de aquisição igual a zero. em relação a tais  juros, não se aplica a isenção dos ganhos de capital decorrentes da alienação de bens de pequeno valor (valor  igual ou inferior a $ 35.000,00).   os juros decorrentes da aplicação com rendimentos auferidos originariamente em reais, quando não sacados,  configuram, para fins do disposto no art. 24 da  nº 2.158-35, de 2001, uma nova aplicação e são  considerados rendimentos auferidos originariamente em moeda estrangeira, sendo o custo de aquisição  destes juros o próprio valor reaplicado.   atenção:    são isentos os ganhos de capital relativos às aplicações financeiras realizadas em moeda  estrangeira pela pessoa física na condição de não residente no brasil correspondentes ao primeiro  crédito de rendimentos ocorrido a partir da data da caracterização da condição de residente no  brasil, na hipótese de aplicação financeira realizada por tempo indeterminado, inclusive depósito  remunerado. os créditos posteriores estarão sujeitos à apuração do ganho de capital.   exemplo:   depósito remunerado no valor de $ 100,000.00, realizado em 03/06/2019 por josé henrique com  rendimentos auferidos originariamente em reais. nesta conta houve quatro operações sujeitas à apuração do  ganho de capital em 2019:    a) créditos de juros no valor de $ 1,000.00 em 28/06/2019 (não sacados);   b) resgate parcial de $ 50,000.00 em 17/10/2019;   c) créditos de juros no valor de $ 600.00 em 20/12/2019 (não sacados); e     d) resgate parcial de $ 30.000,00 em 27/12/2019.    as cotações constantes neste exemplo são fictícias.   cotações do dólar dos estados unidos da américa  :      245    cotação   cotação     03/06/2019 2,42 2,50   28/06/2019 2,80 2,88   17/10/2019 2,00 2,03   20/12/2019 2,10 2,18   27/12/2019 2,30 2,38   apuramos os três ganhos de capital separadamente   a) crédito de juros, não sacados, de $ 1,000.00 em 28/06/2019   tributação dos juros   item cálculo   valor dos juros creditados $ 1,000.00 x 2,80 = $ 2.800,00   ganho de capital $ 2.800,00 – $ 0,00 = $ 2.800,00   imposto sobre a renda (vencimento em 31/07/2019) 0,15 x 2.800,00 = $ 420,00   do saldo da aplicação ($ 101,000.00), $ 100.000,00 são considerados como aplicação realizada com  rendimentos auferidos originariamente em reais e $ 1,000.00 como rendimentos auferidos originariamente  em moeda estrangeira.   b) resgate no valor de $ 50,000.00 em 17/10/2019   aplicação financeira realizada com rendimentos auferidos originariamente parte em reais, parte em moeda  estrangeira.   inicialmente devemos determinar a proporção do resgate correspondente a rendimentos obtidos  originariamente em reais.   item cálculo   resgate (rend. orig. em reais) 50,000.00 x 100,000.00/101,000.00 = $ 49,504.95   resgate (rend. orig. em moeda estrangeira) 50,000.00 - 49,504.95 = $ 495.05   portanto, neste exemplo:   item cálculo   valor do resgate tributável 49,504.95 x 2,00 = $ 99.009,90   valor original 49,504.95 x 2,50 = $ 123.762,38   ganho de capital 99.009,90 - 123.762,38 = - $ 24.752,48   imposto sobre a renda (vencimento em 29/11/2019) perda de capital   do saldo da aplicação ($ 51,000.00), $ 50,495.05 (100,000.00 - 49,504.95) são considerados como  aplicação realizada com rendimentos auferidos originariamente em reais e $ 504.95 (1,000.00 - 495.05)  como aplicação realizada com rendimentos auferidos originariamente em moeda estrangeira.   c) crédito de juros (não sacados) de $ 600.00 em 20/12/2019   aplicação financeira realizada com rendimentos auferidos originariamente parte em reais, parte em moeda  estrangeira.   tributação dos juros   item cálculo   valor dos juros creditados $ 600.00 x 2,10 = $ 1.260,00   ganho de capital $ 1.260,00 – $ 0,00 = $ 1.260,00   imposto sobre a renda (vencimento em 31/01/2020) 0,15 x 1.260,00 = $ 189,00   d) resgate no valor de $ 30,000.00 em 27/12/2019   aplicação financeira realizada com rendimentos auferidos originariamente parte em reais, parte em moeda  estrangeira.   inicialmente devemos determinar a proporção do resgate correspondente a rendimentos obtidos  originariamente em reais.      246   item cálculo   resgate (rend. orig. em reais) 30,000.00 x 50,495.05/51,000.00 = $ 29,702.97   resgate (rend. orig. em moeda estrangeira) 30,000.00 - 29,702.97 = $ 297,03   portanto, neste exemplo:   item cálculo   valor do resgate tributável 29,702.97 x 2,30 = $ 68.316,83   valor original 29,702.97 x 2,50 = $ 74.257,43   ganho de capital 68.316,83 - 74.257,43 = - $ 5.940,60   imposto sobre a renda (vencimento em 31/01/2020) perda de capital   do saldo da aplicação ($ 21,000.00), $ 20,792.03 (50,000.00 - 29,702.97) são considerados como  aplicação realizada com rendimentos auferidos originariamente em reais e $ 207.97 como aplicação  realizada com rendimentos auferidos originariamente em moeda estrangeira.   atenção:    desde 1º de janeiro de 2017 as alíquotas passaram a ser progressivas em função do ganho de  capital apurado.  ',
 'como são considerados os rendimentos produzidos por aplicações financeiras em moeda  estrangeira e o ganho de capital obtido na alienação de bens ou direitos adquiridos em moeda  estrangeira?   os rendimentos produzidos por aplicações financeiras em moeda estrangeira, bem como o ganho de capital  obtido na alienação de bens ou direitos adquiridos em moeda estrangeira, ainda que decorrentes de  rendimentos auferidos originariamente em reais, são considerados rendimentos auferidos originariamente em  moeda estrangeira.  ',
 'os ganhos obtidos com a alienação de moedas “virtuais” são tributados?   os ganhos obtidos com a alienação de moedas virtuais (bitcoins, por exemplo) cujo total alienado no mês seja  superior a $ 35.000,00 são tributados, a título de ganho de capital, segundo alíquotas progressivas  estabelecidas em função do lucro, e o recolhimento do imposto sobre a renda deve ser feito até o último dia  útil do mês seguinte ao da transação.    contribuinte deverá guardar documentação que comprove a autenticidade das operações. ',
 'qual é o tratamento tributário do ganho de capital auferido na alienação de bens ou direitos  situados no brasil por não residente?   é pacífico em direito internacional que os bens ou direitos sujeitam-se à legislação do país onde estão  situados, quer seu proprietário resida ou não no mesmo território. esse critério também está presente no  campo tributário, sendo exemplo disso as cláusulas dos acordos entre estados soberanos, com o fito de evitar  a dupla tributação internacional da renda, os quais, de modo geral, determinam que os lucros obtidos nessas  operações são tributáveis no estado contratante em que os bens estiverem situados.   dessa forma, o ganho de capital auferido por não residente, nas operações com bens situados no brasil,  sujeita-se à legislação tributária brasileira, sofrendo incidência de imposto a alíquotas progressivas  estabelecidas em função do lucro (art. 18 da lei nº 9.249, de 26 de dezembro de 1995, c/c art. 1º da lei nº  13.259, de 16 de março de 2016) ou, na hipótese de haver acordo entre o brasil e o país de residência do  alienante, o determinado nesse ato internacional, se essa hipótese estiver expressa no acordo.    ganho de capital é determinado pela diferença entre o valor de alienação e o custo de aquisição do bem ou  direito, atualizado até 31/12/1995, com base nos índices da tabela de atualização do custo de bens e direitos  anexa à instrução normativa  nº 84, de 11 de outubro de 2001.  custo dos bens e direitos adquiridos a  partir de 01/01/1996 não está sujeito a atualização. não sendo possível comprovar o custo de aquisição  conforme descrito, o valor de aquisição é igual a zero.   na apuração do ganho de capital, não se aplicam as isenções e reduções do imposto previstas para o  residente no brasil.   para implemento dos efeitos tributários aqui tratados, é irrelevante que o proprietário ingresse no brasil para  efetuar a venda do bem, ou o faça por intermédio de procurador devidamente credenciado para esse fim.   atenção:     fonte pagadora adquirente, pessoa física ou jurídica residente ou domiciliada no brasil, deve reter  e recolher o imposto sobre a renda incidente sobre o ganho de capital, sob o código 0473, auferido  por pessoa física ou jurídica residente ou domiciliada no exterior que alienar bens localizados no  brasil, ou o procurador do alienante quando este não der conhecimento à fonte pagadora de que o  proprietário do imóvel, é residente no exterior (lei nº 10.833, de 29 de dezembro de 2003, arts. 26  e 93, inciso ). ',
 'em quais situações é permitida a utilização do percentual de redução sobre o ganho de capital  na venda de imóvel?    – situações nas quais se aplica a redução:    partir do ano-calendário de 2005, a redução sobre o ganho de capital na venda de imóvel aplica-se em  quatro situações:   1 - art. 18 da lei nº 7.713, de 22 de dezembro de 1988:   na alienação de imóvel adquirido até 31/12/1988, por contribuinte residente no brasil, pode ser aplicado um  percentual fixo de redução sobre o ganho de capital, determinado em função do ano de aquisição ou  incorporação do imóvel, de acordo com a tabela abaixo:     redução      alienação   imóvel       aquisição   %    redução       aquisição   %    redução       aquisição   %    redução       aquisição   %    redução       aquisição   %    redução   até 1969 100 1973 80 1977 60 1981 40 1985 20   1970 95 1974 75 1978 55 1982 35 1986 15   1971 90 1975 70 1979 50 1983 30 1987 10   1972 85 1976 65 1980 45 1984 25 1988 5   (instrução normativa  nº 84, de 11 de outubro de 2001, art. 26)   2 - art. 37 da medida provisória nº 252, de 15 de junho de 2005:   na alienação, a qualquer título, de bens imóveis, realizada por pessoa física residente no brasil, pode ser  aplicado fator de redução   do ganho de capital apurado.    fator de redução é determinado pela seguinte fórmula:  = 1 / 1,0035m, onde "m" corresponde ao número  de meses-calendário, ou fração, decorridos entre o mês de janeiro de 1996 ou a data de aquisição do imóvel,  se posterior, e o mês de sua alienação.   essa redução aplica-se às alienações ocorridas entre 16/06/2005 e 13/10/2005, por força do ato declaratório  do presidente da mesa do congresso nacional nº 38, de 14 de outubro de 2005.   3 - inciso  do § 1º e § 2º do art. 40 da lei nº 11.196, de 21 de novembro de 2005:   na alienação, a qualquer título, de bens imóveis, realizada por pessoa física residente no brasil, pode ser  aplicado fator de redução (fr1) do ganho de capital apurado.    fator de redução será determinado pela seguinte fórmula:   fr1 = 1/1,0060m1, onde “m1” corresponde ao número de meses-calendário, ou fração, decorridos entre o mês  de janeiro de 1996 ou a data de aquisição do imóvel, se posterior, e o mês de sua alienação.   essa redução aplica-se às alienações ocorridas entre 14/10/2005 e 30/11/2005.   4 - incisos  e  do § 1º e § 2º do art. 40 da lei nº 11.196, de 21 de novembro de 2005:   na alienação, a qualquer título, de bens imóveis, realizada por pessoa física residente no brasil, podem ser  aplicados fatores de redução (fr1 e fr2) do ganho de capital apurado.     base de cálculo do imposto corresponde à multiplicação do ganho de capital pelos fatores de redução, que  são determinados pelas seguintes fórmulas:    a) fr1 = 1/1,0060m1, onde "m1" corresponde ao número de meses-calendário, ou fração, decorridos entre o  mês de janeiro de 1996 ou a data de aquisição do imóvel, se posterior, e o mês de novembro de 2005, para  imóveis adquiridos até o mês de novembro de 2005; e   b) fr2 = 1/1,0035m2, onde "m2" corresponde ao número de meses-calendário, ou fração, decorridos entre o  mês de dezembro de 2005, ou o mês da aquisição do imóvel, se posterior, e o de sua alienação.   essas reduções aplicam-se às alienações ocorridas a partir de 1º/12/2005.    - forma de utilização da redução:   as reduções aplicam-se sucessivamente e quando cabíveis.    aplicação de cada uma das reduções referidas nos itens 1 a 4 do inciso  acima dá-se sobre o ganho de  capital diminuído das reduções anteriores.    seguir mostra-se a sequência das reduções do ganho de capital para se chegar ao valor do imposto sobre  a renda devido:      250   bc1 = valor alienação – custo aquisição   1 - alienações ocorridas entre 16/06/2005 e 13/10/2005:   redução 1: bc1 – redução da lei nº 7.713, de 1988 = bc2   redução 2: bc2 x  = bc3    = bc3 x 15%   2 - alienações ocorridas entre 14/10/2005 e 30/11/2005:   redução 1: bc1 – redução da lei nº 7.713, de 1988 = bc2;   redução 2: bc2 x fr1 = bc3    = bc3 x 15%   3 - alienações ocorridas a partir de 1º/12/2005:   redução 1: bc1 – redução da lei nº 7.713, de 1988 = bc2;   redução 2: bc2 x fr1 = bc3;   redução 3: bc3 x fr2 = bc4;    = bc4 x 15%.    - quadro resumo:    quadro seguinte resume as quatro situações previstas na legislação:   data de alienação redução   até 15/06/2005 lei nº 7.713, de 1988, art. 18.   de 16/06/2005 a 13/10/2005       lei nº 7.713, de 1988, art. 18;   =1/1,0035m ( nº 252, de 2005, art. 37)   de 14/10/2005 a 30/11/2005      lei nº 7.713, de 1988, art. 18;   fr1=1/1,0060m1 (lei nº 11.196, de 2005, art. 40, §§ 1º e 2º)    partir de 1º/12/2005      lei nº 7.713, de 1988 art. 18;   fr1=1/1,0060m1  (até 11/05); (lei nº 11.196, de 2005, art. 40, § 1º,  e § 2º)   fr2=1/1,0035m2 (a partir de 12/05). (lei nº 11.196, de 2005, art. 40, § 1º, )   a) m e m1 contados a partir de 01/1996 ou da data de aquisição, se posterior;   b) m2 contados a partir de 12/2005 ou da data de aquisição, se posterior. ',
 'é cabível a redução do ganho de capital quando o contribuinte edifica em terreno alheio?   sim. é cabível a redução sobre o ganho de capital na alienação de imóvel construído em terreno alheio, uma  vez que a construção efetuada em propriedade alheia caracteriza-se como bem imóvel, nos termos do art. 79  da lei nº 10.406, de 10 de janeiro de 2002 - código civil, dado seu caráter de permanência.  ',
 'como proceder para aplicar o percentual de redução na hipótese de alienação de imóvel  edificado, ampliado e/ou reformado em data posterior à da aquisição do terreno?   no caso de percentual de redução diferenciado, é necessário que o custo seja desdobrado de acordo com a  data de aquisição, ou seja, o quanto representa o custo parcial do terreno, da edificação, da ampliação ou  reforma em relação ao total de aquisição.   relativamente ao desdobramento do custo acima mencionado, cabe destacar o seguinte:    - lei nº 7.713, de 22 de dezembro de 1988   para efeito da apuração do valor a ser tributado, no caso de edificação, ampliação e/ou reforma em terreno  próprio adquirido até 31/12/1988, o procedimento é o seguinte:   a) edificação, ampliação e/ou reforma iniciada até 31/12/1988:   desde que essas constem da declaração de ajuste anual, ainda que tenham sido concluídas em ano  posterior ao da sua aquisição, é considerado, para todo o imóvel, o ano da aquisição do terreno.   b) edificação, ampliação e/ou reforma iniciada após 31/12/1988:    redução aplica-se apenas em relação ao terreno e às edificações existentes em 31/12/1988.    - lei nº 11.196, de 21 de novembro de 2005   para efeito da apuração do valor a ser tributado, no caso de edificação, ampliação e/ou reforma em terreno  próprio adquirido até 31/12/1995, o procedimento é o seguinte:   a) edificação, ampliação e/ou reforma iniciada até 31/12/1995:   desde que essas constem da declaração de ajuste anual, ainda que tenham sido concluídas em ano  posterior, é considerado o mês da aquisição do terreno.   b) edificação, ampliação e/ou reforma iniciada após 31/12/1995:   desde que essas constem da declaração de ajuste anual, é considerado o mês de sua realização. ',
 'quando se deve apurar o ganho de capital decorrente de desapropriação?   no caso de desapropriação, o ganho de capital deve ser apurado no mês em que esta se consumar, isto é,  no ato do pagamento integral da indenização. portanto, quando se tratar de imóvel desapropriado cujo  pagamento total e final da indenização tenha ocorrido em 2019, considera-se:   a) data de alienação, a do recebimento final do valor fixado;   b) valor de alienação, a soma das parcelas já recebidas como adiantamento e acrescidas do valor final  recebido, e excluídos os juros, que não integram o valor de alienação e são tributados com os demais  rendimentos recebidos no mês no carnê-leão (se pagos por pessoa física), ou na fonte (se pagos por pessoa  jurídica), bem como na declaração de ajuste anual.   excluem-se do valor da indenização os honorários advocatícios nela eventualmente contidos, cujo ônus seja  do expropriado.   no caso de bem objeto de desapropriação em que ainda não tenha sido recebido o valor integral da  indenização, o contribuinte deve preencher o campo “discriminação”, da declaração de bens ou direitos,  informando essa circunstância e especificando os valores recebidos até 31/12/2019. não incluir as parcelas  referentes a juros.   atenção:    1 - tratando-se de desapropriação para fins de reforma agrária, conforme o disposto no § 5º do art.  184 da constituição federal de 1988, não se apura o ganho de capital relativo à terra nua;   2 - em razão ao disposto no art. 19 da lei nº 10.522, de 2002, na portaria conjunta / nº  1, de 2014, e na nota / nº 1.114, de 2012, a secretaria especial da receita federal do  brasil   encontra-se vinculada ao entendimento do superior tribunal de justiça  , ao julgar      252   o recurso especial nº 1.116.460/, no âmbito da sistemática do art. 543- do código de processo  civil  , que entendeu que a indenização decorrente de desapropriação não encerra ganho de  capital, tendo-se em vista que a propriedade é transferida ao poder público por valor justo e  determinado pela justiça a título de indenização, não ensejando lucro, mas mera reposição do valor  do bem expropriado. afastou-se, portanto, a incidência do imposto sobre a renda relativo às verbas  auferidas a título de indenização advinda de desapropriação, seja por utilidade pública ou por  interesse social.   ',
 'como apurar o ganho de capital de imóvel rural?    ganho de capital corresponde à diferença entre o valor de alienação e o custo de aquisição da terra nua  (sem as benfeitorias) e depende da data de aquisição do imóvel rural. caso o custo das benfeitorias (tanto as  adquiridas pelo alienante quanto as por este realizadas) não tenha sido deduzido como custo ou despesa da  atividade rural, o seu valor integra o custo de aquisição para fins de apuração do ganho de capital.    1 - imóveis adquiridos até 31/12/1996   para os imóveis rurais adquiridos até 31/12/1996, aplicam-se as regras para apuração do ganho de capital  vigentes antes da edição da lei nº 9.393, de 19 de dezembro de 1996.   1.1 - custo de aquisição    custo de aquisição, como regra geral, deve ser o valor constante na declaração de bens e direitos.      1.2 - falta de declaração   caso o contribuinte não tenha apresentado declaração nos exercícios de 1992 a 1996, o cálculo do custo de  aquisição deve seguir orientação específica, dependente da obrigatoriedade ou não da apresentação das  referidas declarações, conforme regras que constam na pergunta abaixo indicada.   1.3 - valor de mercado em 31/12/1991 inferior ao custo corrigido   tratando-se de imóvel adquirido até 1991, cujo valor de mercado declarado em 31/12/1991, for inferior ao  custo corrigido, o contribuinte pode atualizar o custo de aquisição, utilizando a tabela de atualização do custo  de bens e direitos da instrução normativa  nº 84, de 11 de outubro de 2001.   1.4 - imóvel adquirido após 31/12/1991   se o contribuinte adquiriu imóvel rural após 31/12/1991, o custo é o valor da escritura (isto é, o valor pago)  corrigido até 31/12/1995, utilizando a tabela de atualização do custo de bens e direitos da instrução  normativa  nº 84, de 11 de outubro de 2001.   1.5 - imóvel adquirido a partir de 01/01/1996   tratando-se de imóvel adquirido após 31/12/1995, não se atribui correção monetária ao seu custo, conforme  a lei nº 9.249, de 26 de dezembro de 1995, art. 17.   1.6 - valor de alienação    valor de alienação, em todos os casos, é o valor efetivo da transação.   2 - imóveis adquiridos a partir de 01/01/1997   com o advento da lei nº 9.393, de 19 de dezembro de 1996, passam a ser considerados como custo de  aquisição e valor de alienação do imóvel rural, o valor da terra nua  , declarado no documento de  informação e apuração do  (diat), respectivamente nos anos da ocorrência de sua aquisição e de sua  alienação. caso não tenham sido entregues os diat relativos aos anos de aquisição ou alienação, ou ambos,  deve-se proceder ao cálculo do ganho de capital com base nos valores reais da transação.   (lei nº 9.393, de 19 de dezembro de 1996, art. 19)   atenção:    se as benfeitorias tiverem sido deduzidas como despesa de custeio na apuração da determinação  da base de cálculo do imposto da atividade rural, o valor de alienação referente a elas será tributado  como receita da atividade rural.   ',
 'que se considera terra nua para fins de apuração do ganho de capital?   considera-se terra nua o imóvel rural, por natureza, que compreende o solo com sua superfície e respectiva  floresta nativa, despojado das construções, instalações e melhoramentos, das culturas permanentes, das  árvores de florestas plantadas e das pastagens cultivadas ou melhoradas, que se classificam como  investimentos (benfeitorias). ',
 'como apurar o ganho de capital quando o valor da terra nua   de compra e o de  alienação são idênticos, como, por exemplo, para o imóvel adquirido antes da entrega do diat e  vendido após a sua entrega, porém no mesmo ano?   em relação à terra nua, não há que se falar em ganho de capital, visto que o valor da terra nua   de  compra e o de venda são coincidentes, salvo se as benfeitorias não tiverem sido deduzidas como despesa de  atividade rural, caso em que se apura o ganho de capital sobre as benfeitorias.  ',
 'como apurar o ganho de capital se o contribuinte adquirir um imóvel rural, a partir de  01/01/1997, após a entrega do diat ou aliená-lo antes da sua entrega ou, em qualquer caso, onde não  se possa apurar o valor da terra nua   de compra ou de venda, ou ambos?    contribuinte deve proceder ao cálculo do ganho de capital com base nos valores reais das transações de  compra e de alienação do imóvel. ',
 'consorciado que no ano-calendário foi contemplado e alienou o veículo pelo valor de  mercado. para tanto, liberou a alienação fiduciária do veículo e assumiu a dívida do consórcio com  a administradora, por meio de nota promissória.  dívida assumida integra o custo do veículo, para  apuração do ganho de capital?   sim. nesse caso, para efeito de apuração do ganho de capital, considera-se como custo o valor das parcelas  pagas até a alienação do bem, corrigido até 31/12/1995, acrescido do valor da dívida assumida. ',
 'proprietário de um imóvel adquire, mediante contrato particular, outro imóvel a ser  construído.  alienação do imóvel que possui, antes do recebimento daquele em construção, se  enquadra na isenção de alienação de único imóvel por valor igual ou inferior a $ 440.000,00?    proprietário ou possuidor de um imóvel que adquire outro imóvel, construído ou em construção, mediante  contrato particular pactuado com a construtora ou agente financeiro, ainda que não tenha desembolsado  qualquer quantia, tem, a partir do contrato, a propriedade de dois imóveis, não podendo, na venda de um  deles, fazer jus à isenção do único imóvel alienado por valor igual ou inferior a $ 440.000,00.    contrato particular firmado entre a construtora/agente financeiro e o adquirente é instrumento válido para  configurar a aquisição do imóvel.  ',
 'para as alienações decorrentes de adiantamento da legítima, dissolução da sociedade  conjugal ou união estável e sucessão causa mortis, aplicam-se as isenções do bem de pequeno  valor, do único imóvel e dos bens adquiridos até 1988?   sim. tais transferências também são beneficiadas com essas isenções, desde que satisfaçam as exigências  legais. ',
 'para as alienações decorrentes de adiantamento da legítima, dissolução da sociedade  conjugal ou união estável e sucessão causa mortis, aplicam-se os fatores de  redução de que trata  a lei nº 11.196, de 21 de novembro de 2005?   sim. tais transferências também são beneficiadas com essas isenções, desde que satisfaçam às exigências  legais. ',
 'como apurar o ganho de capital quando o valor da alienação é recebido parceladamente?    ganho de capital é apurado como alienação à vista e o imposto deve ser pago de acordo com o recebimento  das parcelas, até o último dia útil do mês subsequente ao do recebimento.  ganho de capital diferido é  calculado aplicando-se o percentual resultante da relação entre o ganho de capital total e o valor total da  alienação sobre o valor de cada parcela recebida.   se o parcelamento incluir cláusula de reajuste, qualquer que seja a designação dada à mesma (juros, correção  monetária, reajuste de parcelas etc.), a parte correspondente ao reajuste deve ter tratamento tributário de  juros.',
 'qual é o tratamento tributário das cláusulas de correção para venda parcelada de imóvel,  previstas em contrato de compra e venda?   independentemente da designação dada (juros, correção monetária, reajuste de parcelas etc.), qualquer  acréscimo no valor da venda provocado pela divisão em parcelas do pagamento deve ser tributado em  separado do ganho de capital, na fonte ou mediante recolhimento mensal obrigatório (carnê-leão), conforme  o caso, e na declaração de ajuste anual correspondente ao ano-calendário de seu recebimento.   exemplo:   se no contrato estiver estabelecido que o valor da venda é de $ 100.000,00, em dez parcelas de  $ 10.000,00 corrigidas pela variação do ; $ 100.000,00 são considerados como valor de alienação,  $ 10.000,00 é o valor de cada parcela para fins de diferimento do ganho de capital.  parte correspondente  à atualização da parcela pelo  fica sujeita ao carnê-leão, quando recebida de pessoas físicas, e à  tributação na fonte, quando recebida de pessoas jurídicas, bem como ao ajuste na declaração de ajuste  anual. ',
 'como atualizar os bens ou direitos adquiridos até 1995 e pela primeira vez declarados?   tanto o contribuinte que esteve obrigado a apresentar a declaração de ajuste anual - exercício 1992 e não  apresentou, quanto o que esteve obrigado a apresentar aquela declaração e apresentou, mas não incluiu  determinado(s) bem ou bens, podem atualizá-los conforme tabela de atualização do custo de bens e direitos  constante da instrução normativa  nº 84, de 11 de outubro de 2001, observado o art. 96 da lei nº 8.383,  de 30 de dezembro de 1991. ',
 'quais são as despesas que podem integrar o custo de aquisição de bens e direitos?   podem integrar o custo de aquisição, quando comprovados com documentação hábil e idônea, e  discriminados na declaração de rendimentos do ano-calendário da realização da despesa:   1 - de bens imóveis:   a) os gastos com a construção, ampliação e reforma, desde que os projetos tenham sido aprovados pelos  órgãos municipais competentes;   b) os gastos com pequenas obras, como pintura, reparos em azulejos, encanamentos, pisos, paredes;   c) as despesas com demolição de prédio construído no terreno, desde que seja condição para se efetivar a  alienação;   d) as despesas de corretagem referentes à aquisição do imóvel vendido, desde que suportado o ônus pelo  alienante;   e) os gastos com a realização de obras públicas como colocação de meio-fio, sarjetas, pavimentação de vias,  instalação de rede de esgoto e de eletricidade que tenha beneficiado o imóvel;   f) o valor do imposto de transmissão pago pelo alienante na aquisição do imóvel alienado;   g) o valor da contribuição de melhoria;   h) o valor do laudêmio pago ao senhorio ou proprietário por desistir do seu direito de opção;   i) os juros e demais acréscimos pagos para a aquisição do imóvel;   j) as despesas com a escritura e o registro do imóvel, cujo ônus tenha sido do adquirente, desde que  comprovados com documentação hábil e idônea.      256   2 - de demais bens ou direitos:   os dispêndios realizados com conservação, reparos, comissão ou corretagem, quando não transferido o ônus  ao adquirente, juros e demais acréscimos pagos no financiamento para a aquisição de bens ou direitos, retífica  de motor etc.   atenção:  valor da corretagem, quando suportado pelo alienante, é deduzido do valor da  alienação e, quando se tratar de venda a prazo, com diferimento da tributação, a dedução far-se-á  sobre o valor da parcela do preço recebida no mês do pagamento da referida corretagem.   ',
 'um casal, cujo regime de casamento é o de comunhão parcial de bens, ou companheiros em  união estável, alienou/alienaram um bem comum, sendo que um dos cônjuges possui outro bem  adquirido antes do casamento.  cônjuge ou companheiro que possui apenas o bem comum faz jus  à isenção de alienação do único imóvel?   não.  bem adquirido na constância do casamento em regime de comunhão parcial de bens ou em união  estável, salvo contrato escrito entre as partes, pertence a ambos cônjuges/companheiros. para a apuração do  ganho de capital, deve ser observado se qualquer um dos cônjuges/companheiros possui outro imóvel ou  tenha alienado algum imóvel nos últimos cinco anos. em caso positivo, não se pode considerar como  alienação de único bem para efeito da isenção prevista em lei. assim, o bem comum alienado pelo casal,  sendo um dos cônjuges/companheiros proprietário de outro imóvel, está sujeito ao imposto sobre a renda  sobre o ganho de capital tributável. ',
 'com relação à isenção para alienação de bens ou direitos por valor igual ou inferior a  $ 35.000,00, como proceder na alienação de bens ou direitos possuídos em condomínio?    limite de isenção para alienações de bens ou direitos de pequeno valor ($ 35.000,00) é considerado em  relação à parte de cada condômino.   atenção:    no caso de bens comuns, o limite aplica-se ao valor total dos bens ou conjunto de bens alienados. ',
 'qual é o custo de aquisição de imóvel adquirido pelo sistema financeiro de habitação    e posteriormente alienado com transferência do saldo devedor?    custo de aquisição corresponde ao valor efetivamente pago pela aquisição do imóvel constante do campo   “situação em 31/12/2018 ($)” da sua declaração de bens eu direitos do exercício de 2019, ano-calendário  de 2018, somado às parcelas pagas em 2019. se desobrigado de apresentar a declaração de ajuste anual,  o custo dos bens e direitos adquiridos ou das parcelas pagas até 31/12/1995.',
 'qual é o custo de aquisição para os imóveis adquiridos em 2019 por meio do sistema  financeiro da habitação  , carta de crédito, programa de arrendamento residencial   ou  financiamento direto com a construtora?   considera-se como custo de aquisição o valor das parcelas efetivamente pagas à construtora ou  incorporadora e ao agente financeiro pela aquisição do imóvel.  forma de pagamento de tais valores deve  ser especificada no campo “discriminação” da declaração de bens ou direitos.',
 'no caso de alienação em que o valor da terra nua   do diat do ano da alienação for  menor que o valor efetivo da venda da terra nua do imóvel rural, como deve ser declarada a diferença  entre os valores?    valor de alienação para apuração do ganho de capital é o constante no diat do ano de alienação. assim a  diferença entre o valor de alienação e o declarado no diat deve ser informado como rendimento isento e não  tributável. ',
 'como são tributados os rendimentos de títulos de investimento coletivo?   não se sujeitam ao ganho de capital e são tributados exclusivamente na fonte como aplicação financeira de  renda fixa, incidindo o imposto sobre a renda incidente na fonte sobre o valor dos rendimentos líquidos pagos  ou creditados decorrentes da aquisição de títulos ou contratos de investimento coletivo, tais como os  comumente conhecidos como “boi gordo”, à alíquota de 20%.    retenção do imposto é efetuada por ocasião do pagamento dos rendimentos ou resgate do título ou contrato,  sendo a fonte pagadora dos rendimentos a responsável por essa retenção.   ',
 'qual é o tratamento tributário da cessão de direito sobre passe de atleta?   na hipótese de a pessoa física titular de direito sobre passe de atleta cedê-lo a terceiros, a diferença positiva  entre o preço de aquisição atualizado até 31/12/1995 e o de cessão deve ser tributada como ganho de capital  na alienação de direitos.   esse direito deve estar informado na declaração de bens e direitos de seu titular.   atenção:    as quantias remetidas para o exterior, para pagamento do valor do passe, são tributáveis na fonte,  nos termos dos arts. 741, inciso , e 746 do regulamento do imposto sobre a renda - /2018,  aprovado pelo decreto nº 9.580, de 22 de novembro de 2018. ',
 'que se considera bem de pequeno valor para fins de exclusão do ganho de capital?   considera-se bem de pequeno valor aquele decorrente da alienação de bens ou direitos cujo preço unitário  de alienação ou cessão, no mês de sua efetivação, seja igual ou inferior a, exceto no caso de alienação de  moeda estrangeira mantida em espécie:    - $ 20.000,00, no caso de alienação de ações negociadas no mercado de balcão, inclusive no exterior;    - $ 35.000,00, nos demais casos, inclusive nas alienações de ações negociadas em bolsas de valores no  exterior.   atenção:    1) na determinação do limite deve ser observado que:   a) no caso de alienação de diversos bens ou direitos da mesma natureza, deve ser considerado o  valor do conjunto dos bens ou direitos alienados em um mesmo mês, tais como automóveis e  motocicletas, imóvel urbano e terra nua, quadros e esculturas, etfs (exchange traded funds) e  reits (real estate investment trust), investimentos em bolsa de valores. sendo ultrapassado esse  limite, o ganho de capital deve ser apurado em relação a cada um dos bens;   b) no caso da sociedade conjugal ou união estável (salvo contrato escrito entre os companheiros),  o limite de isenção aplica-se em relação ao valor de cada um dos bens ou direitos possuídos em  comunhão e ao valor do conjunto dos bens ou direitos da mesma natureza, alienados em um  mesmo mês;   c) na alienação de bens ou direitos em condomínio, o limite aplica-se em relação à parte de cada  condômino ou coproprietário;   d) quando se tratar de permuta com recebimento de torna em dinheiro deve ser considerado o valor  total da alienação e não apenas o valor da torna;   2) as alienações de ações em bolsa de valores no brasil estão sujeitas à apuração de ganhos  líquidos em renda variável (consulte a pergunta 656). ',
 'na permuta de imóveis com recebimento de torna em dinheiro, se o valor da torna for igual  ou inferior a $ 35.000,00, pode ser considerada como "bem de pequeno valor"?   não, uma vez que a não incidência alcança o valor total da alienação e não apenas o valor da torna.   para alienações até 15 de junho de 2005, considera-se bem de pequeno valor a alienação até $ 20.000,00.   em relação às alienações de bens ou direitos de pequeno valor ocorridas a partir de 16 de junho de 2005, o  valor da operação é $ 35.000,00.    consulte a pergunta 631          259   imóvel      habitação     consórcio',
 'que se considera bem de pequeno valor no caso de alienação de imóvel financiado pelo  sistema financeiro da habitação  , ou de bem adquirido por consórcio e assemelhados?    valor de alienação compreende apenas o valor recebido, sem o saldo devedor transferido ao adquirente.  dessa forma, se o valor recebido for igual ou inferior a $ 35.000,00, não se tributa o ganho de capital.  ',
 'contribuinte que aliena o único imóvel que possui deve pagar o imposto sobre a renda por  ocasião de sua alienação?   está isento do imposto sobre a renda o ganho de capital auferido na alienação do único imóvel que o titular  possua, cujo valor de alienação seja de até $ 440.000,00, desde que não tenha realizado qualquer outra  alienação de imóvel nos últimos cinco anos.   no caso da sociedade conjugal ou união estável (salvo contrato escrito entre os companheiros), o limite de  isenção aplica-se em relação ao valor total do imóvel alienado.   atenção:   no regime de separação de bens, os requisitos para a isenção devem ser verificados  individualmente, por cônjuge, observada a parcela que lhe couber. ',
 'contribuinte que possui a copropriedade de imóvel e ainda possui outro imóvel em seu  próprio nome, ao vender qualquer um deles pode considerá-lo "único imóvel" para efeito de  exclusão do ganho de capital?   não.  copropriedade caracteriza propriedade de imóvel. nesse caso, possuindo o contribuinte mais de um  imóvel, não há que se falar em “único imóvel”. assim, não podem utilizar, do benefício.',
 'contribuinte possuía dois terrenos contíguos, promoveu a unificação e sobre eles construiu  uma residência, passando a ser proprietário desse único imóvel. pode usufruir da exclusão do  ganho de capital na alienação do único imóvel?    sim.  art. 23 da lei nº 9.250, de 26 de dezembro de 1995, faz referência à alienação do único imóvel que o  titular possua (no mês da alienação). conforme o disposto no art. 111, inciso , do código tributário nacional   , e caso não tenha ocorrido alienação a qualquer título nos últimos 5 anos de outro imóvel, e  considerando que a unificação de imóveis não caracteriza alienação, é cabível a exclusão do ganho de capital  no caso de alienação, como um único imóvel e em um único instrumento, de imóvel resultante de unificação  anterior de outros que o titular possuía, devidamente averbada em cartório de registro de imóveis ',
 'contribuinte, proprietário de um único imóvel, que não alienou nenhum imóvel nos últimos 5  anos, aliena uma parte desse imóvel, permanecendo na propriedade da área remanescente. tal  contribuinte tem direito nessa operação à isenção de alienação de único imóvel?   não. nesse caso, o contribuinte não alienou todo o imóvel, mas parte, permanecendo, na data, como  proprietário de imóvel. não faz jus, portanto, à isenção. ',
 'qual é o custo de aquisição na transferência de ativos recebidos na dissolução da sociedade  conjugal ou da união estável?   na transferência do direito de propriedade em decorrência de dissolução de sociedade conjugal ou da união  estável, os bens e direitos podem ser avaliados pelo valor constante na última declaração de bens e direitos  ou em valor superior àquele declarado (valor de mercado), conforme legislação pertinente.   se a transferência dos bens ou direitos a quem lhe foram atribuídos foi em valor superior àquele pelo qual  constavam na última declaração de quem declarava os bens antes da dissolução da sociedade conjugal ou  união estável, a diferença positiva é tributada à alíquota de 15%, em nome do cônjuge ou companheiro a  quem o ativo foi atribuído. nesse caso, os bens e direitos devem ser incluídos na declaração de bens, pelo  valor atribuído na transferência do direito de propriedade, que constituirá custo para efeito de eventual  alienação futura.   se a transferência for efetuada pelo valor informado na última declaração de bens e direitos, não incide a  cobrança de imposto no ato da transferência.  ex-cônjuge ou ex-companheiro a quem foram atribuídos os  bens ou direitos, deve incluí-los em sua declaração de bens e direitos, pelos valores informados na última  declaração de quem os declarava, antes da dissolução da sociedade conjugal ou da união estável.  ',
 'quem é o contribuinte do imposto sobre o ganho de capital na transferência de ativos em  decorrência de dissolução da sociedade conjugal ou da união estável?   se a transferência for efetuada por valor superior àquele que vinha sendo declarado, a diferença a maior  sujeita-se à incidência de imposto sobre a renda à alíquota de 15% (quinze por cento).     contribuinte do imposto é o ex-cônjuge ou o ex-convivente ao qual o bem ou direito for atribuído.  ',
 'qual é o tratamento tributário na transferência de bens e direitos na doação em adiantamento  da legítima?   na transferência do direito de propriedade por doação em adiantamento de legítima, os bens e direitos podem  ser avaliados pelo valor constante na última declaração de bens e direitos do doador ou por valor superior  àquele declarado (valor de mercado), conforme legislação pertinente.   se os bens ou direitos forem transferidos por valor superior àquele declarado, a diferença positiva entre este  e o valor constante na última declaração de bens e direitos do doador é tributada como ganho de capital à  alíquota de 15% (quinze por cento), em nome do doador.      261   se a transferência for efetuada pelo valor constante na última declaração de bens e direitos do doador, não  há cobrança de imposto no ato da transferência, mas o donatário deve incluir os bens ou direitos, em sua  declaração de bens e direitos, pelo valor constante na declaração do doador, o qual constitui custo para efeito  de apuração de ganho de capital em eventual futura alienação. ',
 'que se considera como incorporação de prédios em condomínio?   considera-se incorporação imobiliária a atividade exercida com o intuito de promover e realizar a construção,  para alienação total ou parcial (antes da conclusão das obras), de edificações ou conjunto de edificações  compostas de unidades autônomas, sob o regime de condomínio.    incorporador vende frações ideais do terreno, vinculadas às unidades autônomas (apartamentos, salas,  conjuntos etc.), em construção ou a serem construídas, obtendo, assim, os recursos necessários para a  edificação. pode também alienar as unidades já construídas.',
 'quais as construções abrangidas e a legislação que se aplica para as incorporações  imobiliárias?    as incorporações imobiliárias abrangem, por exemplo, construções de edifícios de apartamentos ou lojas em  condomínio. estão sujeitas ao disciplinamento imposto pela lei nº 4.591, de 16 de dezembro de 1964.   ',
 'quem é considerado "incorporador"?   considera-se incorporador a pessoa física ou jurídica, comerciante ou não, que, embora não efetuando a  construção, compromisse ou efetive a venda de frações ideais de terreno objetivando a vinculação de tais  frações a unidades autônomas, em edificações a serem construídas ou em construção sob regime  condominial, ou que meramente aceita propostas para efetivação de tais transações, coordenando e levando  a termo a incorporação e responsabilizando-se, conforme o caso, pela entrega, em certo prazo, a preço e em  determinadas condições, das obras concluídas.   estende-se a condição de incorporador aos proprietários e titulares de direitos aquisitivos que contratem a  construção de edifícios que se destinem à constituição em condomínio, sempre que iniciarem as alienações  antes da conclusão das obras.   (lei nº 4.591, de 16 de dezembro de 1964, arts. 29 e 30)   tendo em vista as disposições da lei nº 4.591, de 1964, especificamente os arts. 29 a 32 e 68, é irrelevante  a forma da construção efetuada (vertical, horizontal, autônoma, isolada etc.) para que a pessoa física seja  considerada incorporador e se submeta ao regime de equiparação a pessoa jurídica, para efeitos fiscais,      262   desde que existentes os demais pressupostos para a configuração dessa forma de tributação dos resultados  auferidos nesse empreendimento.   ',
 'que se considera como loteamento e desmembramento de terrenos?   desmembramento é a subdivisão de uma gleba (área) em lotes para edificação, na qual seja aproveitado o  sistema viário oficial local, isto é, não se abrem vias, ruas ou logradouros públicos nem se ampliam ou  modificam os existentes, para que as edificações e os acessos a elas sejam factíveis.   loteamento é a subdivisão de uma gleba (área) em lotes, destinados a edificações, com abertura de novas  vias de circulação e de logradouros públicos ou de ampliação, modificação ou prolongamento dos existentes.   atenção:     promoção de loteamento por pessoa física, seja de terreno urbano ou rural, a equipara a pessoa  jurídica para os efeitos do imposto sobre a renda.    subdivisão ou desmembramento de imóvel rural, havido após 30/06/1977, em mais de 10 (dez)  lotes, ou alienação de mais de 10 (dez) quinhões ou frações ideais do terreno, equipara-se a  loteamento e, em consequência, equipara a pessoa física a pessoa jurídica, exceto se a subdivisão  ou desmembramento se efetivar por força de partilha amigável ou judicial, em decorrência de  herança, legado, doação como adiantamento da legítima, ou extinção de condomínio.   ',
 'pessoa física equiparada a pessoa jurídica em razão de operações com imóveis, sofre dupla  tributação de seus lucros se efetuar alienação eventual de imóveis sujeita ao regime da lei nº 9.250,  de 26 de dezembro de 1995?   não.  alienação dos imóveis não integrantes da incorporação ou loteamento é tributada como ganho de  capital da pessoa física de seu titular, ou não se sujeita à tributação, se atendidas as condições dos arts. 22  (bens de pequeno valor) e 23 (único imóvel) da lei nº 9.250, de 26 de dezembro de 1995.',
 'na hipótese de falecimento de pessoa física equiparada a pessoa jurídica pela promoção de  loteamento, como fica a situação fiscal do cônjuge meeiro, casado em comunhão de bens com o de  cujus, em relação aos créditos vincendos e aos remanescentes?    morte, enquanto termo final da personalidade, implica exclusão do de cujus do mundo jurídico, mas não a  extinção dos efeitos tributários que decorrem do empreendimento imobiliário e alcançam o espólio, o cônjuge  meeiro e os sucessores causa mortis.   assim, se a lei fiscal equiparou a pessoa física responsável pelo empreendimento imobiliário a pessoa jurídica,  a equiparação se prolonga até os sucessores causa mortis, porque, em face das leis que disciplinam o  parcelamento do solo, eles continuam loteadores. é o que se depreende do art. 29 da lei nº 6.766, de 19 de  dezembro de 1979: "aquele que adquirir a propriedade loteada (...) por sucessão causa mortis sucederá o      263   transmitente em todos os direitos e obrigações, ficando obrigado a respeitar os compromissos de compra e  venda ou as promessas de cessão, em todas as cláusulas, sendo nula qualquer disposição em contrário,  ressalvado o direito do herdeiro ou legatário de renunciar à herança ou ao legado".    situação jurídica do cônjuge, em face do regime de casamento, é idêntica àquela desfrutada pelo consorte  falecido; assim sendo, também se equiparou em tal ocasião. dessa forma, impõe-se que ele prossiga  apurando os resultados na condição de pessoa jurídica por equiparação, em relação à parcela do patrimônio  que lhe for adjudicada, cumprindo todas as condições impostas pela legislação tributária. os herdeiros e  legatários, na condição de sucessores, conforme definição do art. 29 da lei nº 6.766, de 1979, devem  constituir-se em pessoa jurídica (por equiparação) a fim de, nesta condição, darem continuidade à apuração  dos resultados, na forma disposta na legislação tributária, em relação à parcela do patrimônio que lhes couber  na partilha. ',
 'as rendas consideradas consumidas e as deduções permitidas em lei, sem comprovação,  podem justificar acréscimo patrimonial?   quando o contribuinte, por determinação legal, tributa unicamente parte do rendimento bruto, a exemplo de  10% e 60% para transporte de carga e de passageiros (caminhoneiro e taxista), respectivamente, e 10% para  garimpeiro, ou efetua qualquer dedução sem necessidade de comprovação de gastos, tais como dedução  com dependentes ou 20% a título de desconto simplificado, considera-se consumida a importância não  tributada ou deduzida, por presunção legal, não podendo justificar acréscimo patrimonial. ',
 'como declarar a quantia recebida como pagamento de empréstimo concedido?   informar, no campo “discriminação” da declaração de bens e direitos, o valor do empréstimo, o nome e o  número de inscrição no cadastro de pessoas físicas   do mutuário e as datas e os valores recebidos  para quitação do mesmo, ainda que o empréstimo tenha sido concedido e integralmente recebido no ano de  2019. nos campos “situação em 31/12/2018 ($)” e “situação em 31/12/2019 ($)” informar os saldos em  31/12/2018 e 31/12/2019, respectivamente.    valor recebido deve ser não só comprovado por meio de documentação hábil e idônea e pelo devido  lançamento do mútuo nas respectivas declarações, como também ser compatível com os rendimentos e  disponibilidades financeiras declaradas pelos mutuantes, nas respectivas datas de entrega e recebimento dos  valores.    simples alegação de que parte ou todo o acréscimo patrimonial é proveniente do recebimento de quantias  anteriormente emprestadas a terceiros não justifica o aumento patrimonial. ',
 'acréscimo patrimonial oriundo de transações ilícitas é tributável?   os rendimentos derivados de atividades ou transações ilícitas ou percebidos com infração à lei são tributáveis  por força do art. 26 da lei nº 4.506, de 30 de novembro de 1964, sem prejuízo das demais sanções legais que  couberem em cada caso. ',
 'quais são as operações realizadas nos mercados financeiro e de capital?   nesses mercados são negociados títulos, valores mobiliários e ativos financeiros que, de acordo com as  características do ativo ou contrato objeto da operação, podem ser classificados em dois grandes segmentos:   1 - mercado de renda variável   compõe-se de ativos de renda variável, quais sejam, aqueles cuja remuneração ou retorno de capital não  pode ser dimensionado no momento da aplicação. são eles as ações, quotas ou quinhões de capital, o  ouro, ativo financeiro, e os contratos negociados nas bolsas de valores, de mercadorias, de futuros e  assemelhadas.   2 - mercado de renda fixa   compõe-se de ativos de renda fixa aqueles cuja remuneração ou retorno de capital pode ser dimensionado  no momento da aplicação. os títulos de renda fixa são públicos ou privados, conforme a condição da  entidade ou empresa que os emite. como títulos de renda fixa públicos citam-se as notas do tesouro  nacional  , os bônus do banco central  , os títulos da dívida agrária  , bem como os  títulos estaduais e municipais. como títulos de renda fixa privados, aqueles emitidos por instituições ou  empresas de direito privado, citam-se as letras de câmbio  , os certificados de depósito bancário   , os recibos de depósito bancário   e as debêntures.   equiparam-se a operações de renda fixa, para fins de incidência do imposto sobre a renda incidente na  fonte, as operações de mútuo e de compra vinculada à revenda, no mercado secundário, tendo por objeto  ouro, ativo financeiro, as operações de financiamento, inclusive box, realizadas em bolsas de valores, de  mercadorias e de futuros e as operações de transferência de dívidas, bem como qualquer rendimento  auferido pela entrega de recursos a pessoa jurídica.   ',
 'quais são as operações do mercado de renda variável?    mercado de renda variável compreende todas as operações realizadas nas bolsas de valores, de  mercadorias, de futuros e assemelhadas, bem como as operações com ouro, ativo financeiro, realizadas fora  de bolsas, com a interveniência de instituições integrantes do sistema financeiro nacional (bancos, corretoras  e distribuidoras), ressalvadas as operações de mútuo e de compra vinculada à revenda com ouro, ativo  financeiro, e as operações de financiamento referidas na pergunta anterior.   ',
 'qual é o tratamento tributário das operações de renda variável?    tratamento tributário conferido a essas operações depende das modalidades em que são negociados os  ativos ou contratos, modalidades essas denominadas mercados à vista, de opções, futuro e a termo.    ',
 'que é ganho líquido no mercado de renda variável?   ganho líquido é o resultado positivo auferido em um conjunto de operações realizadas em cada mês, em um  ou mais mercados de bolsa e em operações com ouro, ativo financeiro, realizadas fora de bolsa.  ',
 'que é operação day trade?   considera-se day trade a operação ou a conjugação de operações iniciadas e encerradas em um mesmo dia,  com o mesmo ativo, em uma mesma instituição intermediadora, em que a quantidade negociada tenha sido  liquidada, total ou parcialmente.   na apuração do resultado da operação day trade são considerados, pela ordem, o primeiro negócio de compra  com o primeiro de venda ou o primeiro negócio de venda com o primeiro de compra, sucessivamente.   será admitida a compensação de perdas incorridas em operações de day trade realizadas no mesmo dia.  ',
 'qual é a alíquota de incidência do  aplicável às operações do mercado de renda variável  realizadas em bolsa?   os ganhos líquidos auferidos em operações realizadas em bolsas de valores, de mercadorias, de futuros, e  assemelhadas, inclusive day trade, serão tributados às seguintes alíquotas:   a) 20%, no caso de operação day trade;   b) 15%, nas operações realizadas nos mercados à vista, a termo, de opções e de futuros.   as operações realizadas em bolsas de valores, de mercadorias, de futuros, e assemelhadas estão sujeitas à  retenção do imposto sobre a renda incidente na fonte à alíquota de 0,005% (cinco milésimos por cento), salvo  se o valor da retenção do imposto seja igual ou inferior a $ 1,00, como antecipação, podendo ser  compensado com o imposto sobre a renda mensal na apuração do ganho líquido.   no caso de incidência do imposto sobre a renda na fonte em operações day trade. ',
 'todas as operações em bolsas estão sujeitas ao ?   não. são isentos do imposto sobre a renda os ganhos líquidos auferidos por pessoa física em operações  efetuadas:    - com ações, no mercado à vista de bolsas de valores ou mercado de balcão, se o total das alienações desse  ativo, realizadas no mês, não exceder a $ 20.000,00 (vinte mil reais);     - com ouro, ativo financeiro, se o total das alienações desse ativo, realizadas no mês, não exceder a  $ 20.000,00 (vinte mil reais);    – com ações de pequenas e médias empresas a que se refere o art. 16 da lei nº 13.043, de 13 de novembro  de 2014.   atenção:    ocorrendo alienação no mesmo mês de ações e de ouro, ativo financeiro, os limites de isenção dos  itens  e  acima aplicam-se separadamente a cada modalidade de ativo.     isenção não se aplica, entre outras, às operações de day trade, às negociações de cotas dos fundos de  investimento em índice de ações, aos resgates de cotas de fundos ou clubes de investimento em ações e à  alienação de ações efetivada em operações de exercício de opções e no vencimento ou liquidação antecipada  de contratos a termo. ',
 'as despesas incorridas nas operações no mercado de renda variável podem ser deduzidas?   sim. as despesas efetivamente pagas destacadas na nota de corretagem ou no extrato da conta-corrente  para a realização de operações de compra ou venda (corretagens, emolumentos etc.) podem ser consideradas  na apuração do ganho líquido, sendo acrescidas ao preço de compra e deduzidas do preço de venda dos  ativos ou contratos negociados.  ',
 'é permitida a compensação de perdas com ganhos em operações de renda variável?   sim. para fins de apuração e pagamento do imposto mensal sobre os ganhos líquidos, as perdas incorridas  nas operações de renda variável nos mercados à vista, de opções, futuros, a termo e assemelhados, poderão  ser compensadas com os ganhos líquidos auferidos, no próprio mês ou nos meses subsequentes, em outras  operações realizadas em qualquer das modalidades operacionais previstas naqueles mercados, operações  comuns.   atenção:   as perdas incorridas em operações de day trade, somente poderão ser compensadas com ganhos  líquidos auferidos em operações da mesma espécie (day trade), realizadas no mês ou meses  subsequentes. do mesmo modo, as perdas incorridas em operações comuns somente são  compensáveis com os ganhos líquidos auferidos nessas operações.',
 'resultado negativo ou perda apurado em um mês pode ser compensado com ganho auferido  em meses anteriores?   não se pode compensar resultados negativos de um mês com ganhos auferidos em meses anteriores, pois a  base de cálculo do imposto é apurada mensalmente.',
 'resultado negativo ou perda apurado em dezembro pode ser compensado com o ganho  auferido em qualquer mês do exercício seguinte?   sim, não há restrição quanto ao mês ou ano de sua utilização.   ressalte-se que, caso se pretenda compensar o resultado negativo (prejuízo) de períodos anteriores, esse  prejuízo deve estar informado no demonstrativo de renda variável (constante da declaração de ajuste anual)  onde ocorreu o prejuízo e nos períodos seguintes, até a sua completa compensação.      267   atenção:    as perdas incorridas em operações iniciadas e encerradas no mesmo dia (day trade) somente são  compensáveis com os ganhos líquidos auferidos nessas operações (day trade), em uma ou mais  modalidades operacionais. do mesmo modo, as perdas incorridas em operações comuns somente  são compensáveis com os ganhos líquidos auferidos nessas operações.   ',
 'os rendimentos auferidos em operações day trade sujeitam-se à incidência do imposto sobre  a renda incidente na fonte?   sim. os rendimentos auferidos em operações day trade realizadas em bolsas de valores, de mercadorias, de  futuros e assemelhadas, por qualquer beneficiário, sujeitam-se à incidência do imposto sobre a renda incidente  na fonte à alíquota de 1%. ',
 'quem é o responsável pela retenção do imposto sobre a renda retido na fonte incidente sobre  operações day trade?    responsável pela retenção e recolhimento do imposto sobre a renda na fonte é a instituição intermediadora  da operação de day trade que receber, diretamente, a ordem do cliente.   ',
 'valor do imposto retido na fonte sobre as operações day trade pode ser deduzido do  imposto incidente sobre os ganhos no mês ou em meses posteriores?   sim.  valor do imposto retido na fonte sobre operações day trade pode ser deduzido do imposto incidente  sobre ganhos líquidos apurados no mês ou compensado com o imposto incidente sobre os ganhos líquidos  apurados nos meses subsequentes se, até o mês de dezembro do ano-calendário da retenção, houver saldo  de imposto retido.  ',
 'valor do imposto retido na fonte durante o ano-calendário sobre rendimentos de day trade  pode ser compensado com o imposto incidente sobre ganhos auferidos em meses do ano- calendário seguinte?   não.  valor do imposto retido na fonte sobre operações day trade somente pode ser compensado até o mês  de dezembro do ano-calendário da retenção.   atenção:    se ao fim do ano-calendário houver saldo de imposto retido na fonte a compensar, fica facultado à  pessoa física solicitar restituição, nos termos previstos na legislação de regência.'  ,
 'quais os ativos que são negociados nos mercados à vista e nos demais mercados?   são negociados nos mercados à vista ações emitidas por companhias abertas e ouro, ativo financeiro, sendo  este último negociado dentro e fora das bolsas, desde que com interveniência de instituição financeira. nos  demais mercados (a termo, opções e futuro) podem ser negociados, além das ações, contratos tendo por  objeto outros ativos, como índices de ações, taxa de juros, dólar, café, boi gordo etc. sobre outros valores  mobiliários.',
 'que se entende por operações realizadas “dentro de bolsas” e “fora de bolsas”?    os mercados de valores mobiliários compreendem os mercados organizados (de bolsa e balcão) e os  mercados de balcão não organizados.    mercado de bolsa – ambiente de negociação em local físico definido, com sistema de negociação eletrônico  (ou viva-voz) e regras adequadas à realização de operações de compra e venda de títulos e valores  mobiliários.   mercado de balcão organizado - ambiente de negociação administrado por instituições auto-reguladoras,  autorizadas e supervisionadas pela comissão de valores mobiliários  , que mantêm sistema de  negociação (eletrônico ou não) e regras adequadas à realização de operações de compra e venda de títulos  e valores mobiliários.    mercado de balcão não organizado - mercado de títulos e valores mobiliários cujos negócios não são  supervisionados por entidade auto-reguladora.   mercados de bolsa e mercados de balcão organizado não se assemelham na medida em que distintos quanto  a suas características de funcionamento, em especial no que diz respeito aos diferentes mecanismos de  formação de preços.  ',
 'qual é o tratamento tributário conferido aos ganhos obtidos nas alienações de ações fora de  bolsa de valores?   os ganhos auferidos por pessoas físicas nas alienações de ações fora de bolsa são tributados como ganho  de capital. se alienadas em bolsa de valores estão sujeitas às normas de apuração de renda variável. ',
 'resultado negativo obtido pela pessoa física na alienação de ações  fora de bolsa pode ser  compensado com o lucro obtido nas operações realizadas em bolsas de valores, de mercadorias e  de futuros?   não. os resultados negativos apurados nas alienações fora de bolsa não são compensáveis, pois estão  sujeitos à apuração de ganho de capital.   para efeitos fiscais, o lucro obtido na alienação de ações realizadas em pregões de bolsa de valores é  conceituado como ganho líquido, enquanto o lucro apurado na alienação de ações realizada no mercado de  balcão é considerado como ganho de capital.',
 'que é o mercado à vista?   é uma modalidade de mercado em que são negociados valores mobiliários e ouro, ativo financeiro, cuja  liquidação física (entrega do ativo pelo vendedor) e financeira (pagamento do ativo pelo comprador) ocorrem,  no máximo, até o 3º dia após ao da negociação.        à  -  líquido',
 'como se calcula o ganho líquido sobre operações nos mercados à vista?    ganho líquido é constituído pela diferença positiva entre o valor de venda do ativo e o seu custo de aquisição.  ',
 'como se calcula o custo de aquisição dos ativos no mercado à vista?    custo de aquisição é calculado pela média ponderada dos custos unitários. ',
 'qual é o custo de aquisição de bonificações recebidas em virtude de incorporação de lucros  e reservas no caso de ações?   1 - no caso de ações recebidas em bonificação, em virtude de incorporação ao capital social da pessoa jurídica  de lucros ou reservas, considera-se custo de aquisição da participação o valor do lucro ou reserva capitalizado  que corresponder ao acionista ou sócio, independentemente da forma de tributação adotada pela empresa.   2 - na hipótese de lucros apurados nos anos-calendário de 1994 e 1995, as ações bonificadas terão custo  zero.  ',
 'qual é o valor do custo de aquisição de ações desdobradas?    custo das ações recebidas em virtude de desdobramento do número de ações originalmente possuídas  pelo investidor é igual a zero, ou seja, aumenta apenas a quantidade de ações e permanece inalterado o valor  total das ações.  ',
 'que é mercado a termo?   é uma modalidade de mercado a prazo em que se negocia a compra ou venda de determinado ativo por preço  e prazo preestabelecidos em contrato (liquidação diferida, geralmente 30, 60, 90 dias).   é exigido das partes contratantes, vendedor e comprador, um depósito de margem em garantia.  ',
 'que se considera ganho líquido no mercado a termo?   1 - situação do comprador    custo de aquisição é o preço do ativo estabelecido no contrato de compra a termo.    ganho líquido é a diferença positiva entre o valor de venda à vista do ativo, na data da liquidação do contrato  a termo ou posteriormente, e o custo de aquisição.   exemplo:    investidor comprou a termo 1.000 ações , ao preço de $ 10,00 por ação, totalizando o valor do contrato  em $ 10.000,00, com vencimento para 30 dias. no vencimento, o investidor vendeu no mercado à vista as  1.000 ações  por $ 12.000,00. assim, sem considerar a corretagem e outras despesas, temos:   valor de venda à vista do ativo      $  12.000,00   custo de aquisição do ativo      ($ 10.000,00)   ganho líquido        $    2.000,00   2 - situação do vendedor descoberto    custo de aquisição é o preço de compra à vista do ativo objeto da liquidação do contrato a termo.    ganho líquido é a diferença positiva entre o preço do ativo recebido constante no contrato a termo e o custo  de aquisição.   exemplo:    investidor vendeu a termo 1.000 ações , ao preço de $ 10,00 por ação, totalizando o valor do contrato  em $ 10.000,00, com vencimento para 30 dias. no vencimento, tendo o mercado registrado movimento de  baixa no período, o investidor comprou no mercado à vista o lote de 1.000 ações  por $ 9.500,00, para  honrar a liquidação do contrato a termo. assim, sem considerar a corretagem e outras despesas, temos:   valor contratual recebido $  10.000,00   custo de aquisição do ativo ($   9.500,00)   ganho líquido $       500,00   atenção:     ganho obtido pelo vendedor coberto nas operações de financiamento realizadas no mercado a  termo com ações é tributado como aplicação de renda fixa.  ',
 'que é mercado de opções?   é uma modalidade de mercado a prazo em que são negociados direitos de comprar ou de vender um  determinado ativo, mediante pagamento pelo comprador (titular) ao vendedor (lançador) de um valor chamado  prêmio, com preço do ativo e prazo de exercício previamente fixados.   é exigido da posição lançadora um depósito de margem em garantia, no caso de lançador descoberto.  ',
 'que se considera ganho líquido no mercado de opções, no caso de operações que tenham  por objeto a negociação da opção?   operações tendo por objeto a negociação das opções de compra ou de venda (sem exercício):   1 - posição titular (direito de compra ou de venda)    custo de aquisição das opções de mesma série é calculado pela média ponderada dos prêmios unitários  pagos.    ganho líquido é obtido pela diferença positiva entre o valor da operação de encerramento das opções de  mesma série (valor recebido pela venda de opções) e o seu custo médio de aquisição.   exemplo:    investidor adquiriu opção de compra de 10.000 ações , pagando o prêmio unitário de $ 1,00, totalizando  o prêmio de $ 10.000,00, com vencimento para 60 dias e preço de exercício de $ 10,00 por ação .   antes do vencimento, em face da tendência favorável do mercado, o investidor decidiu encerrar (zerar) sua  posição compradora, e vendeu opção de compra de 10.000 ações , da mesma série, recebendo o prêmio  total de $ 12.000,00.   desconsiderando a corretagem e outras despesas, temos:   valor do prêmio recebido                $  12.000,00   valor do prêmio pago pela compra ($ 10.000,00)   ganho líquido              $    2.000,00   2 - posição lançadora (obrigação de venda ou de compra)   para apurar o ganho líquido, adote os seguintes procedimentos:   a) some os valores dos prêmios referentes às opções lançadas, recebidos até a data da operação de  encerramento, em opções de mesma série;   b) por ocasião do encerramento, divida o valor encontrado em "a" pela quantidade de opções de mesma série  lançadas até aquela data, apurando o valor médio do prêmio recebido em cada opção;   c) na hipótese de encerramento parcial, o valor das opções remanescentes é ajustado, subtraindo-se do valor  encontrado em "a", o valor calculado em "b", multiplicado pela quantidade de opções objeto da operação de  encerramento.    ganho líquido é obtido pela diferença positiva entre o valor médio do prêmio recebido em cada opção  multiplicado pela quantidade de opções de mesma série objeto da operação de encerramento e o valor desta  operação.   exemplo:    investidor vendeu opção de compra de 10.000 ações , recebendo o prêmio unitário de $ 1,00, totalizando  o prêmio de $ 10.000,00, e, dias depois, vendeu novamente opção de compra de 5.000 ações , da mesma  série, recebendo o prêmio unitário de $ 1,10, totalizando o prêmio de $ 5.500,00. ambas as operações  com vencimento para 60 dias e preço de exercício de $ 10,00 por ação .   antes do vencimento, em face da tendência indefinida do mercado, o investidor decidiu encerrar parcialmente  sua posição vendedora, e adquiriu opção de compra de 12.000 ações , da mesma série, pagando o prêmio  unitário de $ 1,00, totalizando o prêmio de $ 12.000,00.    desconsiderando a corretagem e outras despesas, temos:      272   item cálculo   prêmio total recebido $ 15.500,00   valor médio do prêmio recebido $ 15.500,00 ÷ 15.000 = $ 1,03   valor prêmio rec. / metade encerrada $ 12.000,00  $ 1,03 = $ 12.360,00   valor prêmio pago pela quantidade encerrada $ 12.000,00   ganho líquido $ 360,00   valor do saldo de opções vendidas $ 15.500,00 - $ 12.360,00 = $ 3.140,00.',
 'que se considera ganho líquido no exercício de opções de compra?   1 - titular de opção de compra (comprador)    custo de aquisição é o preço de exercício do ativo acrescido do valor do prêmio pago. considera-se preço  de exercício o valor de compra do ativo acordado para liquidação da operação.    ganho líquido é a diferença positiva entre o valor de venda à vista do ativo, na data do exercício, e o seu  custo de aquisição.   ocorrendo a venda posteriormente à data do exercício, o ganho líquido será a diferença positiva entre o valor  recebido pela venda do ativo e o custo médio de aquisição, apurado conforme estabelecido para o mercado  à vista.   exemplo:    investidor adquiriu opção de compra de 10.000 ações , pagando o prêmio total de $ 10.000,00, com  vencimento para 60 dias e ao preço de exercício de $ 10,00 por ação . no vencimento, estando o preço de  mercado da ação  acima do preço de exercício, o investidor decidiu exercer a opção, mediante manifestação  a sua sociedade corretora com simultânea ordem de venda à vista das 10.000 ações .  venda à vista  totalizou $ 130.000,00, enquanto o preço de exercício totalizou $ 100.000,00.   desconsiderando a corretagem e outras despesas, temos:   valor de venda à vista do ativo $130.000,00   custo de aquisição da operação:   valor prêmio pago $10.000,00   preço exercício pago $ 100.000,00   custo total $ 110.000,00   ganho líquido $20.000,00   2 - lançador de opção de compra (vendedor)    custo de aquisição:    - para o lançador coberto, é o custo médio de aquisição do ativo conforme estabelecido para o mercado à  vista.    - para o lançador descoberto, é o preço de aquisição do ativo objeto do exercício.    ganho líquido é a diferença positiva entre o preço de exercício do ativo, acrescido do valor do prêmio  recebido, e o seu custo de aquisição.   considera-se preço de exercício, o valor de venda do ativo acordado para liquidação da operação. ',
 'que se considera ganho líquido no exercício de opções de venda?   1 - titular de opção de venda (vendedor)    custo de aquisição é o custo médio de aquisição do ativo acrescido do valor do prêmio pago.    ganho líquido é a diferença positiva entre o preço de exercício do ativo e o seu custo de aquisição.   exemplo:    investidor adquiriu opção de venda de 20.000 ações , pagando o prêmio total de $ 20.000,00, com  vencimento para 60 dias e preço de $ 10,00 por ação . no vencimento, estando o preço de mercado da  ação  abaixo do preço de exercício, o investidor decidiu exercer a opção, mediante manifestação a sua  sociedade corretora com simultânea ordem de compra no mercado à vista das 20.000 ações .  compra à  vista totalizou $ 160.000,00, enquanto o preço de exercício totalizou $ 200.000,00.   assim, sem considerar a corretagem e outras despesas, temos:   valor contratual recebido (exercício da opção)  $ 200.000,00   custo de aquisição da operação:   valor prêmio pago $20.000,00   compra à vista do ativo  $160.000,00   custo total  $ 180.000,00   ganho líquido $ 20.000,00   2 - lançador de opção de venda (comprador)    custo de aquisição é o preço de exercício do ativo, diminuído do valor do prêmio recebido.   considera-se preço de exercício o valor de compra do ativo acordado para liquidação da operação.    ganho líquido é a diferença positiva entre o preço de venda à vista do ativo, na data do exercício, e o seu  custo de aquisição.   ocorrendo a venda posteriormente à data do exercício, o ganho líquido é a diferença positiva entre o valor  recebido pela venda do ativo e o custo médio de aquisição, apurado conforme estabelecido para o mercado  à vista.   não ocorrendo o encerramento ou exercício da opção, o valor do prêmio recebido constitui ganho líquido para  o lançador. ',
 'qual é o tratamento dado ao valor do prêmio quando não ocorre o exercício ou o  encerramento da opção?    valor do prêmio constitui ganho para o lançador e perda para o titular na data do vencimento da opção.  ',
 'que é o mercado futuro?   é uma modalidade de mercado a prazo em que são negociados contratos de lotes padronizados, de  determinado ativo, com data de liquidação futura previamente acordada.   é exigida da pos',
 'que se considera ganho líquido nos mercados futuros?     ganho líquido é o resultado positivo da soma algébrica dos ajustes diários por ocasião da liquidação dos  contratos ou da cessão ou encerramento da posição, em cada mês.   os resultados, positivos ou negativos, apurados em cada contrato corresponderão à soma algébrica dos  ajustes diários incorridos entre as datas de abertura e de encerramento ou de liquidação do contrato, total ou  parcial.    resultado é apurado na liquidação da operação, parcial ou total.  ',
 'quem está obrigado a preencher o demonstrativo de apuração de ganhos - renda variável?   este demonstrativo deve ser preenchido, com a utilização do programa irpf2020, pelo contribuinte pessoa  física, residente no brasil, que durante o ano-calendário de 2019 efetuou:   1 - alienação de ações no mercado à vista em bolsa de valores;   2 - alienação de ouro, ativo financeiro, no mercado disponível ou à vista em bolsa de mercadorias e de futuros  ou diretamente junto a instituições financeiras;   3 - operações nos mercados a termo, de opções e futuro, realizadas em bolsa de valores, de mercadorias e  de futuros, com qualquer ativo;   4 - operações realizadas em mercados de liquidação futura, fora de bolsa, inclusive com opções flexíveis.   atenção:    fica dispensado de preencher este demonstrativo o contribuinte que tenha auferido, no ano- calendário, ganhos líquidos nas operações isentas abaixo relacionadas, exceto no caso de  pretender compensar as perdas apuradas com ganhos auferidos em operações realizadas em  bolsa sujeitas à incidência do imposto:    - com ações, no mercado à vista de bolsas de valores ou mercado de balcão, se o total das  alienações desse ativo, realizadas no mês, não exceder a $ 20.000, 00 (vinte mil reais);     - com ouro, ativo financeiro, se o total das alienações desse ativo, realizadas no mês, não exceder  a $ 20.000,00 (vinte mil reais);    – com ações de pequenas e médias empresas a que se refere o art. 16 da lei nº 13.043, de 13  de novembro de 2014.  ',
 'qual é o prazo para o pagamento do  sobre os ganhos líquidos auferidos no mercado de  renda variável?    imposto sobre a renda deve ser pago até o último dia útil do mês subsequente àquele em que os ganhos  houverem sido apurados.    código a ser utilizado no documento de arrecadação das receitas federais (darf) para pagamento desse  tributo é 6015. ',
 'os ganhos líquidos em renda variável devem ser oferecidos à tributação na declaração de  ajuste anual?   não. os ganhos líquidos são apurados e tributados, mês a mês, em separado, e não integram a base de  cálculo do imposto sobre a renda na declaração de ajuste anual. da mesma forma, o imposto pago não pode  ser deduzido do devido na declaração.   ',
 'como são tributados os ganhos obtidos pelos quotistas de fundos de ações?   são tributados no resgate de quotas à alíquota de 15%. esse imposto será retido pelo administrador do fundo  na data do resgate das quotas, sendo considerado exclusivo de fonte. ',
 'como são tributadas as operações no mercado de renda variável e de renda fixa realizadas  direta e individualmente por não residente no brasil?   1 - ressalvados os itens 2 e 3, os rendimentos decorrentes de aplicações financeiras de renda fixa e em  fundos de investimento, os ganhos líquidos auferidos em operações realizadas em bolsas de valores, de  mercadorias, de futuros e assemelhadas, os ganhos líquidos auferidos na alienação de ouro, ativo financeiro,  e em operações realizadas nos mercados de liquidação futura, fora de bolsa, e os rendimentos auferidos nas  operações de swap estão sujeitos às mesmas normas de tributação pelo imposto sobre a renda, previstas  para o residente no brasil.   2 - os rendimentos auferidos por investidor não residente, individual ou coletivo, que realizar operações  financeiras no brasil de acordo com as normas e condições estabelecidas pelo conselho monetário nacional,  sujeitam-se à incidência do imposto sobre a renda às seguintes alíquotas:    - 10%, no caso de aplicações nos fundos de investimento em ações, em operações de swap, registradas  ou não em bolsa, e nas operações realizadas em mercados de liquidação futura, fora de bolsa;    - 15%, nos demais casos, inclusive em aplicações financeiras de renda fixa, realizadas no mercado de  balcão ou em bolsa.   3 - não estão sujeitos à incidência do imposto sobre a renda os ganhos de capital auferidos pelos investidores  não residentes que realizarem operações financeiras no brasil de acordo com as normas e condições  estabelecidas pelo conselho monetário nacional:    - nas operações realizadas em bolsas de valores, de mercadorias, de futuros e assemelhadas, com  exceção das operações conjugadas;    - nas operações com ouro, ativo financeiro, fora de bolsa.   atenção:    esse regime de tributação não se aplica a investimento oriundo de país que não tribute a renda ou  que a tribute a alíquota inferior a 20%, o qual sujeitar-se-á às mesmas regras estabelecidas para os  residentes no brasil.  ',
 'qual é o tratamento tributário dos rendimentos obtidos pela pessoa física nas aplicações de  renda fixa?   os rendimentos produzidos por aplicação financeira de renda fixa são tributados na fonte às alíquotas de:   a) 22,5%, em aplicações com prazo de até seis meses;   b) 20%, em aplicações com prazo de seis meses e um dia até doze meses;   c) 17,5%, em aplicações com prazo de doze meses e um dia até vinte e quatro meses;   d) 15%, em aplicações com prazo acima de vinte e quatro meses.  ',
 'pode ser compensado na declaração anual o imposto sobre a renda retido em aplicação de  renda fixa?   não.  imposto retido é considerado como devido exclusivamente na fonte e os rendimentos dessas  aplicações não integram a base de cálculo do imposto sobre a renda na declaração de ajuste anual.',
 'como são tributados os rendimentos obtidos pelos quotistas de fundos de renda fixa?   se classificados como de longo prazo, os rendimentos são tributados na fonte às alíquotas de:   a) 22,5%, em aplicações com prazo de até seis meses;   b) 22%, em aplicações com prazo de seis meses e um dia até doze meses;   c) 17,5%, em aplicações com prazo de doze meses e um dia até vinte e quatro meses;   d) 15%, em aplicações com prazo acima de vinte e quatro meses.   se classificados como de curto prazo, os  rendimentos são  tributados na fonte às alíquotas de:   a) 22,5%, em aplicações com prazo de até seis meses;   b) 20%, em aplicações com prazo acima de seis meses.   atenção:    1)  alíquota é 0% no caso de rendimentos auferidos por pessoas físicas em operações com cotas  de emissão de fundos em direitos creditórios, observado o art. 2º da lei nº 12.431, de 24 de junho  de 2011:   2) os rendimentos auferidos por cotistas de fundos de índice de renda fixa cujas carteiras sejam  compostas por ativos financeiros que busquem refletir as variações e rentabilidade de índices de  renda fixa nos termos do art. 2º da lei nº 13.043, de 13 de novembro de 2014, sujeitam-se às  seguintes alíquotas:   2.1) 25%, no caso de fundos cuja carteira de ativos financeiros apresente prazo médio de  repactuação igual ou inferior a 180 (cento e oitenta) dias;   2.2) 20%, no caso de fundos cuja carteira de ativos financeiros apresente prazo médio de  repactuação superior a cento e oitenta dias e igual ou inferior a 720 (setecentos e vinte) dias; e   2.3) 15%, no caso de fundos cuja carteira de ativos financeiros apresente prazo médio de  repactuação superior a 720 (setecentos e vinte) dias.   os ganhos de capital auferidos por cotistas dos fundos de índice de renda fixa são tributados  como aplicações financeiras de renda fixa, aplicando-se as alíquotas acima. ',
 'como devem ser tributadas as aplicações em renda variável na bolsa de valores efetuadas  por duas ou mais pessoas?   no caso de cônjuges ou companheiros que operem em bolsa de valores, o limite de isenção a que se refere  a pergunta 656 poderá ser utilizado por ambos, os quais, no decorrer do ano-calendário, devem apurar e  tributar separadamente os ganhos líquidos auferidos por cada um, não sendo permitida apuração e tributação  mensal em conjunto.   na hipótese de os cônjuges ou companheiros apresentarem a declaração de ajuste anual em conjunto, deve  ser preenchido um “demonstrativo de apuração de ganhos - renda variável” para cada cônjuge investidor,  registrando separadamente suas operações.  ',
 'quais são os ativos que produzem rendimentos isentos ou tributados à alíquota zero  de  imposto?   são isentos do imposto sobre a renda ou tributados à alíquota zero:     - os rendimentos auferidos por pessoa física em contas de depósitos de poupança;     - na fonte e na declaração de ajuste anual das pessoas físicas, a remuneração produzida por letras  hipotecárias, certificados de recebíveis imobiliários e letras de crédito imobiliário;     - na fonte e na declaração de ajuste anual das pessoas físicas, a remuneração produzida por certificado  de depósito agropecuário  , warrant agropecuário  , certificado de direitos creditórios do  agronegócio  , letra de crédito do agronegócio   e certificado de recebíveis do agronegócio   ;   atenção:    enquadra-se no conceito de remuneração, para fins da isenção, a parcela da variação cambial paga  pelo certificado de direitos creditórios do agronegócio   e pelo certificado de recebíveis do  agronegócio  , emitidos com cláusula de correção pela variação cambial.    - na fonte e na declaração de ajuste anual das pessoas físicas, a remuneração produzida pela cédula de  produto rural  , com liquidação financeira, instituída pela lei nº 8.929, de 22 de agosto de 1994, com a  redação dada pela lei nº 10.200, de 14 de fevereiro de 2001, desde que negociada no mercado financeiro;    – os rendimentos e ganhos de capital produzidos por debêntures emitidas por sociedade de propósito  específico, por certificados de recebíveis imobiliários e por cotas de emissão de fundo de investimento em  direitos creditórios, emitidos na forma do art. 2º da lei nº 12.431, de 24 de junho de 2011; – os rendimentos e ganhos de capital produzidos pela letra imobiliária garantida   de que trata o art. 63  da lei nº 13.097, de 19 de janeiro de 2015.    isenção não se estende aos ganhos de capital na alienação ou cessão. ',
 'como são tributados os rendimentos oriundos de títulos de capitalização?   são tributados exclusivamente na fonte:   1 - à alíquota de 25%:     os benefícios líquidos resultantes da amortização antecipada, mediante sorteio, dos títulos de  economia denominados capitalização;     os benefícios atribuídos aos portadores de títulos de capitalização nos lucros da empresa emitente.   2 - à alíquota de 20%:     nas demais hipóteses, inclusive no caso de resgate sem ocorrência de sorteio.   3 - à alíquota de 30%:     os prêmios em dinheiro, mediante sorteio, sem amortização antecipada. ',
 'qual é o tratamento tributário dos ganhos em operações de compra e venda de ouro?   quando as operações forem realizadas em bolsas de valores, de mercadorias, de futuros ou pelas instituições  integrantes do sistema financeiro nacional, aplica-se o tratamento tributário previsto para as operações de  renda variável, tendo em vista tratar-se de ouro, ativo financeiro, conforme as perguntas nºs 651, 653 e 654.    já operações de mútuo e de compra vinculada à revenda, tendo por objeto ouro, ativo financeiro, são  equiparadas às operações de renda fixa, para fins de incidência do imposto sobre a renda na fonte.   no caso de operações com ouro não considerado ativo financeiro na forma da lei nº 7.766, de 11 de maio de  1989, o lucro apurado na alienação sujeita-se ao tratamento tributário conferido ao ganho de capital, inclusive  quando o ouro for transformado em joias ou já adquirido sob a forma de joias, para uso pessoal do adquirente  ou de seus familiares.   as disposições acima não alcançam os ganhos com ouro adquirido e revendido em sua forma bruta (minério  bruto, pó, grão, pepita).  lucro resultante é tributado como ganho de capital, se eventual a operação, ou como  o é na pessoa jurídica, se habitual a operação, exceto se efetuada por garimpeiro na venda a empresas  legalmente habilitadas nos termos do art. 10 da lei nº 7.713, de 22 de dezembro de 1988.   atenção:     comprovação dos rendimentos obtidos em operações com ouro não considerado como ativo  financeiro pode ser feita por meio dos documentos normalmente utilizados em negócios de compra  e venda, tais como notas fiscais, contratos ou recibos, desde que neles constem a identificação dos  interessados e demais informações pertinentes às operações, inclusive com a indicação de seus  endereços e dos respectivos números de inscrição no cadastro de pessoas físicas   ou no  cadastro nacional da pessoa jurídica  , conforme o caso. ',
 'qual é o tratamento tributário do lucro obtido na venda de pedras e metais preciosos?    tratamento tributário depende da forma sob a qual o metal ou a pedra for negociado:   1 - adquirido e revendido em sua forma bruta (pedra bruta, pó, grão, pepita):    lucro resultante é tributado como ganho de capital, se eventual a operação ou como o é na pessoa jurídica,  se habitual a operação. as operações de alienação efetuadas pelos garimpeiros a empresas legalmente  habilitadas são tributadas na fonte e na declaração de ajuste anual, em rendimentos tributáveis recebidos  de pessoas jurídicas.      279   2 - adquirido em qualquer das formas referidas no item 1 e transformado em joias ou já adquirido sob a forma  de joias:    lucro apurado em sua venda, se eventual a operação, tem o tratamento tributário de ganho de capital.   atenção:     comprovação dos rendimentos obtidos nessas operações, por tratar-se de negócios comuns e  franqueados a qualquer pessoa física ou jurídica, pode ser feita por meio de documentos  normalmente utilizados nos negócios de compra e venda, tais como notas fiscais, contratos ou  recibos, desde que neles constem a identificação dos interessados e demais informações  pertinentes às operações, inclusive com a indicação de seus endereços e dos respectivos números  de inscrição no cadastro de pessoas físicas   ou no cadastro nacional da pessoa jurídica   , conforme o caso.',
 'qual é o tratamento tributário dos rendimentos produzidos nas aplicações em caderneta de  poupança?   os rendimentos obtidos em caderneta de poupança pela pessoa física estão isentos do imposto sobre a renda,   ainda que em virtude de decisão judicial que tenha determinado a correção dos valores depositados por índice   diferente do fixado pela autoridade monetária.  ',
 'qual é o tratamento tributário dos rendimentos produzidos em caderneta de poupança do tipo  pecúlio?    caderneta de poupança tipo pecúlio, instituída pelo decreto-lei nº 2.301, de 21 de novembro de 1986,  constituída com instituições financeiras integrantes do sistema financeiro da habitação   - sociedades  de crédito imobiliário, caixas econômicas e associações de poupança e empréstimos - e destinada à formação  voluntária de poupança para desfrute durante a aposentadoria do seu titular, tem o mesmo tratamento  tributário determinado para as demais cadernetas de poupança do , ou autorizadas pelo conselho  monetário nacional  , isto é, os rendimentos produzidos estão isentos do imposto sobre a renda.  ',
 'qual é o tratamento tributário dos rendimentos produzidos por caderneta de poupança em  nome de menor, cujo depósito é efetuado em decorrência de ordem judicial?   tratando-se de caderneta de poupança do sistema financeiro de habitação ou autorizada pelo conselho  monetário nacional  , os rendimentos auferidos têm o mesmo tratamento fiscal de qualquer outra  caderneta de poupança; portanto, estão isentos do imposto sobre a renda.  ',
 'as importâncias correspondentes aos juros de caderneta de poupança remetidas para  beneficiário pessoa física não residente no brasil são tributáveis?   não. relativamente aos juros de caderneta de poupança, o não residente sujeita-se às mesmas normas de  tributação previstas para o residente no brasil. assim, os rendimentos correspondentes aos juros creditados  estão isentos.'
 ]

# Shuffle no corpus
corpus = shuffle(corpus)


def make_qa(corpus):
    questions, docs = [],[]
    for text in corpus:
        q_point = text.find('?')+1
        questions.append(text[:q_point].strip())
        docs.append(text[q_point:].strip())
    assert len(questions) == len(docs), f'ERRO len'

    return questions, docs

q, d = make_qa(corpus)

df = pd.DataFrame(
    {
        'question': q,
        'doc': d,
    }
    )
qid_to_question = {k:v for k, v in enumerate(df.question)}
question_to_qid = {k:v for v, k in qid_to_question.items()}

pid_to_doc = {k:v for k, v in enumerate(df.doc)}
doc_to_pid = {k:v for v, k in pid_to_doc.items()}

assert len(qid_to_question) == len(pid_to_doc) == df.shape[0]

df['qid'] = qid_to_question.keys()
df['pid'] = pid_to_doc.keys()
df = df[['qid', 'question', 'pid', 'doc']]
df

,qid,question,pid,doc
0,0,podem ser deduzidas as despesas efetuadas com ...,0,sim. são dedutíveis os pagamentos de despesas ...
1,1,qual é o tratamento tributário dos rendimentos...,1,deve-se verificar se existe ato internacional ...
2,2,qual é o tratamento tributário do valor obtido...,2,valor recebido na venda de produto rural deve ...
3,3,qual é o tratamento tributário no caso de cess...,3,ocorrendo a cessão do exercício do usufruto: ...
4,4,pessoa que passe a receber o crédito educativo...,4,fato de o dependente receber crédito educativo...
...,...,...,...,...
674,674,de quem é a responsabilidade pelo recolhimento...,674,até o término do prazo fixado para a entrega d...
675,675,qual é o tratamento tributário dos rendimentos...,675,os empregados de empresas estatais estrangeira...
676,676,que se considera bem de pequeno valor para fin...,676,considera-se bem de pequeno valor aquele decor...
677,677,que se considera como data e valor de aquisiçã...,677,"nesta hipótese, consideram-se a data e o valor..."


In [19]:
def pt_stop_words(path):
    #read_file with stop_words
    with open(path) as f:
        stop_words = f.readlines()

    pt_stop_words = []
    for w in stop_words:
        #remove break lines and spaces
        pt_stop_words.append(w.replace('\n', '').strip())

    return pt_stop_words

path = '/content/drive/MyDrive/Colab Notebooks/stopwords.txt'
stop_words = pt_stop_words(path)

In [20]:
# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in stop_words:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(df.doc.values):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

In [46]:
x = np.array([4, 3, 7, 1])
x[np.argpartition(x, 0)]

array([1, 3, 7, 4])

In [74]:
def search_one(query, K=20, passages=df.doc.values):
    print("Input question:", query)
    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-K:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print(f"Top-{K} lexical search (BM25) hits")
    
    for hit in bm25_hits[0:K]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))
    
def search_all(queries, K=20, passages=df.doc.values):
    
    topK_pids = {}
    for q in queries:

        ##### BM25 search (lexical search) #####
        bm25_scores = bm25.get_scores(bm25_tokenizer(q))
        top_n = np.argpartition(bm25_scores, -5)[-K:]
        bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
        bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
        
        pids = []
        qid = question_to_qid[q]
        for hit in bm25_hits[0:K]:
            pids.append(doc_to_pid[passages[hit['corpus_id']]])
        
        topK_pids[qid] = pids
    return topK_pids

In [75]:
queries = df.question.values
search_one(query=queries[0])

Input question: podem ser deduzidas as despesas efetuadas com instrução?
Top-20 lexical search (BM25) hits
	18.523	sim. as despesas com instrução efetuadas antes do divórcio podem ser deduzidas desde que os filhos figurem  como dependentes na declaração do ano-calendário relativo ao divórcio. admite-se a dedução até o limite  anual individual de $ 3.561,50 para o ano-calendário de 2019.
	13.788	não. como regra geral, somente são dedutíveis na declaração as despesas médicas e com instrução de  pessoas físicas consideradas dependentes perante a legislação tributária e incluídas na declaração do  responsável em que for considerado dependente.   contudo, podem ser deduzidas na declaração as despesas médicas e com instrução pagas pelo declarante  referentes a alimentandos, desde que em cumprimento de decisão judicial ou acordo homologado  judicialmente, ou por escritura pública, observados os limites legais.
	13.023	via de regra, as despesas médico-hospitalares próprias de um dos cônjuges o

In [84]:
topK_pids = search_all(queries)

for i, (q, pids) in enumerate(topK_pids.items()):
    print(f'Query: {qid_to_question[q]}\n')
    print('Relevant docs:')
    for j, pid in enumerate(pids, 1):
        print(f'pos. {j:>3} -- {pid_to_doc[pid]}')
    print()

    if i == 3: break

Query: podem ser deduzidas as despesas efetuadas com instrução?

Relevant docs:
pos.   1 -- sim. as despesas com instrução efetuadas antes do divórcio podem ser deduzidas desde que os filhos figurem  como dependentes na declaração do ano-calendário relativo ao divórcio. admite-se a dedução até o limite  anual individual de $ 3.561,50 para o ano-calendário de 2019.
pos.   2 -- não. como regra geral, somente são dedutíveis na declaração as despesas médicas e com instrução de  pessoas físicas consideradas dependentes perante a legislação tributária e incluídas na declaração do  responsável em que for considerado dependente.   contudo, podem ser deduzidas na declaração as despesas médicas e com instrução pagas pelo declarante  referentes a alimentandos, desde que em cumprimento de decisão judicial ou acordo homologado  judicialmente, ou por escritura pública, observados os limites legais.
pos.   3 -- via de regra, as despesas médico-hospitalares próprias de um dos cônjuges ou companheiro(a